In [3]:
import pandas as pd
import json
import re
import numpy as np
import spacy

In [59]:
issues = """hit:3484_conv:6969,1,caring,I found a stray dog last night and took him home with me.,244,I found a stray dog last night and took him home with me.,5|5|5_5|5|5,
hit:3484_conv:6969,2,caring,I found a stray dog last night and took him home with me.,447,What a nice thing to do.  Do you know what breed?,5|5|5_5|5|5,
hit:3484_conv:6969,3,caring,I found a stray dog last night and took him home with me.,244,some kind of heeler mix. Not sure.,5|5|5_5|5|5,
hit:3484_conv:6969,4,caring,I found a stray dog last night and took him home with me.,447,Is he a young dog or old?,5|5|5_5|5|5,
hit:3484_conv:6969,5,caring,I found a stray dog last night and took him home with me.,244,He looks young. A year or two old.,5|5|5_5|5|5,
hit:3485_conv:6970,1,excited,Me and my college friends have planned a trip to Miami this upcoming weekend. It is going to be awesome,155,This upcoming weekend me and my college friends planned a trip to Miami. It is going to be a fantastic trip for me,4|5|5_5|5|5,
hit:3485_conv:6970,2,excited,Me and my college friends have planned a trip to Miami this upcoming weekend. It is going to be awesome,389,Awesome!  What are you all going to do there?,4|5|5_5|5|5,
hit:3485_conv:6970,3,excited,Me and my college friends have planned a trip to Miami this upcoming weekend. It is going to be awesome,155,As you know this summer is very hot_comma_ we planned to spend the whole days in the beaches,4|5|5_5|5|5,
hit:3485_conv:6970,4,excited,Me and my college friends have planned a trip to Miami this upcoming weekend. It is going to be awesome,389,That sounds awesome!  Any beer drinking at the beach?  That's my favorite!,4|5|5_5|5|5,
hit:3485_conv:6971,1,prepared,Getting ready for florida,389,Buying some new clothes and packing,5|5|5_4|5|5,
hit:3485_conv:6971,2,prepared,Getting ready for florida,155,Are you also going for a trip?,5|5|5_4|5|5,
hit:3485_conv:6971,3,prepared,Getting ready for florida,389,Yup.  Getting ready for a vacation next month.  Taking my 3yo nephew.,5|5|5_4|5|5,
hit:3485_conv:6971,4,prepared,Getting ready for florida,155,Nice. Where is your destination?,5|5|5_4|5|5,
hit:3485_conv:6971,5,prepared,Getting ready for florida,389,We're heading to Panama City Florida.  Should be an awesome time!,5|5|5_4|5|5,
hit:3487_conv:6974,1,impressed,My dog can now shake hands on command.,244,My dog can now shake hands on command.,5|5|5_5|5|5,
hit:3487_conv:6974,2,impressed,My dog can now shake hands on command.,447,I love when my dog does that.  Did it take you long to train him.,5|5|5_5|5|5,
hit:3487_conv:6974,3,impressed,My dog can now shake hands on command.,244,Not long at all. Once she figures out what you want its pretty easy to get her to remember.,5|5|5_5|5|5,
hit:3487_conv:6974,4,impressed,My dog can now shake hands on command.,447,How do you reward her?,5|5|5_5|5|5,
hit:3487_conv:6974,5,impressed,My dog can now shake hands on command.,244,With a treat.,5|5|5_5|5|5,
hit:3491_conv:6982,1,angry,I asked my co-worker to let me know of any change in the project. He did not even bother himself to inform me,155,I missed this week's meeting and asked a co-worker to e-mail me of any changes about the project. He did not even bother himself to inform me,4|5|5_4|5|5,
hit:3491_conv:6982,2,angry,I asked my co-worker to let me know of any change in the project. He did not even bother himself to inform me,448,That sucks_comma_ so did you waste a lot of time on it?,4|5|5_4|5|5,
hit:3491_conv:6982,3,angry,I asked my co-worker to let me know of any change in the project. He did not even bother himself to inform me,155,I just asked him to e-mail me of any changed so that i could be kept in a loop. I am very furious for a careless act,4|5|5_4|5|5,
hit:3491_conv:6982,4,angry,I asked my co-worker to let me know of any change in the project. He did not even bother himself to inform me,448,Yeah_comma_ I would be too_comma_ it only takes like a couple minutes to email..  So are you going to get in trouble?,4|5|5_4|5|5,
hit:3491_conv:6983,1,excited,I'm excited to go see a concert coming up in September,448,Hey man_comma_ can't wait for september_comma_ going to go see Bon Jovi_comma_ what about you?,4|5|5_4|5|5,
hit:3491_conv:6983,2,excited,I'm excited to go see a concert coming up in September,155,Cool. Are you going alone or with friends?,4|5|5_4|5|5,
hit:3491_conv:6983,3,excited,I'm excited to go see a concert coming up in September,448,A big group of us from work got tickets real cheap_comma_ I'm super excited_comma_ haven't been to a concert in years!,4|5|5_4|5|5,
hit:3491_conv:6983,4,excited,I'm excited to go see a concert coming up in September,155,Good for you. To be honest_comma_ i do not really enjoy attending live concerts,4|5|5_4|5|5,
hit:3493_conv:6986,1,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,449,Hi!  I planned a get together at my house and was surprised.,4|4|5_5|5|5,<HI>
hit:3493_conv:6986,2,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,160,What happened?  Do tell :),4|4|5_5|5|5,
hit:3493_conv:6986,3,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,449,I could only think of 5 people to invite_comma_ and was worried they wouldn't even come. But then...,4|4|5_5|5|5,
hit:3493_conv:6986,4,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,160,Well.. I guess I will never know...,4|4|5_5|5|5,
hit:3493_conv:6986,5,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,449,I was surprised because 15 people showed up and it turned out like a party!,4|4|5_5|5|5,
hit:3493_conv:6986,6,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,160,Oh Snap!  wow... That was a surprise!  ,4|4|5_5|5|5,
hit:3493_conv:6986,7,surprised,I invited some people to my house for a get together and didn't think that many people would come.  In fact_comma_ I could only initially think of 5 people to invite.  The night came_comma_ and so did more than 15 people.  I was surprise at the turn out.,449,It turned out to be a fun night.,4|4|5_5|5|5,
hit:3493_conv:6987,1,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,160,I am in the best of moods... I am going on vacation tomorrow!,5|5|5_4|4|5,
hit:3493_conv:6987,2,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,449,I bet! Where are you going on vacation?,5|5|5_4|4|5,
hit:3493_conv:6987,3,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,160,Hawaii!  I have been saving forever!,5|5|5_4|4|5,
hit:3493_conv:6987,4,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,449,Wow! Hawaii sounds like a great place to vacay.  I have never been there_comma_ have you?,5|5|5_4|4|5,
hit:3493_conv:6987,5,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,160,Nope.. thats why I am so excited.  With the volcano stuff and all it has been sorta grim_comma_ but I can still make the best of it.,5|5|5_4|4|5,
hit:3493_conv:6987,6,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,449,Oh yeah_comma_ I didn't even think about that.  It was terrifying to watch on tv.  I hope you are staying somewhere safe.,5|5|5_4|4|5,
hit:3493_conv:6987,7,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,160,I am it is a mission trip but I am taking part of it as a normal vaca too,5|5|5_4|4|5,
hit:3493_conv:6987,8,excited,I am in such a good mood.  I am going on vacation tomorrow!!!  Going to Hawaii.,449,That's awesome.  Are you traveling with others?,5|5|5_4|4|5,
hit:3494_conv:6988,1,furious,The rain cancelled my plans for today.,244,The rain cancelled my plans for today.,5|5|5_5|5|5,
hit:3494_conv:6988,2,furious,The rain cancelled my plans for today.,160,I am so sorry.  That has been happening a lot lately on the East Coast.,5|5|5_5|5|5,
hit:3494_conv:6988,3,furious,The rain cancelled my plans for today.,244,Yeah_comma_ it sucks.,5|5|5_5|5|5,
hit:3494_conv:6988,4,furious,The rain cancelled my plans for today.,160,I feel your pain I am in Florida.,5|5|5_5|5|5,
hit:3494_conv:6989,1,sad,So yeah.. had bury my goldfish this morning.  Sad times.,160,So yeah.. my goldfish died this morning.,5|5|5_5|5|5,
hit:3494_conv:6989,2,sad,So yeah.. had bury my goldfish this morning.  Sad times.,244,That's sad. Did you give him a burial at sea?,5|5|5_5|5|5,
hit:3494_conv:6989,3,sad,So yeah.. had bury my goldfish this morning.  Sad times.,160,Indeed I did_comma_ hence my emotions.  ,5|5|5_5|5|5,
hit:3494_conv:6989,4,sad,So yeah.. had bury my goldfish this morning.  Sad times.,244,Do you plan on getting another fish?,5|5|5_5|5|5,
hit:3496_conv:6993,1,furious,I am so mad.  My dogs won't stop barking and its making me crazy,160,I am so mad.. my dogs wont stop barking and I am going absolutely nuts,5|5|5_5|5|5,
hit:3496_conv:6993,2,furious,I am so mad.  My dogs won't stop barking and its making me crazy,447,I feel you_comma_ nothing irritates me more than a dog that won't stop barking.  Any plans to fix the problem?,5|5|5_5|5|5,
hit:3496_conv:6993,3,furious,I am so mad.  My dogs won't stop barking and its making me crazy,160,Well I have four dogs.  Sooo.  Do I shoot the dogs or all the people mowing the grass?,5|5|5_5|5|5,
hit:3496_conv:6993,4,furious,I am so mad.  My dogs won't stop barking and its making me crazy,447,Ha_comma_ neither.  Look into some barking collars or maybe some training.,5|5|5_5|5|5,
hit:3497_conv:6994,1,surprised,I got a invite to go camping next weekend.,244,I got a invite to go camping next weekend.,5|5|5_5|5|5,
hit:3497_conv:6994,2,surprised,I got a invite to go camping next weekend.,226,Is it something that you want to do?,5|5|5_5|5|5,
hit:3497_conv:6994,3,surprised,I got a invite to go camping next weekend.,244,yes. I love the outdoors.,5|5|5_5|5|5,
hit:3497_conv:6994,4,surprised,I got a invite to go camping next weekend.,226,That's a great invitation then. Will you be able to go?,5|5|5_5|5|5,
hit:3497_conv:6994,5,surprised,I got a invite to go camping next weekend.,244,Yes I'm going to go.,5|5|5_5|5|5,
hit:3497_conv:6995,1,furious,I wasn't paying attention and locked my keys in the car. I was livid as I did not have a spare.,226,I wasn't paying attention and locked my keys in the car. I was livid as I did not have a spare.,5|5|5_5|5|5,
hit:3497_conv:6995,2,furious,I wasn't paying attention and locked my keys in the car. I was livid as I did not have a spare.,244,Did you pay a locksmith to come get it open for you?,5|5|5_5|5|5,
hit:3497_conv:6995,3,furious,I wasn't paying attention and locked my keys in the car. I was livid as I did not have a spare.,226,Unfortunately I had no other choice. You can bet that I got another key made.,5|5|5_5|5|5,
hit:3497_conv:6995,4,furious,I wasn't paying attention and locked my keys in the car. I was livid as I did not have a spare.,244,I had to go that route once.  and my truck has some sort of special chip in the key. I literally had to have my truck towed to  the dealership.,5|5|5_5|5|5,
hit:3498_conv:6996,1,hopeful,I start my new job this week.  Feeling a little uneasy..,160,I start my new job this week. Feeling a little uneasy..,5|5|5_4|5|5,
hit:3498_conv:6996,2,hopeful,I start my new job this week.  Feeling a little uneasy..,52,Congrats!! That's wonderful. why do you feel unease ?,5|5|5_4|5|5,
hit:3498_conv:6996,3,hopeful,I start my new job this week.  Feeling a little uneasy..,160,You know.. new people.. fresh start.. the unknown.. ,5|5|5_4|5|5,
hit:3498_conv:6996,4,hopeful,I start my new job this week.  Feeling a little uneasy..,52,i understand. It is hard to start but i'm sure you will make new friends at your new job!,5|5|5_4|5|5,
hit:3498_conv:6997,1,faithful,I would never cheat on my wife,52,I love my wife so much_comma_ i would never be able to cheat on her. i don't know how some people can cheat on their spouses,4|5|5_5|5|5,
hit:3498_conv:6997,2,faithful,I would never cheat on my wife,160,Well I love my husband and I have cheated so it really one doesn't have much to do with the other.,4|5|5_5|5|5,
hit:3498_conv:6997,3,faithful,I would never cheat on my wife,52,why would you cheat on him? I will never understand that,4|5|5_5|5|5,
hit:3498_conv:6997,4,faithful,I would never cheat on my wife,160,its not about cheating... you can love someone but if they don't meet certain needs_comma_ not just sexual... you can get it elsewhere but that doesnt negate your love for the person who is unable to fulfill your needs,4|5|5_5|5|5,
hit:3499_conv:6999,1,anxious,Whenever I go to family get together_comma_ I always feel extremely anxious and out of place,448,Anytime I have a family get together coming up_comma_ like during the holidays_comma_ I get super anxious and worry non-stop about it,5|5|5_5|5|5,
hit:3499_conv:6999,2,anxious,Whenever I go to family get together_comma_ I always feel extremely anxious and out of place,447,Same here.  Is it because of needed prepartations or because of family drama?,5|5|5_5|5|5,
hit:3499_conv:6999,3,anxious,Whenever I go to family get together_comma_ I always feel extremely anxious and out of place,448,Yeah for the most part_comma_ I enjoy one side of my family_comma_ but other side_comma_ I could care less for.  Lots of drama and subtle remarks.,5|5|5_5|5|5,
hit:3499_conv:6999,4,anxious,Whenever I go to family get together_comma_ I always feel extremely anxious and out of place,447,Drama is never good.  I usually don't re-invite people who bring it with them.,5|5|5_5|5|5,
hit:3501_conv:7002,1,hopeful,i bought cbd oil in hopes my anxiety would go away,321,Ive suffered from anxiety and sciatica for years _comma_ the anxiety since i was small _comma_ one day i ordered cbd oil in hopes it would go away,5|5|5_5|5|5,
hit:3501_conv:7002,2,hopeful,i bought cbd oil in hopes my anxiety would go away,50,what is CBD oil? I feel bad for you.. i fell you though_comma_ i recently started having bad anxiety attacks_comma_ i will see a doctor in 2 weeks,5|5|5_5|5|5,
hit:3501_conv:7002,3,hopeful,i bought cbd oil in hopes my anxiety would go away,321,my anxiety went away _comma_ and back pain is not ass bad _comma_ cbd oil is the extract of cannabis _comma_ legal in all states trump made it so _comma_ look up the youtube video charlottes web its awesome,5|5|5_5|5|5,<POLITICAL>
hit:3501_conv:7002,4,hopeful,i bought cbd oil in hopes my anxiety would go away,50,I don't think i would ever be comfortable using cannabis. even if it's legal. but i'm really glad it worked for you,5|5|5_5|5|5,
hit:3501_conv:7002,5,hopeful,i bought cbd oil in hopes my anxiety would go away,321,its none psycho active _comma_ it just contains all the healing properties _comma_ but thank you ,5|5|5_5|5|5,
hit:3503_conv:7007,1,devastated,I was trying to do something nice for my son and give him some furniture_comma_ but you know how teenagers are. They never answer their phones_comma_ and he decided that he was going to ignore me.,450,I don't like how older kids ignore your messages.,5|5|5_5|5|5,
hit:3503_conv:7007,2,devastated,I was trying to do something nice for my son and give him some furniture_comma_ but you know how teenagers are. They never answer their phones_comma_ and he decided that he was going to ignore me.,447,It can be infuriating.  Do you call or send texts?,5|5|5_5|5|5,
hit:3503_conv:7007,3,devastated,I was trying to do something nice for my son and give him some furniture_comma_ but you know how teenagers are. They never answer their phones_comma_ and he decided that he was going to ignore me.,450,I send texts. He is my step-son. I tried to give him new furniture_comma_ and he just ignored the text.,5|5|5_5|5|5,
hit:3503_conv:7007,4,devastated,I was trying to do something nice for my son and give him some furniture_comma_ but you know how teenagers are. They never answer their phones_comma_ and he decided that he was going to ignore me.,447,My Mom sends me texts a lot.  But I'm not always looking at my phone.  When she calls though I usually answer.,5|5|5_5|5|5,
hit:3503_conv:7007,5,devastated,I was trying to do something nice for my son and give him some furniture_comma_ but you know how teenagers are. They never answer their phones_comma_ and he decided that he was going to ignore me.,450,Well_comma_ at least you answer.,5|5|5_5|5|5,
hit:3504_conv:7008,1,embarrassed,I tripped in fell face first at a bar last night.,244,I tripped in fell face first at a bar last night.,5|5|5_5|5|5,
hit:3504_conv:7008,2,embarrassed,I tripped in fell face first at a bar last night.,53,Oh no! Were you drunk!,5|5|5_5|5|5,
hit:3504_conv:7008,3,embarrassed,I tripped in fell face first at a bar last night.,244,Maybe.,5|5|5_5|5|5,<UNIGRAM>
hit:3504_conv:7008,4,embarrassed,I tripped in fell face first at a bar last night.,53,Did you wake up with a big bruise?,5|5|5_5|5|5,
hit:3504_conv:7008,5,embarrassed,I tripped in fell face first at a bar last night.,244,Just my pride.,5|5|5_5|5|5,
hit:3504_conv:7008,6,embarrassed,I tripped in fell face first at a bar last night.,53,It happens. Have to be careful sometimes.,5|5|5_5|5|5,
hit:3504_conv:7009,1,sad,My 5 year old is about to start Kindergarten and I'm sad he's growing up so fast.,53,My little boy just turned 5 and he's about to start kindergarten :(,5|5|5_5|5|5,
hit:3504_conv:7009,2,sad,My 5 year old is about to start Kindergarten and I'm sad he's growing up so fast.,244,Look on the bright side_comma_ you'll have some extra free time,5|5|5_5|5|5,
hit:3504_conv:7009,3,sad,My 5 year old is about to start Kindergarten and I'm sad he's growing up so fast.,53,Yes that is definitely a plus!,5|5|5_5|5|5,
hit:3504_conv:7009,4,sad,My 5 year old is about to start Kindergarten and I'm sad he's growing up so fast.,244,I think you'll end up fine.  Its an important milestone.,5|5|5_5|5|5,
hit:3504_conv:7009,5,sad,My 5 year old is about to start Kindergarten and I'm sad he's growing up so fast.,53,Yes! It's very good for him to socialize.,5|5|5_5|5|5,
hit:3505_conv:7010,1,surprised,got offered assistant manager ,321,ive been working as a part timer at a store for 3 years_comma_ recently they offered my management position instead of a full time position _comma_ i was in shock,5|5|5_5|5|5,
hit:3505_conv:7010,2,surprised,got offered assistant manager ,447,Now that's a nice surprise.  You must have really impressed them.,5|5|5_5|5|5,
hit:3505_conv:7010,3,surprised,got offered assistant manager ,321,ive always been a good worker_comma_ i even tell management when they are messing up and correct them _comma_ im still not sure if want management but im still very surprised ,5|5|5_5|5|5,
hit:3505_conv:7010,4,surprised,got offered assistant manager ,447,I'd take it.  You sound like you might be good at it.,5|5|5_5|5|5,
hit:3506_conv:7012,1,annoyed,It hasn't rained here in a while and I have to water my yard today so it dosen't burn.,244,It hasn't rained here in a while and I have to water my yard today so it dosen't die.,5|5|5_5|5|5,
hit:3506_conv:7012,2,annoyed,It hasn't rained here in a while and I have to water my yard today so it dosen't burn.,50,i wish i could send our rain your way! It's been raining here for weeks!!,5|5|5_5|5|5,
hit:3506_conv:7012,3,annoyed,It hasn't rained here in a while and I have to water my yard today so it dosen't burn.,244,Thanks_comma_ we could use it.,5|5|5_5|5|5,
hit:3506_conv:7012,4,annoyed,It hasn't rained here in a while and I have to water my yard today so it dosen't burn.,50,It's been very nasty humid and hot in VA for the past 3 weeks,5|5|5_5|5|5,
hit:3509_conv:7018,1,trusting,I go to church every week without fail. I filled with full of hope whenever I leave from church.,43,I'll filled with hope whenever I've been in church.,4|4|4_4|5|5,
hit:3509_conv:7018,2,trusting,I go to church every week without fail. I filled with full of hope whenever I leave from church.,155,Good for you. Religion can be a source of hope and optimism for many people in times of distress,4|4|4_4|5|5,
hit:3509_conv:7018,3,trusting,I go to church every week without fail. I filled with full of hope whenever I leave from church.,43,Yeah_comma_ very good stress relief. So I never miss any Sunday! ,4|4|4_4|5|5,
hit:3509_conv:7018,4,trusting,I go to church every week without fail. I filled with full of hope whenever I leave from church.,155,I also try to find some time and attend church services whenever possible,4|4|4_4|5|5,
hit:3509_conv:7019,1,afraid,I received some degree of criticism from my supervisor for the quality of work i did. I feel very uncertain about my perspectives in this company,155,In the last weekly meeting_comma_ i received some fair share of criticism from my supervisor for the quality of work i did. I feel very uncertain,4|5|5_4|4|4,
hit:3509_conv:7019,2,afraid,I received some degree of criticism from my supervisor for the quality of work i did. I feel very uncertain about my perspectives in this company,43,oh that's ok_comma_ put more efforts next time. i believe you will come out of flying colors,4|5|5_4|4|4,
hit:3509_conv:7019,3,afraid,I received some degree of criticism from my supervisor for the quality of work i did. I feel very uncertain about my perspectives in this company,155,I don't know why_comma_ but lately i am feeling more sluggish and unmotivated to work hard. I am lost in the middle of emotional chaos,4|5|5_4|4|4,
hit:3509_conv:7019,4,afraid,I received some degree of criticism from my supervisor for the quality of work i did. I feel very uncertain about my perspectives in this company,43,is the work you are doing interests you_comma_ or else you would need to change the job,4|5|5_4|4|4,
hit:3511_conv:7022,1,content,When I take my kids to the pool and they swim and have fun I get to watch them and it's relaxing.,53,Everytime I take my kids to the pool. I feel so content while they run around and have fun.,5|5|5_5|5|5,
hit:3511_conv:7022,2,content,When I take my kids to the pool and they swim and have fun I get to watch them and it's relaxing.,244,Sounds like a good time.  I'm not surprised. Do you take them often?,5|5|5_5|5|5,
hit:3511_conv:7022,3,content,When I take my kids to the pool and they swim and have fun I get to watch them and it's relaxing.,53,Yes_comma_ we go at least once a week.,5|5|5_5|5|5,
hit:3511_conv:7022,4,content,When I take my kids to the pool and they swim and have fun I get to watch them and it's relaxing.,244,That sounds like lots of fun.,5|5|5_5|5|5,
hit:3511_conv:7022,5,content,When I take my kids to the pool and they swim and have fun I get to watch them and it's relaxing.,53,Yes it's very relaxing.,5|5|5_5|5|5,
hit:3511_conv:7023,1,sad,my dog died.,244,my dog died. ,5|5|5_5|5|5,
hit:3511_conv:7023,2,sad,my dog died.,53,Oh that's so rough. I had a dog die a few years ago and it was traumatic.,5|5|5_5|5|5,
hit:3511_conv:7023,3,sad,my dog died.,244,I had her for 10 years.,5|5|5_5|5|5,
hit:3511_conv:7023,4,sad,my dog died.,53,How did she die?,5|5|5_5|5|5,
hit:3511_conv:7023,5,sad,my dog died.,244,She had cancer.,5|5|5_5|5|5,
hit:3511_conv:7023,6,sad,my dog died.,53,I'm sorry. At least she isn't suffering anymore.,5|5|5_5|5|5,
hit:3512_conv:7024,1,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,10,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job,5|5|3_5|5|5,
hit:3512_conv:7024,2,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,447,Takes a good student to get that honar.  Wad he speach good?,5|5|3_5|5|5,
hit:3512_conv:7024,3,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,10,It was okay_comma_ he is scared of public speaking,5|5|3_5|5|5,
hit:3512_conv:7024,4,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,447,I used to be scared of it too.  Trick for me was to memorize my speaches and tune out while giving them.,5|5|3_5|5|5,
hit:3513_conv:7026,1,lonely,I moved out of my parents house and into my own apartment. It is very different without anyone around. It's too quiet.,243,I moved out of my parents house and into my own apartment.,5|5|5_5|5|5,
hit:3513_conv:7026,2,lonely,I moved out of my parents house and into my own apartment. It is very different without anyone around. It's too quiet.,52,that's really good! Huge step :) How do you like your new place?,5|5|5_5|5|5,
hit:3513_conv:7026,3,lonely,I moved out of my parents house and into my own apartment. It is very different without anyone around. It's too quiet.,243,It is very different without anyone around. It's too quiet.,5|5|5_5|5|5,
hit:3513_conv:7026,4,lonely,I moved out of my parents house and into my own apartment. It is very different without anyone around. It's too quiet.,52,Maybe you could get a pet? ,5|5|5_5|5|5,
hit:3513_conv:7027,1,grateful,I'm so happy i found a new job! we've been struggling financially for a couple of months,52,I'm so happy I found a new job! We've been struggling financially for a couple of months and i'm so thankful for my wife who was super supportive,5|5|5_5|5|5,
hit:3513_conv:7027,2,grateful,I'm so happy i found a new job! we've been struggling financially for a couple of months,243,Congrats! I bet that makes you feel great!,5|5|5_5|5|5,
hit:3513_conv:7027,3,grateful,I'm so happy i found a new job! we've been struggling financially for a couple of months,52,yes_comma_ it was really hard to get food on the table for the last couple of months but now things got better,5|5|5_5|5|5,
hit:3513_conv:7027,4,grateful,I'm so happy i found a new job! we've been struggling financially for a couple of months,243,That is really not a good feeling. I am glad you came out of it.,5|5|5_5|5|5,
hit:3514_conv:7028,1,terrified, I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,10, I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,5|5|5_5|4|5,
hit:3514_conv:7028,2,terrified, I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,416,good for you my friend! Darkness is nothing to be scared about,5|5|5_5|4|5,
hit:3514_conv:7028,3,terrified, I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,10,It was at the time_comma_ but I am over it now,5|5|5_5|4|5,
hit:3514_conv:7028,4,terrified, I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,416,I'm so happy for you_comma_ way to go!,5|5|5_5|4|5,
hit:3515_conv:7030,1,hopeful,I cannot wait for the new super smash bros for the nintendo switch_comma_ I hope it is really good!,10,I cannot wait for the new super smash bros for the nintendo switch_comma_ I hope it is really good!,5|5|5_4|5|5,
hit:3515_conv:7030,2,hopeful,I cannot wait for the new super smash bros for the nintendo switch_comma_ I hope it is really good!,391,I hear great reviews_comma_ people who have played it are raving about it,5|5|5_4|5|5,
hit:3515_conv:7030,3,hopeful,I cannot wait for the new super smash bros for the nintendo switch_comma_ I hope it is really good!,10,I have heard that too_comma_ I just wanna get my hands on it,5|5|5_4|5|5,
hit:3515_conv:7030,4,hopeful,I cannot wait for the new super smash bros for the nintendo switch_comma_ I hope it is really good!,391,I wish you get it as soon as possible buddy,5|5|5_4|5|5,
hit:3516_conv:7033,1,furious,I was out with friends last night and my wallet was stolen.  I think it was the waitress.,274,I was at dinner last night with friends and I lost my wallet.,5|5|5_5|5|5,
hit:3516_conv:7033,2,furious,I was out with friends last night and my wallet was stolen.  I think it was the waitress.,416,Oh shoot.. What did you do? did you find it back?,5|5|5_5|5|5,
hit:3516_conv:7033,3,furious,I was out with friends last night and my wallet was stolen.  I think it was the waitress.,274,I think the waitress took it off the table.,5|5|5_5|5|5,
hit:3516_conv:7033,4,furious,I was out with friends last night and my wallet was stolen.  I think it was the waitress.,416,damn.. hope your affairs are in control,5|5|5_5|5|5,
hit:3517_conv:7034,1,prepared,I'm about to have my third child and can't wait to finally have this baby.  The nursery is done_comma_ the clothes are ready_comma_ and I'm packed for the hospital. Good to go.,438,I'm about to have my third child and can't wait to finally have this baby. The nursery is done_comma_ the clothes are ready_comma_ and I'm packed for the hospital. Good to go.,5|5|5_I have clicked the 'End Chat' button thank you!|5|5,
hit:3517_conv:7034,2,prepared,I'm about to have my third child and can't wait to finally have this baby.  The nursery is done_comma_ the clothes are ready_comma_ and I'm packed for the hospital. Good to go.,10,By this time you probably have the process all down pat!,5|5|5_I have clicked the 'End Chat' button thank you!|5|5,
hit:3517_conv:7034,3,prepared,I'm about to have my third child and can't wait to finally have this baby.  The nursery is done_comma_ the clothes are ready_comma_ and I'm packed for the hospital. Good to go.,438,Yeah_comma_ I feel much more comfortable with the process and labor in general.  I have diapers stacked a mile high too...which is really the most important part. ha ha.,5|5|5_I have clicked the 'End Chat' button thank you!|5|5,
hit:3517_conv:7034,4,prepared,I'm about to have my third child and can't wait to finally have this baby.  The nursery is done_comma_ the clothes are ready_comma_ and I'm packed for the hospital. Good to go.,10,I have heard that_comma_ diapers and formula are expensive for no reason,5|5|5_I have clicked the 'End Chat' button thank you!|5|5,
hit:3517_conv:7035,1,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,10,I have heard that_comma_ diapers and formula are expensive for no reason,I have clicked the 'End Chat' button thank you!|5|5_5|5|5,
hit:3517_conv:7035,2,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,438,They sure are! I try to breastfeed but usually can't do that for more than 6 months so I'm hoping I can keep my milk supply up longer. It really does cost a fortune.  I also use reusable diapers at home.,I have clicked the 'End Chat' button thank you!|5|5_5|5|5,
hit:3517_conv:7035,3,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,10,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job,I have clicked the 'End Chat' button thank you!|5|5_5|5|5,
hit:3517_conv:7035,4,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,438,That is so cool!  Also_comma_ its great how proud you were of him.  When did that happen?,I have clicked the 'End Chat' button thank you!|5|5_5|5|5,
hit:3517_conv:7035,5,impressed,I was proud of my friend when he was the high school valedictorian_comma_ he did a great job!,10,That would save you a lot of money_comma_ I know formula specifically is very very pricey. I did not know reusable diapers existed!,I have clicked the 'End Chat' button thank you!|5|5_5|5|5,
hit:3518_conv:7037,1,embarrassed,I was at the movies last night and spilled my drink on some young boy next to me.,274,I went to the movies last night and made a fool of myself.,5|5|5_5|5|5,
hit:3518_conv:7037,2,embarrassed,I was at the movies last night and spilled my drink on some young boy next to me.,50,What happened?,5|5|5_5|5|5,
hit:3518_conv:7037,3,embarrassed,I was at the movies last night and spilled my drink on some young boy next to me.,274,I tripped over one of the seats and ended up spilling my drink all over some young boy.,5|5|5_5|5|5,
hit:3518_conv:7037,4,embarrassed,I was at the movies last night and spilled my drink on some young boy next to me.,50,oh no :( It happens though_comma_ it wasn't your fault!,5|5|5_5|5|5,

hit:832_conv:1665,1,embarrassed,I misspoke some words out in public and lots of people heard.,224, "I misspoke some words out in public and lots of people heard.,5|5|5_5|5|5,
hit:832_conv:1665,2,embarrassed,I misspoke some words out in public and lots of people heard.,45,Oh_comma_ wow_comma_ I guess you were embarrassed? It happens to us all though_comma_ so you just laugh it off!,5|5|5_5|5|5,
hit:832_conv:1665,3,embarrassed,I misspoke some words out in public and lots of people heard.,226,I did laugh. Trouble is sometimes when I start laughing I can't stop. ,5|5|5_5|5|5,
hit:832_conv:1665,4,embarrassed,I misspoke some words out in public and lots of people heard.,45,I do that too! Or laugh at the wrong time_comma_ so I understand!,5|5|5_5|5|5,
hit:833_conv:1666,1,prepared,I had a test in college that I was very nervous for so I studided really hard for it and felt ready when it came around.,217,In my last year of college I had a final exam that I was really scared of and worried about. I spent much of my time studying for it. When it finally came time to take it I felt good about it and was ready to take it on.,4|4|4_5|5|5,
hit:833_conv:1666,2,prepared,I had a test in college that I was very nervous for so I studided really hard for it and felt ready when it came around.,209,You sounded very prepared. How do you feel about it? Think you passed?,4|4|4_5|5|5,
hit:833_conv:1666,3,prepared,I had a test in college that I was very nervous for so I studided really hard for it and felt ready when it came around.,217,Yes I know I did great on it!,4|4|4_5|5|5,
hit:833_conv:1666,4,prepared,I had a test in college that I was very nervous for so I studided really hard for it and felt ready when it came around.,209,Good to hear that you did well. You'll make valedictorian before you know it!,4|4|4_5|5|5,
hit:833_conv:1666,5,prepared,I had a test in college that I was very nervous for so I studided really hard for it and felt ready when it came around.,217,Thank you. I didn't quite make valedictorian and don't think I'm that great. But I can dream I guess.,4|4|4_5|5|5,
hit:833_conv:1667,1,prepared,Yesterday I had my final Physics exam and I think I nailed it_comma_ due to studying so hard.,209,Yesterday I had my final Physics exam and I think I nailed it_comma_ due to studying so hard.,5|5|5_4|4|4,
hit:833_conv:1667,2,prepared,Yesterday I had my final Physics exam and I think I nailed it_comma_ due to studying so hard.,217,That sounds great! It sounds like you really studied hard. I hope you did wonderful on it.,5|5|5_4|4|4,
hit:833_conv:1667,3,prepared,Yesterday I had my final Physics exam and I think I nailed it_comma_ due to studying so hard.,209,Yeah_comma_ me too. Hopefully just as well as you did on yours. I really struggle with mathematics.,5|5|5_4|4|4,
hit:833_conv:1667,4,prepared,Yesterday I had my final Physics exam and I think I nailed it_comma_ due to studying so hard.,217,It sounds like you did great. Studying is the biggest part really.,5|5|5_4|4|4,
hit:834_conv:1668,1,anxious,I had to go to the doctor recently for my foot_comma_ I was definitely scared of the possible outcome.,100,I got my foot looked at recently by a doctor_comma_ boy was I nervous going into it.,5|5|5_4|5|5,
hit:834_conv:1668,2,anxious,I had to go to the doctor recently for my foot_comma_ I was definitely scared of the possible outcome.,234,Oh_comma_ that's annoying! I hate going to the doctor...freaks me out. I always feel like they're going to tell me I'll die in 5 weeks,5|5|5_4|5|5,
hit:834_conv:1668,3,anxious,I had to go to the doctor recently for my foot_comma_ I was definitely scared of the possible outcome.,100,Yeah I feel the same way_comma_ then it ends up being nothing serious usually anyway.,5|5|5_4|5|5,
hit:834_conv:1668,4,anxious,I had to go to the doctor recently for my foot_comma_ I was definitely scared of the possible outcome.,234,Similar to most things_comma_ yea. What was wrong with your foot?,5|5|5_4|5|5,
hit:834_conv:1669,1,annoyed,Once I had a coworker who really liked to pull pranks on me,234,I have this coworker who really likes to pull pranks on people...again..and again. It's always the same stuff,4|5|5_5|5|5,
hit:834_conv:1669,2,annoyed,Once I had a coworker who really liked to pull pranks on me,100,Oh that can be annoying sometimes_comma_ I hope they're not too serious.,4|5|5_5|5|5,
hit:834_conv:1669,3,annoyed,Once I had a coworker who really liked to pull pranks on me,234,Nah_comma_ they're just stupid things like moving stuff around_comma_ putting wacky sticky notes on your desk_comma_ changing your desktop wallpaper_comma_ etc.,4|5|5_5|5|5,
hit:834_conv:1669,4,annoyed,Once I had a coworker who really liked to pull pranks on me,100,Oh well that doesn't sound too bad_comma_ I hope he gets some in return though.,4|5|5_5|5|5,
hit:835_conv:1670,1,anticipating,I'm looking forward to going to the fair in a couple of weeks.,225,Our county fair is coming up in a couple of weeks and I'm looking forward to going.,5|5|5_4|5|5,
hit:835_conv:1670,2,anticipating,I'm looking forward to going to the fair in a couple of weeks.,26,I am supposed to be going to an amusement park with friends this weekend_comma_ hoping they have fair type rides there. ,5|5|5_4|5|5,
hit:835_conv:1670,3,anticipating,I'm looking forward to going to the fair in a couple of weeks.,225,I'd say that they will. In fact_comma_ I'd expect it.,5|5|5_4|5|5,
hit:835_conv:1670,4,anticipating,I'm looking forward to going to the fair in a couple of weeks.,26,What is your favorite ride?,5|5|5_4|5|5,
hit:836_conv:1672,1,joyful,I won a free $50 gift card yesterday!,234,So_comma_ I won a big gift card yesterday which made me very happy!,4|5|5_5|5|5,
hit:836_conv:1672,2,joyful,I won a free $50 gift card yesterday!,225,Well_comma_ how big was it_comma_ and where for?,4|5|5_5|5|5,
hit:836_conv:1672,3,joyful,I won a free $50 gift card yesterday!,234,It was for $50 and it was a prepaid so I could use it anywhere I wanted. I never win anything_comma_ so it was exciting for me,4|5|5_5|5|5,
hit:836_conv:1672,4,joyful,I won a free $50 gift card yesterday!,225,That's the best kind. What are your plans for it ?,4|5|5_5|5|5,
hit:836_conv:1672,5,joyful,I won a free $50 gift card yesterday!,234,To be honest_comma_ I've had my eye on some neat looking jewelry (necklace) that should help pay for quite a bit of it!,4|5|5_5|5|5,
hit:836_conv:1673,1,grateful,I'm always grateful for the love of my wife and children.,225,I'm always really amazed at how my children are wonderful.,5|5|5_4|5|5,
hit:836_conv:1673,2,grateful,I'm always grateful for the love of my wife and children.,234,Are they just great in general or did they do something extra special?,5|5|5_4|5|5,
hit:836_conv:1673,3,grateful,I'm always grateful for the love of my wife and children.,225,They're always great_comma_ I just don't get used to it.,5|5|5_4|5|5,
hit:836_conv:1673,4,grateful,I'm always grateful for the love of my wife and children.,234,Aw :) I wish I had kids. I think it would be worth it,5|5|5_4|5|5,
hit:837_conv:1674,1,joyful,I have met with my close friend from high school after 7 years,155,Yesterday_comma_ i had a dinner with my close friend from high school. I was very glad to finally meet him after 7 years,4|4|4_5|5|5,
hit:837_conv:1674,2,joyful,I have met with my close friend from high school after 7 years,62,that sounds like fun_comma_ how long do you know each other?,4|4|4_5|5|5,
hit:837_conv:1674,3,joyful,I have met with my close friend from high school after 7 years,155,We had been like best friends in high school_comma_ but after graduating I did not get a chance to see him at all,4|4|4_5|5|5,
hit:837_conv:1674,4,joyful,I have met with my close friend from high school after 7 years,62,I see. What kind of restaurant did you guys go? ,4|4|4_5|5|5,
hit:837_conv:1675,1,nostalgic,I went to Disneyland the first time when I was 10 years old_comma_ and I went back last year_comma_ I was so happy,62,I went to the Disneyland again this year_comma_ it was awesome ,5|5|5_4|4|4,
hit:837_conv:1675,2,nostalgic,I went to Disneyland the first time when I was 10 years old_comma_ and I went back last year_comma_ I was so happy,155,To be honest_comma_ i have never visited it yet. But I am very eager to go there,5|5|5_4|4|4,
hit:837_conv:1675,3,nostalgic,I went to Disneyland the first time when I was 10 years old_comma_ and I went back last year_comma_ I was so happy,62,You should visit when you have a chance_comma_ it is a magic place,5|5|5_4|4|4,
hit:837_conv:1675,4,nostalgic,I went to Disneyland the first time when I was 10 years old_comma_ and I went back last year_comma_ I was so happy,155,In which state is the DisneyLand that you had been is located?,5|5|5_4|4|4,
hit:837_conv:1675,5,nostalgic,I went to Disneyland the first time when I was 10 years old_comma_ and I went back last year_comma_ I was so happy,62,California ,5|5|5_4|4|4,<UNIGRAM>
hit:838_conv:1677,1,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,127,I moved out of my beach community in Los Angeles because it was so expensive_comma_ I don't know how my remaining friends there exist!,5|5|5_5|5|5,
hit:838_conv:1677,2,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,242,Theyre clearly fatcats_comma_ thats how. Where did you move to? ,5|5|5_5|5|5,
hit:838_conv:1677,3,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,127,New Orleans. It's a fun city but it's not as nice as California. One of my friends in LA is just a teacher and goes to Hawaii all the time. I've never been. ,5|5|5_5|5|5,
hit:838_conv:1677,4,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,242,isnt california like really hot? Im not sure if id be able to tolerate the temperature to be honest_comma_ do you intend to move back to los angeles in the future?,5|5|5_5|5|5,
hit:838_conv:1677,5,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,127,It's nice at the beach_comma_ like 85 degrees. A cold day is 55_comma_ which are rare. I'd like to move back but can't ever imagine paying 2400 for renting a box. ,5|5|5_5|5|5,
hit:838_conv:1677,6,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,242,i pay 160 euros a month for my apartament lol_comma_ 2400 is insanity tbh. and 85 degrees is like 30c D: that is so hot!,5|5|5_5|5|5,
hit:838_conv:1677,7,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,127,Haha_comma_ it feels perfect to me.  Your rent sounds affordable_comma_ I'll bet you have healthcare_comma_ too. ,5|5|5_5|5|5,
hit:838_conv:1677,8,jealous,My friend who lives in Los Angeles not only gets to live by the beach she can still somehow afford to go to Hawaii all the time.,242,i actually dont_comma_ my country decided to illegally deny it to me_comma_ but most people do. you could move to EU if you really wanted to_comma_ rent sure would be way more afforadable regardless of which country you pick,5|5|5_5|5|5,
hit:839_conv:1678,1,furious,My wife lost her phone_comma_ which i bought for our anniversary,43,My wife lost her phone_comma_ its a new one,4|4|4_3|3|2,
hit:839_conv:1678,2,furious,My wife lost her phone_comma_ which i bought for our anniversary,159,What kind of phone was it?,4|4|4_3|3|2,
hit:839_conv:1678,3,furious,My wife lost her phone_comma_ which i bought for our anniversary,43,Blackberry porsche model_comma_ it was an anniversary gift,4|4|4_3|3|2,
hit:839_conv:1678,4,furious,My wife lost her phone_comma_ which i bought for our anniversary,159,Oh no! How awful. I'm buying a house on Monday!,4|4|4_3|3|2,
hit:839_conv:1679,1,impressed,I bought a house at the young age of 25. I worked really hard to get were I wanted. I deserve the best out of my life. ,159,Congratulations! You deserve it!,3|3|2_4|4|4,
hit:839_conv:1679,2,impressed,I bought a house at the young age of 25. I worked really hard to get were I wanted. I deserve the best out of my life. ,43,/Am not sure_comma_ what i deserve,3|3|2_4|4|4,
hit:839_conv:1679,3,impressed,I bought a house at the young age of 25. I worked really hard to get were I wanted. I deserve the best out of my life. ,159,Buying a house is house is a huge accomplishment!,3|3|2_4|4|4,
hit:839_conv:1679,4,impressed,I bought a house at the young age of 25. I worked really hard to get were I wanted. I deserve the best out of my life. ,43,oh thank you!! for your wishes!!,3|3|2_4|4|4,
hit:839_conv:1679,5,impressed,I bought a house at the young age of 25. I worked really hard to get were I wanted. I deserve the best out of my life. ,159,You are more than welcome :),3|3|2_4|4|4,
hit:840_conv:1680,1,annoyed,My mom called me at the last minute to tell me she was having all the family over for dinner and she was expecting me to be there. I told her I had plans that night and she told me I should change my plans.,243,I had plans to go out with friends the other night and then my mom called me to tell me she was having all our family over for dinner and I was expected to be there. I told her about my plans and she said to change my plans. She does not understand.,5|5|5_5|5|5,
hit:840_conv:1680,2,annoyed,My mom called me at the last minute to tell me she was having all the family over for dinner and she was expecting me to be there. I told her I had plans that night and she told me I should change my plans.,225,So_comma_ did you go to her dinner_comma_ or do what you had planned?,5|5|5_5|5|5,
hit:840_conv:1680,3,annoyed,My mom called me at the last minute to tell me she was having all the family over for dinner and she was expecting me to be there. I told her I had plans that night and she told me I should change my plans.,243,I went to the dinner and then left a little early to go out. She wasn't happy_comma_ but maybe next time she will plan better.,5|5|5_5|5|5,
hit:840_conv:1680,4,annoyed,My mom called me at the last minute to tell me she was having all the family over for dinner and she was expecting me to be there. I told her I had plans that night and she told me I should change my plans.,225,I find that they don't respect you until you start telling them no.,5|5|5_5|5|5,
hit:840_conv:1680,5,annoyed,My mom called me at the last minute to tell me she was having all the family over for dinner and she was expecting me to be there. I told her I had plans that night and she told me I should change my plans.,243,That's what I have found too. She needs to realize I am an adult.,5|5|5_5|5|5,
hit:840_conv:1681,1,surprised,I was startled by a wreck in front of my house yesterday morning.,225,There was a wreck in front of my house yesterday morning early.,5|5|5_5|5|5,
hit:840_conv:1681,2,surprised,I was startled by a wreck in front of my house yesterday morning.,243,Was everyone okay?,5|5|5_5|5|5,
hit:840_conv:1681,3,surprised,I was startled by a wreck in front of my house yesterday morning.,225,Everyone was_comma_ but a deer was killed.,5|5|5_5|5|5,
hit:840_conv:1681,4,surprised,I was startled by a wreck in front of my house yesterday morning.,243,I bet that did some damage to the car.,5|5|5_5|5|5,
hit:841_conv:1682,1,guilty,I lied to my friend about an important situation. It only made things worse in the end.,234,I lied to my friend a few days ago_comma_ and it made me feel really bad,4|5|5_5|5|5,
hit:841_conv:1682,2,guilty,I lied to my friend about an important situation. It only made things worse in the end.,225,Was it about something serious?,4|5|5_5|5|5,
hit:841_conv:1682,3,guilty,I lied to my friend about an important situation. It only made things worse in the end.,234,Yes_comma_ and it actually ended up making things worse in the end as he found out I kept it secret his girlfriend was cheating on him,4|5|5_5|5|5,
hit:841_conv:1682,4,guilty,I lied to my friend about an important situation. It only made things worse in the end.,225,I hope she wasn't cheating with you at least.,4|5|5_5|5|5,
hit:841_conv:1682,5,guilty,I lied to my friend about an important situation. It only made things worse in the end.,234,Thankfully no,4|5|5_5|5|5,
hit:841_conv:1683,1,surprised,I can't believe how fast the grass is growing this year.,225,I seem to be mowing the lawn more frequently this year.,5|5|5_4|5|5,
hit:841_conv:1683,2,surprised,I can't believe how fast the grass is growing this year.,234,Has it rained more often there than usual?,5|5|5_4|5|5,
hit:841_conv:1683,3,surprised,I can't believe how fast the grass is growing this year.,225,Yes_comma_ it has. I know that's the source.,5|5|5_4|5|5,
hit:841_conv:1683,4,surprised,I can't believe how fast the grass is growing this year.,234,My yard has too actually. I wonder if it's due to more sun and more rain across the board,5|5|5_4|5|5,
hit:842_conv:1684,1,lonely,My friends left me out of their plans. I'm a pretty social person so it's not very fun for me,234,My 'friends' left me out of their social plans yesterday which wasn't very fun for me,4|5|5_5|5|5,
hit:842_conv:1684,2,lonely,My friends left me out of their plans. I'm a pretty social person so it's not very fun for me,40,Did you ask them why? ,4|5|5_5|5|5,
hit:842_conv:1684,3,lonely,My friends left me out of their plans. I'm a pretty social person so it's not very fun for me,234,No_comma_ but I think it's because we had a major argument and they all took the other person's side,4|5|5_5|5|5,
hit:842_conv:1684,4,lonely,My friends left me out of their plans. I'm a pretty social person so it's not very fun for me,40,I would let things cool down for awhile and maybe they will have a change of heart. If not _comma_ apologize and see if that helps.,4|5|5_5|5|5,
hit:842_conv:1685,1,anticipating,I am getting ready to make reservations at the beach for next month for a whole week. I can't wait. ,40,I can't wait to go on vacation next. Getting ready to make reservations online.,5|5|5_4|5|5,
hit:842_conv:1685,2,anticipating,I am getting ready to make reservations at the beach for next month for a whole week. I can't wait. ,234,Sounds fun! What area did you have in mind_comma_ since you're already about to make the reservations,5|5|5_4|5|5,
hit:842_conv:1685,3,anticipating,I am getting ready to make reservations at the beach for next month for a whole week. I can't wait. ,40,South Carolina area. I have some cousins that live there_comma_ so it will be fun seeing them also.,5|5|5_4|5|5,
hit:842_conv:1685,4,anticipating,I am getting ready to make reservations at the beach for next month for a whole week. I can't wait. ,234,I bet! I've been to South Carolina several times. It's very beautiful there_comma_ and I'm sure you'll have fun,5|5|5_4|5|5,
hit:842_conv:1685,5,anticipating,I am getting ready to make reservations at the beach for next month for a whole week. I can't wait. ,40,It's a long drive from where I live but I love the beach and water. ,5|5|5_4|5|5,
hit:843_conv:1686,1,furious,My mother stole my wife's jewelry.,225,My mother stole my wife's jewelry while she was in the hospital.,5|5|5_4|4|4,
hit:843_conv:1686,2,furious,My mother stole my wife's jewelry.,158,Was it a nice steal or a mean one?,5|5|5_4|4|4,
hit:843_conv:1686,3,furious,My mother stole my wife's jewelry.,225,well_comma_ we found out when we found her pawn shop receipt years later.,5|5|5_4|4|4,
hit:843_conv:1686,4,furious,My mother stole my wife's jewelry.,158,Did you ask her about it?,5|5|5_4|4|4,
hit:843_conv:1686,5,furious,My mother stole my wife's jewelry.,225,she was dead when we found it.,5|5|5_4|4|4,
hit:844_conv:1688,1,afraid,I've been hearing noises around the house at night.,225,I've been hearing some strange noises around the house at night.,5|5|5_5|5|5,
hit:844_conv:1688,2,afraid,I've been hearing noises around the house at night.,50,oh no! That's scary! What do you think it is?,5|5|5_5|5|5,
hit:844_conv:1688,3,afraid,I've been hearing noises around the house at night.,225,I don't know_comma_ that's what's making me anxious.,5|5|5_5|5|5,
hit:844_conv:1688,4,afraid,I've been hearing noises around the house at night.,50,I'm sorry to hear that. I wish I could help you figure it out,5|5|5_5|5|5,
hit:845_conv:1690,1,guilty,i took my son to the park yesterday and he fell and hit his face,52,hey. i took my son to the park yesterday and he was going down the slide and i didn't catch him. he fell and hit his face _comma_ i feel so bad about it,5|5|5_3|0|1,
hit:845_conv:1690,2,guilty,i took my son to the park yesterday and he fell and hit his face,17,Oh no! I hate to hear that he is hurt. But you shouldn't feel bad. It happens all the time. Kids will find a thousand ways to get hurt. ,5|5|5_3|0|1,
hit:845_conv:1690,3,guilty,i took my son to the park yesterday and he fell and hit his face,52,tell me about it :) he keeps bumping his head_comma_ scraping his knees. such a busy body! ,5|5|5_3|0|1,
hit:845_conv:1690,4,guilty,i took my son to the park yesterday and he fell and hit his face,17,My daughter is the same way! She gets bumped and scraped way more than my son_comma_ but she just goes right on. How is your son today?,5|5|5_3|0|1,
hit:845_conv:1691,1,sad,I felt sad the other day when my best friend left. She visited for several days and I got used to her being here_comma_ but then she left and I miss her.,17,My best friend moved away last year. A couple weeks ago she came to town and she and her son stayed with me and my family for 9 days. She left 3 days ago and I feel like a piece of me is missing.,3|0|1_5|5|5,
hit:845_conv:1691,2,sad,I felt sad the other day when my best friend left. She visited for several days and I got used to her being here_comma_ but then she left and I miss her.,52,i know how you feel... My best friend lives in a different country,3|0|1_5|5|5,
hit:845_conv:1691,3,sad,I felt sad the other day when my best friend left. She visited for several days and I got used to her being here_comma_ but then she left and I miss her.,17,That must be so hard. How often do you see each other?,3|0|1_5|5|5,
hit:845_conv:1691,4,sad,I felt sad the other day when my best friend left. She visited for several days and I got used to her being here_comma_ but then she left and I miss her.,52,not often at all. I saw her in 2011. we skype but it's not the same,3|0|1_5|5|5,
hit:845_conv:1691,5,sad,I felt sad the other day when my best friend left. She visited for several days and I got used to her being here_comma_ but then she left and I miss her.,17,No it isn't at all. My BFF and I text and talk on the phone constantly_comma_ but there is nothing like spending actual time together. My son and her son are best friends too_comma_ and my little boy has been sad ever since they left.,3|0|1_5|5|5,
hit:846_conv:1692,1,nostalgic,I recently downloaded the Quest for Glory game series I used to play as a kid and felt it all rushing back to me.,216,Do you ever replay a video game or re-watch a movie you really enjoyed as a kid? The memories of your initial experience can really come rushing back.,4|5|5_5|5|5,
hit:846_conv:1692,2,nostalgic,I recently downloaded the Quest for Glory game series I used to play as a kid and felt it all rushing back to me.,225,I do that all the time. I can remember lots of times watching things from long ago.,4|5|5_5|5|5,
hit:846_conv:1692,3,nostalgic,I recently downloaded the Quest for Glory game series I used to play as a kid and felt it all rushing back to me.,216,It's always a bizarre but welcome experience. I downloaded Quest for Glory_comma_ a game I used to adore watching my father play. Now_comma_ as an adult_comma_ it isn't that great_comma_ but the memories are still there.,4|5|5_5|5|5,
hit:846_conv:1692,4,nostalgic,I recently downloaded the Quest for Glory game series I used to play as a kid and felt it all rushing back to me.,225,I still play some games from the 1990s.,4|5|5_5|5|5,
hit:846_conv:1692,5,nostalgic,I recently downloaded the Quest for Glory game series I used to play as a kid and felt it all rushing back to me.,216,Isn't it great? Sure_comma_ the graphics or interface might be outdated. But there is just a certain charm to reliving those moments from your past again.,4|5|5_5|5|5,
hit:846_conv:1693,1,sad,I'm blue that my parents have dementia now.,225,I'm upset these days that both of my parents have dementia now and most of the time don't even know who I am. Makes me feel very alone.,5|5|5_4|5|5,
hit:846_conv:1693,2,sad,I'm blue that my parents have dementia now.,216,That must be very hard on you_comma_ I'm sorry to hear that. It must be very scary to experience.,5|5|5_4|5|5,
hit:846_conv:1693,3,sad,I'm blue that my parents have dementia now.,225,It is like they are dead already without the funeral.,5|5|5_4|5|5,
hit:846_conv:1693,4,sad,I'm blue that my parents have dementia now.,216,Is there anything that can be done from a medical standpoint? ,5|5|5_4|5|5,
hit:847_conv:1694,1,proud,I passed all of my summer classes! Those are the hardest.,27,I took classes this summer and passed all of them.,5|5|5_2|4|4,
hit:847_conv:1694,2,proud,I passed all of my summer classes! Those are the hardest.,17,Woo hoo! Congratulations! You must be super proud of yourse.f,5|5|5_2|4|4,
hit:847_conv:1694,3,proud,I passed all of my summer classes! Those are the hardest.,27,I am. Summer is the hardest because the classes are shorter and all of the parties.,5|5|5_2|4|4,
hit:847_conv:1694,4,proud,I passed all of my summer classes! Those are the hardest.,17,What classes did you take?,5|5|5_2|4|4,
hit:847_conv:1694,5,proud,I passed all of my summer classes! Those are the hardest.,27,spanish_comma_ math_comma_ psych,5|5|5_2|4|4,
hit:847_conv:1694,6,proud,I passed all of my summer classes! Those are the hardest.,17,Which math class did you take? ,5|5|5_2|4|4,
hit:847_conv:1694,7,proud,I passed all of my summer classes! Those are the hardest.,27,it was basic college math. ,5|5|5_2|4|4,
hit:847_conv:1694,8,proud,I passed all of my summer classes! Those are the hardest.,17,Oh okay. I bet psychology was an interesting course.,5|5|5_2|4|4,
hit:847_conv:1695,1,annoyed,My kids are not behaving at all today. They are really getting on my nerves.,17,So my kids are on my nerves so much today. They are not behaving at all_comma_ and are making it impossible for me to get any work done.,2|4|4_5|5|5,
hit:847_conv:1695,2,annoyed,My kids are not behaving at all today. They are really getting on my nerves.,27,That sucks. ,2|4|4_5|5|5,
hit:847_conv:1695,3,annoyed,My kids are not behaving at all today. They are really getting on my nerves.,17,Yes_comma_ it does. They are bored_comma_ I'm sure. But I can't take them anywhere until later.,2|4|4_5|5|5,
hit:847_conv:1695,4,annoyed,My kids are not behaving at all today. They are really getting on my nerves.,27,My kids are taking a nap now. The run this house lol,2|4|4_5|5|5,
hit:847_conv:1695,5,annoyed,My kids are not behaving at all today. They are really getting on my nerves.,17,Kids have a terrible way of doing that.,2|4|4_5|5|5,
hit:848_conv:1696,1,confident,today when i took my spouse to her interview. i think she got it for sure! ,11,Today my spouse had an interview_comma_ and said it went very well! ,5|5|5_5|5|5,
hit:848_conv:1696,2,confident,today when i took my spouse to her interview. i think she got it for sure! ,52,awesome! My husband had an interview as well. I hope both of our spouses get a job,5|5|5_5|5|5,
hit:848_conv:1696,3,confident,today when i took my spouse to her interview. i think she got it for sure! ,11,That is amazing! I bet they will both get it for sure! :) Fingers Crossed. ,5|5|5_5|5|5,
hit:848_conv:1696,4,confident,today when i took my spouse to her interview. i think she got it for sure! ,52,I hope so! My husband was so nervous but he thinks it went well,5|5|5_5|5|5,
hit:848_conv:1697,1,angry,Somebody stole my mail,52,It was my birthday and my friends sent me birthday cards in the mail. I received all of them but not the one who contained a gift card. i'm so angry about it,5|5|5_5|5|5,
hit:848_conv:1697,2,angry,Somebody stole my mail,11,Ugh_comma_ oh no.... is there someway you can have the person who sent it call the gift card to see what can be done? ,5|5|5_5|5|5,
hit:848_conv:1697,3,angry,Somebody stole my mail,52,I don't want to bother her about it. but i did call usps and they will investigate it,5|5|5_5|5|5,
hit:848_conv:1697,4,angry,Somebody stole my mail,11,True true_comma_ well im glad they are looking into it for you. I hope they find it! ,5|5|5_5|5|5,
hit:849_conv:1698,1,surprised,I was told i have a trip coming up. its to a place ive been wanting to go ,11,So I was told I am being taken on a trip to the beach as a gift... pretty exciting. ,5|5|5_5|5|5,
hit:849_conv:1698,2,surprised,I was told i have a trip coming up. its to a place ive been wanting to go ,225,Where are you going ?,5|5|5_5|5|5,
hit:849_conv:1698,3,surprised,I was told i have a trip coming up. its to a place ive been wanting to go ,11,They are taking me to the keys in FL! Unexpected.,5|5|5_5|5|5,
hit:849_conv:1698,4,surprised,I was told i have a trip coming up. its to a place ive been wanting to go ,225,Oh_comma_ that should be nice. I hope you have fun.,5|5|5_5|5|5,
hit:849_conv:1699,1,surprised,I was startled by a car hitting a deer in front of my house yesterday.,225,I was startled by a car hitting a deer in front of my house yesterday.,5|5|5_5|5|5,
hit:849_conv:1699,2,surprised,I was startled by a car hitting a deer in front of my house yesterday.,11,Wow... I would have been as well! Was anyone injured? ,5|5|5_5|5|5,
hit:849_conv:1699,3,surprised,I was startled by a car hitting a deer in front of my house yesterday.,225,I don't think so_comma_ but the deer is dead for sure.,5|5|5_5|5|5,
hit:849_conv:1699,4,surprised,I was startled by a car hitting a deer in front of my house yesterday.,11,Bummer but glad the people are okay! ,5|5|5_5|5|5,
hit:850_conv:1700,1,annoyed,My kids wouldn't be quiet lasy night when I tried to go to bead early.,27,I woke up late and it's my kids fault.,5|5|5_5|5|5,
hit:850_conv:1700,2,annoyed,My kids wouldn't be quiet lasy night when I tried to go to bead early.,11,Oh no... are they in trouble? lol ,5|5|5_5|5|5,
hit:850_conv:1700,3,annoyed,My kids wouldn't be quiet lasy night when I tried to go to bead early.,27,no. They are sleep and I can get some things done.,5|5|5_5|5|5,
hit:850_conv:1700,4,annoyed,My kids wouldn't be quiet lasy night when I tried to go to bead early.,11,Maybe tomorrow morning you should wake them up super early as a surprise. :p,5|5|5_5|5|5,
hit:850_conv:1700,5,annoyed,My kids wouldn't be quiet lasy night when I tried to go to bead early.,27,Sounds like a plan,5|5|5_5|5|5,
hit:850_conv:1701,1,caring,my wife decided to take me out for dinner after a long shift. it was nice of her ,11,I had a really bad day at work yesterday_comma_ and my wife decided to take me out to my favorite place after work. ,5|5|5_5|5|5,
hit:850_conv:1701,2,caring,my wife decided to take me out for dinner after a long shift. it was nice of her ,27,That was nice. ,5|5|5_5|5|5,
hit:850_conv:1701,3,caring,my wife decided to take me out for dinner after a long shift. it was nice of her ,11,It was really thoughtful of her_comma_ Im thankful for it. I needed that. ,5|5|5_5|5|5,
hit:850_conv:1701,4,caring,my wife decided to take me out for dinner after a long shift. it was nice of her ,27,Wives are awesome like that.,5|5|5_5|5|5,
hit:851_conv:1702,1,impressed,MY daughter just learned to play the ukelele over the summer. She taught herself.,27,My daughter just learned to play the ukelele.,5|5|5_5|5|5,
hit:851_conv:1702,2,impressed,MY daughter just learned to play the ukelele over the summer. She taught herself.,52,how fun! I bet she is good at it,5|5|5_5|5|5,
hit:851_conv:1702,3,impressed,MY daughter just learned to play the ukelele over the summer. She taught herself.,27,she actually is and she taught herself,5|5|5_5|5|5,
hit:851_conv:1702,4,impressed,MY daughter just learned to play the ukelele over the summer. She taught herself.,52,wow_comma_ that's amazing. She is very talented. I would never be able to do that,5|5|5_5|5|5,
hit:851_conv:1702,5,impressed,MY daughter just learned to play the ukelele over the summer. She taught herself.,27,me either. She is awesome,5|5|5_5|5|5,
hit:851_conv:1703,1,devastated,I lost my job and i'm the only provider in my family i'm so stressed,52,I lost my job and my wife is a stay at home mom. I don't know how to pay for rent_comma_ i'm so stressed,5|5|5_5|5|5,
hit:851_conv:1703,2,devastated,I lost my job and i'm the only provider in my family i'm so stressed,27,oh no. i'm sorry to here that.,5|5|5_5|5|5,
hit:851_conv:1703,3,devastated,I lost my job and i'm the only provider in my family i'm so stressed,52,thank you. I had an interview today so i hope i will get that job,5|5|5_5|5|5,
hit:851_conv:1703,4,devastated,I lost my job and i'm the only provider in my family i'm so stressed,27,I hope you do too. It will work out in the end.,5|5|5_5|5|5,

hit:1068_conv:2136,1,confident,Looking forward to the start of the college semester. It's my senior year and I feel like I have a grip on things.,273,Looking forward to the start of the college semester. It's my senior year and I feel like I have a grip on things,2|2|2_4|4|5,
hit:1068_conv:2136,2,confident,Looking forward to the start of the college semester. It's my senior year and I feel like I have a grip on things.,43,Well then It's time to dominate. LOL ,2|2|2_4|4|5,
hit:1068_conv:2136,3,confident,Looking forward to the start of the college semester. It's my senior year and I feel like I have a grip on things.,273,Yeah_comma_ that's exactly how I'm feeling_comma_ haha. Feels like I'm ready to enter a zone. What about you? You in school?,2|2|2_4|4|5,
hit:1068_conv:2136,4,confident,Looking forward to the start of the college semester. It's my senior year and I feel like I have a grip on things.,43,I'm working now. Enjoy your own time and also be responsible to your juniors. ,2|2|2_4|4|5,
hit:1068_conv:2137,1,surprised,My friend asked me to get ready for her birthday party. I told her that I was unable to attend the party. She get sad about it. But I went to her home in the midnight to wish her.,43,I unable to attend my friend's birthday party. ,4|4|5_2|2|2,
hit:1068_conv:2137,2,surprised,My friend asked me to get ready for her birthday party. I told her that I was unable to attend the party. She get sad about it. But I went to her home in the midnight to wish her.,273,That's such a bummer. How did he take the bad news?,4|4|5_2|2|2,
hit:1068_conv:2137,3,surprised,My friend asked me to get ready for her birthday party. I told her that I was unable to attend the party. She get sad about it. But I went to her home in the midnight to wish her.,43,I lied to her. But I went to her home in the night. I wished to her in the midnight with the biggest surprise.,4|4|5_2|2|2,
hit:1068_conv:2137,4,surprised,My friend asked me to get ready for her birthday party. I told her that I was unable to attend the party. She get sad about it. But I went to her home in the midnight to wish her.,273,Oh.. interesting. Yeah_comma_ I can't really comment on the lying_comma_ since I don't know you. But at least you went back to her to comfort her.,4|4|5_2|2|2,
hit:1069_conv:2138,1,apprehensive,thinking about a career change_comma_ but I'm not sure what to do about it yet...,273,thinking about a career change_comma_ but I'm not sure what to do about it yet...,4|4|4_5|5|5,
hit:1069_conv:2138,2,apprehensive,thinking about a career change_comma_ but I'm not sure what to do about it yet...,274,What are you most interested in doing?,4|4|4_5|5|5,
hit:1069_conv:2138,3,apprehensive,thinking about a career change_comma_ but I'm not sure what to do about it yet...,273,I love cooking. Thinking about becoming a chef. I love the way food brings people together.,4|4|4_5|5|5,
hit:1069_conv:2138,4,apprehensive,thinking about a career change_comma_ but I'm not sure what to do about it yet...,274,You are absolutely right there.  If its truly a passion then you should go for it.  Life is too short to be stuck in a rut.,4|4|4_5|5|5,
hit:1069_conv:2139,1,guilty,My wife does most of the chores at home.  I am retired and feel as though I should do more to help out.,274,I recently retired from my job.  Now I stay at home all day but my wife still does most of the chores.  I feel as though I should be doing more.,5|5|5_4|4|4,
hit:1069_conv:2139,2,guilty,My wife does most of the chores at home.  I am retired and feel as though I should do more to help out.,273,That's rough. Why do you think it is that you don't do more?,5|5|5_4|4|4,
hit:1069_conv:2139,3,guilty,My wife does most of the chores at home.  I am retired and feel as though I should do more to help out.,274,Partial laziness but I do most of the traditional man chores like yardwork and clean the cars,5|5|5_4|4|4,
hit:1069_conv:2139,4,guilty,My wife does most of the chores at home.  I am retired and feel as though I should do more to help out.,273,I guess that's good. What did you used to do for work_comma_ if you don't mind me asking?,5|5|5_4|4|4,
hit:1069_conv:2139,5,guilty,My wife does most of the chores at home.  I am retired and feel as though I should do more to help out.,274,I was a first responder ,5|5|5_4|4|4,
hit:1070_conv:2141,1,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,20,I was walking my pet and saw a gross dead animal on the sidewalk.,4|5|5_5|5|5,
hit:1070_conv:2141,2,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,260,Did you pet try to eat it?,4|5|5_5|5|5,
hit:1070_conv:2141,3,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,20,No way! I turned around as soon as I saw it.,4|5|5_5|5|5,
hit:1070_conv:2141,4,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,260,What do you think happened to it?,4|5|5_5|5|5,
hit:1070_conv:2141,5,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,20,I think as bird or another animal of some sort ate it. That's pretty gross.,4|5|5_5|5|5,
hit:1070_conv:2141,6,disgusted,I was so grossed out when I was walking my dog and saw a dead animal on the sidewalk.,260,I agree. However it is the circle of life and all that_comma_ and we are a part of it too.,4|5|5_5|5|5,
hit:1071_conv:2143,1,angry,I asked my brother to change my oil over a month ago_comma_ and he still won't help me. He promised_comma_ and yet he keeps putting it off. Kind of annoying.,273,I asked my brother to change my oil over a month ago_comma_ and he still won't help me. He promised_comma_ and yet he keeps putting it off. Kind of annoying.,4|4|4_5|5|5,
hit:1071_conv:2143,2,angry,I asked my brother to change my oil over a month ago_comma_ and he still won't help me. He promised_comma_ and yet he keeps putting it off. Kind of annoying.,250,That sucks. I can't count on my own family for favors either.,4|4|4_5|5|5,
hit:1071_conv:2143,3,angry,I asked my brother to change my oil over a month ago_comma_ and he still won't help me. He promised_comma_ and yet he keeps putting it off. Kind of annoying.,273,Well_comma_ he's usually dependable. And it is a favor to begin with_comma_ so I can't get too mad. It's just I asked for it as the only thing I'd like for my birthday_comma_ and I was kind of hoping for it to be done.,4|4|4_5|5|5,
hit:1071_conv:2143,4,angry,I asked my brother to change my oil over a month ago_comma_ and he still won't help me. He promised_comma_ and yet he keeps putting it off. Kind of annoying.,250,Maybe you should tell him how urgent the oil change is. Maybe he will understand.,4|4|4_5|5|5,
hit:1072_conv:2144,1,caring,I think I want to rescue a cat.,27,I want to adopt a cat,5|5|5_5|5|5,
hit:1072_conv:2144,2,caring,I think I want to rescue a cat.,4,I love cats.,5|5|5_5|5|5,
hit:1072_conv:2144,3,caring,I think I want to rescue a cat.,27,me too. they are so sassy. I like them better than dogs.,5|5|5_5|5|5,
hit:1072_conv:2144,4,caring,I think I want to rescue a cat.,4,I prefer dogs but I still like cats.,5|5|5_5|5|5,
hit:1072_conv:2144,5,caring,I think I want to rescue a cat.,27,aw. I thought we would be friends.,5|5|5_5|5|5,
hit:1072_conv:2145,1,furious,I am so mad that school is back_comma_ traffic will be horrible again.,4,I am so mad school is back_comma_ traffic will be horrible.,5|5|5_5|5|5,
hit:1072_conv:2145,2,furious,I am so mad that school is back_comma_ traffic will be horrible again.,27,But kids will be busy.,5|5|5_5|5|5,
hit:1072_conv:2145,3,furious,I am so mad that school is back_comma_ traffic will be horrible again.,4,Yes but the buses create extra traffic.,5|5|5_5|5|5,
hit:1072_conv:2145,4,furious,I am so mad that school is back_comma_ traffic will be horrible again.,27,Thar's true. My daughter rides the bus.,5|5|5_5|5|5,
hit:1073_conv:2146,1,angry,I watched a scary movie lasy. I had so many nightmares.,27,I don't know why I watched a scary movie.,5|5|5_5|5|5,
hit:1073_conv:2146,2,angry,I watched a scary movie lasy. I had so many nightmares.,11,what movie was it? ,5|5|5_5|5|5,
hit:1073_conv:2146,3,angry,I watched a scary movie lasy. I had so many nightmares.,27,i don't know I didn't get far. The was a weird kid and rats.,5|5|5_5|5|5,
hit:1073_conv:2146,4,angry,I watched a scary movie lasy. I had so many nightmares.,11,lol sounds terrible! ,5|5|5_5|5|5,
hit:1073_conv:2146,5,angry,I watched a scary movie lasy. I had so many nightmares.,27,it was.,5|5|5_5|5|5,
hit:1073_conv:2147,1,angry,when i moved into my new apartment ,11,I had such a bad experience moving into this apartment months ago... ,5|5|5_5|5|5,
hit:1073_conv:2147,2,angry,when i moved into my new apartment ,27,Why? What happened?,5|5|5_5|5|5,
hit:1073_conv:2147,3,angry,when i moved into my new apartment ,11,cockroaches everywhere... gross,5|5|5_5|5|5,
hit:1073_conv:2147,4,angry,when i moved into my new apartment ,27,ok no. I can't even.,5|5|5_5|5|5,
hit:1074_conv:2148,1,prepared,I have already bought all my books for this semester.,27,I bought all of my books for this semester.,5|5|5_5|5|5,
hit:1074_conv:2148,2,prepared,I have already bought all my books for this semester.,274,I am sure that is a huge relief but quite expensive.,5|5|5_5|5|5,
hit:1074_conv:2148,3,prepared,I have already bought all my books for this semester.,27,They are super expensive.,5|5|5_5|5|5,
hit:1074_conv:2148,4,prepared,I have already bought all my books for this semester.,274,Have you ever tried to rent them or get the e-books?,5|5|5_5|5|5,
hit:1074_conv:2148,5,prepared,I have already bought all my books for this semester.,27,I have some of those and hardbacks too.,5|5|5_5|5|5,
hit:1074_conv:2149,1,sentimental,My wife and I will be celebrating 15 years of marriage this year.  I am a very lucky man as she is a wonderful partner.,274,It is my 15 year anniversary coming up.  I am a lucky man!,5|5|5_5|5|5,
hit:1074_conv:2149,2,sentimental,My wife and I will be celebrating 15 years of marriage this year.  I am a very lucky man as she is a wonderful partner.,27,Congratulations. I want that some day.,5|5|5_5|5|5,
hit:1074_conv:2149,3,sentimental,My wife and I will be celebrating 15 years of marriage this year.  I am a very lucky man as she is a wonderful partner.,274,She has been extremely supportive throughout my career as well as mother.,5|5|5_5|5|5,
hit:1074_conv:2149,4,sentimental,My wife and I will be celebrating 15 years of marriage this year.  I am a very lucky man as she is a wonderful partner.,27,You are truly lucky,5|5|5_5|5|5,
hit:1075_conv:2150,1,disappointed,When my aunt did not come to my graduation_comma_ it made me sad. I just wanted her to be there,126,I was upset my aunt did not come to my graduation. I wanted her to be there,3|4|3_5|5|5,
hit:1075_conv:2150,2,disappointed,When my aunt did not come to my graduation_comma_ it made me sad. I just wanted her to be there,267,Oh I am sorry you are upset. I hope you have another opportunityto see her soon.,3|4|3_5|5|5,
hit:1075_conv:2150,3,disappointed,When my aunt did not come to my graduation_comma_ it made me sad. I just wanted her to be there,126,She passed away shortly after that,3|4|3_5|5|5,
hit:1075_conv:2150,4,disappointed,When my aunt did not come to my graduation_comma_ it made me sad. I just wanted her to be there,267,I am so sorry to hear that.,3|4|3_5|5|5,
hit:1075_conv:2151,1,caring,I had a little neighbor girl that was out wandering around outside without a parent and I was really concerned about her.,267,I wonder what happened.  ,5|5|5_3|4|3,
hit:1075_conv:2151,2,caring,I had a little neighbor girl that was out wandering around outside without a parent and I was really concerned about her.,126,What do you mean,5|5|5_3|4|3,
hit:1075_conv:2151,3,caring,I had a little neighbor girl that was out wandering around outside without a parent and I was really concerned about her.,267,I was so worried about a little girl that was wandering around the neighborhood without her parents.,5|5|5_3|4|3,
hit:1075_conv:2151,4,caring,I had a little neighbor girl that was out wandering around outside without a parent and I was really concerned about her.,126,That does sound bad,5|5|5_3|4|3,
hit:1075_conv:2151,5,caring,I had a little neighbor girl that was out wandering around outside without a parent and I was really concerned about her.,267,Luckily she knew where she lived but the parents didn't seem concerned and she was only about 2 years old,5|5|5_3|4|3,
hit:1076_conv:2152,1,proud,I won the trophy in the chess competition which is held between the departments in the office. It's the biggest win for me.,43,I received a trophy for the chess finals match.,4|4|4_5|5|5,
hit:1076_conv:2152,2,proud,I won the trophy in the chess competition which is held between the departments in the office. It's the biggest win for me.,27,Was it first place?,4|4|4_5|5|5,
hit:1076_conv:2152,3,proud,I won the trophy in the chess competition which is held between the departments in the office. It's the biggest win for me.,43,Yeah_comma_ I won it after a long period. Since I played chess in my school. After a few years I played only in this match.,4|4|4_5|5|5,
hit:1076_conv:2152,4,proud,I won the trophy in the chess competition which is held between the departments in the office. It's the biggest win for me.,27,Congratulations. I learned to play chess in third grade.,4|4|4_5|5|5,
hit:1076_conv:2153,1,sad,I cant believe game of thrones is ending.,27,I hate that game of thrones is ending.,5|5|5_4|4|4,
hit:1076_conv:2153,2,sad,I cant believe game of thrones is ending.,43,ah me too. Hearing last season will release in April 2019,5|5|5_4|4|4,
hit:1076_conv:2153,3,sad,I cant believe game of thrones is ending.,27,Yeah I really missed it this summer.,5|5|5_4|4|4,
hit:1076_conv:2153,4,sad,I cant believe game of thrones is ending.,43,it is still such a long wait though,5|5|5_4|4|4,
hit:1076_conv:2153,5,sad,I cant believe game of thrones is ending.,27,I am going to cry like I did with the office.,5|5|5_4|4|4,
hit:1077_conv:2154,1,guilty,I feel horrible I lost my dog.,4,I feel horrible I lost my dog.,5|5|5_8|5|5,
hit:1077_conv:2154,2,guilty,I feel horrible I lost my dog.,20,Oh no what happened to it??,5|5|5_8|5|5,
hit:1077_conv:2154,3,guilty,I feel horrible I lost my dog.,4,He ran away and I couldn't catch him.,5|5|5_8|5|5,
hit:1077_conv:2154,4,guilty,I feel horrible I lost my dog.,20,I hope you find them! If anyone hurt it I would be furious.,5|5|5_8|5|5,
hit:1077_conv:2155,1,angry,I was so mad at my boss when he scheduled me on my day off.,20,I got really grumpy when my boss scheduled me for my day off.,8|5|5_5|5|5,
hit:1077_conv:2155,2,angry,I was so mad at my boss when he scheduled me on my day off.,4,That isn't right. I wouldn't of gone in.,8|5|5_5|5|5,
hit:1077_conv:2155,3,angry,I was so mad at my boss when he scheduled me on my day off.,20,I wasn't going to but he said I would be fired. It's so unfair!,8|5|5_5|5|5,
hit:1077_conv:2155,4,angry,I was so mad at my boss when he scheduled me on my day off.,4,Did you complain to HR?,8|5|5_5|5|5,
hit:1077_conv:2155,5,angry,I was so mad at my boss when he scheduled me on my day off.,20,I am thinking about doing just that.,8|5|5_5|5|5,
hit:1078_conv:2156,1,embarrassed,I was at work recently and had an interesting thing occur.  About halfway through the day_comma_ one of my buddies came over to my desk and told me my fly was undone.  I had been walking around all day with an open fly,282,I was told by a co-worker the other day that my fly was open.  Appearantly I had been walking around for half the day with an open fly.  My face sure was red.,4|5|5_5|5|5,
hit:1078_conv:2156,2,embarrassed,I was at work recently and had an interesting thing occur.  About halfway through the day_comma_ one of my buddies came over to my desk and told me my fly was undone.  I had been walking around all day with an open fly,274,I can imagine how embarrassing that must have been.  I hope you didn't have to give a speech today.,4|5|5_5|5|5,
hit:1078_conv:2156,3,embarrassed,I was at work recently and had an interesting thing occur.  About halfway through the day_comma_ one of my buddies came over to my desk and told me my fly was undone.  I had been walking around all day with an open fly,282,I actually was supposed to give one later on that day at work.  I am glad I was told before the presentation!,4|5|5_5|5|5,
hit:1078_conv:2156,4,embarrassed,I was at work recently and had an interesting thing occur.  About halfway through the day_comma_ one of my buddies came over to my desk and told me my fly was undone.  I had been walking around all day with an open fly,274,Whew. Certainly must appreciate your co-workers candidness.  It happens to all of us.,4|5|5_5|5|5,
hit:1078_conv:2157,1,caring,My friend has a major surgery today.  He has never been through something like this and I am worried for him as well.,274,My friend is having a minor surgery today but he is apprehensive about it. I am worried for him as well too.,5|5|5_4|5|5,
hit:1078_conv:2157,2,caring,My friend has a major surgery today.  He has never been through something like this and I am worried for him as well.,282,Minor surgery is still surgery.  I understand that it can be scary for anyone.,5|5|5_4|5|5,
hit:1078_conv:2157,3,caring,My friend has a major surgery today.  He has never been through something like this and I am worried for him as well.,274,Especially if its your first.  He will have to be anesthetized. ,5|5|5_4|5|5,
hit:1078_conv:2157,4,caring,My friend has a major surgery today.  He has never been through something like this and I am worried for him as well.,282,That is a surreal feeling_comma_ I understand his apprehension.  ,5|5|5_4|5|5,
hit:1079_conv:2158,1,terrified,my daughter choked on food ,11,I had such a bad anxiety attack the other day. ,5|5|5_5|5|5,
hit:1079_conv:2158,2,terrified,my daughter choked on food ,277,I am sorry to hear that.  Maybe you should go see a counselor for help.,5|5|5_5|5|5,
hit:1079_conv:2158,3,terrified,my daughter choked on food ,11,it wasnt over stress_comma_ it was because my daughter choked on food ,5|5|5_5|5|5,
hit:1079_conv:2158,4,terrified,my daughter choked on food ,277,That would cause anyone to have an anxiety attack.  I don't blame you.,5|5|5_5|5|5,
hit:1079_conv:2159,1,nostalgic,I found some of my old childhood toys in the attic and introduced them to my kid.  She loved playing with them.,277,I found some of my old childhood toys in the attic and introduced them to my kid. She loved playing with them.,5|5|5_5|5|5,
hit:1079_conv:2159,2,nostalgic,I found some of my old childhood toys in the attic and introduced them to my kid.  She loved playing with them.,11,awh thats awesome what did you find? ,5|5|5_5|5|5,
hit:1079_conv:2159,3,nostalgic,I found some of my old childhood toys in the attic and introduced them to my kid.  She loved playing with them.,277,It was a bunch of my old Lego sets from the late 80's and early 90's!  Talk about memories!,5|5|5_5|5|5,
hit:1079_conv:2159,4,nostalgic,I found some of my old childhood toys in the attic and introduced them to my kid.  She loved playing with them.,11,so cool! for real thats awesome ,5|5|5_5|5|5,
hit:1080_conv:2160,1,proud,I was proud when my brother graduated college. He really did work hard for it,126,I was happy when my brother graduated college. He worked hard for it,5|4|4_4|4|4,
hit:1080_conv:2160,2,proud,I was proud when my brother graduated college. He really did work hard for it,283,That's great to hear! What was his major? ,5|4|4_4|4|4,
hit:1080_conv:2160,3,proud,I was proud when my brother graduated college. He really did work hard for it,126,It was in english,5|4|4_4|4|4,
hit:1080_conv:2160,4,proud,I was proud when my brother graduated college. He really did work hard for it,283,Awesome. I hope he has a successful career because of it. ,5|4|4_4|4|4,
hit:1080_conv:2161,1,proud,Graduating High School! ,283,I was so happy I graduated High School. I didn't think I would make it! ,4|4|4_5|4|4,
hit:1080_conv:2161,2,proud,Graduating High School! ,126,I am sure that was a good day for you,4|4|4_5|4|4,
hit:1080_conv:2161,3,proud,Graduating High School! ,283,It really was. I was overcome with joy!,4|4|4_5|4|4,
hit:1080_conv:2161,4,proud,Graduating High School! ,126,I am happy for you,4|4|4_5|4|4,
hit:1082_conv:2164,1,furious,I was driving through the parking lot of the grocery store.  Some lazy lady looking for a close parking spot backed into my new car.,274,My brand new car was hit in the grocery store parking lot yesterday.,5|5|5_5|5|5,
hit:1082_conv:2164,2,furious,I was driving through the parking lot of the grocery store.  Some lazy lady looking for a close parking spot backed into my new car.,238,did you catch the person?,5|5|5_5|5|5,
hit:1082_conv:2164,3,furious,I was driving through the parking lot of the grocery store.  Some lazy lady looking for a close parking spot backed into my new car.,274,Yes. She got mad when I told her that I was filing a police report.,5|5|5_5|5|5,
hit:1082_conv:2164,4,furious,I was driving through the parking lot of the grocery store.  Some lazy lady looking for a close parking spot backed into my new car.,238,what did she expect you to do?,5|5|5_5|5|5,
hit:1082_conv:2165,1,impressed,Was impressive that Lebron went to the lakers,238,Was impressive that Lebron went to the lakers,5|5|5_5|5|5,
hit:1082_conv:2165,2,impressed,Was impressive that Lebron went to the lakers,274,I was sad to see him leave Cleveland.,5|5|5_5|5|5,
hit:1082_conv:2165,3,impressed,Was impressive that Lebron went to the lakers,238,He does whats best for him and his family.  I admire that,5|5|5_5|5|5,
hit:1082_conv:2165,4,impressed,Was impressive that Lebron went to the lakers,274,He was the face of Cleveland. ,5|5|5_5|5|5,
hit:1083_conv:2166,1,lonely,My wife left me and I have no one else.,4,My wife left me and I have no one else.,5|5|5_5|5|5,
hit:1083_conv:2166,2,lonely,My wife left me and I have no one else.,274,I am so sorry to hear that. ,5|5|5_5|5|5,
hit:1083_conv:2166,3,lonely,My wife left me and I have no one else.,4,Yes_comma_ she didn't even give me a reason why.,5|5|5_5|5|5,
hit:1083_conv:2166,4,lonely,My wife left me and I have no one else.,274,That must be tough.  Do you have supportive friends to help you through this tough time?,5|5|5_5|5|5,
hit:1083_conv:2167,1,grateful,I have an extremely wonderful wife.  She is very supportive and is the most caring mother I know.,274,My wife_comma_ on the other hand_comma_ is very supportive and is the most caring person I have ever met.,5|5|5_5|5|5,
hit:1083_conv:2167,2,grateful,I have an extremely wonderful wife.  She is very supportive and is the most caring mother I know.,4,That is good_comma_ how long have you been married?,5|5|5_5|5|5,
hit:1083_conv:2167,3,grateful,I have an extremely wonderful wife.  She is very supportive and is the most caring mother I know.,274,We have been married almost 15 years.,5|5|5_5|5|5,
hit:1083_conv:2167,4,grateful,I have an extremely wonderful wife.  She is very supportive and is the most caring mother I know.,4,That is a long time.,5|5|5_5|5|5,
hit:1084_conv:2168,1,terrified,Two ago_comma_ I was attacked by a robber during night. He had a knife. I had choice to give him my wallet.,7,Two ago_comma_ I was attacked by a robber during night. He had a knife. I had choice to give him my wallet. Now_comma_ I always watching behind when I walk in the street.,5|5|5_5|5|5,
hit:1084_conv:2168,2,terrified,Two ago_comma_ I was attacked by a robber during night. He had a knife. I had choice to give him my wallet.,1,Oh no. There's terrible!,5|5|5_5|5|5,
hit:1084_conv:2168,3,terrified,Two ago_comma_ I was attacked by a robber during night. He had a knife. I had choice to give him my wallet.,7,It is was very scary situation. It make you behave weirdly for weeks.,5|5|5_5|5|5,
hit:1084_conv:2168,4,terrified,Two ago_comma_ I was attacked by a robber during night. He had a knife. I had choice to give him my wallet.,1,I'm sorry that happened and hope you don't ever have to be in that situation ever again.,5|5|5_5|5|5,
hit:1084_conv:2169,1,surprised,It was my birthday last weekend. Instead of dinner as planned_comma_ they gave me a weekend trip to NYC.,1,It was my birthday last weekend. Instead of dinner as planned_comma_ they gave me a weekend trip to NYC.,5|5|5_5|5|5,
hit:1084_conv:2169,2,surprised,It was my birthday last weekend. Instead of dinner as planned_comma_ they gave me a weekend trip to NYC.,7,It is nice gift for bithday. That is very good.,5|5|5_5|5|5,
hit:1084_conv:2169,3,surprised,It was my birthday last weekend. Instead of dinner as planned_comma_ they gave me a weekend trip to NYC.,1,Yes. It was very kind of them. I was not expecting it at all!,5|5|5_5|5|5,
hit:1084_conv:2169,4,surprised,It was my birthday last weekend. Instead of dinner as planned_comma_ they gave me a weekend trip to NYC.,7,It is even better when you don't expect it.,5|5|5_5|5|5,
hit:1085_conv:2170,1,angry,I recently discovered that my wife of 15 years is cheating on me.  She has been seeing my best friend of 20 years.  I am devastated.,274,My wife has been cheating on me_comma_ worst yet is that it was with my best friend of 20 years.,5|5|5_5|5|5,
hit:1085_conv:2170,2,angry,I recently discovered that my wife of 15 years is cheating on me.  She has been seeing my best friend of 20 years.  I am devastated.,11,Wow sounds like what i just seen on the news in NC... im so sorry !! ,5|5|5_5|5|5,
hit:1085_conv:2170,3,angry,I recently discovered that my wife of 15 years is cheating on me.  She has been seeing my best friend of 20 years.  I am devastated.,274,Thank you for the sentiment.  I have a great support system to help me through this.,5|5|5_5|5|5,
hit:1085_conv:2170,4,angry,I recently discovered that my wife of 15 years is cheating on me.  She has been seeing my best friend of 20 years.  I am devastated.,11,thats always important to have and i am glad you have that ,5|5|5_5|5|5,
hit:1085_conv:2171,1,lonely,i miss my dad ,11,I really miss my dad. ,5|5|5_5|5|5,
hit:1085_conv:2171,2,lonely,i miss my dad ,274,Has he passed or does he just live far away?,5|5|5_5|5|5,
hit:1085_conv:2171,3,lonely,i miss my dad ,11,he has passed away_comma_ and i just feel alone ,5|5|5_5|5|5,
hit:1085_conv:2171,4,lonely,i miss my dad ,274,I can only imagine.  Are you close with your mother?,5|5|5_5|5|5,
hit:1086_conv:2172,1,disappointed,My home computer stopped working,244,My main home computer stopped working yesterday?,5|5|5_5|5|5,
hit:1086_conv:2172,2,disappointed,My home computer stopped working,99,Oh no_comma_ that sounds super annoying! What was wrong with it?,5|5|5_5|5|5,
hit:1086_conv:2172,3,disappointed,My home computer stopped working,244,Bad power supply,5|5|5_5|5|5,
hit:1086_conv:2172,4,disappointed,My home computer stopped working,99,Oh no! Sounds expensive,5|5|5_5|5|5,
hit:1086_conv:2172,5,disappointed,My home computer stopped working,244,This one was. I didn't buy a cheap parts when I built that computer :(,5|5|5_5|5|5,
hit:1086_conv:2172,6,disappointed,My home computer stopped working,99,I'm sorry you had to go through that!,5|5|5_5|5|5,
hit:1086_conv:2172,7,disappointed,My home computer stopped working,244,Atleast its not something more important like the Hard drive or motherboard.,5|5|5_5|5|5,
hit:1086_conv:2173,1,proud,My daughter recently successfully potty trained and I was very proud of both her and myself for accomplishing this feat!,99,My daughter recently finished potty training! ,5|5|5_5|5|5,
hit:1086_conv:2173,2,proud,My daughter recently successfully potty trained and I was very proud of both her and myself for accomplishing this feat!,244,Awesome. How old is she?,5|5|5_5|5|5,
hit:1086_conv:2173,3,proud,My daughter recently successfully potty trained and I was very proud of both her and myself for accomplishing this feat!,99,She is 2! I was happy to get this accomplished before she turned 3. My friends son is 3 and still isn't trained.,5|5|5_5|5|5,
hit:1086_conv:2173,4,proud,My daughter recently successfully potty trained and I was very proud of both her and myself for accomplishing this feat!,244,Great to hear she's progressing well.  Maybe she'll be motivation for your friends son.,5|5|5_5|5|5,
hit:1087_conv:2174,1,joyful,I was happy when I went on vacation from work,238,I was happy when I went on vacation from work,5|5|5_4|4|5,
hit:1087_conv:2174,2,joyful,I was happy when I went on vacation from work,284,That is such a great feeling did you have a great time?,5|5|5_4|4|5,
hit:1087_conv:2174,3,joyful,I was happy when I went on vacation from work,238,Well it was 111 degrees on my first day camping with the family but it got cooler after that initial day,5|5|5_4|4|5,
hit:1087_conv:2174,4,joyful,I was happy when I went on vacation from work,284,This heat is very dangerous hope yall was very hydrated with water to stay cool.,5|5|5_4|4|5,
hit:1088_conv:2176,1,hopeful,I pray I pass the bar exam next week.,4,I pray I pass the bar exam.,5|5|5_5|5|5,
hit:1088_conv:2176,2,hopeful,I pray I pass the bar exam next week.,267,I remember when my aunt was studying for that.  Did you already take it?,5|5|5_5|5|5,
hit:1088_conv:2176,3,hopeful,I pray I pass the bar exam next week.,4,No I take it next week.,5|5|5_5|5|5,
hit:1088_conv:2176,4,hopeful,I pray I pass the bar exam next week.,267,Oh my!  I wonder if things have changed since 15 years ago.   How is the studying going?,5|5|5_5|5|5,
hit:1088_conv:2177,1,nostalgic,I remember when my grandma passed my family went to her house to clean it out and we found a lot of memorabilia,267,I remember when my grandma passed my family went to her house to clean it out and we found a lot of memorabilia,5|5|5_5|5|5,
hit:1088_conv:2177,2,nostalgic,I remember when my grandma passed my family went to her house to clean it out and we found a lot of memorabilia,4,Sorry to hear about that.,5|5|5_5|5|5,
hit:1088_conv:2177,3,nostalgic,I remember when my grandma passed my family went to her house to clean it out and we found a lot of memorabilia,267,Thank you.  It was really neat to find all those memories and old pictures with the family.,5|5|5_5|5|5,
hit:1088_conv:2177,4,nostalgic,I remember when my grandma passed my family went to her house to clean it out and we found a lot of memorabilia,4,What did you do with them?,5|5|5_5|5|5,
hit:1089_conv:2178,1,content,Finally got a job that pays enough for the bills and a bit left over to save for emergencies.,1,Finally got a job that pays enough for the bills and a bit left over to save for emergencies.,5|5|5_5|5|5,
hit:1089_conv:2178,2,content,Finally got a job that pays enough for the bills and a bit left over to save for emergencies.,238,congrats. what type of job is it?,5|5|5_5|5|5,
hit:1089_conv:2178,3,content,Finally got a job that pays enough for the bills and a bit left over to save for emergencies.,1,It's for marketing. Thank you!,5|5|5_5|5|5,
hit:1089_conv:2178,4,content,Finally got a job that pays enough for the bills and a bit left over to save for emergencies.,238,sounds like a sweet gig.,5|5|5_5|5|5,
hit:1089_conv:2179,1,prepared,I was ready when that last storm hit us,238,I was ready when that last storm hit us,5|5|5_5|5|5,
hit:1089_conv:2179,2,prepared,I was ready when that last storm hit us,1,Storm? What area do you live in? Was it bad?,5|5|5_5|5|5,
hit:1089_conv:2179,3,prepared,I was ready when that last storm hit us,238,I live in central texas.  It wasnt as bad as they predicted but some flooding,5|5|5_5|5|5,
hit:1089_conv:2179,4,prepared,I was ready when that last storm hit us,1,I'm glad you were ready for it. Is weather generally bad over there in the summer?,5|5|5_5|5|5,
hit:1090_conv:2180,1,prepared,I am ready for my test tomorrow.,4,I am ready for my test tomorrow.,5|5|5_4|5|5,
hit:1090_conv:2180,2,prepared,I am ready for my test tomorrow.,43,Well_comma_ good to see your confidence.,5|5|5_4|5|5,
hit:1090_conv:2180,3,prepared,I am ready for my test tomorrow.,4,I studied hard.,5|5|5_4|5|5,
hit:1090_conv:2180,4,prepared,I am ready for my test tomorrow.,43,you'll get a very good result for your hard work. All the best for your test.,5|5|5_4|5|5,
hit:1090_conv:2181,1,furious,I enraged at the taxi driver. He drove the vehicle in a high speed. I insisted him to slow down the car. But he didn't.,43,I had enraged on driver who was doing rash driving.,4|5|5_5|5|5,
hit:1090_conv:2181,2,furious,I enraged at the taxi driver. He drove the vehicle in a high speed. I insisted him to slow down the car. But he didn't.,4,What did you do?,4|5|5_5|5|5,
hit:1090_conv:2181,3,furious,I enraged at the taxi driver. He drove the vehicle in a high speed. I insisted him to slow down the car. But he didn't.,43,I insisted him to slow down the speed. But he didn't.,4|5|5_5|5|5,
hit:1090_conv:2181,4,furious,I enraged at the taxi driver. He drove the vehicle in a high speed. I insisted him to slow down the car. But he didn't.,4,Did you call the police?,4|5|5_5|5|5,

hit:1243_conv:2487,1,guilty,I lied to my girlfriend once about how much education I had because I felt she would leave me otherwise,273,"I lied to my girlfriend once about how much education I had because I felt she would leave me otherwise,4|4|4_5|5|5,
hit:1243_conv:2487,2,guilty,I lied to my girlfriend once about how much education I had because I felt she would leave me otherwise,27,Did you ever come clean?,4|4|4_5|5|5,
hit:1243_conv:2487,3,guilty,I lied to my girlfriend once about how much education I had because I felt she would leave me otherwise,273,Yeah_comma_ I had to. It was too big of a secret. We had a really_comma_ really long talk. But she loves me for me_comma_ so she was able to accept it_comma_ and I've started working towards almost finishing the education I lied about having in the first place.,4|4|4_5|5|5,
hit:1243_conv:2487,4,guilty,I lied to my girlfriend once about how much education I had because I felt she would leave me otherwise,27,That's good. I always like about not understanfing football when I meet a guy. It makes them feel good lol,4|4|4_5|5|5,
hit:1244_conv:2488,1,devastated,I lost my dog the other day. They were my best friend since 12. Don't know what I'm going to do...,273,I lost my dog the other day. They were my best friend since 12. Don't know what I'm going to do..,5|5|5_5|5|5,
hit:1244_conv:2488,2,devastated,I lost my dog the other day. They were my best friend since 12. Don't know what I'm going to do...,218,I am so sorry.  Our pets are not just pets... they are part of our family.  Go through the grieving process_comma_ even if it takes time.  You deserve that.,5|5|5_5|5|5,
hit:1244_conv:2488,3,devastated,I lost my dog the other day. They were my best friend since 12. Don't know what I'm going to do...,273,i really appreciate that. It'll be hard... Especially for my son. He doesn't really understand that they're not coming back_comma_ since he's just 2,5|5|5_5|5|5,
hit:1244_conv:2488,4,devastated,I lost my dog the other day. They were my best friend since 12. Don't know what I'm going to do...,218,That is difficult.  My son didn't experience until age 10.,5|5|5_5|5|5,
hit:1245_conv:2490,1,jealous,I really don't like people with new cars because I cant affors one.,27,I can't afford a new car.,5|5|5_5|5|5,
hit:1245_conv:2490,2,jealous,I really don't like people with new cars because I cant affors one.,274,Are you deep in debt with medical expenses?,5|5|5_5|5|5,
hit:1245_conv:2490,3,jealous,I really don't like people with new cars because I cant affors one.,27,Deep in debt and poor.,5|5|5_5|5|5,
hit:1245_conv:2490,4,jealous,I really don't like people with new cars because I cant affors one.,274,Have you considered talking to a debt counselor?,5|5|5_5|5|5,
hit:1245_conv:2490,5,jealous,I really don't like people with new cars because I cant affors one.,27,Actually Im meeting with one soon,5|5|5_5|5|5,
hit:1245_conv:2491,1,joyful,I am excited to be starting law school soon.  I will be attending Syracuse University in the Spring.,274,I will be getting into some major debt myself soon with law school.,5|5|5_5|5|5,
hit:1245_conv:2491,2,joyful,I am excited to be starting law school soon.  I will be attending Syracuse University in the Spring.,27,Law school Wow,5|5|5_5|5|5,
hit:1245_conv:2491,3,joyful,I am excited to be starting law school soon.  I will be attending Syracuse University in the Spring.,274,Though the tuition is enormous_comma_ I am excited about the challenge that lies ahead.,5|5|5_5|5|5,
hit:1245_conv:2491,4,joyful,I am excited to be starting law school soon.  I will be attending Syracuse University in the Spring.,27,I hope you do well,5|5|5_5|5|5,
hit:1246_conv:2493,1,embarrassed,I wasted a beer all of myself at a party and couldnt get dry.,27,I wasted beer all over myself at a party the other night.,5|5|5_5|5|5,
hit:1246_conv:2493,2,embarrassed,I wasted a beer all of myself at a party and couldnt get dry.,122,That sounds like a sticky_comma_ stinky mess.,5|5|5_5|5|5,
hit:1246_conv:2493,3,embarrassed,I wasted a beer all of myself at a party and couldnt get dry.,27,It was all over me and I had no way to get it dry.,5|5|5_5|5|5,
hit:1246_conv:2493,4,embarrassed,I wasted a beer all of myself at a party and couldnt get dry.,122,What did you do?,5|5|5_5|5|5,
hit:1246_conv:2493,5,embarrassed,I wasted a beer all of myself at a party and couldnt get dry.,27,Went outside til it dried enough. Its warm at night here.,5|5|5_5|5|5,
hit:1247_conv:2494,1,excited,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,273,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,3|3|3_5|5|5,
hit:1247_conv:2494,2,excited,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,274,I completely understand.  I will be starting law school myself very soon. ,3|3|3_5|5|5,
hit:1247_conv:2494,3,excited,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,273,Law school... Yeah_comma_ my dad wanted me to be a lawyer_comma_ since he's one. I said no way. I couldn't handle that stress_comma_ and I don't have a natural interest in it. But good on you. It's a noble profession.,3|3|3_5|5|5,
hit:1247_conv:2494,4,excited,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,274,What year of college are you in?,3|3|3_5|5|5,
hit:1247_conv:2494,5,excited,School is starting soon. Getting that nervous feeling of being back in college again and having to make sure I can keep up with all of my assignments.,273,My senior year!,3|3|3_5|5|5,
hit:1247_conv:2495,1,lonely,Everyone has gone out for the evening and I am home alone.  It feels good to have peace and quiet but it sure seems lonely.,274,It is nights like this when I am alone that I will cherish when I start back to school.  But tonight seems a bit lonely with no one around.,5|5|5_3|3|3,
hit:1247_conv:2495,2,lonely,Everyone has gone out for the evening and I am home alone.  It feels good to have peace and quiet but it sure seems lonely.,273,Sorry to hear that. I'm in the same boat. At least I have two dogs with me. Do you have any friends around,5|5|5_3|3|3,
hit:1247_conv:2495,3,lonely,Everyone has gone out for the evening and I am home alone.  It feels good to have peace and quiet but it sure seems lonely.,274,No but I too have a dog that is great company.,5|5|5_3|3|3,
hit:1247_conv:2495,4,lonely,Everyone has gone out for the evening and I am home alone.  It feels good to have peace and quiet but it sure seems lonely.,273,Yeah_comma_ they can be. Except these guys get really rowdy when there's a thunderstorm (like there will be tonight_comma_ sadly...). What about yours?,5|5|5_3|3|3,
hit:1248_conv:2496,1,terrified,I get really scared when dogs are around.,4,I really get scared when I am around dogs.,5|5|5_4|4|4,
hit:1248_conv:2496,2,terrified,I get really scared when dogs are around.,297,Some dogs can be very intimidating. I can understand that. Did something bad happen?,5|5|5_4|4|4,
hit:1248_conv:2496,3,terrified,I get really scared when dogs are around.,4,I got bite by a dog when I was little.,5|5|5_4|4|4,
hit:1248_conv:2496,4,terrified,I get really scared when dogs are around.,297,That can be traumatizing to a young person. I hope you can one day overcome your fear. Most dogs are very friendly.,5|5|5_4|4|4,
hit:1248_conv:2497,1,guilty,I ate the rest of a co-workers sandwich that he was saving. Im feeling pretty bad about it.,297,So I have something to admit. I ate the rest of a co-workers sandwich the other day. I havent told anyone. Not my proudest moment.,4|4|4_5|5|5,
hit:1248_conv:2497,2,guilty,I ate the rest of a co-workers sandwich that he was saving. Im feeling pretty bad about it.,4,That isn't nice.,4|4|4_5|5|5,
hit:1248_conv:2497,3,guilty,I ate the rest of a co-workers sandwich that he was saving. Im feeling pretty bad about it.,297,Yea I know. Someone ate my lunch last week and I remember how mad I was. I know that is not a good enough excuse for what I did. Im feeling pretty bad about it right now.,4|4|4_5|5|5,
hit:1248_conv:2497,4,guilty,I ate the rest of a co-workers sandwich that he was saving. Im feeling pretty bad about it.,4,That isn't nice.,4|4|4_5|5|5,
hit:1249_conv:2498,1,angry,MY kids came in and wasted my last Dr Pepper. Thats my coffee/,27,My kids are so terrible sometimes.,5|5|5_4|4|4,
hit:1249_conv:2498,2,angry,MY kids came in and wasted my last Dr Pepper. Thats my coffee/,273,That's rough. I've been around my siblings kids long enough to know how rough they can be_comma_ even though I don't have my own. How old are they_comma_ if you don't mind me asking?,5|5|5_4|4|4,
hit:1249_conv:2498,3,angry,MY kids came in and wasted my last Dr Pepper. Thats my coffee/,27,3 and 4. They came running in here and wasted my last Dr Pepper. ,5|5|5_4|4|4,
hit:1249_conv:2498,4,angry,MY kids came in and wasted my last Dr Pepper. Thats my coffee/,273,Dang.. I love Dr. Pepper too. They can be selfish sometimes_comma_ but we just have to remember that they have developed the parts of their brain to make them care about more than just themselves.,5|5|5_4|4|4,
hit:1249_conv:2498,5,angry,MY kids came in and wasted my last Dr Pepper. Thats my coffee/,27,I just want them to go to bed and be infants again. They were sweet then.,5|5|5_4|4|4,
hit:1249_conv:2499,1,joyful,I get to see my long distance girlfriend in a week! Really happy we managed to afford a plane ticket so we could meet again,273,I get to see my long distance girlfriend in a week! Really happy we managed to afford a plane ticket so we could meet again,4|4|4_5|5|5,
hit:1249_conv:2499,2,joyful,I get to see my long distance girlfriend in a week! Really happy we managed to afford a plane ticket so we could meet again,27,That's awesome. Long distance can be tough.,4|4|4_5|5|5,
hit:1249_conv:2499,3,joyful,I get to see my long distance girlfriend in a week! Really happy we managed to afford a plane ticket so we could meet again,273,It really is. We've been doing it for 5 years though_comma_ since I've been in school.,4|4|4_5|5|5,
hit:1249_conv:2499,4,joyful,I get to see my long distance girlfriend in a week! Really happy we managed to afford a plane ticket so we could meet again,27,Wow. I could never do that. I dont think I could.,4|4|4_5|5|5,
hit:1250_conv:2500,1,joyful,My husband was a drug addict and after 15 years of being with him he got clean. He has been clean now for 4 years. I am so proud of him.,228,My husband has been clean from drugs for 4 years now.,5|5|5_5|5|5,
hit:1250_conv:2500,2,joyful,My husband was a drug addict and after 15 years of being with him he got clean. He has been clean now for 4 years. I am so proud of him.,273,Dang_comma_ that's quite an achievement. Everyone loves a good redemption story. How did you cope with it?,5|5|5_5|5|5,
hit:1250_conv:2500,3,joyful,My husband was a drug addict and after 15 years of being with him he got clean. He has been clean now for 4 years. I am so proud of him.,228,It was really hard. I have been with him for 21 years and we have 4 kids_comma_ but I am so happy for him and what he has accomplished.,5|5|5_5|5|5,
hit:1250_conv:2500,4,joyful,My husband was a drug addict and after 15 years of being with him he got clean. He has been clean now for 4 years. I am so proud of him.,273,That's crazy! You deserve just as much props as he does! I'm glad you guys stuck it through. That's incredibly rare.,5|5|5_5|5|5,
hit:1250_conv:2500,5,joyful,My husband was a drug addict and after 15 years of being with him he got clean. He has been clean now for 4 years. I am so proud of him.,228,Yes it is rare. He will be getting his 4 year chip at the NA meeting on Wednesday. It is awesome!,5|5|5_5|5|5,
hit:1250_conv:2501,1,afraid,My apartment is getting infested with bugs. I'm not so sure we'll be able to get rid of them entirely...,273,My apartment is getting infested with bugs. I'm not so sure we'll be able to get rid of them entirely..,5|5|5_5|5|5,
hit:1250_conv:2501,2,afraid,My apartment is getting infested with bugs. I'm not so sure we'll be able to get rid of them entirely...,228,That is horrible. I would hate to have to deal with that when it's not my fault.,5|5|5_5|5|5,
hit:1250_conv:2501,3,afraid,My apartment is getting infested with bugs. I'm not so sure we'll be able to get rid of them entirely...,273,Yeah_comma_ not a lot you can do when they just come from other apartments_comma_ even if you kill them in yours. It's gross to think about.,5|5|5_5|5|5,
hit:1250_conv:2501,4,afraid,My apartment is getting infested with bugs. I'm not so sure we'll be able to get rid of them entirely...,228,Yeah_comma_ they do have those sticky traps that you can put out to catch them once they spray really good. We dealt with roaches before_comma_ but got rid of them.,5|5|5_5|5|5,
hit:1251_conv:2502,1,disgusted,Stepped in dog poop and tracked it all throughout my parent's house because I didn't know I had done it. Then I had too clean it up. Really annoying...,273,Stepped in dog poop and tracked it all throughout my parent's house because I didn't know I had done it. Then I had too clean it up. Really annoying...,3|3|3_5|5|5,
hit:1251_conv:2502,2,disgusted,Stepped in dog poop and tracked it all throughout my parent's house because I didn't know I had done it. Then I had too clean it up. Really annoying...,4,When did this happen?,3|3|3_5|5|5,
hit:1251_conv:2502,3,disgusted,Stepped in dog poop and tracked it all throughout my parent's house because I didn't know I had done it. Then I had too clean it up. Really annoying...,273,Happened earlier today! The house still stinks.,3|3|3_5|5|5,
hit:1251_conv:2502,4,disgusted,Stepped in dog poop and tracked it all throughout my parent's house because I didn't know I had done it. Then I had too clean it up. Really annoying...,4,You should buy some spray.,3|3|3_5|5|5,
hit:1251_conv:2503,1,faithful,I know my Falcons will win the Super Bowl this year.,4,I know my Falcons will win the Super Bowl this year.,5|5|5_3|3|3,
hit:1251_conv:2503,2,faithful,I know my Falcons will win the Super Bowl this year.,273,Falcons fan_comma_ huh? That's cool. I like the Green Bay Packers. You guys knocked us out of the playoffs a couple years ago because of our bad defense :(,5|5|5_3|3|3,
hit:1251_conv:2503,3,faithful,I know my Falcons will win the Super Bowl this year.,4,Yes_comma_ I was at that game.,5|5|5_3|3|3,
hit:1251_conv:2503,4,faithful,I know my Falcons will win the Super Bowl this year.,273,Are you serious? I bet that was fun_comma_ to see you guys get to the Super Bowl on that. Memorable night.,5|5|5_3|3|3,
hit:1252_conv:2504,1,prepared,I did all of my kids school shopping weeks ago. Beat the crowds.,27,I beat all of the crazy crowds for back to school shopping.,5|5|5_5|5|5,
hit:1252_conv:2504,2,prepared,I did all of my kids school shopping weeks ago. Beat the crowds.,4,That was smart.,5|5|5_5|5|5,
hit:1252_conv:2504,3,prepared,I did all of my kids school shopping weeks ago. Beat the crowds.,27,I went shopping weeks ago. They need the same stuff every year.,5|5|5_5|5|5,
hit:1252_conv:2504,4,prepared,I did all of my kids school shopping weeks ago. Beat the crowds.,4,I hope you didn't pay full price.,5|5|5_5|5|5,
hit:1252_conv:2504,5,prepared,I did all of my kids school shopping weeks ago. Beat the crowds.,27,I haven't paid full price for anything since the late 90s,5|5|5_5|5|5,
hit:1252_conv:2505,1,anxious,I am really worried I will let my father down.,4,I am really worried I will let my father down.,5|5|5_5|5|5,
hit:1252_conv:2505,2,anxious,I am really worried I will let my father down.,27,Why would you think that?,5|5|5_5|5|5,
hit:1252_conv:2505,3,anxious,I am really worried I will let my father down.,4,Cause I usually mess up.,5|5|5_5|5|5,
hit:1252_conv:2505,4,anxious,I am really worried I will let my father down.,27,Try hard not too. That's the best you can do,5|5|5_5|5|5,
hit:1253_conv:2506,1,guilty,Ate the last biscuit this morning during breakfast. Then my child came in asking for another one and I felt really bad. Oh well. Maybe I'll get her something as a treat tomorroow.,273,Ate the last biscuit this morning during breakfast. Then my child came in asking for another one and I felt really bad. Oh well. Maybe I'll get her something as a treat tomorroow.,4|4|4_5|5|5,
hit:1253_conv:2506,2,guilty,Ate the last biscuit this morning during breakfast. Then my child came in asking for another one and I felt really bad. Oh well. Maybe I'll get her something as a treat tomorroow.,122,It really makes a parent feel bad when that happens. I live with what my neighbor calls mom guilt.,4|4|4_5|5|5,
hit:1253_conv:2506,3,guilty,Ate the last biscuit this morning during breakfast. Then my child came in asking for another one and I felt really bad. Oh well. Maybe I'll get her something as a treat tomorroow.,273,Hah. Yeah_comma_ I'm the same way. But sometimes it's hard to plan how much they want to eat!,4|4|4_5|5|5,
hit:1253_conv:2506,4,guilty,Ate the last biscuit this morning during breakfast. Then my child came in asking for another one and I felt really bad. Oh well. Maybe I'll get her something as a treat tomorroow.,122,very difficult. If you hadn't eaten it the child most likely wouldn't have wanted it.,4|4|4_5|5|5,
hit:1254_conv:2508,1,disappointed,Didn't get to exercise today. I feel like I'm lazy.,273,Didn't get to exercise today. I feel like I'm lazy,5|5|5_5|5|5,
hit:1254_conv:2508,2,disappointed,Didn't get to exercise today. I feel like I'm lazy.,4,You can still exercise now.,5|5|5_5|5|5,
hit:1254_conv:2508,3,disappointed,Didn't get to exercise today. I feel like I'm lazy.,273,It's almost 9_comma_ and I still have work to do. I want to go jogging_comma_ but I don't think I have time :(,5|5|5_5|5|5,
hit:1254_conv:2508,4,disappointed,Didn't get to exercise today. I feel like I'm lazy.,4,You should do it_comma_ even 15 minutes.,5|5|5_5|5|5,
hit:1254_conv:2509,1,lonely,I wish I had more friends.,4,I wish I had  more friends.,5|5|5_5|5|5,
hit:1254_conv:2509,2,lonely,I wish I had more friends.,273,I'm sorry to hear that. Do you think you struggle to communicate with people?,5|5|5_5|5|5,
hit:1254_conv:2509,3,lonely,I wish I had more friends.,4,I am very sky.,5|5|5_5|5|5,
hit:1254_conv:2509,4,lonely,I wish I had more friends.,273,Shy? Yeah_comma_ I get that. I was too_comma_ until I started getting out more. The ability to socialize is a muscle you have to work at.,5|5|5_5|5|5,
hit:1255_conv:2510,1,trusting,Really hoping my brother comes through and can fix my car so that I don't have to spend as much money to fix it. Otherwise_comma_ I don't know how I'll pay rent.,273,Really hoping my brother comes through and can fix my car so that I don't have to spend as much money to fix it. Otherwise_comma_ I don't know how I'll pay rent,3|3|3_5|5|5,
hit:1255_conv:2510,2,trusting,Really hoping my brother comes through and can fix my car so that I don't have to spend as much money to fix it. Otherwise_comma_ I don't know how I'll pay rent.,7,It is stressful situation. I hope your brother find time to help you.,3|3|3_5|5|5,
hit:1255_conv:2510,3,trusting,Really hoping my brother comes through and can fix my car so that I don't have to spend as much money to fix it. Otherwise_comma_ I don't know how I'll pay rent.,273,Yeah_comma_ no kidding. If he can't_comma_ I'm doomed.,3|3|3_5|5|5,
hit:1255_conv:2510,4,trusting,Really hoping my brother comes through and can fix my car so that I don't have to spend as much money to fix it. Otherwise_comma_ I don't know how I'll pay rent.,7,Sorry to hear that,3|3|3_5|5|5,
hit:1255_conv:2511,1,terrified,When I was a kid_comma_ I was pulling water from well in my grand father house. When the bucket was out of the well_comma_ a live snake was floating inside of it.,7,When I was a kid_comma_ I was pulling water from well in my grand father house. When the bucket was out of the well_comma_ a live snake was floating inside of it.,5|5|5_3|3|3,
hit:1255_conv:2511,2,terrified,When I was a kid_comma_ I was pulling water from well in my grand father house. When the bucket was out of the well_comma_ a live snake was floating inside of it.,273,that's crazy. Snakes are one of my worst fears. Was it poisonous_comma_ do you know?,5|5|5_3|3|3,
hit:1255_conv:2511,3,terrified,When I was a kid_comma_ I was pulling water from well in my grand father house. When the bucket was out of the well_comma_ a live snake was floating inside of it.,7,No_comma_ I started shouting. I was so scared.,5|5|5_3|3|3,
hit:1255_conv:2511,4,terrified,When I was a kid_comma_ I was pulling water from well in my grand father house. When the bucket was out of the well_comma_ a live snake was floating inside of it.,273,Hah. I would have to. I would have booked it the second I saw it. People could prank me easily with fake snakes if they wanted to.,5|5|5_3|3|3,
hit:1256_conv:2512,1,annoyed,The other day at work_comma_ i kept having to do everyone elses just.  It was frustrating to do someone elses work for them,301,Its frustrating to get paid less than other people when you are doing their job,5|5|5_5|5|5,
hit:1256_conv:2512,2,annoyed,The other day at work_comma_ i kept having to do everyone elses just.  It was frustrating to do someone elses work for them,273,Wow_comma_ you are? How did you find out?,5|5|5_5|5|5,
hit:1256_conv:2512,3,annoyed,The other day at work_comma_ i kept having to do everyone elses just.  It was frustrating to do someone elses work for them,301,They were just being lazy_comma_ and the work had to get done_comma_ so I had to do it,5|5|5_5|5|5,
hit:1256_conv:2512,4,annoyed,The other day at work_comma_ i kept having to do everyone elses just.  It was frustrating to do someone elses work for them,273,What do you do_comma_ if you don't mind me asking?,5|5|5_5|5|5,
hit:1256_conv:2512,5,annoyed,The other day at work_comma_ i kept having to do everyone elses just.  It was frustrating to do someone elses work for them,301,I work at a retail store and we had a closing checklist that needed to be done,5|5|5_5|5|5,
hit:1256_conv:2513,1,sentimental,Even though they're in their 60s and have been together for the past 30 years_comma_ my mom and dad still get giddy around each other. Makes me happy.,273,Even though they're in their 60s and have been together for the past 30 years_comma_ my mom and dad still get giddy around each other. Makes me happy,5|5|5_5|5|5,
hit:1256_conv:2513,2,sentimental,Even though they're in their 60s and have been together for the past 30 years_comma_ my mom and dad still get giddy around each other. Makes me happy.,301,Thats really awesome that they are still so much in love.  do you have a significant other?,5|5|5_5|5|5,
hit:1256_conv:2513,3,sentimental,Even though they're in their 60s and have been together for the past 30 years_comma_ my mom and dad still get giddy around each other. Makes me happy.,273,I do! We've been together for six years. I still feel the same way about her as I did the first day. How about you?,5|5|5_5|5|5,
hit:1256_conv:2513,4,sentimental,Even though they're in their 60s and have been together for the past 30 years_comma_ my mom and dad still get giddy around each other. Makes me happy.,301,Awesome! my wife and I got married 2 years ago_comma_ I hope we make it 30 years like your parents!,5|5|5_5|5|5,
hit:1257_conv:2514,1,grateful,My dog ran away and some kind person brought him back to me.,99,My dog ran away and some kind person brought him back to me,5|5|5_1|5|5,
hit:1257_conv:2514,2,grateful,My dog ran away and some kind person brought him back to me.,296,That's so awesome! So many times they don't come back. ,5|5|5_1|5|5,
hit:1257_conv:2514,3,grateful,My dog ran away and some kind person brought him back to me.,99,I know I was so thankful and happy to have him back,5|5|5_1|5|5,
hit:1257_conv:2514,4,grateful,My dog ran away and some kind person brought him back to me.,296,Fantastic! What kind of dog is he?,5|5|5_1|5|5,
hit:1257_conv:2515,1,proud,I was very proud of my mom when she got through her surgery.,296,My mom went through a bad surgery and has recovered well.,1|5|5_5|5|5,
hit:1257_conv:2515,2,proud,I was very proud of my mom when she got through her surgery.,99,I'm glad to hear. What kind of surgery?,1|5|5_5|5|5,
hit:1257_conv:2515,3,proud,I was very proud of my mom when she got through her surgery.,296,She had stomach surgery for holes. She is older and now she  is changing her diet,1|5|5_5|5|5,
hit:1257_conv:2515,4,proud,I was very proud of my mom when she got through her surgery.,99,I'm happy she is doing well and making changes!,1|5|5_5|5|5,
hit:1257_conv:2515,5,proud,I was very proud of my mom when she got through her surgery.,296,Thank you_comma_ you're so nice!,1|5|5_5|5|5,
hit:1258_conv:2516,1,disgusted,MY son refuses to go to the potty. He is 3 1/2. I am tired of dirty diapers.,27,I am so tired of cleaning dirty diapers.,5|5|5_4|4|4,
hit:1258_conv:2516,2,disgusted,MY son refuses to go to the potty. He is 3 1/2. I am tired of dirty diapers.,273,I hear you. I've had to change the diapers of my nieces and nephews for the past 15 years_comma_ as they've come and gone.,5|5|5_4|4|4,
hit:1258_conv:2516,3,disgusted,MY son refuses to go to the potty. He is 3 1/2. I am tired of dirty diapers.,27,MY son is 3 1/2 those are baby poops. He refuses to use the potty though.,5|5|5_4|4|4,
hit:1258_conv:2516,4,disgusted,MY son refuses to go to the potty. He is 3 1/2. I am tired of dirty diapers.,273,I've never potty trained them. At what age does that normally happen? 3?,5|5|5_4|4|4,
hit:1258_conv:2516,5,disgusted,MY son refuses to go to the potty. He is 3 1/2. I am tired of dirty diapers.,27,Two if they try. He is stubborn,5|5|5_4|4|4,
hit:1258_conv:2517,1,annoyed,There's a fly inside this house and I can't_comma_ for the life of me_comma_ kill it. I'm trying to work peacefully and it just keeps buzzing around.,273,There's a fly inside this house and I can't_comma_ for the life of me_comma_ kill it. I'm trying to work peacefully and it just keeps buzzing around,4|4|4_5|5|5,
hit:1258_conv:2517,2,annoyed,There's a fly inside this house and I can't_comma_ for the life of me_comma_ kill it. I'm trying to work peacefully and it just keeps buzzing around.,27,I hate that. Flies and gnats are so annoying,4|4|4_5|5|5,
hit:1258_conv:2517,3,annoyed,There's a fly inside this house and I can't_comma_ for the life of me_comma_ kill it. I'm trying to work peacefully and it just keeps buzzing around.,273,Yeah_comma_ they are. Especially when they seem to be particularly skilled at being elusive for whatever reason. Like some of them just got additional expert training or whatever,4|4|4_5|5|5,
hit:1258_conv:2517,4,annoyed,There's a fly inside this house and I can't_comma_ for the life of me_comma_ kill it. I'm trying to work peacefully and it just keeps buzzing around.,27,Lol. It does seem like that sometimes. They can sit on the blinds for forever and move at the last second when you swing the swatter. Like Seal Team Six flies or something.,4|4|4_5|5|5,
hit:1259_conv:2518,1,grateful,I was grateful to be alive after I survived a car wreck.,296,I was grateful I was live after a car wreck.,5|5|5_5|5|5,
hit:1259_conv:2518,2,grateful,I was grateful to be alive after I survived a car wreck.,4,I am glad your ok.,5|5|5_5|5|5,
hit:1259_conv:2518,3,grateful,I was grateful to be alive after I survived a car wreck.,296,Thank you! Sorry I meant alive- It is very scary to be in the hospital and having others take care of you,5|5|5_5|5|5,
hit:1259_conv:2518,4,grateful,I was grateful to be alive after I survived a car wreck.,4,I was in a car wreck once.,5|5|5_5|5|5,
hit:1259_conv:2518,5,grateful,I was grateful to be alive after I survived a car wreck.,296,Oh no! Glad you are ok too!,5|5|5_5|5|5,
hit:1259_conv:2519,1,excited,I can not wait for preseason football Thursday.,4,I can't wait for preseason football Thursday.,5|5|5_5|5|5,
hit:1259_conv:2519,2,excited,I can not wait for preseason football Thursday.,296,Are you a big football fan?,5|5|5_5|5|5,
hit:1259_conv:2519,3,excited,I can not wait for preseason football Thursday.,4,Yes_comma_ I love it more than any other sport.,5|5|5_5|5|5,
hit:1259_conv:2519,4,excited,I can not wait for preseason football Thursday.,296,Do you paint your face and buy everything from your favorite team?,5|5|5_5|5|5,
hit:1260_conv:2521,1,joyful,My little sister is getting married!  I am so happy for her and can't wait for the wedding!,277,My little sister is getting married!  I am so happy for her and can't wait for the wedding!,5|5|5_5|5|5,
hit:1260_conv:2521,2,joyful,My little sister is getting married!  I am so happy for her and can't wait for the wedding!,218,How exciting!  When is the wedding?,5|5|5_5|5|5,
hit:1260_conv:2521,3,joyful,My little sister is getting married!  I am so happy for her and can't wait for the wedding!,277,August 11!  I t will be such a happy event.  The wedding is in New Orleans!,5|5|5_5|5|5,
hit:1260_conv:2521,4,joyful,My little sister is getting married!  I am so happy for her and can't wait for the wedding!,218,Wow that's right around the corner!  New Orleans is the PERFECT place for a celebration_comma_ too!,5|5|5_5|5|5,
hit:1261_conv:2522,1,surprised,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,301,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,4|5|5_5|5|5,
hit:1261_conv:2522,2,surprised,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,278,I haven't won on a ticket in a long time_comma_ but I once one $300!! Spend it on anything fun?,4|5|5_5|5|5,
hit:1261_conv:2522,3,surprised,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,301,I was shocked!  I am currently saving up money for a vacation_comma_ so it all just went towards that ,4|5|5_5|5|5,
hit:1261_conv:2522,4,surprised,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,278,Hey that's still fun! I spent mine on paying off a credit card in part_comma_ which is boring but lead to more money in the long run.,4|5|5_5|5|5,
hit:1261_conv:2522,5,surprised,The other day I won $100 on a scratch off lottery ticket and I couldn't believe my luck,301,Thats what im currently doing.  Getting rid of that debt,4|5|5_5|5|5,
hit:1262_conv:2524,1,prepared,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up.,273,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up,3|3|3_5|5|5,
hit:1262_conv:2524,2,prepared,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up.,4,Why did you lie?,3|3|3_5|5|5,
hit:1262_conv:2524,3,prepared,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up.,273,I was just scared in the moment. It was a complete gut reaction. I thought I didn't deserve to go to jail. But now I realize I do.,3|3|3_5|5|5,
hit:1262_conv:2524,4,prepared,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up.,4,How long will you go for?,3|3|3_5|5|5,
hit:1262_conv:2524,5,prepared,Getting ready to make the toughest decision of my life tomorrow. Have to turn myself in for a hit and run that I did 2 days ago. Guilt has been eating me up.,273,I have no idea... It's something I will find out tomorrow. Any time is too much time.,3|3|3_5|5|5,
hit:1262_conv:2525,1,ashamed,I can't believe I used salt instead of sugar in the cake I baked.,4,I can't believe I used salt instead of sugar in the cake I baked.,5|5|5_3|3|3,
hit:1262_conv:2525,2,ashamed,I can't believe I used salt instead of sugar in the cake I baked.,273,Whoaa.... You probably had to throw that away immediately_comma_ huh? Don't worry_comma_ my sister-in-law forgot to put eggs in a cake one time and it came out like a brick.,5|5|5_3|3|3,
hit:1262_conv:2525,3,ashamed,I can't believe I used salt instead of sugar in the cake I baked.,4,Yes_comma_ it was a disaster.,5|5|5_3|3|3,
hit:1262_conv:2525,4,ashamed,I can't believe I used salt instead of sugar in the cake I baked.,273,Well_comma_ you live and you learn. At least you can bake_comma_ or try to. I don't hardly cook at all_cmma_ besides simple things!,5|5|5_3|3|3,


hit:1264_conv:2529,1,content,Had a really good spaghetti dinner tonight. Just relaxing and getting some work done now. Feeling pretty accomplished today.,273,"Had a really good spaghetti dinner tonight. Just relaxing and getting some work done now. Feeling pretty accomplished today,5|5|5_4|5|5,
hit:1264_conv:2529,2,content,Had a really good spaghetti dinner tonight. Just relaxing and getting some work done now. Feeling pretty accomplished today.,302,Big dinners are always a great way to tie everything up at the end of the day. Was it a pretty long day for you?,5|5|5_4|5|5,
hit:1264_conv:2529,3,content,Had a really good spaghetti dinner tonight. Just relaxing and getting some work done now. Feeling pretty accomplished today.,273,With work_comma_ yeah. It was also nice because we had everyone in the family there_comma_ about 9 of us_comma_ with lots of kids. Very lively dinner table.,5|5|5_4|5|5,
hit:1264_conv:2529,4,content,Had a really good spaghetti dinner tonight. Just relaxing and getting some work done now. Feeling pretty accomplished today.,302,Haha I bet! Always great to have everyone together_comma_ though_comma_ really gets your mind off all the stresses of daily life.,5|5|5_4|5|5,
hit:1265_conv:2530,1,sad,I didn't do well on my job interview.,220,It's such a bummer that I did badly on the job interview I had today.,5|5|5_5|5|5,
hit:1265_conv:2530,2,sad,I didn't do well on my job interview.,27,I'm sorry. Are you sure you did that bad?,5|5|5_5|5|5,
hit:1265_conv:2530,3,sad,I didn't do well on my job interview.,220,I am and I prepared so much for it! It's all quite upsetting.,5|5|5_5|5|5,
hit:1265_conv:2530,4,sad,I didn't do well on my job interview.,27,They'll be other interviews.,5|5|5_5|5|5,
hit:1265_conv:2531,1,sad,All of the good shows are ending. It really ruins tv for me.,27,Every good show on tv is ending or has already ended.,5|5|5_5|5|5,
hit:1265_conv:2531,2,sad,All of the good shows are ending. It really ruins tv for me.,220,What shows are you thinking of specifically?,5|5|5_5|5|5,
hit:1265_conv:2531,3,sad,All of the good shows are ending. It really ruins tv for me.,27,Game of thrones is the top one. I want it to last forever,5|5|5_5|5|5,
hit:1265_conv:2531,4,sad,All of the good shows are ending. It really ruins tv for me.,220,I didn't realize it was ending? You must be disappointed.,5|5|5_5|5|5,
hit:1265_conv:2531,5,sad,All of the good shows are ending. It really ruins tv for me.,27,I am. It better end well because there wasnt even a season this year.,5|5|5_5|5|5,
hit:1266_conv:2532,1,afraid,I was afraid the other night when gunshots went off,296,I was scared the other night when I heard several gunshots in a row.,5|5|5_4|5|5,
hit:1266_conv:2532,2,afraid,I was afraid the other night when gunshots went off,269,Oh wow_comma_ I hope you are fine!,5|5|5_4|5|5,
hit:1266_conv:2532,3,afraid,I was afraid the other night when gunshots went off,296,I am. I am in a very dangerous neighborhood so you just never know if it's going to get you and the police don't show up.,5|5|5_4|5|5,
hit:1266_conv:2532,4,afraid,I was afraid the other night when gunshots went off,269,Yikes. If I was you_comma_ I would get my own gun! Better to have it and not need it_comma_ then have not have it then need it!,5|5|5_4|5|5,
hit:1266_conv:2532,5,afraid,I was afraid the other night when gunshots went off,296,Yes_comma_ I have thought about it. I have pepper spray and a few big dogs- I am saving up to move out of here soon and I have cameras,5|5|5_4|5|5,
hit:1266_conv:2533,1,anticipating,I can't wait until I get paid on Friday.,269,I can't wait to get paid on Friday!,4|5|5_5|5|5,
hit:1266_conv:2533,2,anticipating,I can't wait until I get paid on Friday.,296,payday is the best day! What are you doing to do?,4|5|5_5|5|5,
hit:1266_conv:2533,3,anticipating,I can't wait until I get paid on Friday.,269,I'm thinking save half of it and then spend the rest to get a standing desk. I sit way too much and my hips pay the price unfortunately!,4|5|5_5|5|5,
hit:1266_conv:2533,4,anticipating,I can't wait until I get paid on Friday.,296,OH wow! I feel you there and it's totally worth it! If you have to sit- get a donut pillow to not sit on your tailbone.,4|5|5_5|5|5,
hit:1267_conv:2534,1,sad,I am upset Brian Lawyer died.,4,I am upset Brian Lawyer died.,5|5|5_2|5|5,
hit:1267_conv:2534,2,sad,I am upset Brian Lawyer died.,286,Oh that's horrible. Who's Brian Lawyer?,5|5|5_2|5|5,
hit:1267_conv:2534,3,sad,I am upset Brian Lawyer died.,4,He was a WWE wrestler.,5|5|5_2|5|5,
hit:1267_conv:2534,4,sad,I am upset Brian Lawyer died.,286,I've never heard of him_comma_ but that sucks. How'd he die?,5|5|5_2|5|5,
hit:1267_conv:2534,5,sad,I am upset Brian Lawyer died.,4,He hung himself.,5|5|5_2|5|5,
hit:1267_conv:2534,6,sad,I am upset Brian Lawyer died.,286,Oh wow that's horrible.,5|5|5_2|5|5,
hit:1267_conv:2535,1,hopeful,I started taking a new body building supplement and I've been feeling stronger_comma_ so I'm hopeful that it works well.,286,I've just started taking a new body building supplement and it's working wonders so far.,2|5|5_5|5|5,
hit:1267_conv:2535,2,hopeful,I started taking a new body building supplement and I've been feeling stronger_comma_ so I'm hopeful that it works well.,4,What is the name of it?,2|5|5_5|5|5,
hit:1267_conv:2535,3,hopeful,I started taking a new body building supplement and I've been feeling stronger_comma_ so I'm hopeful that it works well.,286,It's creatine. I'm hoping I continue to make progress while using it.,2|5|5_5|5|5,
hit:1267_conv:2535,4,hopeful,I started taking a new body building supplement and I've been feeling stronger_comma_ so I'm hopeful that it works well.,4,I hear that stuff is bad for you.,2|5|5_5|5|5,
hit:1267_conv:2535,5,hopeful,I started taking a new body building supplement and I've been feeling stronger_comma_ so I'm hopeful that it works well.,286,It's very well researched and is proven safe and effective. One of the few supplements that actually work.,2|5|5_5|5|5,
hit:1268_conv:2536,1,confident,I know I will be able to get my speeding ticket overturned.,4,I know I will be able to get my speeding ticket overturned.,5|5|5_4|3|3,
hit:1268_conv:2536,2,confident,I know I will be able to get my speeding ticket overturned.,294,Ohh no_comma_ were you pulled over wrongly?,5|5|5_4|3|3,
hit:1268_conv:2536,3,confident,I know I will be able to get my speeding ticket overturned.,4,Yes_comma_ his radar was way off.,5|5|5_4|3|3,
hit:1268_conv:2536,4,confident,I know I will be able to get my speeding ticket overturned.,294,How fast did they say you were going?,5|5|5_4|3|3,
hit:1268_conv:2537,1,proud,My daughter learned to read and I am so unbelievably happy about it!,294,My daughter has learned to read and it is so rewarding to see her continue to grow!,4|3|3_5|5|5,
hit:1268_conv:2537,2,proud,My daughter learned to read and I am so unbelievably happy about it!,4,Thats great news.,4|3|3_5|5|5,
hit:1268_conv:2537,3,proud,My daughter learned to read and I am so unbelievably happy about it!,294,Yeah_comma_ she is really getting to be so grown up.,4|3|3_5|5|5,
hit:1268_conv:2537,4,proud,My daughter learned to read and I am so unbelievably happy about it!,4,That is great news.,4|3|3_5|5|5,
hit:1269_conv:2539,1,anticipating,My brother is about to find out that I had to steal $20 from him to pay for gas_comma_ since I was in such a bind. He's gonna be so mad at me.,273,My brother is about to find out that I had to steal $20 from him to pay for gas_comma_ since I was in such a bind. He's gonna be so mad at me,4|4|4_5|5|5,
hit:1269_conv:2539,2,anticipating,My brother is about to find out that I had to steal $20 from him to pay for gas_comma_ since I was in such a bind. He's gonna be so mad at me.,122,Did you really steal it or was it more like borrowing without asking first?,4|4|4_5|5|5,
hit:1269_conv:2539,3,anticipating,My brother is about to find out that I had to steal $20 from him to pay for gas_comma_ since I was in such a bind. He's gonna be so mad at me.,273,Kind of the latter... But still_comma_ I feel grimy about it. I just had no other alternative. I'm between jobs_comma_ and I needed gas to get to a job interview. We'll see how he reacts_comma_ but he has a short fuse.,4|4|4_5|5|5,
hit:1269_conv:2539,4,anticipating,My brother is about to find out that I had to steal $20 from him to pay for gas_comma_ since I was in such a bind. He's gonna be so mad at me.,122,Hmmmm. with no job and no way to pay it back I sure hope he can be understanding.,4|4|4_5|5|5,
hit:1270_conv:2540,1,nostalgic,Watched an old TV show that I used to watch with my ex-girlfriend. Made me yearn for those past times. But trying not to get too down in the dumps about it.,273,Watched an old TV show that I used to watch with my ex-girlfriend. Made me yearn for those past times. But trying not to get too down in the dumps about it.,3|3|3_5|5|5,
hit:1270_conv:2540,2,nostalgic,Watched an old TV show that I used to watch with my ex-girlfriend. Made me yearn for those past times. But trying not to get too down in the dumps about it.,27,What show was it?,3|3|3_5|5|5,
hit:1270_conv:2540,3,nostalgic,Watched an old TV show that I used to watch with my ex-girlfriend. Made me yearn for those past times. But trying not to get too down in the dumps about it.,273,It was Dr. Who. She was a nerd who love to dress up as those characters and go to conventions.,3|3|3_5|5|5,
hit:1270_conv:2540,4,nostalgic,Watched an old TV show that I used to watch with my ex-girlfriend. Made me yearn for those past times. But trying not to get too down in the dumps about it.,27,Oh. I've never seen that show. I know it has die hard fans though.,3|3|3_5|5|5,
hit:1270_conv:2541,1,guilty,I spend way too much money on food. Its getting bad.,27,I spend way too much money on food.,5|5|5_3|3|3,
hit:1270_conv:2541,2,guilty,I spend way too much money on food. Its getting bad.,273,That's a tough one. Do you eat out a lot? That's usually the culprit. I had to learn how to cook for myself so I could afford to live.,5|5|5_3|3|3,
hit:1270_conv:2541,3,guilty,I spend way too much money on food. Its getting bad.,27,I get food delivered several times a week. It's terrible.,5|5|5_3|3|3,
hit:1270_conv:2541,4,guilty,I spend way too much money on food. Its getting bad.,273,Oh man_comma_ that's doubly terrible! Yeah_comma_ if you can_comma_ you should try to make a budget and get better at cooking for yourself. Huge money saver_comma_ but I'm sure you already knew that. I hope it works for you_comma_ though.,5|5|5_3|3|3,
hit:1270_conv:2541,5,guilty,I spend way too much money on food. Its getting bad.,27,I know. I have to do better. It is a real waste.,5|5|5_3|3|3,
hit:1271_conv:2542,1,anxious,Really don't want to start my new job in a week. It's at a fast food place and I don't know if I'll be able to keep up with demand.,273,Really don't want to start my new job in a week. It's at a fast food place and I don't know if I'll be able to keep up with demand,4|4|4_5|5|5,
hit:1271_conv:2542,2,anxious,Really don't want to start my new job in a week. It's at a fast food place and I don't know if I'll be able to keep up with demand.,296,Those jobs are hard. You can always try it and see if you like it. I recommend a temp agency they often have nice desk jobs. Some even offer work from home jobs.,4|4|4_5|5|5,
hit:1271_conv:2542,3,anxious,Really don't want to start my new job in a week. It's at a fast food place and I don't know if I'll be able to keep up with demand.,273,Yeah. But maybe it will instill some good work ethic in me_comma_ and teach me how to be good at multi-tasking. We'll see.,4|4|4_5|5|5,
hit:1271_conv:2542,4,anxious,Really don't want to start my new job in a week. It's at a fast food place and I don't know if I'll be able to keep up with demand.,296,MOst work requires multi tasking. Yes that is true. You sound like you already have a good head on your shoulders.,4|4|4_5|5|5,
hit:1271_conv:2543,1,surprised,When someone stole my lawn mower.,296,I had someone steal my lawn mower. I never had anything stolen since I lived here.,5|5|5_4|4|4,
hit:1271_conv:2543,2,surprised,When someone stole my lawn mower.,273,that's terrible. Was it an expensive one?,5|5|5_4|4|4,
hit:1271_conv:2543,3,surprised,When someone stole my lawn mower.,296,I think they all are. At least 100 or 200. I was shocked because I have a fenced in yard and dogs people are scared of- They WILL bite and are big- They were very brave and stupid.,5|5|5_4|4|4,
hit:1271_conv:2543,4,surprised,When someone stole my lawn mower.,273,No kidding... Who goes out of their way to get a lawnmower? Like... Why risk getting arrested and going to jail for a lawnmower... Oh well... Humanity. Sorry to hear that happened to you.,5|5|5_4|4|4,
hit:1271_conv:2543,5,surprised,When someone stole my lawn mower.,296,Thank you! Yes it's been a trial since- Can't afford another. I hate thieves- Lots of druggies around.,5|5|5_4|4|4,
hit:1272_conv:2544,1,lonely,My dad just passed away. Even though I haven't lived near him to be able to see him often in a long time_comma_ I really feel his missed presence. It just feels... strange.,273,My dad just passed away. Even though I haven't lived near him to be able to see him often in a long time_comma_ I really feel his missed presence. It just feels... strange,4|4|4_5|5|5,
hit:1272_conv:2544,2,lonely,My dad just passed away. Even though I haven't lived near him to be able to see him often in a long time_comma_ I really feel his missed presence. It just feels... strange.,269,I'm sorry to hear that_comma_ I hope he is in a better place now! How old was he?,4|4|4_5|5|5,
hit:1272_conv:2544,3,lonely,My dad just passed away. Even though I haven't lived near him to be able to see him often in a long time_comma_ I really feel his missed presence. It just feels... strange.,273,He was only 67. He struggled with a heart condition most of his life. Thanks for the kind words.,4|4|4_5|5|5,
hit:1272_conv:2544,4,lonely,My dad just passed away. Even though I haven't lived near him to be able to see him often in a long time_comma_ I really feel his missed presence. It just feels... strange.,269,Oh wow_comma_ I can't imagine losing my dad at 67. I hope you make it through these hard times_comma_ stay strong!,4|4|4_5|5|5,
hit:1272_conv:2545,1,impressed,I was in awe of the driving ability of my old aunt_comma_ she's old but she still has it!,269,I was in awe at the driving ability of my aunt_comma_ she's old but she still has it!,5|5|5_4|4|4,
hit:1272_conv:2545,2,impressed,I was in awe of the driving ability of my old aunt_comma_ she's old but she still has it!,273,Right on! That's cool to hear. Everyone likes to rag on older people driving. Glad to hear she's the exception. What kind of car does she drive?,5|5|5_4|4|4,
hit:1272_conv:2545,3,impressed,I was in awe of the driving ability of my old aunt_comma_ she's old but she still has it!,269,Thank you! She drives an old cadillac and she drives it like a Ferrari. It's pretty grand watching her drive sometimes!,5|5|5_4|4|4,
hit:1272_conv:2545,4,impressed,I was in awe of the driving ability of my old aunt_comma_ she's old but she still has it!,273,Haha... That sounds awesome. Wish I could carpool with her. I hate driving_comma_ but it sounds like she loves it.,5|5|5_4|4|4,
hit:1273_conv:2546,1,prepared,I'm having a party this weekend_comma_ I went out and got the food ahead of time.,220,I'm having a party this weekend and I already got my grocery shopping done for it! I refuse to be last minute like last time.,5|5|5_5|5|5,
hit:1273_conv:2546,2,prepared,I'm having a party this weekend_comma_ I went out and got the food ahead of time.,218,I have a friend who plans ahead like that.  I'm a last minute gal.  I think you will be less stressed by planning ahead.,5|5|5_5|5|5,
hit:1273_conv:2546,3,prepared,I'm having a party this weekend_comma_ I went out and got the food ahead of time.,220,I will be and be able to enjoy myself more in the days leading up to the party.,5|5|5_5|5|5,
hit:1273_conv:2546,4,prepared,I'm having a party this weekend_comma_ I went out and got the food ahead of time.,218,Not to mention enjoy the actual party_comma_ too!  Are you celebrating anything in particular?,5|5|5_5|5|5,
hit:1274_conv:2548,1,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,273,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,5|5|5_5|5|5,
hit:1274_conv:2548,2,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,286,Oh my god_comma_ I would have been so embarrassed! How'd you react?,5|5|5_5|5|5,
hit:1274_conv:2548,3,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,273,mortified. Tried to pretend it didn't happen. At first no one noticed_comma_ but then they all figured it out. I left at lunch time :(,5|5|5_5|5|5,
hit:1274_conv:2548,4,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,286,So you left work at lunch time? Did you come back or were you too ashamed?,5|5|5_5|5|5,
hit:1274_conv:2548,5,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,273,I came back! Was worried about getting fired. People will probably tease me for a bit_comma_ but oh well.,5|5|5_5|5|5,
hit:1274_conv:2548,6,embarrassed,So I was at work today_comma_ wearing some slightly tight jeans_comma_ since they're slim fit_comma_ and I bent over and they split. Just like in the movies. I meant people could see my undies...,286,Yeah_comma_ I mean things happen. Better to put it in the back of your head and keep trucking along.,5|5|5_5|5|5,
hit:1274_conv:2549,1,excited,I have a two week vacation coming out and I'm really excited about it.,286,I have a two week vacation coming out and I'm friggin' excited! ,5|5|5_5|5|5,
hit:1274_conv:2549,2,excited,I have a two week vacation coming out and I'm really excited about it.,273,Dang_comma_ a whole two weeks? Going anywhere_comma_ or just hanging out and really enjoying yourself at home?,5|5|5_5|5|5,
hit:1274_conv:2549,3,excited,I have a two week vacation coming out and I'm really excited about it.,286,Plan on doing both_comma_ actually. We're going to Florida for about a week and then after we return from that trip I'm going to just bum out for a week.,5|5|5_5|5|5,
hit:1274_conv:2549,4,excited,I have a two week vacation coming out and I'm really excited about it.,273,That's a good way to do it. especially because it's easy to get those post-vacation blues. That's actually really smart,5|5|5_5|5|5,
hit:1274_conv:2549,5,excited,I have a two week vacation coming out and I'm really excited about it.,286,Yeah_comma_ I think it'll really allow me to recoup and be 110% ready to attack work by the time the vacation is over.,5|5|5_5|5|5,
hit:1275_conv:2550,1,confident,About to take my driver's license test in a few days. Been practicing for a while now. I know I'm about to smash it,273,About to take my driver's license test in a few days. Been practicing for a while now. I know I'm about to smash it,4|4|4_5|5|5,
hit:1275_conv:2550,2,confident,About to take my driver's license test in a few days. Been practicing for a while now. I know I'm about to smash it,286,Good to hear you're confident! I'm sure you'll do well if you've been studying.,4|4|4_5|5|5,
hit:1275_conv:2550,3,confident,About to take my driver's license test in a few days. Been practicing for a while now. I know I'm about to smash it,273,Yeah_comma_ been getting a lot of practice in behind the wheel. That way my adrenaline doesn't spike too high. Wish I could tell you how it goes_comma_ but that's alright,4|4|4_5|5|5,
hit:1275_conv:2550,4,confident,About to take my driver's license test in a few days. Been practicing for a while now. I know I'm about to smash it,286,You don't have to tell me_comma_ I know you'll ace it! You sound well prepared.,4|4|4_5|5|5,
hit:1275_conv:2551,1,guilty,I lied to a cop about why I was speeding and immediately felt guilty about it.,286,I recently got caught speeding and lied to the cop about why_comma_ so he let me go. I feel kind of guilty/,5|5|5_4|4|4,
hit:1275_conv:2551,2,guilty,I lied to a cop about why I was speeding and immediately felt guilty about it.,273,Yeah_comma_ everyone speeds. Terrible to get caught. Hopefully you weren't speeding in a neighborhood_comma_ were you?,5|5|5_4|4|4,
hit:1275_conv:2551,3,guilty,I lied to a cop about why I was speeding and immediately felt guilty about it.,286,No_comma_ it was a highway and I wasn't going much over the limit. I still felt bad_comma_ though. I don't know why I lied_comma_ it just came out!,5|5|5_4|4|4,
hit:1275_conv:2551,4,guilty,I lied to a cop about why I was speeding and immediately felt guilty about it.,273,Yeah_comma_ I know. Adrenaline gets to you. You're human. At least you were doing it in a sort of safe way_comma_ it sounds like. Some cops are just out to get you.,5|5|5_4|4|4,
hit:1276_conv:2552,1,terrified,just heard a thump upstairs in a house I'm house sitting all alone at. Really don't want to go check it out..,273,just heard a thump upstairs in a house I'm house sitting all alone at. Really don't want to go check it out,3|3|3_5|5|5,
hit:1276_conv:2552,2,terrified,just heard a thump upstairs in a house I'm house sitting all alone at. Really don't want to go check it out..,296,Don't go check it out. That never turns out well in horror movies. Bring a gun then or another person.,3|3|3_5|5|5,
hit:1276_conv:2552,3,terrified,just heard a thump upstairs in a house I'm house sitting all alone at. Really don't want to go check it out..,273,I don't have a gun! Or another person! Ah man... Guess I'll just sleep in the bathroom with the door locked,3|3|3_5|5|5,
hit:1276_conv:2552,4,terrified,just heard a thump upstairs in a house I'm house sitting all alone at. Really don't want to go check it out..,296,I would do that! Or call a friend over- Get a weapon too.,3|3|3_5|5|5,
hit:1276_conv:2553,1,surprised,I was surprised out water company started on my street and tore up our whole street replacing water pipes,296,Our water company decided to tear our street up_comma_ start on mine for like the third time in a year to replace pipes again,5|5|5_3|3|3,
hit:1276_conv:2553,2,surprised,I was surprised out water company started on my street and tore up our whole street replacing water pipes,273,Yikes... sounds like they need to hire better people. Hopefully the noise isn't too loud in the morning,5|5|5_3|3|3,
hit:1276_conv:2553,3,surprised,I was surprised out water company started on my street and tore up our whole street replacing water pipes,296,They want to come in my yard and house and I have dogs. They just have done this many times now trying to stop the led and I am so over it- No one is coming in my house again.,5|5|5_3|3|3,
hit:1276_conv:2553,4,surprised,I was surprised out water company started on my street and tore up our whole street replacing water pipes,273,Man.. Good luck with that. Sounds like it will be hard to fight them. But I hope it works out for you.,5|5|5_3|3|3,
hit:1276_conv:2553,5,surprised,I was surprised out water company started on my street and tore up our whole street replacing water pipes,296,I don't think I have to let them in my house but they are out everyday. it's hard with dogs.,5|5|5_3|3|3,
hit:1277_conv:2554,1,surprised,Got hit with a parking ticket the other day as the lady was writing it. I begged her to stop writing it in the middle of it... and wouldn't you know it_comma_ she did! So thankful.,273,Got hit with a parking ticket the other day as the lady was writing it. I begged her to stop writing it in the middle of it... and wouldn't you know it_comma_ she did! So thankful.,3|3|3_5|5|5,
hit:1277_conv:2554,2,surprised,Got hit with a parking ticket the other day as the lady was writing it. I begged her to stop writing it in the middle of it... and wouldn't you know it_comma_ she did! So thankful.,99,That is lucky!,3|3|3_5|5|5,
hit:1277_conv:2554,3,surprised,Got hit with a parking ticket the other day as the lady was writing it. I begged her to stop writing it in the middle of it... and wouldn't you know it_comma_ she did! So thankful.,273,Yeah_comma_ only would have saved me like $20_comma_ but still! It's the thought that counts. Some people have hearts out there.,3|3|3_5|5|5,
hit:1277_conv:2554,4,surprised,Got hit with a parking ticket the other day as the lady was writing it. I begged her to stop writing it in the middle of it... and wouldn't you know it_comma_ she did! So thankful.,99,I'm happy you got to avoid that!,3|3|3_5|5|5,
hit:1277_conv:2555,1,furious,My coworker blamed his bad work mistake on me.,99,My coworker made a mistake and blamed it on me!,5|5|5_3|3|3,
hit:1277_conv:2555,2,furious,My coworker blamed his bad work mistake on me.,273,Seriously? How? What was the mistake? Did you talk to him about it_comma_ or your boss?,5|5|5_3|3|3,
hit:1277_conv:2555,3,furious,My coworker blamed his bad work mistake on me.,99,He used my computer to put in his notes for a ticket he messed up on so it was under my name and not his!,5|5|5_3|3|3,
hit:1277_conv:2555,4,furious,My coworker blamed his bad work mistake on me.,273,That's sneaky... and hard to explain. I'm sorry you have to deal with that hard situation. What do you think you'll do about it?,5|5|5_3|3|3,
hit:1278_conv:2556,1,disgusted,Sitting next to my baby brother who's two-years-old_comma_ and he's just ripping farts. Really wish I didn't have to stay with my parents during the summer while college is out.,273,Sitting next to my baby brother who's two-years-old_comma_ and he's just ripping farts. Really wish I didn't have to stay with my parents during the summer while college is out,4|4|4_5|5|5,
hit:1278_conv:2556,2,disgusted,Sitting next to my baby brother who's two-years-old_comma_ and he's just ripping farts. Really wish I didn't have to stay with my parents during the summer while college is out.,47,Wap you must be disgusted with the farting. were you mad? or are you used to this kind of situations?,4|4|4_5|5|5,
hit:1278_conv:2556,3,disgusted,Sitting next to my baby brother who's two-years-old_comma_ and he's just ripping farts. Really wish I didn't have to stay with my parents during the summer while college is out.,273,Kind of used to it.Comes with the territory of being with a kid brother. How about you? Any siblings?,4|4|4_5|5|5,
hit:1278_conv:2556,4,disgusted,Sitting next to my baby brother who's two-years-old_comma_ and he's just ripping farts. Really wish I didn't have to stay with my parents during the summer while college is out.,47,Yes i do have 2 siblings_comma_ older brother and younger sister_comma_ but we dont fart in front of eachother_comma_ i love them!,4|4|4_5|5|5,
hit:1279_conv:2558,1,nostalgic,Watched an old Tv show I watched as a kid--a cartoon. Reminded me of the time I used to spend with my brother after school_comma_ when there was nothing we had to care or worry about,273,Watched an old Tv show I watched as a kid--a cartoon. Reminded me of the time I used to spend with my brother after school_comma_ when there was nothing we had to care or worry about,5|5|5_5|5|5,
hit:1279_conv:2558,2,nostalgic,Watched an old Tv show I watched as a kid--a cartoon. Reminded me of the time I used to spend with my brother after school_comma_ when there was nothing we had to care or worry about,286,Sounds like the good old days seemed more relaxing and care free.,5|5|5_5|5|5,
hit:1279_conv:2558,3,nostalgic,Watched an old Tv show I watched as a kid--a cartoon. Reminded me of the time I used to spend with my brother after school_comma_ when there was nothing we had to care or worry about,273,Yeah. It was the show Dragon Ball Z_comma_ some kids cartoon. We're not that close anymore_comma_ but I wish we could go back to those days for a moment_comma_ sometimes.,5|5|5_5|5|5,
hit:1279_conv:2558,4,nostalgic,Watched an old Tv show I watched as a kid--a cartoon. Reminded me of the time I used to spend with my brother after school_comma_ when there was nothing we had to care or worry about,286,Oh man! I grew up on that cartoon too!,5|5|5_5|5|5,
hit:1279_conv:2559,1,guilty,I pirated a few games that I'm really enjoying and feel guilty for not supporting the developers.,286,I pirated a few games recently and I'm enjoying the hell out of them. I sort of feel guilty for not supporting the developers though.,5|5|5_5|5|5,
hit:1279_conv:2559,2,guilty,I pirated a few games that I'm really enjoying and feel guilty for not supporting the developers.,273,Yeah... that's a tough one. I've pirated stuff before. Kind of how it goes as a poor college student. Maybe when you get older you can pay for things appropriately.,5|5|5_5|5|5,
hit:1279_conv:2559,3,guilty,I pirated a few games that I'm really enjoying and feel guilty for not supporting the developers.,286,Older? I'm thirty! But now I have kids and money is tight sometimes. I might still buy the guys soon.,5|5|5_5|5|5,
hit:1279_conv:2559,4,guilty,I pirated a few games that I'm really enjoying and feel guilty for not supporting the developers.,273,Hahaha... Good point. I understand. Some things are just more important_comma_ fair point. My fault on assuming your age.,5|5|5_5|5|5,
hit:1279_conv:2559,5,guilty,I pirated a few games that I'm really enjoying and feel guilty for not supporting the developers.,286,Hey_comma_ it's all good. I didn't mean it in a confrontational way_comma_ I just thought it was funny.,5|5|5_5|5|5,
hit:1280_conv:2560,1,proud,Saw my brother graduate college last month! Felt so happy for him finally achieving what he's worked hard for. Hope he gets a job soon_comma_ though.,273,Saw my brother graduate college last month! Felt so happy for him finally achieving what he's worked hard for. Hope he gets a job soon_comma_ though,3|3|3_5|5|5,
hit:1280_conv:2560,2,proud,Saw my brother graduate college last month! Felt so happy for him finally achieving what he's worked hard for. Hope he gets a job soon_comma_ though.,269,That's awesome_comma_ the exact opposite is true for me. My little brother is just about to start his first year of college next month!,3|3|3_5|5|5,
hit:1280_conv:2560,3,proud,Saw my brother graduate college last month! Felt so happy for him finally achieving what he's worked hard for. Hope he gets a job soon_comma_ though.,273,Ah_comma_ he'll be under a lot of stress. Hopefully you guys can maintain your relationship through it all_comma_ as college tends to sweep people away and take up all their attention.,3|3|3_5|5|5,
hit:1280_conv:2560,4,proud,Saw my brother graduate college last month! Felt so happy for him finally achieving what he's worked hard for. Hope he gets a job soon_comma_ though.,269,Yes_comma_ I hope so. Well_comma_ I hope your brother can put his degree to good use and make some moolah!,3|3|3_5|5|5,
hit:1280_conv:2561,1,annoyed,I've been pretty agitated about the slow internet speeds. I think it's the weather?,269,I've been pretty pissed off about the slow internet speeds lately. I think it's the weather? ,5|5|5_3|3|3,
hit:1280_conv:2561,2,annoyed,I've been pretty agitated about the slow internet speeds. I think it's the weather?,273,Who knows.. I've dealt with slow internet all my life. Wish I could finally get a decent ISP.,5|5|5_3|3|3,
hit:1280_conv:2561,3,annoyed,I've been pretty agitated about the slow internet speeds. I think it's the weather?,269,I think it's time to get a new ISP! Any recommendations?,5|5|5_3|3|3,
hit:1280_conv:2561,4,annoyed,I've been pretty agitated about the slow internet speeds. I think it's the weather?,273,Not really_comma_ especially since they vary sometimes by region. We should just move to Europe_comma_ I hear they have great internet there because they actually view it as important.,5|5|5_3|3|3,
hit:1281_conv:2562,1,terrified,I have anxiety so I get terrible panick attacks sometimes.,296,I have bad panic attacks about death after experiencing so much of it the past year,5|5|5_5|5|5,
hit:1281_conv:2562,2,terrified,I have anxiety so I get terrible panick attacks sometimes.,122,I am so sorry to hear that. I assume lots of folks close to you have entered eternity lately?,5|5|5_5|5|5,
hit:1281_conv:2562,3,terrified,I have anxiety so I get terrible panick attacks sometimes.,296,Yes people and pets. Too soon. So it definitely makes you think everyone is going to die any minute and life is meaningless.,5|5|5_5|5|5,
hit:1281_conv:2562,4,terrified,I have anxiety so I get terrible panick attacks sometimes.,122,sadly we all know that everyone and everything is going to die_comma_ but that is why we need to really live while we are able. Don't retreat. Embrace everyone who is close to you.,5|5|5_5|5|5,
hit:1281_conv:2562,5,terrified,I have anxiety so I get terrible panick attacks sometimes.,296,yes very true. I try but when you go through trauma it affects you in different ways. Mine is to be terrified and want to die too at times.,5|5|5_5|5|5,


hit:1559_conv:3118,1,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,226,"My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,4|4|5_5|5|5,
hit:1559_conv:3118,2,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,1,How many kids would that be now?,4|4|5_5|5|5,
hit:1559_conv:3118,3,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,226,That makes 3 girls. They keep me on my toes.,4|4|5_5|5|5,
hit:1559_conv:3118,4,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,1,hahaha I bet they do! ,4|4|5_5|5|5,
hit:1559_conv:3119,1,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,1,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,5|5|5_4|4|5,
hit:1559_conv:3119,2,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,226,Do you have another way of contacting them?,5|5|5_4|4|5,
hit:1559_conv:3119,3,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,1,I asked their roommates but they don't know anything.,5|5|5_4|4|5,
hit:1559_conv:3119,4,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,226,You must be worried sick. The roommates sound suspicious.,5|5|5_4|4|5,
hit:1560_conv:3120,1,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,1,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,5|5|5_5|5|5,
hit:1560_conv:3120,2,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,296,Wow! Thats amazing. What are you going to school for?,5|5|5_5|5|5,
hit:1560_conv:3120,3,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,1,I'm in marketing. I find what influences people interesting.,5|5|5_5|5|5,
hit:1560_conv:3120,4,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,296,Oh that's great. Marketing is very lucrative!,5|5|5_5|5|5,
hit:1560_conv:3121,1,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,I recently started work again and I am also picking back up my business. It is making me feel happy about the future,5|5|5_5|5|5,
hit:1560_conv:3121,2,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,1,I hope everything works out. That all sounds very promising!,5|5|5_5|5|5,
hit:1560_conv:3121,3,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,Thank you! yes it is. It's been a really terrible year with lots of deaths so it's a miracle I am even here.,5|5|5_5|5|5,
hit:1560_conv:3121,4,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,1,Oh no_comma_ I wish people didn't have to go through the bad stuff. At least_comma_ they make you appreciate the good times more.,5|5|5_5|5|5,
hit:1560_conv:3121,5,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,yes they do! Thank you,5|5|5_5|5|5,
hit:1561_conv:3123,1,terrified,I was outside when lightning struck a tree that was close to me.,226,I was outside when lightning struck a tree that was close to me.,4|4|5_5|5|5,
hit:1561_conv:3123,2,terrified,I was outside when lightning struck a tree that was close to me.,60,That must have been scary. ,4|4|5_5|5|5,
hit:1561_conv:3123,3,terrified,I was outside when lightning struck a tree that was close to me.,226,I thought I was a goner.,4|4|5_5|5|5,
hit:1561_conv:3123,4,terrified,I was outside when lightning struck a tree that was close to me.,60,When you hear thunder do you hide now?,4|4|5_5|5|5,
hit:1562_conv:3125,1,surprised,I went to register my daughter for school today and the place was empty!,27,I went to register my youngest daughter for school today.,5|5|5_5|5|5,
hit:1562_conv:3125,2,surprised,I went to register my daughter for school today and the place was empty!,60,Was it stressful? ,5|5|5_5|5|5,
hit:1562_conv:3125,3,surprised,I went to register my daughter for school today and the place was empty!,27,It maybe would have been if anyone was at the school. I really don't want her to go there but the private school had too long of a waiting list for this year.,5|5|5_5|5|5,
hit:1562_conv:3125,4,surprised,I went to register my daughter for school today and the place was empty!,60,They school was locked up? I can't imagine. My first born is due in December and i'm all ready stressed about school. ,5|5|5_5|5|5,
hit:1562_conv:3125,5,surprised,I went to register my daughter for school today and the place was empty!,27,It was open and they just left flyers. I don't even know who my childs teacher is. The public schools here are trash.,5|5|5_5|5|5,
hit:1562_conv:3125,6,surprised,I went to register my daughter for school today and the place was empty!,60,Wow_comma_ that sounds like where I went. Have you considered a charter school?,5|5|5_5|5|5,
hit:1562_conv:3125,7,surprised,I went to register my daughter for school today and the place was empty!,27,I'm going to get her in the private school my oldest went to. It will be fine she is only ine pre k now.,5|5|5_5|5|5,
hit:1562_conv:3125,8,surprised,I went to register my daughter for school today and the place was empty!,60,I would be scared. They are so impressionable at that age.,5|5|5_5|5|5,
hit:1563_conv:3126,1,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,243,We have some very bad storms headed our way this evening_comma_ possible tornadoes too.,5|5|5_5|5|5,
hit:1563_conv:3126,2,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,226,Please be prepared ahead of time. I know that situation can be scary so it's best to be ready.,5|5|5_5|5|5,
hit:1563_conv:3126,3,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,243,I have cleared a safe spot in the basement and also taken down a tote full of food_comma_ water_comma_ and other emergency supplies.,5|5|5_5|5|5,
hit:1563_conv:3126,4,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,226,Keep alert and be safe.,5|5|5_5|5|5,
hit:1563_conv:3127,1,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,226,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries,5|5|5_5|5|5,
hit:1563_conv:3127,2,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,243,Oh wow! That is wonderful! I bet you were so grateful.,5|5|5_5|5|5,
hit:1563_conv:3127,3,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,226,I couldn't believe that someone would do that for me.,5|5|5_5|5|5,
hit:1563_conv:3127,4,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,243,That was so nice. Maybe someday you will be able to pay it forward.,5|5|5_5|5|5,
hit:1564_conv:3129,1,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,Well I rescued another dog the other day and thankfully she is very calm and sweet but I was so worried when I saw her as I already have more then enough animals.,5|5|5_5|5|5,
hit:1564_conv:3129,2,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,That's so awesome. Is she little or big?,5|5|5_5|5|5,
hit:1564_conv:3129,3,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,She is in between. Just a pocket pit. She was so starved and abused and maybe pregnant. I had to help her. Even though it makes me worried- I am over the city limit.,5|5|5_5|5|5,
hit:1564_conv:3129,4,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,But she needs you. It's similar to mine. My found my pit eating out of the dumpster. What did you name her?,5|5|5_5|5|5,
hit:1564_conv:3129,5,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,I know. And rescues never help me. So many are not nice anyway to people. I will make sure she is ok. Her name is sweetie. She is so docile and scared of everything. Most docile terrier ever.,5|5|5_5|5|5,
hit:1564_conv:3129,6,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,That's sad I'm sure she will come around soon.,5|5|5_5|5|5,
hit:1565_conv:3130,1,caring,I love seeing all of the kids excited about school. They are so sweet.,27,I love seeing all the kids excited about school starting again.,5|5|5_5|5|5,
hit:1565_conv:3130,2,caring,I love seeing all of the kids excited about school. They are so sweet.,226,It is always such an exciting time for them. So many new things to discover.,5|5|5_5|5|5,
hit:1565_conv:3130,3,caring,I love seeing all of the kids excited about school. They are so sweet.,27,Especially the elementary kids. I want them to love school forever. That's a pipe dream.,5|5|5_5|5|5,
hit:1565_conv:3130,4,caring,I love seeing all of the kids excited about school. They are so sweet.,226,If they get a really good teacher it can instill a love of learning that will stay with them.,5|5|5_5|5|5,
hit:1565_conv:3130,5,caring,I love seeing all of the kids excited about school. They are so sweet.,27,MY daughter had a teacher like that in pre k. We send her a card every year. She is in tehe 8th grade now!,5|5|5_5|5|5,
hit:1565_conv:3131,1,afraid,We had a bad storm and the power was out for hours during the night.,226,We had a bad storm and the power was out for hours during the night,5|5|5_5|5|5,
hit:1565_conv:3131,2,afraid,We had a bad storm and the power was out for hours during the night.,27,It just started storming here. I hope we don't lose power.,5|5|5_5|5|5,
hit:1565_conv:3131,3,afraid,We had a bad storm and the power was out for hours during the night.,226,Storms are so unpredictable. It's best to be prepared when you know they are coming.,5|5|5_5|5|5,
hit:1565_conv:3131,4,afraid,We had a bad storm and the power was out for hours during the night.,27,I keep saying we will get some flashlights_comma_ the ones only phone are only good if the phone is charged.,5|5|5_5|5|5,
hit:1566_conv:3132,1,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,1,Checked out my old yearbook. It was so much easier making friends back then.,5|5|5_5|5|5,
hit:1566_conv:3132,2,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,296,Yes typically kids are nicer and more open. Adults have so many secrets and issues :),5|5|5_5|5|5,
hit:1566_conv:3132,3,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,1,Yea_comma_ I guess that happens with time. Wish it wasn't so though.,5|5|5_5|5|5,
hit:1566_conv:3132,4,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,296,I here you! But it's better to make a good friend then many that aren't that good. Maybe try church or meetups?,5|5|5_5|5|5,
hit:1566_conv:3133,1,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,Well I used to live in a hotel down south and I was scared all the time. I had too many pets and scared I would be found out a lot and kicked out. It was like living while hiding things from the nazis. I'd never live that way again.,5|5|5_5|5|5,
hit:1566_conv:3133,2,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,1,Oh no_comma_ how many pets do you have?,5|5|5_5|5|5,
hit:1566_conv:3133,3,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,Too many! lol- I had more at the time- but they only wanted you to have 2- they don't like pets down there very much. I rescue- so lots! :),5|5|5_5|5|5,
hit:1566_conv:3133,4,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,1,That's incredible. I love animals and those who help them!,5|5|5_5|5|5,
hit:1566_conv:3133,5,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,:) It's hard work! Thats why I work from home but worth it,5|5|5_5|5|5,
hit:1567_conv:3134,1,excited,Going on a date with my crush this weekend. I'm kinda nervous!,1,Going on a date with my crush this weekend. I'm kinda nervous!,5|5|5_5|5|5,
hit:1567_conv:3134,2,excited,Going on a date with my crush this weekend. I'm kinda nervous!,226,Those butterflies are good for you. It will keep things interesting.,5|5|5_5|5|5,
hit:1567_conv:3134,3,excited,Going on a date with my crush this weekend. I'm kinda nervous!,1,Butterflies are great. It means I like her_comma_ but I just hope I don't make a doofus of myself.,5|5|5_5|5|5,
hit:1567_conv:3134,4,excited,Going on a date with my crush this weekend. I'm kinda nervous!,226,Don't be afraid to be yourself. She will love it if you do.,5|5|5_5|5|5,
hit:1567_conv:3135,1,sad,My nieces pet guinea pig died yesterday.,226,My nieces pet guinea pig died yesterday,5|5|5_5|5|5,
hit:1567_conv:3135,2,sad,My nieces pet guinea pig died yesterday.,1,Oh no_comma_ the poor little thing. Your nieces must be devastated.,5|5|5_5|5|5,
hit:1567_conv:3135,3,sad,My nieces pet guinea pig died yesterday.,226,She was upset. But I feel for the one left behind. They were sisters.,5|5|5_5|5|5,
hit:1567_conv:3135,4,sad,My nieces pet guinea pig died yesterday.,1,Oh no_comma_ that happened to my dogs. One died before the other_comma_ and the other was so depressed_comma_ she quickly followed.,5|5|5_5|5|5,
hit:1568_conv:3136,1,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,We have our water pipes being redone in our corrupt city for the third or fourth time. They blocked my street and the mailman still isn't delivering and the garbage was not picked up. ,5|5|5_5|5|5,
hit:1568_conv:3136,2,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,Yikes that sounds like a pain. I would dump my trash in front of city hall.,5|5|5_5|5|5,
hit:1568_conv:3136,3,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Thats true lol. I could do that. But they'd likely harm me- One of those corrupt cities where they don't care about crime and rapes but care about the little dumb things,5|5|5_5|5|5,
hit:1568_conv:3136,4,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,Time to make your town facebook famous or something. Shame them into fixing things. ,5|5|5_5|5|5,
hit:1568_conv:3136,5,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Oh it's flint_comma_ MI- they are very famous for being corrupt. Nothing I can do but save up to move out :),5|5|5_5|5|5,
hit:1568_conv:3136,6,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,I was robbed in Flint. I grew up just down the road from Fenton. You need to move. ,5|5|5_5|5|5,
hit:1568_conv:3136,7,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Wow! yes- it was good for a time but it is SO corrupt here and bills so high- I thought it was cheap no- Yes I am saving to move and working on a business too! outta here soon- it's so dangerous and it IS the ghetto.,5|5|5_5|5|5,
hit:1569_conv:3138,1,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,1,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,5|5|5_5|5|5,
hit:1569_conv:3138,2,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,220,Wow_comma_ I wonder why he was being so generous,5|5|5_5|5|5,
hit:1569_conv:3138,3,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,1,We think he's a great guy. I hope work doesn't change him.,5|5|5_5|5|5,
hit:1569_conv:3138,4,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,220,I hope not either!,5|5|5_5|5|5,
hit:1569_conv:3139,1,guilty,I asked my friend for a loan but I used the money for a different reason.,220,I told my friend I needed to borrow money for some car repairs but I used it for something else completely. I feel bad about it.,5|5|5_5|5|5,
hit:1569_conv:3139,2,guilty,I asked my friend for a loan but I used the money for a different reason.,1,Oh no. What did you use it on?,5|5|5_5|5|5,
hit:1569_conv:3139,3,guilty,I asked my friend for a loan but I used the money for a different reason.,220,I don't even want to say_comma_ I'm so ashamed.,5|5|5_5|5|5,
hit:1569_conv:3139,4,guilty,I asked my friend for a loan but I used the money for a different reason.,1,Oh I'm so sorry. Hopefully_comma_ you've learned and won't do it again_comma_ yea?,5|5|5_5|5|5,
hit:1570_conv:3140,1,lonely,My house will be empty soon. Kids are almost grown.,27,MY nest will be empty soon. I don't know what I'm going to do.,5|5|5_5|5|5,
hit:1570_conv:3140,2,lonely,My house will be empty soon. Kids are almost grown.,60,New hobby. Maybe surfing or hiking.,5|5|5_5|5|5,
hit:1570_conv:3140,3,lonely,My house will be empty soon. Kids are almost grown.,27,I'm too old for that. I think I'll buy a bunch of jugsaw puzzles.,5|5|5_5|5|5,
hit:1570_conv:3140,4,lonely,My house will be empty soon. Kids are almost grown.,60,Your never to old. I thought my father to surf at 67 years old. Your die sitting in front of a puzzle. How about Yoga or painting class? ,5|5|5_5|5|5,
hit:1570_conv:3140,5,lonely,My house will be empty soon. Kids are almost grown.,27,Yoga? oh lord. Painting sounds fun though,5|5|5_5|5|5,
hit:1570_conv:3140,6,lonely,My house will be empty soon. Kids are almost grown.,60,Yoga is low impact. They have classes for all ages and levels. It will be a reason to get up and help you stay healthy longer.,5|5|5_5|5|5,
hit:1570_conv:3140,7,lonely,My house will be empty soon. Kids are almost grown.,27,I think I'll look into it. Seriously.,5|5|5_5|5|5,
hit:1571_conv:3142,1,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,1,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,5|5|5_5|5|5,
hit:1571_conv:3142,2,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,294,That is so incredibly nice of them.  It is nice to have good neighbors...and good pears!,5|5|5_5|5|5,
hit:1571_conv:3142,3,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,1,They are great people. They had so many that instead of throwing them out_comma_ they took the time to collect them and share it with us!,5|5|5_5|5|5,
hit:1571_conv:3142,4,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,294,Wow_comma_ I wish I had neighbors like that.  I'm stuck going to the store and paying $2.99 a pound on them.  You're right_comma_ it is expensive.,5|5|5_5|5|5,
hit:1571_conv:3143,1,content,I am really happy with how things are going at work right now.,294,I'm really happy with how things are going at work right now.  I just got a raise_comma_ and a promotion.,5|5|5_5|5|5,
hit:1571_conv:3143,2,content,I am really happy with how things are going at work right now.,1,That's great! I hope things continue to go well for ya.,5|5|5_5|5|5,
hit:1571_conv:3143,3,content,I am really happy with how things are going at work right now.,294,I hope so too.  Things don't normally go this well so I'm enjoying while it lasts.,5|5|5_5|5|5,
hit:1571_conv:3143,4,content,I am really happy with how things are going at work right now.,1,ah always gotta prepare for those unexpected rainy days.,5|5|5_5|5|5,
hit:1572_conv:3144,1,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,1,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money,5|5|5_5|5|5,
hit:1572_conv:3144,2,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,226,Were you able to reschedule with your friend?,5|5|5_5|5|5,
hit:1572_conv:3144,3,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,1,No. She's not taking my calls currently.,5|5|5_5|5|5,
hit:1572_conv:3144,4,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,226,Maybe you can make it up to them.,5|5|5_5|5|5,
hit:1572_conv:3145,1,sentimental,My girls went on an out of town trip and I was home alone,226,My girls went on an out of town trip and I was home alone,5|5|5_5|5|5,
hit:1572_conv:3145,2,sentimental,My girls went on an out of town trip and I was home alone,1,Oh no. Being home alone can be good though. Finally everything to yourself!,5|5|5_5|5|5,
hit:1572_conv:3145,3,sentimental,My girls went on an out of town trip and I was home alone,226,You would think but I really missed them.,5|5|5_5|5|5,
hit:1572_conv:3145,4,sentimental,My girls went on an out of town trip and I was home alone,1,I feel ya. How long will they be gone for?,5|5|5_5|5|5,
hit:1573_conv:3147,1,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,243,My mom had to put our family dog to sleep last week.,5|5|5_3|4|5,
hit:1573_conv:3147,2,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,66,Oh_comma_ that is so tough. I am sorry. Are you all right?,5|5|5_3|4|5,
hit:1573_conv:3147,3,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,243,It's so hard because we have had her since I was in 1st grade. She was so old and sick.,5|5|5_3|4|5,
hit:1573_conv:3147,4,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,66,That's where you really do have to make the decision for the animal_comma_ not us -- that doesn't make it easier when it happens. Do you have other pets?,5|5|5_3|4|5,
hit:1574_conv:3148,1,anticipating,Ready to go on my vacation!,220,I'm leaving for vacation soon! I'm so pumped! It can't come soon enough!,5|5|5_5|5|5,
hit:1574_conv:3148,2,anticipating,Ready to go on my vacation!,1,Yes! Vacation. Where ya headed?,5|5|5_5|5|5,
hit:1574_conv:3148,3,anticipating,Ready to go on my vacation!,220,Disney World_comma_ I've never been!,5|5|5_5|5|5,
hit:1574_conv:3148,4,anticipating,Ready to go on my vacation!,1,Oh you'll have lots of fun!,5|5|5_5|5|5,
hit:1574_conv:3149,1,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,1,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,5|5|5_5|5|5,
hit:1574_conv:3149,2,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,220,I'm sorry_comma_ that's life for most of us_comma_ though.,5|5|5_5|5|5,
hit:1574_conv:3149,3,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,1,Yes_comma_ it definitely is. Gotta work with what ya got.,5|5|5_5|5|5,
hit:1574_conv:3149,4,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,220,Maybe you'll hit the lottery one day!,5|5|5_5|5|5,
hit:1575_conv:3150,1,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,1,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,5|5|5_5|5|5,
hit:1575_conv:3150,2,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,226,That had to be disappointing. Will you be able to go at another time?,5|5|5_5|5|5,
hit:1575_conv:3150,3,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,1,Yea definitely. I hope we get another time sooner than later.,5|5|5_5|5|5,
hit:1575_conv:3150,4,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,226,At least you have something to look forward to.,5|5|5_5|5|5,
hit:1575_conv:3151,1,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,226,I was crossing a wet road and slipped. I fell with a thud right on my rear end,5|5|5_5|5|5,
hit:1575_conv:3151,2,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,1,Oh no. How is your butt now?,5|5|5_5|5|5,
hit:1575_conv:3151,3,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,226,Very funny. All I could do is sit there and laugh.,5|5|5_5|5|5,
hit:1575_conv:3151,4,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,1,hahaha I hope you recover soon.,5|5|5_5|5|5,
hit:1576_conv:3152,1,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,296,I heard one of my favorite youtubers talking about the 90's the other day and how old we were lol. But it reminded me of fun times as a teen,5|5|5_5|5|5,
hit:1576_conv:3152,2,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,1,90s were good time. What is your favorite memory?,5|5|5_5|5|5,
hit:1576_conv:3152,3,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,296,I am not sure I can pick on favorite but I loved how there were no bills lol and it was just freedom to be creative and listen to music all night and hang out with friends talking until the late hours :),5|5|5_5|5|5,
hit:1576_conv:3152,4,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,1,Oh yea. No bills is definitely great. You can still do all those things y'know.,5|5|5_5|5|5,
hit:1576_conv:3153,1,trusting,My friend left me with her dog. Never met this little guy.,1,My friend left me with her dog. Never met this little guy,5|5|5_5|5|5,
hit:1576_conv:3153,2,trusting,My friend left me with her dog. Never met this little guy.,296,Aw! Well I hope you can help him?,5|5|5_5|5|5,
hit:1576_conv:3153,3,trusting,My friend left me with her dog. Never met this little guy.,1,Yep. We are getting along so far. Looks like he misses his mommy though.,5|5|5_5|5|5,
hit:1576_conv:3153,4,trusting,My friend left me with her dog. Never met this little guy.,296,Oh I bet! Are you just babysitting?,5|5|5_5|5|5,
hit:1577_conv:3154,1,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,1,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,5|5|5_5|5|5,
hit:1577_conv:3154,2,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,273,That's got to be a pretty good feeling_comma_ to see our kids succeed and hopefully do better than us. I'm happy for you! What does he research?,5|5|5_5|5|5,
hit:1577_conv:3154,3,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,1,He's working on Parkinson actually. He has such a great heart.,5|5|5_5|5|5,
hit:1577_conv:3154,4,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,273,Parkinson's disease? That's pretty noble. Imagine if he were to be the one to help cause a breakthrough on curing it... How crazy would that be. Would add a nice legacy to your family.,5|5|5_5|5|5,
hit:1577_conv:3155,1,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,273,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,5|5|5_5|5|5,
hit:1577_conv:3155,2,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,1,Oh man_comma_ siblings are such hit or miss sometimes. Some get really good ones and some get_comma_ well_comma_ yours. Sorry to say.,5|5|5_5|5|5,
hit:1577_conv:3155,3,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,273,If I'm being honest_comma_ they're not all bad. The older one helps me with a lot of things. And the younger ones can be funny. But yeah_comma_ sometimes they just.. .get to me.,5|5|5_5|5|5,
hit:1577_conv:3155,4,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,1,Everyone has their faults I guess. I'm glad you don't focus on their faults.,5|5|5_5|5|5,
hit:1578_conv:3156,1,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,226,My favorite team has a new Quarterback. Maybe this will be a better season for them,4|4|5_5|5|5,
hit:1578_conv:3156,2,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,274,Who is your favorite team?,4|4|5_5|5|5,
hit:1578_conv:3156,3,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,226,I like college football. I root for Arkansas.,4|4|5_5|5|5,
hit:1578_conv:3156,4,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,274,Who is the quarterback now,4|4|5_5|5|5,
hit:1578_conv:3157,1,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,274,After a long hard day at work_comma_ I came home to an empty house.,5|5|5_4|4|5,
hit:1578_conv:3157,2,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,226,Is that a good thing or a bad thing?,5|5|5_4|4|5,
hit:1578_conv:3157,3,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,274,Bad as there is no one to share my thoughts and relieve stress with.My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,4|4|5_5|5|5,
hit:1578_conv:3157,4,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,226,Are there any social groups you can join to have some company?,5|5|5_4|4|5,



hit:1559_conv:3118,1,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,226,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,4|4|5_5|5|5,
hit:1559_conv:3118,2,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,1,How many kids would that be now?,4|4|5_5|5|5,
hit:1559_conv:3118,3,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,226,That makes 3 girls. They keep me on my toes.,4|4|5_5|5|5,
hit:1559_conv:3118,4,surprised,My oldest girls are just over a year apart. I wasn't expecting to have another baby so soon.,1,hahaha I bet they do! ,4|4|5_5|5|5,
hit:1559_conv:3119,1,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,1,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,5|5|5_4|4|5,
hit:1559_conv:3119,2,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,226,Do you have another way of contacting them?,5|5|5_4|4|5,
hit:1559_conv:3119,3,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,1,I asked their roommates but they don't know anything.,5|5|5_4|4|5,
hit:1559_conv:3119,4,afraid,My friend hasn't answered me calls for 2 weeks now. I don't know what's going on.,226,You must be worried sick. The roommates sound suspicious.,5|5|5_4|4|5,
hit:1560_conv:3120,1,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,1,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,5|5|5_5|5|5,
hit:1560_conv:3120,2,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,296,Wow! Thats amazing. What are you going to school for?,5|5|5_5|5|5,
hit:1560_conv:3120,3,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,1,I'm in marketing. I find what influences people interesting.,5|5|5_5|5|5,
hit:1560_conv:3120,4,grateful,I got a $1_comma_000 scholarship for tuition. Doesn't cover everything_comma_ but it's something.,296,Oh that's great. Marketing is very lucrative!,5|5|5_5|5|5,
hit:1560_conv:3121,1,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,I recently started work again and I am also picking back up my business. It is making me feel happy about the future,5|5|5_5|5|5,
hit:1560_conv:3121,2,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,1,I hope everything works out. That all sounds very promising!,5|5|5_5|5|5,
hit:1560_conv:3121,3,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,Thank you! yes it is. It's been a really terrible year with lots of deaths so it's a miracle I am even here.,5|5|5_5|5|5,
hit:1560_conv:3121,4,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,1,Oh no_comma_ I wish people didn't have to go through the bad stuff. At least_comma_ they make you appreciate the good times more.,5|5|5_5|5|5,
hit:1560_conv:3121,5,hopeful,i am feeling hopeful about my future after coming back to work and starting my business back up,296,yes they do! Thank you,5|5|5_5|5|5,
hit:1561_conv:3123,1,terrified,I was outside when lightning struck a tree that was close to me.,226,I was outside when lightning struck a tree that was close to me.,4|4|5_5|5|5,
hit:1561_conv:3123,2,terrified,I was outside when lightning struck a tree that was close to me.,60,That must have been scary. ,4|4|5_5|5|5,
hit:1561_conv:3123,3,terrified,I was outside when lightning struck a tree that was close to me.,226,I thought I was a goner.,4|4|5_5|5|5,
hit:1561_conv:3123,4,terrified,I was outside when lightning struck a tree that was close to me.,60,When you hear thunder do you hide now?,4|4|5_5|5|5,
hit:1562_conv:3125,1,surprised,I went to register my daughter for school today and the place was empty!,27,I went to register my youngest daughter for school today.,5|5|5_5|5|5,
hit:1562_conv:3125,2,surprised,I went to register my daughter for school today and the place was empty!,60,Was it stressful? ,5|5|5_5|5|5,
hit:1562_conv:3125,3,surprised,I went to register my daughter for school today and the place was empty!,27,It maybe would have been if anyone was at the school. I really don't want her to go there but the private school had too long of a waiting list for this year.,5|5|5_5|5|5,
hit:1562_conv:3125,4,surprised,I went to register my daughter for school today and the place was empty!,60,They school was locked up? I can't imagine. My first born is due in December and i'm all ready stressed about school. ,5|5|5_5|5|5,
hit:1562_conv:3125,5,surprised,I went to register my daughter for school today and the place was empty!,27,It was open and they just left flyers. I don't even know who my childs teacher is. The public schools here are trash.,5|5|5_5|5|5,
hit:1562_conv:3125,6,surprised,I went to register my daughter for school today and the place was empty!,60,Wow_comma_ that sounds like where I went. Have you considered a charter school?,5|5|5_5|5|5,
hit:1562_conv:3125,7,surprised,I went to register my daughter for school today and the place was empty!,27,I'm going to get her in the private school my oldest went to. It will be fine she is only ine pre k now.,5|5|5_5|5|5,
hit:1562_conv:3125,8,surprised,I went to register my daughter for school today and the place was empty!,60,I would be scared. They are so impressionable at that age.,5|5|5_5|5|5,
hit:1563_conv:3126,1,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,243,We have some very bad storms headed our way this evening_comma_ possible tornadoes too.,5|5|5_5|5|5,
hit:1563_conv:3126,2,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,226,Please be prepared ahead of time. I know that situation can be scary so it's best to be ready.,5|5|5_5|5|5,
hit:1563_conv:3126,3,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,243,I have cleared a safe spot in the basement and also taken down a tote full of food_comma_ water_comma_ and other emergency supplies.,5|5|5_5|5|5,
hit:1563_conv:3126,4,prepared,We have some bad storms headed our way. I have cleared out a safe place in the basement and also packed a tote of food_comma_ water_comma_ medications and other emergency supplies to keep down there just in case.,226,Keep alert and be safe.,5|5|5_5|5|5,
hit:1563_conv:3127,1,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,226,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries,5|5|5_5|5|5,
hit:1563_conv:3127,2,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,243,Oh wow! That is wonderful! I bet you were so grateful.,5|5|5_5|5|5,
hit:1563_conv:3127,3,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,226,I couldn't believe that someone would do that for me.,5|5|5_5|5|5,
hit:1563_conv:3127,4,grateful,I was at the store and did not have enough money to pay for my items. The person behind be paid for my groceries..,243,That was so nice. Maybe someday you will be able to pay it forward.,5|5|5_5|5|5,
hit:1564_conv:3129,1,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,Well I rescued another dog the other day and thankfully she is very calm and sweet but I was so worried when I saw her as I already have more then enough animals.,5|5|5_5|5|5,
hit:1564_conv:3129,2,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,That's so awesome. Is she little or big?,5|5|5_5|5|5,
hit:1564_conv:3129,3,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,She is in between. Just a pocket pit. She was so starved and abused and maybe pregnant. I had to help her. Even though it makes me worried- I am over the city limit.,5|5|5_5|5|5,
hit:1564_conv:3129,4,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,But she needs you. It's similar to mine. My found my pit eating out of the dumpster. What did you name her?,5|5|5_5|5|5,
hit:1564_conv:3129,5,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,296,I know. And rescues never help me. So many are not nice anyway to people. I will make sure she is ok. Her name is sweetie. She is so docile and scared of everything. Most docile terrier ever.,5|5|5_5|5|5,
hit:1564_conv:3129,6,anxious,I felt anxious when I saw a new dog and knew she needed help but I didn't want another dog.,60,That's sad I'm sure she will come around soon.,5|5|5_5|5|5,
hit:1565_conv:3130,1,caring,I love seeing all of the kids excited about school. They are so sweet.,27,I love seeing all the kids excited about school starting again.,5|5|5_5|5|5,
hit:1565_conv:3130,2,caring,I love seeing all of the kids excited about school. They are so sweet.,226,It is always such an exciting time for them. So many new things to discover.,5|5|5_5|5|5,
hit:1565_conv:3130,3,caring,I love seeing all of the kids excited about school. They are so sweet.,27,Especially the elementary kids. I want them to love school forever. That's a pipe dream.,5|5|5_5|5|5,
hit:1565_conv:3130,4,caring,I love seeing all of the kids excited about school. They are so sweet.,226,If they get a really good teacher it can instill a love of learning that will stay with them.,5|5|5_5|5|5,
hit:1565_conv:3130,5,caring,I love seeing all of the kids excited about school. They are so sweet.,27,MY daughter had a teacher like that in pre k. We send her a card every year. She is in tehe 8th grade now!,5|5|5_5|5|5,
hit:1565_conv:3131,1,afraid,We had a bad storm and the power was out for hours during the night.,226,We had a bad storm and the power was out for hours during the night,5|5|5_5|5|5,
hit:1565_conv:3131,2,afraid,We had a bad storm and the power was out for hours during the night.,27,It just started storming here. I hope we don't lose power.,5|5|5_5|5|5,
hit:1565_conv:3131,3,afraid,We had a bad storm and the power was out for hours during the night.,226,Storms are so unpredictable. It's best to be prepared when you know they are coming.,5|5|5_5|5|5,
hit:1565_conv:3131,4,afraid,We had a bad storm and the power was out for hours during the night.,27,I keep saying we will get some flashlights_comma_ the ones only phone are only good if the phone is charged.,5|5|5_5|5|5,
hit:1566_conv:3132,1,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,1,Checked out my old yearbook. It was so much easier making friends back then.,5|5|5_5|5|5,
hit:1566_conv:3132,2,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,296,Yes typically kids are nicer and more open. Adults have so many secrets and issues :),5|5|5_5|5|5,
hit:1566_conv:3132,3,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,1,Yea_comma_ I guess that happens with time. Wish it wasn't so though.,5|5|5_5|5|5,
hit:1566_conv:3132,4,nostalgic,Checked out my old yearbook. It was so much easier making friends back then.,296,I here you! But it's better to make a good friend then many that aren't that good. Maybe try church or meetups?,5|5|5_5|5|5,
hit:1566_conv:3133,1,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,Well I used to live in a hotel down south and I was scared all the time. I had too many pets and scared I would be found out a lot and kicked out. It was like living while hiding things from the nazis. I'd never live that way again.,5|5|5_5|5|5,
hit:1566_conv:3133,2,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,1,Oh no_comma_ how many pets do you have?,5|5|5_5|5|5,
hit:1566_conv:3133,3,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,Too many! lol- I had more at the time- but they only wanted you to have 2- they don't like pets down there very much. I rescue- so lots! :),5|5|5_5|5|5,
hit:1566_conv:3133,4,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,1,That's incredible. I love animals and those who help them!,5|5|5_5|5|5,
hit:1566_conv:3133,5,afraid,When I used to live in a hotel I was afraid all the time of being found out with all my pets,296,:) It's hard work! Thats why I work from home but worth it,5|5|5_5|5|5,
hit:1567_conv:3134,1,excited,Going on a date with my crush this weekend. I'm kinda nervous!,1,Going on a date with my crush this weekend. I'm kinda nervous!,5|5|5_5|5|5,
hit:1567_conv:3134,2,excited,Going on a date with my crush this weekend. I'm kinda nervous!,226,Those butterflies are good for you. It will keep things interesting.,5|5|5_5|5|5,
hit:1567_conv:3134,3,excited,Going on a date with my crush this weekend. I'm kinda nervous!,1,Butterflies are great. It means I like her_comma_ but I just hope I don't make a doofus of myself.,5|5|5_5|5|5,
hit:1567_conv:3134,4,excited,Going on a date with my crush this weekend. I'm kinda nervous!,226,Don't be afraid to be yourself. She will love it if you do.,5|5|5_5|5|5,
hit:1567_conv:3135,1,sad,My nieces pet guinea pig died yesterday.,226,My nieces pet guinea pig died yesterday,5|5|5_5|5|5,
hit:1567_conv:3135,2,sad,My nieces pet guinea pig died yesterday.,1,Oh no_comma_ the poor little thing. Your nieces must be devastated.,5|5|5_5|5|5,
hit:1567_conv:3135,3,sad,My nieces pet guinea pig died yesterday.,226,She was upset. But I feel for the one left behind. They were sisters.,5|5|5_5|5|5,
hit:1567_conv:3135,4,sad,My nieces pet guinea pig died yesterday.,1,Oh no_comma_ that happened to my dogs. One died before the other_comma_ and the other was so depressed_comma_ she quickly followed.,5|5|5_5|5|5,
hit:1568_conv:3136,1,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,We have our water pipes being redone in our corrupt city for the third or fourth time. They blocked my street and the mailman still isn't delivering and the garbage was not picked up. ,5|5|5_5|5|5,
hit:1568_conv:3136,2,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,Yikes that sounds like a pain. I would dump my trash in front of city hall.,5|5|5_5|5|5,
hit:1568_conv:3136,3,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Thats true lol. I could do that. But they'd likely harm me- One of those corrupt cities where they don't care about crime and rapes but care about the little dumb things,5|5|5_5|5|5,
hit:1568_conv:3136,4,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,Time to make your town facebook famous or something. Shame them into fixing things. ,5|5|5_5|5|5,
hit:1568_conv:3136,5,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Oh it's flint_comma_ MI- they are very famous for being corrupt. Nothing I can do but save up to move out :),5|5|5_5|5|5,
hit:1568_conv:3136,6,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,60,I was robbed in Flint. I grew up just down the road from Fenton. You need to move. ,5|5|5_5|5|5,
hit:1568_conv:3136,7,surprised,I was very surprised today when my mail wasn't delivered for a second day and the garbage men did not pick anything up.,296,Wow! yes- it was good for a time but it is SO corrupt here and bills so high- I thought it was cheap no- Yes I am saving to move and working on a business too! outta here soon- it's so dangerous and it IS the ghetto.,5|5|5_5|5|5,
hit:1569_conv:3138,1,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,1,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,5|5|5_5|5|5,
hit:1569_conv:3138,2,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,220,Wow_comma_ I wonder why he was being so generous,5|5|5_5|5|5,
hit:1569_conv:3138,3,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,1,We think he's a great guy. I hope work doesn't change him.,5|5|5_5|5|5,
hit:1569_conv:3138,4,impressed,We're behind on a work project_comma_ but our boss told us not to worry about it and go home for the day.,220,I hope not either!,5|5|5_5|5|5,
hit:1569_conv:3139,1,guilty,I asked my friend for a loan but I used the money for a different reason.,220,I told my friend I needed to borrow money for some car repairs but I used it for something else completely. I feel bad about it.,5|5|5_5|5|5,
hit:1569_conv:3139,2,guilty,I asked my friend for a loan but I used the money for a different reason.,1,Oh no. What did you use it on?,5|5|5_5|5|5,
hit:1569_conv:3139,3,guilty,I asked my friend for a loan but I used the money for a different reason.,220,I don't even want to say_comma_ I'm so ashamed.,5|5|5_5|5|5,
hit:1569_conv:3139,4,guilty,I asked my friend for a loan but I used the money for a different reason.,1,Oh I'm so sorry. Hopefully_comma_ you've learned and won't do it again_comma_ yea?,5|5|5_5|5|5,
hit:1570_conv:3140,1,lonely,My house will be empty soon. Kids are almost grown.,27,MY nest will be empty soon. I don't know what I'm going to do.,5|5|5_5|5|5,
hit:1570_conv:3140,2,lonely,My house will be empty soon. Kids are almost grown.,60,New hobby. Maybe surfing or hiking.,5|5|5_5|5|5,
hit:1570_conv:3140,3,lonely,My house will be empty soon. Kids are almost grown.,27,I'm too old for that. I think I'll buy a bunch of jugsaw puzzles.,5|5|5_5|5|5,
hit:1570_conv:3140,4,lonely,My house will be empty soon. Kids are almost grown.,60,Your never to old. I thought my father to surf at 67 years old. Your die sitting in front of a puzzle. How about Yoga or painting class? ,5|5|5_5|5|5,
hit:1570_conv:3140,5,lonely,My house will be empty soon. Kids are almost grown.,27,Yoga? oh lord. Painting sounds fun though,5|5|5_5|5|5,
hit:1570_conv:3140,6,lonely,My house will be empty soon. Kids are almost grown.,60,Yoga is low impact. They have classes for all ages and levels. It will be a reason to get up and help you stay healthy longer.,5|5|5_5|5|5,
hit:1570_conv:3140,7,lonely,My house will be empty soon. Kids are almost grown.,27,I think I'll look into it. Seriously.,5|5|5_5|5|5,
hit:1571_conv:3142,1,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,1,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,5|5|5_5|5|5,
hit:1571_conv:3142,2,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,294,That is so incredibly nice of them.  It is nice to have good neighbors...and good pears!,5|5|5_5|5|5,
hit:1571_conv:3142,3,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,1,They are great people. They had so many that instead of throwing them out_comma_ they took the time to collect them and share it with us!,5|5|5_5|5|5,
hit:1571_conv:3142,4,grateful,Fruits have been so expensive lately_comma_ so our neighbors always brings us a basket of pears from their trees!,294,Wow_comma_ I wish I had neighbors like that.  I'm stuck going to the store and paying $2.99 a pound on them.  You're right_comma_ it is expensive.,5|5|5_5|5|5,
hit:1571_conv:3143,1,content,I am really happy with how things are going at work right now.,294,I'm really happy with how things are going at work right now.  I just got a raise_comma_ and a promotion.,5|5|5_5|5|5,
hit:1571_conv:3143,2,content,I am really happy with how things are going at work right now.,1,That's great! I hope things continue to go well for ya.,5|5|5_5|5|5,
hit:1571_conv:3143,3,content,I am really happy with how things are going at work right now.,294,I hope so too.  Things don't normally go this well so I'm enjoying while it lasts.,5|5|5_5|5|5,
hit:1571_conv:3143,4,content,I am really happy with how things are going at work right now.,1,ah always gotta prepare for those unexpected rainy days.,5|5|5_5|5|5,
hit:1572_conv:3144,1,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,1,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money,5|5|5_5|5|5,
hit:1572_conv:3144,2,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,226,Were you able to reschedule with your friend?,5|5|5_5|5|5,
hit:1572_conv:3144,3,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,1,No. She's not taking my calls currently.,5|5|5_5|5|5,
hit:1572_conv:3144,4,guilty,I had plans with my friend to go to dinner_comma_ but I chose to stay at work to make more money.,226,Maybe you can make it up to them.,5|5|5_5|5|5,
hit:1572_conv:3145,1,sentimental,My girls went on an out of town trip and I was home alone,226,My girls went on an out of town trip and I was home alone,5|5|5_5|5|5,
hit:1572_conv:3145,2,sentimental,My girls went on an out of town trip and I was home alone,1,Oh no. Being home alone can be good though. Finally everything to yourself!,5|5|5_5|5|5,
hit:1572_conv:3145,3,sentimental,My girls went on an out of town trip and I was home alone,226,You would think but I really missed them.,5|5|5_5|5|5,
hit:1572_conv:3145,4,sentimental,My girls went on an out of town trip and I was home alone,1,I feel ya. How long will they be gone for?,5|5|5_5|5|5,
hit:1573_conv:3147,1,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,243,My mom had to put our family dog to sleep last week.,5|5|5_3|4|5,
hit:1573_conv:3147,2,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,66,Oh_comma_ that is so tough. I am sorry. Are you all right?,5|5|5_3|4|5,
hit:1573_conv:3147,3,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,243,It's so hard because we have had her since I was in 1st grade. She was so old and sick.,5|5|5_3|4|5,
hit:1573_conv:3147,4,sad,My mom had to put our family dog to sleep the other day. He was very old and sick. We got her when I was in 1st grade.,66,That's where you really do have to make the decision for the animal_comma_ not us -- that doesn't make it easier when it happens. Do you have other pets?,5|5|5_3|4|5,
hit:1574_conv:3148,1,anticipating,Ready to go on my vacation!,220,I'm leaving for vacation soon! I'm so pumped! It can't come soon enough!,5|5|5_5|5|5,
hit:1574_conv:3148,2,anticipating,Ready to go on my vacation!,1,Yes! Vacation. Where ya headed?,5|5|5_5|5|5,
hit:1574_conv:3148,3,anticipating,Ready to go on my vacation!,220,Disney World_comma_ I've never been!,5|5|5_5|5|5,
hit:1574_conv:3148,4,anticipating,Ready to go on my vacation!,1,Oh you'll have lots of fun!,5|5|5_5|5|5,
hit:1574_conv:3149,1,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,1,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,5|5|5_5|5|5,
hit:1574_conv:3149,2,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,220,I'm sorry_comma_ that's life for most of us_comma_ though.,5|5|5_5|5|5,
hit:1574_conv:3149,3,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,1,Yes_comma_ it definitely is. Gotta work with what ya got.,5|5|5_5|5|5,
hit:1574_conv:3149,4,annoyed,I wish I was born wealthy. I have to give up spending time with my family to get more hours at work in.,220,Maybe you'll hit the lottery one day!,5|5|5_5|5|5,
hit:1575_conv:3150,1,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,1,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,5|5|5_5|5|5,
hit:1575_conv:3150,2,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,226,That had to be disappointing. Will you be able to go at another time?,5|5|5_5|5|5,
hit:1575_conv:3150,3,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,1,Yea definitely. I hope we get another time sooner than later.,5|5|5_5|5|5,
hit:1575_conv:3150,4,lonely,My friends left on a trip_comma_ but I couldn't go with them because I have so many projects to work on.,226,At least you have something to look forward to.,5|5|5_5|5|5,
hit:1575_conv:3151,1,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,226,I was crossing a wet road and slipped. I fell with a thud right on my rear end,5|5|5_5|5|5,
hit:1575_conv:3151,2,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,1,Oh no. How is your butt now?,5|5|5_5|5|5,
hit:1575_conv:3151,3,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,226,Very funny. All I could do is sit there and laugh.,5|5|5_5|5|5,
hit:1575_conv:3151,4,embarrassed,I was crossing a wet road and slipped. I fell with a thud right on my rear end.,1,hahaha I hope you recover soon.,5|5|5_5|5|5,
hit:1576_conv:3152,1,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,296,I heard one of my favorite youtubers talking about the 90's the other day and how old we were lol. But it reminded me of fun times as a teen,5|5|5_5|5|5,
hit:1576_conv:3152,2,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,1,90s were good time. What is your favorite memory?,5|5|5_5|5|5,
hit:1576_conv:3152,3,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,296,I am not sure I can pick on favorite but I loved how there were no bills lol and it was just freedom to be creative and listen to music all night and hang out with friends talking until the late hours :),5|5|5_5|5|5,
hit:1576_conv:3152,4,nostalgic,I heard one of my favorite youtubers talking about the 90's and how old we were lol. Made me remember good times as a teen,1,Oh yea. No bills is definitely great. You can still do all those things y'know.,5|5|5_5|5|5,
hit:1576_conv:3153,1,trusting,My friend left me with her dog. Never met this little guy.,1,My friend left me with her dog. Never met this little guy,5|5|5_5|5|5,
hit:1576_conv:3153,2,trusting,My friend left me with her dog. Never met this little guy.,296,Aw! Well I hope you can help him?,5|5|5_5|5|5,
hit:1576_conv:3153,3,trusting,My friend left me with her dog. Never met this little guy.,1,Yep. We are getting along so far. Looks like he misses his mommy though.,5|5|5_5|5|5,
hit:1576_conv:3153,4,trusting,My friend left me with her dog. Never met this little guy.,296,Oh I bet! Are you just babysitting?,5|5|5_5|5|5,
hit:1577_conv:3154,1,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,1,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,5|5|5_5|5|5,
hit:1577_conv:3154,2,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,273,That's got to be a pretty good feeling_comma_ to see our kids succeed and hopefully do better than us. I'm happy for you! What does he research?,5|5|5_5|5|5,
hit:1577_conv:3154,3,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,1,He's working on Parkinson actually. He has such a great heart.,5|5|5_5|5|5,
hit:1577_conv:3154,4,proud,After all these years as a single mom_comma_ my kid is now a successful researcher. He's pretty happy in life.,273,Parkinson's disease? That's pretty noble. Imagine if he were to be the one to help cause a breakthrough on curing it... How crazy would that be. Would add a nice legacy to your family.,5|5|5_5|5|5,
hit:1577_conv:3155,1,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,273,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,5|5|5_5|5|5,
hit:1577_conv:3155,2,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,1,Oh man_comma_ siblings are such hit or miss sometimes. Some get really good ones and some get_comma_ well_comma_ yours. Sorry to say.,5|5|5_5|5|5,
hit:1577_conv:3155,3,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,273,If I'm being honest_comma_ they're not all bad. The older one helps me with a lot of things. And the younger ones can be funny. But yeah_comma_ sometimes they just.. .get to me.,5|5|5_5|5|5,
hit:1577_conv:3155,4,annoyed,Someone ate the last piece of pizza this afternoon_comma_ when I was away all day and looking forward to eating it. I hate living with family during the summer_comma_ because my siblings eat and take things that aren't theirs!,1,Everyone has their faults I guess. I'm glad you don't focus on their faults.,5|5|5_5|5|5,
hit:1578_conv:3156,1,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,226,My favorite team has a new Quarterback. Maybe this will be a better season for them,4|4|5_5|5|5,
hit:1578_conv:3156,2,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,274,Who is your favorite team?,4|4|5_5|5|5,
hit:1578_conv:3156,3,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,226,I like college football. I root for Arkansas.,4|4|5_5|5|5,
hit:1578_conv:3156,4,hopeful,My favorite team has a new Quarterback. Maybe this will be a better season for them.,274,Who is the quarterback now,4|4|5_5|5|5,
hit:1578_conv:3157,1,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,274,After a long hard day at work_comma_ I came home to an empty house.,5|5|5_4|4|5,
hit:1578_conv:3157,2,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,226,Is that a good thing or a bad thing?,5|5|5_4|4|5,
hit:1578_conv:3157,3,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,274,Bad as there is no one to share my thoughts and relieve stress with.,5|5|5_4|4|5,
hit:1578_conv:3157,4,lonely,After being at work all day_comma_ I am now home relaxing with no one around.  There is no one to share my day with.,226,Are there any social groups you can join to have some company?,5|5|5_4|4|5,


hit:2444_conv:4888,1,joyful,i was happy when Lebron signed with the lakers,238,"i was happy when Lebron signed with the lakers,5|5|5_4|5|5,
hit:2444_conv:4888,2,joyful,i was happy when Lebron signed with the lakers,390,Oh yeah! So was I. That was so awesome! ,5|5|5_4|5|5,
hit:2444_conv:4888,3,joyful,i was happy when Lebron signed with the lakers,238,yea i hope they take down the warriors,5|5|5_4|5|5,
hit:2444_conv:4888,4,joyful,i was happy when Lebron signed with the lakers,390,Woo hoo! That would be so awesome! ,5|5|5_4|5|5,
hit:2444_conv:4889,1,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,390,I get so frantic when I am late for appointments. I had an eye doctor appointment and was 15 minutes late. ,4|5|5_5|5|5,
hit:2444_conv:4889,2,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,238,what ended up happening,4|5|5_5|5|5,
hit:2444_conv:4889,3,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,390,I thought the eye doctor was going to be upset_comma_ but they were fine with it. ,4|5|5_5|5|5,
hit:2444_conv:4889,4,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,238,oh thats cool,4|5|5_5|5|5,
hit:2445_conv:4890,1,faithful,I have always shopped at the stores who give me good customer service.,27,Good customer service is important to me.,5|5|5_3|5|5,
hit:2445_conv:4890,2,faithful,I have always shopped at the stores who give me good customer service.,209,It's important to me too_comma_ since I work in customer service. ,5|5|5_3|5|5,
hit:2445_conv:4890,3,faithful,I have always shopped at the stores who give me good customer service.,27,I have always stayed loyal to stores who have great customer service. I can't spend my money somewhere where I am treated poorly,5|5|5_3|5|5,
hit:2445_conv:4890,4,faithful,I have always shopped at the stores who give me good customer service.,209,I know. The customer is always right! Or so they teach us.,5|5|5_3|5|5,
hit:2445_conv:4890,5,faithful,I have always shopped at the stores who give me good customer service.,27,To an extent. You just don't have to be rude when people just want help,5|5|5_3|5|5,
hit:2445_conv:4891,1,prepared,I have an interview for my potential job tomorrow. Been practicing in front of a mirror all day.,209,I have an interview for my potential job tomorrow and have been practicing in front of the mirror all day.,3|5|5_5|5|5,
hit:2445_conv:4891,2,prepared,I have an interview for my potential job tomorrow. Been practicing in front of a mirror all day.,27,I hope you get the job. Just don't be nervous . It shoes.,3|5|5_5|5|5,
hit:2445_conv:4891,3,prepared,I have an interview for my potential job tomorrow. Been practicing in front of a mirror all day.,209,I feel quite prepared. Hopefully all this practicing pays off.,3|5|5_5|5|5,
hit:2445_conv:4891,4,prepared,I have an interview for my potential job tomorrow. Been practicing in front of a mirror all day.,27,Good luck. ,3|5|5_5|5|5,
hit:2446_conv:4892,1,terrified,I went skydiving the other day and was so scared to hop out of the plane. It was a chilling experience.,209,I went skydiving the other day and was so frightened to take the leap out of the plane.,5|5|5_5|5|5,
hit:2446_conv:4892,2,terrified,I went skydiving the other day and was so scared to hop out of the plane. It was a chilling experience.,269,I feel you_comma_ that feeling is the worse.,5|5|5_5|5|5,
hit:2446_conv:4892,3,terrified,I went skydiving the other day and was so scared to hop out of the plane. It was a chilling experience.,209,It was indeed. I took the jump and was screaming the whole way down.,5|5|5_5|5|5,
hit:2446_conv:4892,4,terrified,I went skydiving the other day and was so scared to hop out of the plane. It was a chilling experience.,269,It's like that feeling when you hit a bump on the road but 100x worse!,5|5|5_5|5|5,
hit:2446_conv:4893,1,jealous,Whenever I see a very expensive car on the road_comma_ I get so envious! One day...,269,Whenever I see a very expensive car on the road_comma_ I get so envious! One day...,5|5|5_5|5|5,
hit:2446_conv:4893,2,jealous,Whenever I see a very expensive car on the road_comma_ I get so envious! One day...,209,I do too. What is your ultimate dream car?,5|5|5_5|5|5,
hit:2446_conv:4893,3,jealous,Whenever I see a very expensive car on the road_comma_ I get so envious! One day...,269,I love Porsches!,5|5|5_5|5|5,
hit:2446_conv:4893,4,jealous,Whenever I see a very expensive car on the road_comma_ I get so envious! One day...,209,Porches are pretty nice. Not nice as Ford sedans_comma_ though.,5|5|5_5|5|5,
hit:2447_conv:4894,1,prepared,I'm going hiking with my friends tomorrow. I have everything I need and I forget bug spray like last time! I am ready!,269,I'm going hiking with my friends tomorrow. I have everything I need and I forget bug spray like last time! I am ready!,5|5|5_5|5|5,
hit:2447_conv:4894,2,prepared,I'm going hiking with my friends tomorrow. I have everything I need and I forget bug spray like last time! I am ready!,209,Sounds pretty neat! Where are you going to hike?,5|5|5_5|5|5,
hit:2447_conv:4894,3,prepared,I'm going hiking with my friends tomorrow. I have everything I need and I forget bug spray like last time! I am ready!,269,A local trail about 5 mi from us. Last time I went_comma_ I forgot so many things. I am more than prepared this time!,5|5|5_5|5|5,
hit:2447_conv:4894,4,prepared,I'm going hiking with my friends tomorrow. I have everything I need and I forget bug spray like last time! I am ready!,209,That's good to hear. Just don't forget that bug spray!,5|5|5_5|5|5,
hit:2447_conv:4895,1,guilty,I ate 24 burritos at Chipotle the other day. Felt so bad about myself.,209,So_comma_ I have a dark confession to make. I ate 24 burritos from Chipotle the other day. Felt so bad about myself.,5|5|5_5|5|5,
hit:2447_conv:4895,2,guilty,I ate 24 burritos at Chipotle the other day. Felt so bad about myself.,269,I hate overeating especially when I'm on a diet. I feel so guilty about it all the time.,5|5|5_5|5|5,
hit:2447_conv:4895,3,guilty,I ate 24 burritos at Chipotle the other day. Felt so bad about myself.,209,I know_comma_ felt the same thing yesterday. They had to lift me to my car because I couldn't move at all.,5|5|5_5|5|5,
hit:2447_conv:4895,4,guilty,I ate 24 burritos at Chipotle the other day. Felt so bad about myself.,269,24 burritos_comma_ my god. You are a eating terminator!,5|5|5_5|5|5,
hit:2448_conv:4896,1,guilty,I managed to bag 50 squirrels when I was hunting yesterday. Kinda felt bad for the little critters.,209,I managed to bag 50 squirrels the other day when I was hunting. I felt marginally bad.,5|5|5_5|5|5,
hit:2448_conv:4896,2,guilty,I managed to bag 50 squirrels when I was hunting yesterday. Kinda felt bad for the little critters.,269,Are you going to eat them?,5|5|5_5|5|5,
hit:2448_conv:4896,3,guilty,I managed to bag 50 squirrels when I was hunting yesterday. Kinda felt bad for the little critters.,209,Of course! I am going to make a 25 pound squirrel pot pie and submit it to the country fair. I am also planning on making an exquisite fur coat to sell on etsy. I hope some hillbilly will buy it.,5|5|5_5|5|5,
hit:2448_conv:4896,4,guilty,I managed to bag 50 squirrels when I was hunting yesterday. Kinda felt bad for the little critters.,269,Hillbillies love that stuff! They love roadkill hahaha,5|5|5_5|5|5,
hit:2448_conv:4897,1,disgusted,I stepped in dried dog poop today on my way to my car. It was so nasty!,269,I stepped in dried dog poop today on my way to my car. It was so nasty!,5|5|5_5|5|5,
hit:2448_conv:4897,2,disgusted,I stepped in dried dog poop today on my way to my car. It was so nasty!,209,Dang_comma_ that is some nasty stuff. Did you take your shoe and shove it in the face of your sworn enemies?,5|5|5_5|5|5,
hit:2448_conv:4897,3,disgusted,I stepped in dried dog poop today on my way to my car. It was so nasty!,269,Haha_comma_ maybe next time,5|5|5_5|5|5,
hit:2448_conv:4897,4,disgusted,I stepped in dried dog poop today on my way to my car. It was so nasty!,209,I guess you shouldn't clean the poop off your shoe then. Save it for when the time comes.,5|5|5_5|5|5,
hit:2449_conv:4898,1,impressed,My son spoke his first word yesterday. I was so proud.,209,My son spoke his first word yesterday. I was so impressed and proud.,5|5|5_5|5|5,
hit:2449_conv:4898,2,impressed,My son spoke his first word yesterday. I was so proud.,269,Congrats! How old is your son?,5|5|5_5|5|5,
hit:2449_conv:4898,3,impressed,My son spoke his first word yesterday. I was so proud.,209,He turns 1 this month! He is growing up so fast.,5|5|5_5|5|5,
hit:2449_conv:4898,4,impressed,My son spoke his first word yesterday. I was so proud.,269,What was his first word?,5|5|5_5|5|5,
hit:2449_conv:4899,1,excited,I can't wait to get paid on Saturday! I worked overtime 4 times this weekend and my check should be fat!,269,I can't wait to get paid on Saturday! I worked overtime 4 times this weekend and my check should be fat!,5|5|5_5|5|5,
hit:2449_conv:4899,2,excited,I can't wait to get paid on Saturday! I worked overtime 4 times this weekend and my check should be fat!,209,Dang_comma_ those are some long hours_comma_ my friend. Do you work in a Nike sweatshop? I hear they have some primo health benefits.,5|5|5_5|5|5,
hit:2449_conv:4899,3,excited,I can't wait to get paid on Saturday! I worked overtime 4 times this weekend and my check should be fat!,269,Yes I get great health benefits but I also love money!,5|5|5_5|5|5,
hit:2449_conv:4899,4,excited,I can't wait to get paid on Saturday! I worked overtime 4 times this weekend and my check should be fat!,209,I love money too. Are you planning on spending it on anything once you get those fat stacks?,5|5|5_5|5|5,
hit:2450_conv:4900,1,prepared,I felt ready when I went to work,238,I felt ready when I went to work,5|5|5_5|5|5,
hit:2450_conv:4900,2,prepared,I felt ready when I went to work,9,Work is always so boring for me.,5|5|5_5|5|5,
hit:2450_conv:4900,3,prepared,I felt ready when I went to work,238,thats why when you got your stuff in order you can knock it out,5|5|5_5|5|5,
hit:2450_conv:4900,4,prepared,I felt ready when I went to work,9,Work is always miserable for me.,5|5|5_5|5|5,
hit:2450_conv:4901,1,grateful,I was really glad I got to keep my computer when I moved.,9,I was really happen I got to keep my computer when i moved.,5|5|5_5|5|5,
hit:2450_conv:4901,2,grateful,I was really glad I got to keep my computer when I moved.,238,why is that,5|5|5_5|5|5,
hit:2450_conv:4901,3,grateful,I was really glad I got to keep my computer when I moved.,9,I did not think my girlfriend would let me take it.,5|5|5_5|5|5,
hit:2450_conv:4901,4,grateful,I was really glad I got to keep my computer when I moved.,238,dont let her control you!,5|5|5_5|5|5,
hit:2451_conv:4902,1,grateful,I love that my extended family likes to keep everyone together,267,I love that my extended family likes to keep everyone together,5|5|5_5|5|5,
hit:2451_conv:4902,2,grateful,I love that my extended family likes to keep everyone together,389,Yes that is awesome!  I bet it makes for good times!,5|5|5_5|5|5,
hit:2451_conv:4902,3,grateful,I love that my extended family likes to keep everyone together,267,It really does.  They always plan the best activities,5|5|5_5|5|5,
hit:2451_conv:4902,4,grateful,I love that my extended family likes to keep everyone together,389,Are the children having fun with each other?,5|5|5_5|5|5,
hit:2451_conv:4903,1,guilty,Said something mean to someone on facebook,389,I was under the influence and facebooking LOL.  Not something that should go together but I was kinda mean to an ex coworker when she asked me to write a letter of recommendation for her,5|5|5_5|5|5,
hit:2451_conv:4903,2,guilty,Said something mean to someone on facebook,267,oh wow.  Haha  Did you talk to them and make it right?,5|5|5_5|5|5,
hit:2451_conv:4903,3,guilty,Said something mean to someone on facebook,389,Yeah I sure did.  felt bad the next day after I had seen the conversation.  I then wrote that letter she wanted.,5|5|5_5|5|5,
hit:2451_conv:4903,4,guilty,Said something mean to someone on facebook,267,That is great of you,5|5|5_5|5|5,
hit:2452_conv:4904,1,furious,I was upset when I didnt win the lottery!,238,I was upset when i didnt win the lottery!,5|5|5_5|5|5,
hit:2452_conv:4904,2,furious,I was upset when I didnt win the lottery!,269,That sucks_comma_ I hope you win next time!,5|5|5_5|5|5,
hit:2452_conv:4904,3,furious,I was upset when I didnt win the lottery!,238,#metoo,5|5|5_5|5|5,<UNIGRAM>
hit:2452_conv:4904,4,furious,I was upset when I didnt win the lottery!,269,Just don't get too addicted and turn into a degenerate gambler!,5|5|5_5|5|5,
hit:2452_conv:4905,1,anxious,Public speaking makes me so nervous! I get very sweaty and start to shake uncontrollably...,269,Public speaking makes me so nervous! I get very sweaty and start to shake uncontrollably...,5|5|5_5|5|5,
hit:2452_conv:4905,2,anxious,Public speaking makes me so nervous! I get very sweaty and start to shake uncontrollably...,238,I hear ya.. practice makes perfect tho!,5|5|5_5|5|5,
hit:2452_conv:4905,3,anxious,Public speaking makes me so nervous! I get very sweaty and start to shake uncontrollably...,269,I'm trying to improve with self help books and online courses,5|5|5_5|5|5,
hit:2452_conv:4905,4,anxious,Public speaking makes me so nervous! I get very sweaty and start to shake uncontrollably...,238,good luck with that_comma_ i hope it works out!,5|5|5_5|5|5,
hit:2453_conv:4906,1,prepared,My wedding is tomorrow and I've been practicing my vows. Feel so ready for this day!,209,My wedding is tomorrow and I've been going over my vows over and over. Never felt so ready!,5|5|5_5|5|5,
hit:2453_conv:4906,2,prepared,My wedding is tomorrow and I've been practicing my vows. Feel so ready for this day!,27,Congratulations,5|5|5_5|5|5,<UNIGRAM>
hit:2453_conv:4906,3,prepared,My wedding is tomorrow and I've been practicing my vows. Feel so ready for this day!,209,Thanks man. Feels like I've been preparing for this day my whole life.,5|5|5_5|5|5,
hit:2453_conv:4906,4,prepared,My wedding is tomorrow and I've been practicing my vows. Feel so ready for this day!,27,How long you been together?,5|5|5_5|5|5,
hit:2453_conv:4907,1,anxious,I can't wait to find a new show to bing on,27,I ove binge watching tv shows,5|5|5_5|5|5,
hit:2453_conv:4907,2,anxious,I can't wait to find a new show to bing on,209,Me too. What kind of tv shows?,5|5|5_5|5|5,
hit:2453_conv:4907,3,anxious,I can't wait to find a new show to bing on,27,I love dramas. They leave you with a cliffhanger but you can watch the next show immediately lol,5|5|5_5|5|5,
hit:2453_conv:4907,4,anxious,I can't wait to find a new show to bing on,209,Yeah_comma_ seems like all tv shows are like that. My favorite is the Teletubbies.,5|5|5_5|5|5,
hit:2453_conv:4907,5,anxious,I can't wait to find a new show to bing on,27,Does that still come on?,5|5|5_5|5|5,
hit:2454_conv:4908,1,grateful,I am so thankful for simple things like clean water_comma_ some people in Africa have to walk miles just to get that!,269,I am so thankful for simple things like clean water_comma_ some people in Africa have to walk miles just to get that!,5|5|5_5|5|5,
hit:2454_conv:4908,2,grateful,I am so thankful for simple things like clean water_comma_ some people in Africa have to walk miles just to get that!,52,tell me about it! It's good that you appreciate these things,5|5|5_5|5|5,
hit:2454_conv:4908,3,grateful,I am so thankful for simple things like clean water_comma_ some people in Africa have to walk miles just to get that!,269,I feel so grateful. I usually take the simple things in life for granted...,5|5|5_5|5|5,
hit:2454_conv:4908,4,grateful,I am so thankful for simple things like clean water_comma_ some people in Africa have to walk miles just to get that!,52,It's hard not to! We get used to all the nice things we have,5|5|5_5|5|5,
hit:2454_conv:4909,1,ashamed,I gained so much weight ,52,i gained so much weight and i look so horrible... it's embarrasing,5|5|5_5|5|5,
hit:2454_conv:4909,2,ashamed,I gained so much weight ,269,I know that feeling! Eating is so addicting...,5|5|5_5|5|5,
hit:2454_conv:4909,3,ashamed,I gained so much weight ,52,yes_comma_ and i also stress eat. i can't control it,5|5|5_5|5|5,
hit:2454_conv:4909,4,ashamed,I gained so much weight ,269,Me too_comma_ you are not alone. It's a very common problem,5|5|5_5|5|5,
hit:2455_conv:4910,1,grateful,I am happy to have a home in a good neighborhood,267,I am happy to have a home in a good neighborhood,5|5|5_5|5|5,
hit:2455_conv:4910,2,grateful,I am happy to have a home in a good neighborhood,238,how long have you been there?,5|5|5_5|5|5,
hit:2455_conv:4910,3,grateful,I am happy to have a home in a good neighborhood,267,3 years.  Costs more than I can afford but my kids are safe,5|5|5_5|5|5,
hit:2455_conv:4910,4,grateful,I am happy to have a home in a good neighborhood,238,thats what matters,5|5|5_5|5|5,
hit:2455_conv:4911,1,faithful,I believe the lakers will crush the warriors!,238,I believe the lakers will crush the warriors!,5|5|5_5|5|5,
hit:2455_conv:4911,2,faithful,I believe the lakers will crush the warriors!,267,You really think so? ,5|5|5_5|5|5,
hit:2455_conv:4911,3,faithful,I believe the lakers will crush the warriors!,238,no but i hope so!,5|5|5_5|5|5,
hit:2455_conv:4911,4,faithful,I believe the lakers will crush the warriors!,267,I do too honestly,5|5|5_5|5|5,
hit:2456_conv:4913,1,grateful,I am so thankful for invention of cars! Can you imagine have to walk or take a bike to places very far away?,269,I am so thankful for invention of cars! Can you imagine have to walk or take a bike to places very far away?,5|5|5_3|5|5,
hit:2456_conv:4913,2,grateful,I am so thankful for invention of cars! Can you imagine have to walk or take a bike to places very far away?,221,I couldn't. I would love to be able to walk to work_comma_ but it's just too far.,5|5|5_3|5|5,
hit:2456_conv:4913,3,grateful,I am so thankful for invention of cars! Can you imagine have to walk or take a bike to places very far away?,269,Me too_comma_ everything is so far nowadays. I am so grateful!,5|5|5_3|5|5,
hit:2456_conv:4913,4,grateful,I am so thankful for invention of cars! Can you imagine have to walk or take a bike to places very far away?,221,If we had a better public transportation system where I live_comma_ I'd probably choose that over cars.,5|5|5_3|5|5,
hit:2457_conv:4914,1,surprised,My son used the potty all by himself today,27,MY son went to the potty all by himself today.,5|5|5_5|5|5,
hit:2457_conv:4914,2,surprised,My son used the potty all by himself today,238,oh thats awesome_comma_,5|5|5_5|5|5,
hit:2457_conv:4914,3,surprised,My son used the potty all by himself today,27,I ws surprised because he is so stubborn,5|5|5_5|5|5,
hit:2457_conv:4914,4,surprised,My son used the potty all by himself today,238,thats usually when it happens,5|5|5_5|5|5,
hit:2457_conv:4914,5,surprised,My son used the potty all by himself today,27,I wonder how he will do tomorrow,5|5|5_5|5|5,
hit:2457_conv:4915,1,excited,I am happy that the NFL season is starting!,238,I am happy that the NFL season is starting!,5|5|5_5|5|5,
hit:2457_conv:4915,2,excited,I am happy that the NFL season is starting!,27,I only watch when guys are around. ,5|5|5_5|5|5,
hit:2457_conv:4915,3,excited,I am happy that the NFL season is starting!,238,oh thats cool,5|5|5_5|5|5,
hit:2457_conv:4915,4,excited,I am happy that the NFL season is starting!,27,And beer.. There has to be beer.,5|5|5_5|5|5,
hit:2458_conv:4916,1,sentimental,Had to leave my dog with a sitter for a couple days,389,I had to attend a funeral and left my dog with a sitter.  When I came home 2 days later he was so happy to see me!  It warmed my heart ;),5|5|5_5|5|5,
hit:2458_conv:4916,2,sentimental,Had to leave my dog with a sitter for a couple days,262,Dogs are loyal like that.,5|5|5_5|5|5,
hit:2458_conv:4916,3,sentimental,Had to leave my dog with a sitter for a couple days,389,I loved it.  He is so sweet.  Do you have any dogs?,5|5|5_5|5|5,
hit:2458_conv:4916,4,sentimental,Had to leave my dog with a sitter for a couple days,262,I did when I was a kid. Her name was Terror. she was nice though,5|5|5_5|5|5,
hit:2458_conv:4917,1,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,262,It is so loney out here. My closest neighbor is two miles away,5|5|5_5|5|5,
hit:2458_conv:4917,2,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,389,Hey that makes for peaceful living doesn't it?,5|5|5_5|5|5,
hit:2458_conv:4917,3,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,262,Its lonely and I always think I'm going to be murdered and end up on 20/20,5|5|5_5|5|5,
hit:2458_conv:4917,4,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,389,Wow.   WHY?  LOL.  That's crazy,5|5|5_5|5|5,
hit:2458_conv:4917,5,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,262,I watch those shows too much,5|5|5_5|5|5,
hit:2458_conv:4917,6,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,389,haha.  Murder documentaries are my favorite too.,5|5|5_5|5|5,
hit:2458_conv:4917,7,lonely,I can't believe how far I moved from everything. Even people. I am by myself out here,262,They are too good to not watch.,5|5|5_5|5|5,
hit:2459_conv:4918,1,content,I don't have a car anymore but I'm ok with that. I'm perfectly fine with my bike because everything I need is very close to me,269,I don't have a car anymore but I'm ok with that. I'm perfectly fine with my bike because everything I need is very close to me,5|5|5_5|5|5,
hit:2459_conv:4918,2,content,I don't have a car anymore but I'm ok with that. I'm perfectly fine with my bike because everything I need is very close to me,277,That is a great place to be.  Sometimes less is more.  ,5|5|5_5|5|5,
hit:2459_conv:4918,3,content,I don't have a car anymore but I'm ok with that. I'm perfectly fine with my bike because everything I need is very close to me,269,I agree. I am perfectly content with having to ride a bike at the moment.,5|5|5_5|5|5,
hit:2459_conv:4918,4,content,I don't have a car anymore but I'm ok with that. I'm perfectly fine with my bike because everything I need is very close to me,277,That is an awesome outlook on life.  It is important to be content with what you have.,5|5|5_5|5|5,
hit:2459_conv:4919,1,faithful,I have always and will always be true to my loved ones.  I would never jeopardize their trust.,277,I have always and will always be true to my loved ones. I would never jeopardize their trust.,5|5|5_5|5|5,
hit:2459_conv:4919,2,faithful,I have always and will always be true to my loved ones.  I would never jeopardize their trust.,269,You sound like a very faithful and loyal person!,5|5|5_5|5|5,
hit:2459_conv:4919,3,faithful,I have always and will always be true to my loved ones.  I would never jeopardize their trust.,277,I strive to be that kind of person. It is quite important to me.,5|5|5_5|5|5,
hit:2459_conv:4919,4,faithful,I have always and will always be true to my loved ones.  I would never jeopardize their trust.,269,Those are some great qualities to have_comma_ that's for sure.,5|5|5_5|5|5,
hit:2460_conv:4921,1,content,Feeling lucky to not have plans,267,Feeling lucky to not have plans,5|5|5_5|5|5,
hit:2460_conv:4921,2,content,Feeling lucky to not have plans,221,I like not having plans too. I enjoy just relaxing sometimes.,5|5|5_5|5|5,
hit:2460_conv:4921,3,content,Feeling lucky to not have plans,267,You got that right,5|5|5_5|5|5,
hit:2460_conv:4921,4,content,Feeling lucky to not have plans,221,I had plans today_comma_ but they got canceled. I wasn't too upset haha,5|5|5_5|5|5,
hit:2461_conv:4922,1,devastated,My dad was recently diagnosed with diabetes. I'm really worried what this will mean for his health going forward. He's in his mid-60's and I want him to live as long as possible_comma_ but this obviously gets in the way of that.,273,My dad was recently diagnosed with diabetes. I'm really worried what this will mean for his health going forward. He's in his mid-60's and I want him to live as long as possible_comma_ but this obviously gets in the way of that.,4|4|4_5|5|5,
hit:2461_conv:4922,2,devastated,My dad was recently diagnosed with diabetes. I'm really worried what this will mean for his health going forward. He's in his mid-60's and I want him to live as long as possible_comma_ but this obviously gets in the way of that.,269,My dad has diabetes too but he was born with it. I heard veggies help a lot.,4|4|4_5|5|5,
hit:2461_conv:4922,3,devastated,My dad was recently diagnosed with diabetes. I'm really worried what this will mean for his health going forward. He's in his mid-60's and I want him to live as long as possible_comma_ but this obviously gets in the way of that.,273,Yeah_comma_ my dad has never been a veggies guy. Always sneaking too many desserts. He had a stroke earlier on in his life_comma_ too_comma_ which has always hampered him_comma_ and I think the medications for his heart condition led to the development of the diabetes. I feel sad for him.,4|4|4_5|5|5,
hit:2461_conv:4922,4,devastated,My dad was recently diagnosed with diabetes. I'm really worried what this will mean for his health going forward. He's in his mid-60's and I want him to live as long as possible_comma_ but this obviously gets in the way of that.,269,Sounds like a very stressful time for you. I hope everything turns out well and wish you the best of luck,4|4|4_5|5|5,
hit:2461_conv:4923,1,annoyed,There has been a fly in my room that won't stop landing on my face. I am about to lose it...,269,There has been a fly in my room that won't stop landing on my face. I am about to lose it...,5|5|5_4|4|4,
hit:2461_conv:4923,2,annoyed,There has been a fly in my room that won't stop landing on my face. I am about to lose it...,273,That's one of those things that never gets old to complain about_comma_ because it's one of life's time-tested annoyances. And what's worse is you open the door or window for them_comma_ and they still refuse to leave...,5|5|5_4|4|4,
hit:2461_conv:4923,3,annoyed,There has been a fly in my room that won't stop landing on my face. I am about to lose it...,269,I know!!! It's like the fly loves to annoy me lol,5|5|5_4|4|4,
hit:2461_conv:4923,4,annoyed,There has been a fly in my room that won't stop landing on my face. I am about to lose it...,273,And you never have a fly-swatter when you need one. Like_comma_ who keeps one of those around? Although I always have flip-flops because I live near the coast_comma_ so that makes a decent fly-swatter.,5|5|5_4|4|4,
hit:2462_conv:4924,1,disgusted,So_comma_ I was eating dinner and my dad kept laughing during it_comma_ and I wondered why. Turns out he accidentally dropped the plate of meat on the ground outside after grilling it_comma_ but decided to serve it to us anyway!,273,So_comma_ I was eating dinner and my dad kept laughing during it_comma_ and I wondered why. Turns out he accidentally dropped the plate of meat on the ground outside after grilling it_comma_ but decided to serve it to us anyway!,3|3|3_5|5|5,
hit:2462_conv:4924,2,disgusted,So_comma_ I was eating dinner and my dad kept laughing during it_comma_ and I wondered why. Turns out he accidentally dropped the plate of meat on the ground outside after grilling it_comma_ but decided to serve it to us anyway!,27,That is hilarious,3|3|3_5|5|5,
hit:2462_conv:4924,3,disgusted,So_comma_ I was eating dinner and my dad kept laughing during it_comma_ and I wondered why. Turns out he accidentally dropped the plate of meat on the ground outside after grilling it_comma_ but decided to serve it to us anyway!,273,Not to me it wasn't! I felt like something was off about the taste_comma_ too_comma_ like there was little grains of dirt or gravel or rock or something. And now I feel gross thinking about what else it could have touched_comma_ since our dogs poop in that yard.,3|3|3_5|5|5,
hit:2462_conv:4924,4,disgusted,So_comma_ I was eating dinner and my dad kept laughing during it_comma_ and I wondered why. Turns out he accidentally dropped the plate of meat on the ground outside after grilling it_comma_ but decided to serve it to us anyway!,27,OH no. Im sorry I laughed then.,3|3|3_5|5|5,
hit:2462_conv:4925,1,guilty,I ate all of the ice cream before my kids could even have any.,27,I ate all of the ice cream last night.,5|5|5_3|3|3,
hit:2462_conv:4925,2,guilty,I ate all of the ice cream before my kids could even have any.,273,All of it! Dang.. I bet someone wasn't happy about that. Unless you live alone_comma_ in which case_comma_ no big deal. Did you just feel like eating it_comma_ or were you feeling sad about something?,5|5|5_3|3|3,
hit:2462_conv:4925,3,guilty,I ate all of the ice cream before my kids could even have any.,27,I don't know. I do feel bad though because my kids didn't get any. Now I have to buy more,5|5|5_3|3|3,
hit:2462_conv:4925,4,guilty,I ate all of the ice cream before my kids could even have any.,273,Ah_comma_ the kids. Well_comma_ maybe that's a good excuse for them to skip the sugar for a few days. I'm sure you'll make it up to them anyhow.,5|5|5_3|3|3,
hit:2462_conv:4925,5,guilty,I ate all of the ice cream before my kids could even have any.,27,They don't need any sugar anyways,5|5|5_3|3|3,
hit:2463_conv:4926,1,confident,I am going to get a new car this year. I just know it,262,Im geting a new car this year.,5|5|5_5|5|5,
hit:2463_conv:4926,2,confident,I am going to get a new car this year. I just know it,269,Nice_comma_ what type of car?,5|5|5_5|5|5,
hit:2463_conv:4926,3,confident,I am going to get a new car this year. I just know it,262,I don't know. I don't have any money yet either. I jus know I am determined to get one before the year is over,5|5|5_5|5|5,
hit:2463_conv:4926,4,confident,I am going to get a new car this year. I just know it,269,Nice_comma_ it all starts with determination first. I believe in you man!,5|5|5_5|5|5,
hit:2463_conv:4926,5,confident,I am going to get a new car this year. I just know it,262,See that's it,5|5|5_5|5|5,
hit:2463_conv:4927,1,sad,My mom has been very depressed lately with her work lately. They randomly changed her schedule and she is having a hard time adapting.,269,My mom has been very depressed lately with her work lately. They randomly changed her schedule and she is having a hard time adapting.,5|5|5_5|5|5,
hit:2463_conv:4927,2,sad,My mom has been very depressed lately with her work lately. They randomly changed her schedule and she is having a hard time adapting.,262,that sucks,5|5|5_5|5|5,
hit:2463_conv:4927,3,sad,My mom has been very depressed lately with her work lately. They randomly changed her schedule and she is having a hard time adapting.,269,Yeah_comma_ she's not getting any younger too.,5|5|5_5|5|5,
hit:2463_conv:4927,4,sad,My mom has been very depressed lately with her work lately. They randomly changed her schedule and she is having a hard time adapting.,262,Tell her its okay,5|5|5_5|5|5,
hit:2464_conv:4928,1,devastated,I can't believe they didn't open our pool the entire summer. My kids were so hurt.,27,WE didn't swim all summer!,5|5|5_4|5|5,
hit:2464_conv:4928,2,devastated,I can't believe they didn't open our pool the entire summer. My kids were so hurt.,221,That sucks! Why not?,5|5|5_4|5|5,
hit:2464_conv:4928,3,devastated,I can't believe they didn't open our pool the entire summer. My kids were so hurt.,27,They never opened our pool.,5|5|5_4|5|5,
hit:2464_conv:4928,4,devastated,I can't believe they didn't open our pool the entire summer. My kids were so hurt.,221,Dang. It's so hot_comma_ swimming seems to be the only way to beat the heat.,5|5|5_4|5|5,
hit:2464_conv:4928,5,devastated,I can't believe they didn't open our pool the entire summer. My kids were so hurt.,27,I know and my kids really wanted to,5|5|5_4|5|5,
hit:2465_conv:4930,1,nostalgic,I love watching old movies I've seen a long time ago. I notice things I didn't pick up on when I was young. It was like reliving the past.,269,I love watching old movies I've seen a long time ago. I notice things I didn't pick up on when I was young. It was like reliving the past.,5|5|5_5|5|5,
hit:2465_conv:4930,2,nostalgic,I love watching old movies I've seen a long time ago. I notice things I didn't pick up on when I was young. It was like reliving the past.,262,Me too. I do it all the time.,5|5|5_5|5|5,
hit:2465_conv:4930,3,nostalgic,I love watching old movies I've seen a long time ago. I notice things I didn't pick up on when I was young. It was like reliving the past.,269,It was a very nostalgic for sure.,5|5|5_5|5|5,
hit:2465_conv:4930,4,nostalgic,I love watching old movies I've seen a long time ago. I notice things I didn't pick up on when I was young. It was like reliving the past.,262,Im like that with food too,5|5|5_5|5|5,



hit:3397_conv:6794,1,excited,One of my favorite actors is doing a singing gig locally and I am going to get to go see him. I can hardly wait.,226,"One of my favorite actors is doing a singing gig locally and I am going to get to go see him. I can hardly wait.,4|4|5_4|5|5,
hit:3397_conv:6794,2,excited,One of my favorite actors is doing a singing gig locally and I am going to get to go see him. I can hardly wait.,386,That is great! What actor is it?,4|4|5_4|5|5,
hit:3397_conv:6794,3,excited,One of my favorite actors is doing a singing gig locally and I am going to get to go see him. I can hardly wait.,226,Someone who I have watched perform in movies and on TV and never knew he could sing.,4|4|5_4|5|5,
hit:3397_conv:6794,4,excited,One of my favorite actors is doing a singing gig locally and I am going to get to go see him. I can hardly wait.,386,That sounds like a great time. I hope you have fun.,4|4|5_4|5|5,
hit:3399_conv:6799,1,terrified,When I was young_comma_ I was attacked by dogs. Now_comma_ I am so scared of dogs.,7,When I was young_comma_ I was attacked by dogs. Now_comma_ I am so scared of dogs.,5|5|5_5|5|5,
hit:3399_conv:6799,2,terrified,When I was young_comma_ I was attacked by dogs. Now_comma_ I am so scared of dogs.,326,I hear this from a lot of people. You should get over that fear somehow.,5|5|5_5|5|5,
hit:3399_conv:6799,3,terrified,When I was young_comma_ I was attacked by dogs. Now_comma_ I am so scared of dogs.,7,I understand. It is difficult to get over it.,5|5|5_5|5|5,
hit:3399_conv:6799,4,terrified,When I was young_comma_ I was attacked by dogs. Now_comma_ I am so scared of dogs.,326,Yes_comma_ but not impossible.,5|5|5_5|5|5,
hit:3400_conv:6800,1,anxious,I was very nervous going to jury duty. I just did not want to go,126,I was nervous going to jury duty. I just did not want to go,3|2|2_5|5|5,
hit:3400_conv:6800,2,anxious,I was very nervous going to jury duty. I just did not want to go,63,oh yea i can imagine,3|2|2_5|5|5,
hit:3400_conv:6800,3,anxious,I was very nervous going to jury duty. I just did not want to go,126,It was okay in the end_comma_ but at the time I did not feel good,3|2|2_5|5|5,
hit:3400_conv:6800,4,anxious,I was very nervous going to jury duty. I just did not want to go,63,yea seems very boring,3|2|2_5|5|5,
hit:3400_conv:6801,1,content,I stayed in curled up watching netflix all night today.,63,I stayed in curled up watching netflix all night today which was great,5|5|5_3|2|2,
hit:3400_conv:6801,2,content,I stayed in curled up watching netflix all night today.,126,I do like netflix as well,5|5|5_3|2|2,
hit:3400_conv:6801,3,content,I stayed in curled up watching netflix all night today.,63,yea it was very nice and cozy,5|5|5_3|2|2,
hit:3400_conv:6801,4,content,I stayed in curled up watching netflix all night today.,126,Those are the best days,5|5|5_3|2|2,
hit:3401_conv:6802,1,anxious,I am waiting to hear back on some test results. I feel pretty stressed about it.,366,I am feeling very stressed out. I have been waiting to hear back on some test results.,5|5|5_5|5|5,
hit:3401_conv:6802,2,anxious,I am waiting to hear back on some test results. I feel pretty stressed about it.,20,Oh I am sorry to hear that. What was the test?,5|5|5_5|5|5,
hit:3401_conv:6802,3,anxious,I am waiting to hear back on some test results. I feel pretty stressed about it.,366,It was some blood work and testing insulin levels. I worry about my health a lot.,5|5|5_5|5|5,
hit:3401_conv:6802,4,anxious,I am waiting to hear back on some test results. I feel pretty stressed about it.,20,I hope everything turns out well for you.,5|5|5_5|5|5,
hit:3401_conv:6802,5,anxious,I am waiting to hear back on some test results. I feel pretty stressed about it.,366,Thanks_comma_ any type of test results can be stressful!,5|5|5_5|5|5,
hit:3401_conv:6803,1,disgusted,I saw someone hurt themselves at work. It really grossed me out so bad I almost puked.,20,I saw someone hurt themselves at work and it was so gross!,5|5|5_5|5|5,
hit:3401_conv:6803,2,disgusted,I saw someone hurt themselves at work. It really grossed me out so bad I almost puked.,366,Oh no! How was your co-worker doing afterwards?,5|5|5_5|5|5,
hit:3401_conv:6803,3,disgusted,I saw someone hurt themselves at work. It really grossed me out so bad I almost puked.,20,He was okay after two weeks at home.,5|5|5_5|5|5,
hit:3401_conv:6803,4,disgusted,I saw someone hurt themselves at work. It really grossed me out so bad I almost puked.,366,He had a good amount of time to recuperate. That's good to hear.,5|5|5_5|5|5,
hit:3402_conv:6804,1,nostalgic,When I see a certain movie on tv_comma_ it makes me think of being a kid around my parents,126,I saw a movie last night that made me think of my dad_comma_ and when I was young,4|3|4_5|5|5,
hit:3402_conv:6804,2,nostalgic,When I see a certain movie on tv_comma_ it makes me think of being a kid around my parents,447,What movie was it?,4|3|4_5|5|5,
hit:3402_conv:6804,3,nostalgic,When I see a certain movie on tv_comma_ it makes me think of being a kid around my parents,126,The outlaw josey wales,4|3|4_5|5|5,
hit:3402_conv:6804,4,nostalgic,When I see a certain movie on tv_comma_ it makes me think of being a kid around my parents,447,Oh a western.  I used to watch westerns with my dad as well.,4|3|4_5|5|5,
hit:3403_conv:6806,1,angry,I was very mad when my dad lost his job. I just felt like he was wronged,126,I was mad when my dad lost job years ago. I felt like he was wronged,1|1|1_4|4|4,
hit:3403_conv:6806,2,angry,I was very mad when my dad lost his job. I just felt like he was wronged,268,ooh sorry how is it now does he have a job,1|1|1_4|4|4,
hit:3403_conv:6806,3,angry,I was very mad when my dad lost his job. I just felt like he was wronged,126,He has one now_comma_ but it was maddening back then,1|1|1_4|4|4,
hit:3403_conv:6806,4,angry,I was very mad when my dad lost his job. I just felt like he was wronged,268,ok sorry for that ,1|1|1_4|4|4,
hit:3403_conv:6807,1,disgusted,i saw a girl pee in front of a car,268,it was nasty and disgusting,4|4|4_1|1|1,
hit:3403_conv:6807,2,disgusted,i saw a girl pee in front of a car,126,What was?,4|4|4_1|1|1,
hit:3403_conv:6807,3,disgusted,i saw a girl pee in front of a car,268,so bad to see a girl pee,4|4|4_1|1|1,
hit:3403_conv:6807,4,disgusted,i saw a girl pee in front of a car,126,I am not sure what you are talking about,4|4|4_1|1|1,
hit:3404_conv:6808,1,sentimental,Lately_comma_ my wife is little bit depressed. So I bought her a gift to show her how important she is to me.,7,Lately_comma_ my wife is little bit depressed. So I bought her a gift to show her how important she is to me.,5|5|5_4|5|5,
hit:3404_conv:6808,2,sentimental,Lately_comma_ my wife is little bit depressed. So I bought her a gift to show her how important she is to me.,20,That is very kind of you. You sound like a good husband.,5|5|5_4|5|5,
hit:3404_conv:6808,3,sentimental,Lately_comma_ my wife is little bit depressed. So I bought her a gift to show her how important she is to me.,7,Thanks. She was very happy. It helped her.,5|5|5_4|5|5,
hit:3404_conv:6808,4,sentimental,Lately_comma_ my wife is little bit depressed. So I bought her a gift to show her how important she is to me.,20,What did you get for her?,5|5|5_4|5|5,
hit:3404_conv:6809,1,disappointed,I didn't get the position I applied for. I really wanted that job.,20,I didn't get the job position I applied for and I really wanted it.,4|5|5_5|5|5,
hit:3404_conv:6809,2,disappointed,I didn't get the position I applied for. I really wanted that job.,7,Sorry to hear that. Did you try to find similar job position on the job market?,4|5|5_5|5|5,
hit:3404_conv:6809,3,disappointed,I didn't get the position I applied for. I really wanted that job.,20,I did but this was my dream job. It was pretty upsetting.,4|5|5_5|5|5,
hit:3404_conv:6809,4,disappointed,I didn't get the position I applied for. I really wanted that job.,7,I understand.,4|5|5_5|5|5,
hit:3405_conv:6810,1,excited,I was so looking forward to Christmas last year. I could not wait to see my family,126,I couldnt wait for Christmas last year. I was happy to see my family,3|3|4_4|4|4,
hit:3405_conv:6810,2,excited,I was so looking forward to Christmas last year. I could not wait to see my family,268,wow me too we had fun,3|3|4_4|4|4,
hit:3405_conv:6810,3,excited,I was so looking forward to Christmas last year. I could not wait to see my family,126,Yeah_comma_ its always a fun time of the year,3|3|4_4|4|4,
hit:3405_conv:6810,4,excited,I was so looking forward to Christmas last year. I could not wait to see my family,268,yeah it was fun too,3|3|4_4|4|4,
hit:3405_conv:6811,1,jealous,i saw my girlfriend hug somebody,268,it so bad to see a girlfriend hug somebody you don't know,4|4|4_3|3|4,
hit:3405_conv:6811,2,jealous,i saw my girlfriend hug somebody,126,That would be weird,4|4|4_3|3|4,
hit:3405_conv:6811,3,jealous,i saw my girlfriend hug somebody,268,yeah and so bad,4|4|4_3|3|4,
hit:3405_conv:6811,4,jealous,i saw my girlfriend hug somebody,126,Sorry you had to see that,4|4|4_3|3|4,
hit:3406_conv:6812,1,embarrassed,I dropped my food plate at dinner.,59,So I was at dinner at my girlfriend's parents' house and I dropped my food plate.,4|4|5_3|3|4,
hit:3406_conv:6812,2,embarrassed,I dropped my food plate at dinner.,126,I bet that was embarrassing,4|4|5_3|3|4,
hit:3406_conv:6812,3,embarrassed,I dropped my food plate at dinner.,59,It was. I got red faced and started sweating because her parents were scolding me.,4|4|5_3|3|4,
hit:3406_conv:6812,4,embarrassed,I dropped my food plate at dinner.,126,I am sorry_comma_ bad on them,4|4|5_3|3|4,
hit:3406_conv:6813,1,devastated,I felt lost when I had to give my dog away last summer. It was just sad to me,126,I felt lost when I had to give away my dog. It was just sad,3|3|4_4|4|5,
hit:3406_conv:6813,2,devastated,I felt lost when I had to give my dog away last summer. It was just sad to me,59,Why did you have to give the dog away? It must have been very upsetting.,3|3|4_4|4|5,
hit:3406_conv:6813,3,devastated,I felt lost when I had to give my dog away last summer. It was just sad to me,126,The place I was going did not allow dogs,3|3|4_4|4|5,
hit:3406_conv:6813,4,devastated,I felt lost when I had to give my dog away last summer. It was just sad to me,59,Oh I see_comma_ but you can still see it right? If you visit the new owner.,3|3|4_4|4|5,
hit:3407_conv:6814,1,guilty,I stole some money from someone at a party. I feel really bad about it now.,20,I stole money from someone at a party years ago and I still feel bad about it.,5|5|5_5|5|5,
hit:3407_conv:6814,2,guilty,I stole some money from someone at a party. I feel really bad about it now.,59,Why did you do that? You must feel terribly guilty for stealing someone's money.,5|5|5_5|5|5,
hit:3407_conv:6814,3,guilty,I stole some money from someone at a party. I feel really bad about it now.,20,I really do. I was in a bad spot at that time in my life.,5|5|5_5|5|5,
hit:3407_conv:6814,4,guilty,I stole some money from someone at a party. I feel really bad about it now.,59,Well_comma_ its good that you realize it was a bad action. I would say pass some money on to a person in need.,5|5|5_5|5|5,
hit:3407_conv:6815,1,sentimental,I was going through my old wedding picturs.,59,Hi_comma_ I was going through my old wedding pictures yesterday and felt so many emotions. ,5|5|5_5|5|5,<HI>
hit:3407_conv:6815,2,sentimental,I was going through my old wedding picturs.,20,That must have been a very special moment in your life.,5|5|5_5|5|5,
hit:3407_conv:6815,3,sentimental,I was going through my old wedding picturs.,59,Yea it was_comma_ it was my most happiest moment being there with my wife.,5|5|5_5|5|5,
hit:3407_conv:6815,4,sentimental,I was going through my old wedding picturs.,20,Did you get professional photos taken?,5|5|5_5|5|5,
hit:3408_conv:6816,1,proud,i was given a chance to speak in public,268,it was so motivating i liked it,4|4|4_2|1|1,
hit:3408_conv:6816,2,proud,i was given a chance to speak in public,126,What did you like,4|4|4_2|1|1,
hit:3408_conv:6816,3,proud,i was given a chance to speak in public,268,the experience of public speaking,4|4|4_2|1|1,
hit:3408_conv:6816,4,proud,i was given a chance to speak in public,126,Oh that sounds nervewracking,4|4|4_2|1|1,
hit:3408_conv:6817,1,surprised,I was surprised when my dad got my a trampoline as a kid. I was not expecting it,126,I was surprised when my dad got my a trampoline as a kid. I was not expecting it,2|1|1_4|4|4,
hit:3408_conv:6817,2,surprised,I was surprised when my dad got my a trampoline as a kid. I was not expecting it,268,so good,2|1|1_4|4|4,
hit:3408_conv:6817,3,surprised,I was surprised when my dad got my a trampoline as a kid. I was not expecting it,126,Yeah it was a fun time,2|1|1_4|4|4,
hit:3408_conv:6817,4,surprised,I was surprised when my dad got my a trampoline as a kid. I was not expecting it,268,good i dint get as a kid,2|1|1_4|4|4,
hit:3409_conv:6818,1,hopeful,I was confident my basketball team was going to win last weekend. We practiced well all week,126,I just knew my basketball team was going to win last weekend. We practice well all week,4|3|3_5|5|5,
hit:3409_conv:6818,2,hopeful,I was confident my basketball team was going to win last weekend. We practiced well all week,59,You must have been well prepared then!,4|3|3_5|5|5,
hit:3409_conv:6818,3,hopeful,I was confident my basketball team was going to win last weekend. We practiced well all week,126,We were and played well,4|3|3_5|5|5,
hit:3409_conv:6818,4,hopeful,I was confident my basketball team was going to win last weekend. We practiced well all week,59,Good_comma_ I wish my teammates would practice as much.,4|3|3_5|5|5,
hit:3409_conv:6819,1,angry,My cousin lost my cell phone.,59,I let my cousin borrow my phone to play games and he lost it!,5|5|5_4|3|3,
hit:3409_conv:6819,2,angry,My cousin lost my cell phone.,126,That is not good,5|5|5_4|3|3,
hit:3409_conv:6819,3,angry,My cousin lost my cell phone.,59,I agree_comma_ I am so livid right now. It was an expensive phone!,5|5|5_4|3|3,
hit:3409_conv:6819,4,angry,My cousin lost my cell phone.,126,I am sorry_comma_ you will get another one,5|5|5_4|3|3,
hit:3410_conv:6820,1,angry,I was so mad when my mom wanted to divorce my dad. I thought she was crazy,126,I was pretty angry when my mom wanted to divorce my dad. I thought she was crazy,1|2|1_4|4|4,
hit:3410_conv:6820,2,angry,I was so mad when my mom wanted to divorce my dad. I thought she was crazy,268,ooooh thats so bad what happened,1|2|1_4|4|4,
hit:3410_conv:6820,3,angry,I was so mad when my mom wanted to divorce my dad. I thought she was crazy,126,They stayed together_comma_ but it wasnt the same,1|2|1_4|4|4,
hit:3410_conv:6820,4,angry,I was so mad when my mom wanted to divorce my dad. I thought she was crazy,268,sorry now how is the situation,1|2|1_4|4|4,
hit:3410_conv:6821,1,grateful,iwas given a promotion,268,it was such a good time,4|4|4_1|2|1,
hit:3410_conv:6821,2,grateful,iwas given a promotion,126,What was a good time?,4|4|4_1|2|1,
hit:3410_conv:6821,3,grateful,iwas given a promotion,268,yeah i really enjoyed,4|4|4_1|2|1,
hit:3410_conv:6821,4,grateful,iwas given a promotion,126,I am not sure what you enjoyed,4|4|4_1|2|1,
hit:3411_conv:6822,1,faithful,I am so very happy with my girlfriend. I want to be with her forever.,20,I am so very happy with my girlfriend. I want to be with her forever.,2|3|2_4|4|4,
hit:3411_conv:6822,2,faithful,I am so very happy with my girlfriend. I want to be with her forever.,268,yeah me too,2|3|2_4|4|4,
hit:3411_conv:6822,3,faithful,I am so very happy with my girlfriend. I want to be with her forever.,20,It is nice to have someone I can rely on to make me a better person.,2|3|2_4|4|4,
hit:3411_conv:6822,4,faithful,I am so very happy with my girlfriend. I want to be with her forever.,268,yeah its such nice,2|3|2_4|4|4,
hit:3411_conv:6823,1,guilty,i was a girl die,268,so  bad,4|4|4_2|3|2,
hit:3411_conv:6823,2,guilty,i was a girl die,20,What is so bad?,4|4|4_2|3|2,
hit:3411_conv:6823,3,guilty,i was a girl die,268,to see some body die,4|4|4_2|3|2,
hit:3411_conv:6823,4,guilty,i was a girl die,20,Did you see this for yourself?,4|4|4_2|3|2,
hit:3412_conv:6824,1,trusting,I felt like I could count on my dad when I was sick. He always cared,126,I always knew I could count on my dad when I was sick. He always cared,3|2|3_4|5|5,
hit:3412_conv:6824,2,trusting,I felt like I could count on my dad when I was sick. He always cared,447,That's sweet.  What kinds of things would he do to make you feel better?,3|2|3_4|5|5,
hit:3412_conv:6824,3,trusting,I felt like I could count on my dad when I was sick. He always cared,126,He made sure I took my medicine,3|2|3_4|5|5,
hit:3412_conv:6824,4,trusting,I felt like I could count on my dad when I was sick. He always cared,447,When I was a kid my dad would put my medication on a small plate guarded by a green army man.,3|2|3_4|5|5,
hit:3413_conv:6826,1,angry,I bought four new tires and waited while they put them on. When I got home I discovered that I had my old tires still on and not my new ones.,226,I bought four new tires and waited while they put them on. When I got home I discovered that I had my old tires still on and not my new ones.,1|1|1_I have clicked the 'End Chat' button thank you!|4|4,
hit:3413_conv:6826,2,angry,I bought four new tires and waited while they put them on. When I got home I discovered that I had my old tires still on and not my new ones.,268,haha that was serious ,1|1|1_I have clicked the 'End Chat' button thank you!|4|4,
hit:3413_conv:6826,3,angry,I bought four new tires and waited while they put them on. When I got home I discovered that I had my old tires still on and not my new ones.,226,I went back and the man on duty was very condescending towards me until he went outside and looked and then knew I was telling the truth. I was furious. ,1|1|1_I have clicked the 'End Chat' button thank you!|4|4,
hit:3413_conv:6826,4,angry,I bought four new tires and waited while they put them on. When I got home I discovered that I had my old tires still on and not my new ones.,268,ok what happened next,1|1|1_I have clicked the 'End Chat' button thank you!|4|4,
hit:3413_conv:6827,1,faithful,i had a serious relationship,268,what did you do about it,I have clicked the 'End Chat' button thank you!|4|4_1|1|1,
hit:3413_conv:6827,2,faithful,i had a serious relationship,226,I'm not sure what we are talking about here. ,I have clicked the 'End Chat' button thank you!|4|4_1|1|1,
hit:3413_conv:6827,3,faithful,i had a serious relationship,268,she cheated on me,I have clicked the 'End Chat' button thank you!|4|4_1|1|1,
hit:3413_conv:6827,4,faithful,i had a serious relationship,226,That's terrible. Did you kick her to the curb?,I have clicked the 'End Chat' button thank you!|4|4_1|1|1,
hit:3413_conv:6827,5,faithful,i had a serious relationship,268,we were talking about relationship ,I have clicked the 'End Chat' button thank you!|4|4_1|1|1,
hit:3414_conv:6828,1,excited,I was so happy for thanksgiving this last year. I was going with friends,126,I was happy about thanksgiving last year. I was going with friends,4|4|5_4|5|5,
hit:3414_conv:6828,2,excited,I was so happy for thanksgiving this last year. I was going with friends,20,Family time is a very important part of life.,4|4|5_4|5|5,
hit:3414_conv:6828,3,excited,I was so happy for thanksgiving this last year. I was going with friends,126,Yeah it_comma_ always important,4|4|5_4|5|5,
hit:3414_conv:6828,4,excited,I was so happy for thanksgiving this last year. I was going with friends,20,What did you have for Thanksgiving dinner?,4|4|5_4|5|5,
hit:3414_conv:6829,1,apprehensive,I found a great deal for a computer online. I don't know if it's too good to be true though.,20,I found a great deal online for a computer but it almost seems too good.,4|5|5_4|4|5,
hit:3414_conv:6829,2,apprehensive,I found a great deal for a computer online. I don't know if it's too good to be true though.,126,I am glad you found it,4|5|5_4|4|5,
hit:3414_conv:6829,3,apprehensive,I found a great deal for a computer online. I don't know if it's too good to be true though.,20,I am not sure if it is a scam or not.,4|5|5_4|4|5,
hit:3414_conv:6829,4,apprehensive,I found a great deal for a computer online. I don't know if it's too good to be true though.,126,You can always research it more,4|5|5_4|4|5,
hit:3415_conv:6830,1,guilty,I stole money when i was a teen.,59,Hi_comma_ I stole money from an old lady when I was a teen. I still feel bad about it.,4|5|5_4|5|4,<HI>
hit:3415_conv:6830,2,guilty,I stole money when i was a teen.,126,Its okay_comma_ you were young,4|5|5_4|5|4,
hit:3415_conv:6830,3,guilty,I stole money when i was a teen.,59,But I should have known better. I cry about it every day.,4|5|5_4|5|4,
hit:3415_conv:6830,4,guilty,I stole money when i was a teen.,126,You shouldnt do that,4|5|5_4|5|4,
hit:3415_conv:6831,1,furious,I was so mad when my brother said he was moving out. I felt like he was leaving me,126,I was mad when my brother was moving out. I felt like he was abadoning me,4|5|4_4|5|5,
hit:3415_conv:6831,2,furious,I was so mad when my brother said he was moving out. I felt like he was leaving me,59,Sometimes we have to part ways with our loved ones. Dont be mad at him.,4|5|4_4|5|5,
hit:3415_conv:6831,3,furious,I was so mad when my brother said he was moving out. I felt like he was leaving me,126,I am not anymore,4|5|4_4|5|5,
hit:3415_conv:6831,4,furious,I was so mad when my brother said he was moving out. I felt like he was leaving me,59,You are not what anymore?,4|5|4_4|5|5,
hit:3416_conv:6832,1,devastated,My phone was burnt in a fire.,59,My cousin threw my phone in the fire because he thought it was firewood. ,5|4|4_4|4|4,
hit:3416_conv:6832,2,devastated,My phone was burnt in a fire.,268,hahahahahaha he must be cunning,5|4|4_4|4|4,
hit:3416_conv:6832,3,devastated,My phone was burnt in a fire.,59,I am still so mad at him. It was my only phone.,5|4|4_4|4|4,
hit:3416_conv:6832,4,devastated,My phone was burnt in a fire.,268,ooooh no i feel sorry but punish him,5|4|4_4|4|4,
hit:3416_conv:6833,1,sad,i was not payed on time,268,i usually feel mad when am not payed on time,4|4|4_5|4|4,
hit:3416_conv:6833,2,sad,i was not payed on time,59,You should be. You should be paid on time for your hard work.,4|4|4_5|4|4,
hit:3416_conv:6833,3,sad,i was not payed on time,268,yeah but some dont care about that,4|4|4_5|4|4,
hit:3416_conv:6833,4,sad,i was not payed on time,59,You should report them to the police.,4|4|4_5|4|4,
hit:3417_conv:6834,1,disgusted,When my cousin returned my car to me_comma_ it was filled with garbage.,59,I let my cousin borrow my car for the weekend and when I got it back_comma_ it was filled with mcdonalds fries.,5|4|4_5|5|5,
hit:3417_conv:6834,2,disgusted,When my cousin returned my car to me_comma_ it was filled with garbage.,20,That is pretty gross! I hope he cleaned it up later.,5|4|4_5|5|5,
hit:3417_conv:6834,3,disgusted,When my cousin returned my car to me_comma_ it was filled with garbage.,59,He didnt. He said it wasnt his fault. I am beyond disgusted rigth now.,5|4|4_5|5|5,
hit:3417_conv:6834,4,disgusted,When my cousin returned my car to me_comma_ it was filled with garbage.,20,That should be the last time you let him borrow it.,5|4|4_5|5|5,
hit:3417_conv:6835,1,joyful,My girlfriend and I are getting married. We are so happy planning the wedding.,20,My girlfriend and I are getting married! I am so excited!,5|5|5_5|4|4,
hit:3417_conv:6835,2,joyful,My girlfriend and I are getting married. We are so happy planning the wedding.,59,That is so nice to hear. How long have you been together?,5|5|5_5|4|4,
hit:3417_conv:6835,3,joyful,My girlfriend and I are getting married. We are so happy planning the wedding.,20,We have been together for three years.,5|5|5_5|4|4,
hit:3417_conv:6835,4,joyful,My girlfriend and I are getting married. We are so happy planning the wedding.,59,That is a long time. I wish the both of you well and excitement! ,5|5|5_5|4|4,
hit:3418_conv:6836,1,angry,My boss fired me for no reason.,59,My boss fired me for no reason. I am absolutely livid.,5|4|5_4|5|4,
hit:3418_conv:6836,2,angry,My boss fired me for no reason.,126,I am sorry that happened_comma_ you will find another job,5|4|5_4|5|4,
hit:3418_conv:6836,3,angry,My boss fired me for no reason.,59,I hope so_comma_ especially since I was the best employee at my last job.,5|4|5_4|5|4,
hit:3418_conv:6836,4,angry,My boss fired me for no reason.,126,You just need to start looking,5|4|5_4|5|4,
hit:3418_conv:6837,1,excited,I was looking forward to seeing a movie last weekend. I wanted to see it all summer,126,I couldnt wait to see a movie last weekend. I had wanted to see it all summer,4|5|4_5|4|5,
hit:3418_conv:6837,2,excited,I was looking forward to seeing a movie last weekend. I wanted to see it all summer,59,It seems you were anticipating for a long time. Which movie did you watch/,4|5|4_5|4|5,
hit:3418_conv:6837,3,excited,I was looking forward to seeing a movie last weekend. I wanted to see it all summer,126,It was the new mission impossible movie,4|5|4_5|4|5,
hit:3418_conv:6837,4,excited,I was looking forward to seeing a movie last weekend. I wanted to see it all summer,59,I have heard its good. Did you like it?,4|5|4_5|4|5,
hit:3419_conv:6838,1,proud,My son came first in English class.,59,Hi_comma_ so my son came first in his English class!,5|5|5_5|5|5,<HI>
hit:3419_conv:6838,2,proud,My son came first in English class.,447,Congratulations to you both.  You must be a very proud parent.,5|5|5_5|5|5,
hit:3419_conv:6838,3,proud,My son came first in English class.,59,Yes I am. I will buy him some chocolate now!,5|5|5_5|5|5,
hit:3419_conv:6838,4,proud,My son came first in English class.,447,I bet he will enoy that immensely.  A good reward for succeeding at a difficult challenge.,5|5|5_5|5|5,

hit:3420_conv:6840,1,afraid,I was afraid when my dad got sick. It was the first time I ever saw him like that,126,I was afraid when my dad was sick. It was the first time I had seen him like that,4|3|3_5|5|5,
hit:3420_conv:6840,2,afraid,I was afraid when my dad got sick. It was the first time I ever saw him like that,59,Oh my_comma_ is he alright?,4|3|3_5|5|5,
hit:3420_conv:6840,3,afraid,I was afraid when my dad got sick. It was the first time I ever saw him like that,126,He was okay in the end,4|3|3_5|5|5,
hit:3420_conv:6840,4,afraid,I was afraid when my dad got sick. It was the first time I ever saw him like that,59,That is good to hear. You must be strong for him! ,4|3|3_5|5|5,

hit:3420_conv:6841,1,annoyed,My brother wont stop laughing at my pimple.,59,I woke up with a big red pimple and my brother wont stop making fun of me.,5|5|5_4|3|3,
hit:3420_conv:6841,2,annoyed,My brother wont stop laughing at my pimple.,126,I am sure it will go away,5|5|5_4|3|3,
hit:3420_conv:6841,3,annoyed,My brother wont stop laughing at my pimple.,59,I hope so_comma_ because he wont stop annoying me till it does.,5|5|5_4|3|3,
hit:3420_conv:6841,4,annoyed,My brother wont stop laughing at my pimple.,126,That does sound annoying,5|5|5_4|3|3,

hit:3421_conv:6842,1,angry,I was pretty angry when I failed my history final. I should have done better,126,I was angry at myself when I failed a history test. I could have done better,4|5|4_4|4|4,
hit:3421_conv:6842,2,angry,I was pretty angry when I failed my history final. I should have done better,268,oh sorry ,4|5|4_4|4|4,
hit:3421_conv:6842,3,angry,I was pretty angry when I failed my history final. I should have done better,126,Yeah_comma_ it was a bad time,4|5|4_4|4|4,
hit:3421_conv:6842,4,angry,I was pretty angry when I failed my history final. I should have done better,268,did you improve,4|5|4_4|4|4,

hit:3421_conv:6843,1,proud,to best worker,268,how important is your work to you,4|4|4_4|5|4,
hit:3421_conv:6843,2,proud,to best worker,126,I do love to work,4|4|4_4|5|4,
hit:3421_conv:6843,3,proud,to best worker,268,great me too,4|4|4_4|5|4,
hit:3421_conv:6843,4,proud,to best worker,126,I hope you work hard going forward,4|4|4_4|5|4,

hit:3422_conv:6844,1,proud,I got a promotion at work.,59,So I got a big promotion at work. My hard work paid off!,5|5|5_4|3|3,
hit:3422_conv:6844,2,proud,I got a promotion at work.,126,I am happy for you,5|5|5_4|3|3,
hit:3422_conv:6844,3,proud,I got a promotion at work.,59,Thanks_comma_ I am really proud of myself and will celebrate now.,5|5|5_4|3|3,
hit:3422_conv:6844,4,proud,I got a promotion at work.,126,That sounds fun,5|5|5_4|3|3,

hit:3422_conv:6845,1,surprised,I was shocked when my brother said he was moving back home. I was just happy he was,126,I was shocked when my brother moved home. I did not expect him to do that,4|3|3_5|5|5,
hit:3422_conv:6845,2,surprised,I was shocked when my brother said he was moving back home. I was just happy he was,59,Well life can be surprising! Why did he move home?,4|3|3_5|5|5,
hit:3422_conv:6845,3,surprised,I was shocked when my brother said he was moving back home. I was just happy he was,126,He was home sick,4|3|3_5|5|5,
hit:3422_conv:6845,4,surprised,I was shocked when my brother said he was moving back home. I was just happy he was,59,Oh I see_comma_ did he go to college?,4|3|3_5|5|5,

hit:3423_conv:6846,1,embarrassed,I slipped on my wedding day.,59,Hi_comma_ I slipped on my wedding day whilst walking on the aisle.,5|5|5_4|5|5,<HI>
hit:3423_conv:6846,2,embarrassed,I slipped on my wedding day.,126,Were you okay,5|5|5_4|5|5,
hit:3423_conv:6846,3,embarrassed,I slipped on my wedding day.,59,Yea but I got really red faced and embarrassed.,5|5|5_4|5|5,
hit:3423_conv:6846,4,embarrassed,I slipped on my wedding day.,126,I am sure everyone forgot about it,5|5|5_4|5|5,

hit:3423_conv:6847,1,confident,I just knew I was going to pass my science test. I studied hard for it,126,I knew I was going to pass a history test. I studied hard for it,4|5|5_5|5|5,
hit:3423_conv:6847,2,confident,I just knew I was going to pass my science test. I studied hard for it,59,You must have been well prepared for it! Good on you,4|5|5_5|5|5,
hit:3423_conv:6847,3,confident,I just knew I was going to pass my science test. I studied hard for it,126,I was and it turned out well,4|5|5_5|5|5,
hit:3423_conv:6847,4,confident,I just knew I was going to pass my science test. I studied hard for it,59,Did you get an A?,4|5|5_5|5|5,

hit:3424_conv:6848,1,surprised,My friends threw me a party on my birthday.,59,I came home from work and found out my friends threw me a party!,5|4|5_4|5|4,
hit:3424_conv:6848,2,surprised,My friends threw me a party on my birthday.,126,I bet that was fun,5|4|5_4|5|4,
hit:3424_conv:6848,3,surprised,My friends threw me a party on my birthday.,59,It was_comma_ I even drank some wine because I was so surprised! ,5|4|5_4|5|4,
hit:3424_conv:6848,4,surprised,My friends threw me a party on my birthday.,126,Wine can be good,5|4|5_4|5|4,

hit:3424_conv:6849,1,disappointed,I was disappointed when my parents were getting a divorce. I wanted better for them,126,I was saddened when my parents were getting a divorce. I wanted better for them,4|5|4_5|4|5,
hit:3424_conv:6849,2,disappointed,I was disappointed when my parents were getting a divorce. I wanted better for them,59,That is sad to hear. But these things happen in life,4|5|4_5|4|5,
hit:3424_conv:6849,3,disappointed,I was disappointed when my parents were getting a divorce. I wanted better for them,126,That is true,4|5|4_5|4|5,
hit:3424_conv:6849,4,disappointed,I was disappointed when my parents were getting a divorce. I wanted better for them,59,You just have to move on and be positive!,4|5|4_5|4|5,

hit:3425_conv:6850,1,angry,My favorite team lost the final.,59,My favorite soccer team just lost their cup final. ,5|5|4_4|3|3,
hit:3425_conv:6850,2,angry,My favorite team lost the final.,126,Thats not good_comma_ at least they got there,5|5|4_4|3|3,
hit:3425_conv:6850,3,angry,My favorite team lost the final.,59,Yep but I am still angry because they did not win the trophy! ,5|5|4_4|3|3,
hit:3425_conv:6850,4,angry,My favorite team lost the final.,126,There is always next year,5|5|4_4|3|3,

hit:3425_conv:6851,1,surprised,I was surprised but happy when my dad got me my first truck. I was happy,126,I was surprised when my dad got me my first car. I was not expecting it,4|3|3_5|5|4,
hit:3425_conv:6851,2,surprised,I was surprised but happy when my dad got me my first truck. I was happy,59,That must have been exciting for you. What car was it?,4|3|3_5|5|4,
hit:3425_conv:6851,3,surprised,I was surprised but happy when my dad got me my first truck. I was happy,126,It was a nissan frontier,4|3|3_5|5|4,
hit:3425_conv:6851,4,surprised,I was surprised but happy when my dad got me my first truck. I was happy,59,Nice do you still have it or did you sell it?,4|3|3_5|5|4,

hit:3426_conv:6852,1,impressed,I was impressed at this workshop I went to over the weekend_comma_ I heard some good speeches,126,I was impressed at what I heard at a workshop last week. There were some good speeches,4|5|4_4|4|4,
hit:3426_conv:6852,2,impressed,I was impressed at this workshop I went to over the weekend_comma_ I heard some good speeches,268,that's cool what was it a about,4|5|4_4|4|4,
hit:3426_conv:6852,3,impressed,I was impressed at this workshop I went to over the weekend_comma_ I heard some good speeches,126,It was just work stuff that was motivating,4|5|4_4|4|4,
hit:3426_conv:6852,4,impressed,I was impressed at this workshop I went to over the weekend_comma_ I heard some good speeches,268,ok cool how was it,4|5|4_4|4|4,

hit:5996_conv:11992,1,content,I was tired when I got home from work yesterday so I just sat on the couch for the rest of the evening and watched tv.,244,"I was tired when I got home from work yesterday so I just sat on the couch for the rest of the evening and watched tv.,5|5|5_5|5|5,
hit:5996_conv:11992,2,content,I was tired when I got home from work yesterday so I just sat on the couch for the rest of the evening and watched tv.,561,Are you glad you watched TV all night or did you wish you'd done something else?,5|5|5_5|5|5,
hit:5996_conv:11992,3,content,I was tired when I got home from work yesterday so I just sat on the couch for the rest of the evening and watched tv.,244,I was pretty happy to just watch tv and relax,5|5|5_5|5|5,
hit:5996_conv:11992,4,content,I was tired when I got home from work yesterday so I just sat on the couch for the rest of the evening and watched tv.,561,Oh_comma_ I know how that feels. I've done that myself nearly every night.,5|5|5_5|5|5,

hit:5996_conv:11993,1,sad,I found out my niece may be autistic. ,561,I recently found out that my niece may be autistic.,5|5|5_5|5|5,
hit:5996_conv:11993,2,sad,I found out my niece may be autistic. ,244,I'm sorry to hear that. have they done any tests?,5|5|5_5|5|5,
hit:5996_conv:11993,3,sad,I found out my niece may be autistic. ,561,She's currently going to appointments and things. So they'll find out more soon. ,5|5|5_5|5|5,
hit:5996_conv:11993,4,sad,I found out my niece may be autistic. ,244,I hope everything turns out for the better.,5|5|5_5|5|5,

hit:5997_conv:11994,1,disgusted,i felt so irritated with his presence,25,we plan to meet at the party just to see my date so unkept and unpresentable,4|4|4_4|4|4,
hit:5997_conv:11994,2,disgusted,i felt so irritated with his presence,422,Well that's unfortunate. Do you think the party will be fun?,4|4|4_4|4|4,
hit:5997_conv:11994,3,disgusted,i felt so irritated with his presence,25,No_comma_ not after i got irritated by him,4|4|4_4|4|4,
hit:5997_conv:11994,4,disgusted,i felt so irritated with his presence,422,Oh that sucks. I hope you guys can work it out.,4|4|4_4|4|4,

hit:5997_conv:11995,1,anticipating,I'm going to Virginia next weekend. I'm really looking forward to the trip!,422,I'm going out of town next weekend with my husband and no kids. I can't wait!,4|4|4_4|4|4,
hit:5997_conv:11995,2,anticipating,I'm going to Virginia next weekend. I'm really looking forward to the trip!,25,wow! that's what i call couple day out!,4|4|4_4|4|4,
hit:5997_conv:11995,3,anticipating,I'm going to Virginia next weekend. I'm really looking forward to the trip!,422,Yeah. I'm taking a couple days off. We're going to Virginia. I'm really looking forward to it!,4|4|4_4|4|4,
hit:5997_conv:11995,4,anticipating,I'm going to Virginia next weekend. I'm really looking forward to the trip!,25,can i come with my wife too?,4|4|4_4|4|4,

hit:5998_conv:11997,1,impressed,I have only been at my company for one year and I got promoted last week. Im really surprised about myself.,297,I got promoted at work and I have only been there for about a year. I am really happy and surprised with what I have achieved.,4|4|5_5|4|4,
hit:5998_conv:11997,2,impressed,I have only been at my company for one year and I got promoted last week. Im really surprised about myself.,44,thats amazing_comma_ im happy for you too,4|4|5_5|4|4,
hit:5998_conv:11997,3,impressed,I have only been at my company for one year and I got promoted last week. Im really surprised about myself.,297,I am very proud of myself and somewhat surprised.,4|4|5_5|4|4,
hit:5998_conv:11997,4,impressed,I have only been at my company for one year and I got promoted last week. Im really surprised about myself.,44,keep up the good work,4|4|5_5|4|4,

hit:5999_conv:11999,1,impressed,A friend of mine is going back to school to get her second masters degree. She's setting the bar really high!,422,A friend of mine is setting the bar really high by going to get her second masters degree.,5|5|5_5|5|5,
hit:5999_conv:11999,2,impressed,A friend of mine is going back to school to get her second masters degree. She's setting the bar really high!,313,Oh man! Good luck to her_comma_ are you envious?,5|5|5_5|5|5,
hit:5999_conv:11999,3,impressed,A friend of mine is going back to school to get her second masters degree. She's setting the bar really high!,422,I am_comma_ but I'm inspired more than anything. She's setting such a great example.,5|5|5_5|5|5,
hit:5999_conv:11999,4,impressed,A friend of mine is going back to school to get her second masters degree. She's setting the bar really high!,313,I'm so glad that you're happy for her! Start the climb yourself! Maybe ask her for some help?,5|5|5_5|5|5,

hit:6000_conv:12000,1,guilty,Last week I took $20 from my wifes wallet. I havent told her about it but I feel bad,506,Last week I took $20 from my wifes wallet. ,3|3|3_|5|5,
hit:6000_conv:12000,2,guilty,Last week I took $20 from my wifes wallet. I havent told her about it but I feel bad,43,did you inform her?,3|3|3_|5|5,
hit:6000_conv:12000,3,guilty,Last week I took $20 from my wifes wallet. I havent told her about it but I feel bad,506,I havent told her about it but I feel bad,3|3|3_|5|5,
hit:6000_conv:12000,4,guilty,Last week I took $20 from my wifes wallet. I havent told her about it but I feel bad,43,Apologize to her. Don't feel guilty.,3|3|3_|5|5,

hit:6000_conv:12001,1,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,43,I bought a plant last week for my garden.,|5|5_3|3|3,
hit:6000_conv:12001,2,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,506,What kind is it?,|5|5_3|3|3,
hit:6000_conv:12001,3,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,43,Yeah_comma_ thank you!!,|5|5_3|3|3,
hit:6000_conv:12001,4,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,506,You are quite welcome,|5|5_3|3|3,
hit:6000_conv:12001,5,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,43,5,|5|5_3|3|3,<UNIGRAM> <NUMERAL>

hit:6001_conv:12002,1,impressed,I went to a soccer game the other night.  the talent that some of these players had was amazing,301,I am always amazed when I see people who poses raw talent,5|5|5_5|5|5,
hit:6001_conv:12002,2,impressed,I went to a soccer game the other night.  the talent that some of these players had was amazing,506,Can you give an example as to someone who amazed you recently,5|5|5_5|5|5,
hit:6001_conv:12002,3,impressed,I went to a soccer game the other night.  the talent that some of these players had was amazing,301,I was at a soccer game recently.  These guys were really talented with what they could do with the ball,5|5|5_5|5|5,
hit:6001_conv:12002,4,impressed,I went to a soccer game the other night.  the talent that some of these players had was amazing,506,Have you ever played yourself?,5|5|5_5|5|5,

hit:6001_conv:12003,1,grateful,My parents gifted me $50_comma_000 to start my own business. I have never been so appreciative in my life,506,My parents gifted me $50_comma_000 to start my own business. ,5|5|5_5|5|5,
hit:6001_conv:12003,2,grateful,My parents gifted me $50_comma_000 to start my own business. I have never been so appreciative in my life,301,Wow! that was incredibly nice of them.  What kind of business is it?,5|5|5_5|5|5,
hit:6001_conv:12003,3,grateful,My parents gifted me $50_comma_000 to start my own business. I have never been so appreciative in my life,506,It is a salon_comma_ I have never been so appreciative in my life,5|5|5_5|5|5,
hit:6001_conv:12003,4,grateful,My parents gifted me $50_comma_000 to start my own business. I have never been so appreciative in my life,301,I need a haircut soon myself.  I hope you do really well,5|5|5_5|5|5,

hit:6002_conv:12004,1,devastated,I just found out that my employer is laying me off!  This will destroy my finances!  The is the worst news I could get!,277,I just found out that my employer is laying me off!  This will destroy my finances!This is the worst news I could get!,5|5|5_5|5|5,
hit:6002_conv:12004,2,devastated,I just found out that my employer is laying me off!  This will destroy my finances!  The is the worst news I could get!,225,How long do you have ?,5|5|5_5|5|5,
hit:6002_conv:12004,3,devastated,I just found out that my employer is laying me off!  This will destroy my finances!  The is the worst news I could get!,277,Only until the end of August!  It is horrible timing!  I just purchased a house!,5|5|5_5|5|5,
hit:6002_conv:12004,4,devastated,I just found out that my employer is laying me off!  This will destroy my finances!  The is the worst news I could get!,225,No_comma_ it's good timing. If you had waited_comma_ you wouldn't have been approved for the mortgage. Don't forget to file for unemployment insurance.,5|5|5_5|5|5,

hit:6002_conv:12005,1,proud,I'm pleased with my two daughters school work.,225,My two daughters are off to a good start with school this year.,5|5|5_5|5|5,
hit:6002_conv:12005,2,proud,I'm pleased with my two daughters school work.,277,That is always good!  What happened?,5|5|5_5|5|5,
hit:6002_conv:12005,3,proud,I'm pleased with my two daughters school work.,225,Nothing_comma_ it's completely normal. I'm just happy about it.,5|5|5_5|5|5,
hit:6002_conv:12005,4,proud,I'm pleased with my two daughters school work.,277,Normal is a great place to be!  There is something to be said for just being happy and content with what you have!,5|5|5_5|5|5,

hit:6003_conv:12006,1,terrified,I feel like our country is becoming unsafe each day. I am afraid to step outside sometimes,506,I feel like our country is becoming unsafe each day,5|5|5_5|5|5,
hit:6003_conv:12006,2,terrified,I feel like our country is becoming unsafe each day. I am afraid to step outside sometimes,422,I don't disagree. Why do you feel that way?,5|5|5_5|5|5,
hit:6003_conv:12006,3,terrified,I feel like our country is becoming unsafe each day. I am afraid to step outside sometimes,506,Just seems like tensions are building from all of these foreign polices and newly imposed tariffs,5|5|5_5|5|5,
hit:6003_conv:12006,4,terrified,I feel like our country is becoming unsafe each day. I am afraid to step outside sometimes,422,I can see that. There's a lot to be worried about.,5|5|5_5|5|5,

hit:6003_conv:12007,1,annoyed,It's election season and I keep getting robocalls. It's driving me crazy!,422,Man_comma_ these candidate robo-calls are driving me nuts!,5|5|5_5|5|5,
hit:6003_conv:12007,2,annoyed,It's election season and I keep getting robocalls. It's driving me crazy!,506,How many do you get per day?,5|5|5_5|5|5,
hit:6003_conv:12007,3,annoyed,It's election season and I keep getting robocalls. It's driving me crazy!,422,Some days I get 3 or 4. I don't even know how they got my number. I'll be so glad when election season is over!,5|5|5_5|5|5,<POLITICAL>
hit:6003_conv:12007,4,annoyed,It's election season and I keep getting robocalls. It's driving me crazy!,506,Should be ending soon and you can go back to your normal life without those pesky calls.,5|5|5_5|5|5,

hit:6004_conv:12008,1,furious,Last Halloween a few kids Egged my house. I was so angry I wanted to catch them!,506,Last Halloween a few kids Egged my house.,5|5|5_5|5|5,
hit:6004_conv:12008,2,furious,Last Halloween a few kids Egged my house. I was so angry I wanted to catch them!,301,Thats super mean of them.  Did you give them bad candy?,5|5|5_5|5|5,
hit:6004_conv:12008,3,furious,Last Halloween a few kids Egged my house. I was so angry I wanted to catch them!,506,I didnt catch them in time so I couldnt get my revenge.,5|5|5_5|5|5,
hit:6004_conv:12008,4,furious,Last Halloween a few kids Egged my house. I was so angry I wanted to catch them!,301,Too bad_comma_ You couldve done something really good to them.,5|5|5_5|5|5,

hit:6004_conv:12009,1,surprised,I went to the store to get groceries and I won a free $20 gift card in a contest.  I couldnt believe it,301,I never win anything but my luck just changed,5|5|5_5|5|5,
hit:6004_conv:12009,2,surprised,I went to the store to get groceries and I won a free $20 gift card in a contest.  I couldnt believe it,506,How did it change?,5|5|5_5|5|5,
hit:6004_conv:12009,3,surprised,I went to the store to get groceries and I won a free $20 gift card in a contest.  I couldnt believe it,301,I went to the store to buy some groceries and I won a $20 gift card in a contest.  I was shocked,5|5|5_5|5|5,
hit:6004_conv:12009,4,surprised,I went to the store to get groceries and I won a free $20 gift card in a contest.  I couldnt believe it,506,Well congratulations_comma_ spoil yourself with it!,5|5|5_5|5|5,

hit:6005_conv:12010,1,lonely,I moved to a new city last month. I have no one around me anymore,506,I moved to a new city last month.,5|5|5_5|5|5,
hit:6005_conv:12010,2,lonely,I moved to a new city last month. I have no one around me anymore,438,Exciting!  How are you settling in?  It might take a while but you'll start making friends and setting up a community around you.,5|5|5_5|5|5,
hit:6005_conv:12010,3,lonely,I moved to a new city last month. I have no one around me anymore,506, I have no one around me anymore so its left me a bit depressed,5|5|5_5|5|5,
hit:6005_conv:12010,4,lonely,I moved to a new city last month. I have no one around me anymore,438,I can understand that.  It is usually very isolating to move_comma_ especially if its away from family and friends.  I did that as well and found I had to consistently get out and try to meet people.  It wasn't always a success but after about 2 months_comma_ I started to have things to do.  Keep your chin up and go explore your new city!,5|5|5_5|5|5,

hit:6005_conv:12011,1,grateful,I have been ill with an autoimmune disease for years but just changed my diet to plant based and am feeling so great and healing for the first time since trying everything under the sun.,438,I have been ill with an autoimmune disease for years but just changed my diet to plant based and am feeling so great and healing for the first time since trying everything under the sun.,5|5|5_5|5|5,
hit:6005_conv:12011,2,grateful,I have been ill with an autoimmune disease for years but just changed my diet to plant based and am feeling so great and healing for the first time since trying everything under the sun.,506,That is great to hear. Will you be taking advantage of your life now?,5|5|5_5|5|5,
hit:6005_conv:12011,3,grateful,I have been ill with an autoimmune disease for years but just changed my diet to plant based and am feeling so great and healing for the first time since trying everything under the sun.,438,I already am!  Though my muscles are still weak_comma_ I am starting to nike and bike with my family and am wanting to get the whole family into yoga too.  I am so thankful to have a new lease on life and it will only get better now.,5|5|5_5|5|5,
hit:6005_conv:12011,4,grateful,I have been ill with an autoimmune disease for years but just changed my diet to plant based and am feeling so great and healing for the first time since trying everything under the sun.,506,That is great_comma_ I am so happy for you!,5|5|5_5|5|5,

hit:6006_conv:12012,1,ashamed,I cheated on a Spanish test in High school. I felt so bad about doing it.,506,I cheated on a Spanish test in High school.,5|5|5_5|5|5,
hit:6006_conv:12012,2,ashamed,I cheated on a Spanish test in High school. I felt so bad about doing it.,44,did you get caught,5|5|5_5|5|5,
hit:6006_conv:12012,3,ashamed,I cheated on a Spanish test in High school. I felt so bad about doing it.,506,No_comma_ but I do feel bad about doing it,5|5|5_5|5|5,
hit:6006_conv:12012,4,ashamed,I cheated on a Spanish test in High school. I felt so bad about doing it.,44,its okay,5|5|5_5|5|5,

hit:6007_conv:12014,1,lonely,My daughter has been at her grandmother's all week. The house is quiet_comma_ and I miss her terribly.,422,My daughter has been at her grandparents' house this week. The house is so quiet and empty.,5|5|5_4|5|5,
hit:6007_conv:12014,2,lonely,My daughter has been at her grandmother's all week. The house is quiet_comma_ and I miss her terribly.,507,Do you find some along time good? do you like being alone every once in a while?,5|5|5_4|5|5,
hit:6007_conv:12014,3,lonely,My daughter has been at her grandmother's all week. The house is quiet_comma_ and I miss her terribly.,422,I thought I would enjoy it much more than I did. I just miss her like crazy.,5|5|5_4|5|5,
hit:6007_conv:12014,4,lonely,My daughter has been at her grandmother's all week. The house is quiet_comma_ and I miss her terribly.,507,I can imagine_comma_ when my daughter leaves with her friends on the weekend i miss her like crazy too,5|5|5_4|5|5,

hit:6008_conv:12016,1,sentimental,I keep a baseball card of my grandfather in my wallet at all times. He meant so much to me and I am proud of his accomplishments,506,I keep a baseball card of my grandfather in my wallet at all times.,5|5|5_4|3|4,
hit:6008_conv:12016,2,sentimental,I keep a baseball card of my grandfather in my wallet at all times. He meant so much to me and I am proud of his accomplishments,25,whats that for?,5|5|5_4|3|4,
hit:6008_conv:12016,3,sentimental,I keep a baseball card of my grandfather in my wallet at all times. He meant so much to me and I am proud of his accomplishments,506,He meant so much to me and I am proud of his accomplishments,5|5|5_4|3|4,
hit:6008_conv:12016,4,sentimental,I keep a baseball card of my grandfather in my wallet at all times. He meant so much to me and I am proud of his accomplishments,25,So what did he accomplish?,5|5|5_4|3|4,

hit:6008_conv:12017,1,angry,I had an important business meeting to attend_comma_got to the airport _comma_only to find out that my flight had been cancelled,25,i requested A REFUND TO PICK THE NEXT AVAILABLE FLIGHT _comma_YET IT WAS DECLINED,4|3|4_5|5|5,
hit:6008_conv:12017,2,angry,I had an important business meeting to attend_comma_got to the airport _comma_only to find out that my flight had been cancelled,506,Could you fight them more for the refund?,4|3|4_5|5|5,
hit:6008_conv:12017,3,angry,I had an important business meeting to attend_comma_got to the airport _comma_only to find out that my flight had been cancelled,25,it got me screaming at such insensitivity,4|3|4_5|5|5,
hit:6008_conv:12017,4,angry,I had an important business meeting to attend_comma_got to the airport _comma_only to find out that my flight had been cancelled,506,I am sure_comma_ I hope you can get a resolution,4|3|4_5|5|5,

hit:6009_conv:12018,1,disappointed,I found out that my father cheated on my mother. I felt so betrayed by him.,506,I found out that my father cheated on my mother.,5|5|5_5|5|5,
hit:6009_conv:12018,2,disappointed,I found out that my father cheated on my mother. I felt so betrayed by him.,244,That sucks.  My parents are divorced.  Same reason.,5|5|5_5|5|5,
hit:6009_conv:12018,3,disappointed,I found out that my father cheated on my mother. I felt so betrayed by him.,506,I felt so betrayed by him,5|5|5_5|5|5,
hit:6009_conv:12018,4,disappointed,I found out that my father cheated on my mother. I felt so betrayed by him.,244,Its never good for anyone. ,5|5|5_5|5|5,

hit:6009_conv:12019,1,sentimental,I found the ID braclet I wore in the hospital when I was born.,244,I found the ID braclet I wore in the hospital when I was born.,5|5|5_5|5|5,
hit:6009_conv:12019,2,sentimental,I found the ID braclet I wore in the hospital when I was born.,506,That is awesome_comma_ keep it in a safe place.,5|5|5_5|5|5,
hit:6009_conv:12019,3,sentimental,I found the ID braclet I wore in the hospital when I was born.,244,I will. I hope I don't lose it again.,5|5|5_5|5|5,
hit:6009_conv:12019,4,sentimental,I found the ID braclet I wore in the hospital when I was born.,506,I do not think you will anymore,5|5|5_5|5|5,
hit:6009_conv:12019,5,sentimental,I found the ID braclet I wore in the hospital when I was born.,244,I've got a good spot to keep it in now.,5|5|5_5|5|5,

hit:6010_conv:12020,1,anticipating,Each week I look forward to one day and one day only. Fridays are what get us all through the rest of the week,506,Each week I look forward to one day and one day only. ,5|5|5_5|5|5,
hit:6010_conv:12020,2,anticipating,Each week I look forward to one day and one day only. Fridays are what get us all through the rest of the week,243,What day is that?,5|5|5_5|5|5,
hit:6010_conv:12020,3,anticipating,Each week I look forward to one day and one day only. Fridays are what get us all through the rest of the week,506,Friday! Fridays are what get us all through the rest of the week,5|5|5_5|5|5,
hit:6010_conv:12020,4,anticipating,Each week I look forward to one day and one day only. Fridays are what get us all through the rest of the week,243,Yes Fridays are wonderful!,5|5|5_5|5|5,

hit:6010_conv:12021,1,surprised,My coworkers had a birthday party for my last week.,243,My coworkers had a birthday party for me last week.,5|5|5_5|5|5,
hit:6010_conv:12021,2,surprised,My coworkers had a birthday party for my last week.,506,Was it a surprise?,5|5|5_5|5|5,
hit:6010_conv:12021,3,surprised,My coworkers had a birthday party for my last week.,243,Yes! Somehow they planned it all without me knowing!,5|5|5_5|5|5,
hit:6010_conv:12021,4,surprised,My coworkers had a birthday party for my last week.,506,That is such a great feeling_comma_ they must really care!,5|5|5_5|5|5,

hit:6011_conv:12022,1,nostalgic,A moment ago I was thinking about my entire Pokemon collection I had as a child,506,A moment ago I was thinking about my entire Pokemon collection I had as a child,5|5|5_|5|5,
hit:6011_conv:12022,2,nostalgic,A moment ago I was thinking about my entire Pokemon collection I had as a child,43,it's a rose plant. I'm waiting for the blossom of the first flower in that plant.,5|5|5_|5|5,
hit:6011_conv:12022,3,nostalgic,A moment ago I was thinking about my entire Pokemon collection I had as a child,506,I am sure it will look beautiful,5|5|5_|5|5,

hit:6011_conv:12023,1,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,43,4,|5|5_5|5|5,<UNIGRAM> <NUMERAL>
hit:6011_conv:12023,2,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,506,Could you explain?,|5|5_5|5|5,
hit:6011_conv:12023,3,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,43,i bought a plant last week,|5|5_5|5|5,
hit:6011_conv:12023,4,anxious,I bought a plant last week. I'm eagerly waiting to see the flower in it.,506,Where did you buy it?,|5|5_5|5|5,

hit:6012_conv:12024,1,prepared,I stayed up all night studying for this test.  I dont think I could be more ready than I am right now,301,I have never felt more ready for something than I do right now,5|5|5_4|5|5,
hit:6012_conv:12024,2,prepared,I stayed up all night studying for this test.  I dont think I could be more ready than I am right now,438,That's awesome!  Anything particular you're ready to accomplish?,5|5|5_4|5|5,
hit:6012_conv:12024,3,prepared,I stayed up all night studying for this test.  I dont think I could be more ready than I am right now,301,I was up all night studying for this history test.  Ive got this!,5|5|5_4|5|5,
hit:6012_conv:12024,4,prepared,I stayed up all night studying for this test.  I dont think I could be more ready than I am right now,438,That's great.  I love the feeling of being prepared and ready to crush something!  History can be tough too with all those dates.,5|5|5_4|5|5,

hit:6012_conv:12025,1,impressed,I went to a piano recital the other day and watched all the younger kids play.  Then a young man started playing and he played the most complicated piece I've ever heard!  Its amazing the talent some people have and amazing how people can write that kind of music too!,438,I went to a piano recital the other day and watched all the younger kids play. Then a young man started playing and he played the most complicated piece I've ever heard! Its amazing the talent some people have and amazing how people can write that kind of music too!,4|5|5_5|5|5,
hit:6012_conv:12025,2,impressed,I went to a piano recital the other day and watched all the younger kids play.  Then a young man started playing and he played the most complicated piece I've ever heard!  Its amazing the talent some people have and amazing how people can write that kind of music too!,301,Wow thats awesome.  I tried learning the piano once.  I wasnt good.  So i can really appreciate good talent.  Im always impressed,4|5|5_5|5|5,
hit:6012_conv:12025,3,impressed,I went to a piano recital the other day and watched all the younger kids play.  Then a young man started playing and he played the most complicated piece I've ever heard!  Its amazing the talent some people have and amazing how people can write that kind of music too!,438,I took piano for 7 years and was pretty good but never really wanted to use it for anything so it kind of wasn't worth it.  I think this guy is trying to get into an orchestra.  It is inspiring!,4|5|5_5|5|5,
hit:6012_conv:12025,4,impressed,I went to a piano recital the other day and watched all the younger kids play.  Then a young man started playing and he played the most complicated piece I've ever heard!  Its amazing the talent some people have and amazing how people can write that kind of music too!,301,Yea he must be really good then.  7 years is a long time.  I bet you can impress people with your skills,4|5|5_5|5|5,

hit:6013_conv:12027,1,anticipating,I have been through a lot in my life_comma_ but I am anticipating that God will bless me.,540,I've been through a lot of hard times in life_comma_ but I know that God will bless me soon.,5|4|5_5|5|5,
hit:6013_conv:12027,2,anticipating,I have been through a lot in my life_comma_ but I am anticipating that God will bless me.,507,i bet he will_comma_ everything always gets better dont worry,5|4|5_5|5|5,
hit:6013_conv:12027,3,anticipating,I have been through a lot in my life_comma_ but I am anticipating that God will bless me.,540,Thank you for that encouragement. I am trying to be as patient as I can be during these hard times.,5|4|5_5|5|5,
hit:6013_conv:12027,4,anticipating,I have been through a lot in my life_comma_ but I am anticipating that God will bless me.,507,like some one once said_comma_ time heals everything just have a positive attitude,5|4|5_5|5|5,

hit:6014_conv:12028,1,devastated,I was left at the alter by the bride to be this weekend.,294,I was left at the alter by the bride to be this weekend.,5|5|5_5|5|5,
hit:6014_conv:12028,2,devastated,I was left at the alter by the bride to be this weekend.,422,Oh no! I'm so sorry to hear that. That must have been awful.,5|5|5_5|5|5,
hit:6014_conv:12028,3,devastated,I was left at the alter by the bride to be this weekend.,294,It was and still is.  I was left with the task of returning all the gifts.  All this in the midst of the worst pain I've ever felt in my life.,5|5|5_5|5|5,
hit:6014_conv:12028,4,devastated,I was left at the alter by the bride to be this weekend.,422,I can't even imagine what you're going through. I hope your friends and family can help you.,5|5|5_5|5|5,

hit:6014_conv:12029,1,angry,My doctor cancelled my appointment an hour before it was scheduled. Now I have to wait 2 weeks for a new one at a worse time. I'm so mad.,422,I had a doctor's appointment the other day and they cancelled it an hour before it was scheduled. I hate that!,5|5|5_5|5|5,
hit:6014_conv:12029,2,angry,My doctor cancelled my appointment an hour before it was scheduled. Now I have to wait 2 weeks for a new one at a worse time. I'm so mad.,294,They should be required to pay you for that.  They would charge you if you tried to cancel that close to the appointment time.  So annoying!  Did they give you a reason?,5|5|5_5|5|5,
hit:6014_conv:12029,3,angry,My docto1r cancelled my appointment an hour before it was scheduled. Now I have to wait 2 weeks for a new one at a worse time. I'm so mad.,422,I agree! No reason. And now I have to wait another two weeks for a new appointment. I'm so mad!,5|5|5_5|5|5,
hit:6014_conv:12029,4,angry,My doctor cancelled my appointment an hour before it was scheduled. Now I have to wait 2 weeks for a new one at a worse time. I'm so mad.,294,Ridiculous.  Doctor's think that they are above everyone else and it drives me crazy!,5|5|5_5|5|5,

hit:6015_conv:12030,1,disappointed,As a politician i was promised a political seat by my party_comma_but they reneged on their promise,25,i was truly a party faithful_comma_had work rigorously for the election my party won,4|3|4_5|5|5,<POLITICAL>
hit:6015_conv:12030,2,disappointed,As a politician i was promised a political seat by my party_comma_but they reneged on their promise,313,Amazing! I'm so proud of you for your dedication! ,4|3|4_5|5|5,
hit:6015_conv:12030,3,disappointed,As a politician i was promised a political seat by my party_comma_but they reneged on their promise,25,You can say that again! but the irony of it was that all the appointments i was promised never came through,4|3|4_5|5|5,
hit:6015_conv:12030,4,disappointed,As a politician i was promised a political seat by my party_comma_but they reneged on their promise,313,Oh dang! That's always the worst isn't it,4|3|4_5|5|5,

hit:6016_conv:12032,1,anxious,I'm anxious for school to start again_comma_ It's my senior year of college!,448,I'm anxious for school to start again_comma_ It's my senior year of college!,5|5|5_5|5|5,
hit:6016_conv:12032,2,anxious,I'm anxious for school to start again_comma_ It's my senior year of college!,422,That's so exciting! It's almost over! But I can understand why it'd make you anxious_comma_ too.,5|5|5_5|5|5,
hit:6016_conv:12032,3,anxious,I'm anxious for school to start again_comma_ It's my senior year of college!,448,Yeah! I can't wait for it to be over with_comma_ but I'm going to have to study hard this last year.,5|5|5_5|5|5,
hit:6016_conv:12032,4,anxious,I'm anxious for school to start again_comma_ It's my senior year of college!,422,Yeah. That last year is really important. But I'm sure you'll do great!,5|5|5_5|5|5,

hit:6016_conv:12033,1,embarrassed,I couldn't remember my friend's husband's name. He knew mine_comma_ so I felt really bad about it.,422,I was at a party with some friends and I could not remember my friend's husband's name to save my life.,5|5|5_5|5|5,
hit:6016_conv:12033,2,embarrassed,I couldn't remember my friend's husband's name. He knew mine_comma_ so I felt really bad about it.,448,Uh oh_comma_ did you try to play it off or did you have to ask your husband?,5|5|5_5|5|5,
hit:6016_conv:12033,3,embarrassed,I couldn't remember my friend's husband's name. He knew mine_comma_ so I felt really bad about it.,422,I tried to play it off_comma_ but I was so embarrassed. I know he's told me what it is multiple times before.,5|5|5_5|5|5,
hit:6016_conv:12033,4,embarrassed,I couldn't remember my friend's husband's name. He knew mine_comma_ so I felt really bad about it.,448,Haha_comma_ that happens_comma_ hopefully he wasn't offended by it or anything.,5|5|5_5|5|5,

hit:6017_conv:12034,1,sad,I was really upset when I watched the final episode of the office.  I felt so connected to those characters.  I cried a little,301,Do you ever get really emotional about people and things youve never met?,5|5|5_5|5|5,
hit:6017_conv:12034,2,sad,I was really upset when I watched the final episode of the office.  I felt so connected to those characters.  I cried a little,438,I certainly do.  I think it shows you care and have compassion for their situations.  So many people lose that compassion and even practice indifference. How about you?,5|5|5_5|5|5,
hit:6017_conv:12034,3,sad,I was really upset when I watched the final episode of the office.  I felt so connected to those characters.  I cried a little,301,I got really upset when I watched the finale of the office.  I felt like I was part of their team and when the whole thing was over I cried a little.  It was like I was losing some friends.,5|5|5_5|5|5,
hit:6017_conv:12034,4,sad,I was really upset when I watched the final episode of the office.  I felt so connected to those characters.  I cried a little,438,I can understand that.  I mean_comma_ you're with them as you watch everything they go through and you start to really like them.  When it ends its like_comma_ was this not real?  I hate coming to the end of shows that run for a long time.  So sad.,5|5|5_5|5|5,

hit:6017_conv:12035,1,caring,I just started taking my younger kids to an elderly assisted living home to try to brighten people's day.  I think its important to show kindness to others and teach my kids that.,438,I just started taking my younger kids to an elderly assisted living home to try to brighten people's day. I think its important to show kindness to others and teach my kids that.,5|5|5_5|5|5,
hit:6017_conv:12035,2,caring,I just started taking my younger kids to an elderly assisted living home to try to brighten people's day.  I think its important to show kindness to others and teach my kids that.,301,Thats really sweet of you and I bet it means the world to those people.  Its great to show kids compassion and being nice to people,5|5|5_5|5|5,
hit:6017_conv:12035,3,caring,I just started taking my younger kids to an elderly assisted living home to try to brighten people's day.  I think its important to show kindness to others and teach my kids that.,438,They really enjoy going there and the people absolutely light up when the kids come up to talk to them. So many of them have no family coming to visit.  I think it also helps kids not be afraid of older people who might have oxygen on or something.,5|5|5_5|5|5,
hit:6017_conv:12035,4,caring,I just started taking my younger kids to an elderly assisted living home to try to brighten people's day.  I think its important to show kindness to others and teach my kids that.,301,I remember being scared the first time I went to visit my great grandmother in a nursing home.  But once you see that they are just people it makes everything alright,5|5|5_5|5|5,

hit:7600_conv:15200,1,terrified,I saw a plane once that was flying so low I thought it was going to crash. I was beyond scared.,548,"I saw a plane once that was flying so low I thought it was going to crash. I was beyond scared.,3|3|5_5|5|5,
hit:7600_conv:15200,2,terrified,I saw a plane once that was flying so low I thought it was going to crash. I was beyond scared.,196,Did it crash?,3|3|5_5|5|5,
hit:7600_conv:15200,3,terrified,I saw a plane once that was flying so low I thought it was going to crash. I was beyond scared.,548,No_comma_ but it sure did look like it was going to.,3|3|5_5|5|5,
hit:7600_conv:15200,4,terrified,I saw a plane once that was flying so low I thought it was going to crash. I was beyond scared.,196,Wow. Must have been terrifying,3|3|5_5|5|5,
hit:7600_conv:15201,1,hopeful,I have a job interview next week. I hope I get it.,196,I have a job interview next week.,5|5|5_3|3|5,
hit:7600_conv:15201,2,hopeful,I have a job interview next week. I hope I get it.,548,That can be exciting. Are you nervous?,5|5|5_3|3|5,
hit:7600_conv:15201,3,hopeful,I have a job interview next week. I hope I get it.,196,A little. I really hope I get it,5|5|5_3|3|5,
hit:7600_conv:15201,4,hopeful,I have a job interview next week. I hope I get it.,548,Have you prepared for any questions that they may ask?,5|5|5_3|3|5,
hit:7601_conv:15202,1,impressed,I was amazed at how beautiful San Sebastian was when I went ,196,I was amazed at how beautiful san sebastian was when I went for the first time,5|5|5_5|5|5,
hit:7601_conv:15202,2,impressed,I was amazed at how beautiful San Sebastian was when I went ,238,what is that ? a town? resort?,5|5|5_5|5|5,
hit:7601_conv:15202,3,impressed,I was amazed at how beautiful San Sebastian was when I went ,196,It's a city on the north coast of Spain. Near the French border. The Spanish royal family have a summer palace there.,5|5|5_5|5|5,
hit:7601_conv:15202,4,impressed,I was amazed at how beautiful San Sebastian was when I went ,238,oh wow that sounds like a wonderful place,5|5|5_5|5|5,
hit:7601_conv:15202,5,impressed,I was amazed at how beautiful San Sebastian was when I went ,196,It really is. The food is amongst the best in teh world and the buildings are beautiful. Not to mention the long beaches and cheap beer,5|5|5_5|5|5,
hit:7601_conv:15203,1,joyful,i was happy to see that i got the day off from work,238,i was happy to see that i got the day off from work,5|5|5_5|5|5,
hit:7601_conv:15203,2,joyful,i was happy to see that i got the day off from work,196,Well done. What are you going to do with it?,5|5|5_5|5|5,
hit:7601_conv:15203,3,joyful,i was happy to see that i got the day off from work,238,im going to go clubbing!,5|5|5_5|5|5,
hit:7601_conv:15203,4,joyful,i was happy to see that i got the day off from work,196,During the day?!!?,5|5|5_5|5|5,
hit:7602_conv:15204,1,joyful,I am going to the park later.  I am so happy about it.,322,I am going to the park later!,4|4|4_5|5|5,
hit:7602_conv:15204,2,joyful,I am going to the park later.  I am so happy about it.,377,I love the park! What are you going to do there?,4|4|4_5|5|5,
hit:7602_conv:15204,3,joyful,I am going to the park later.  I am so happy about it.,322,I am going to relax.  I am so happy about it!,4|4|4_5|5|5,
hit:7602_conv:15204,4,joyful,I am going to the park later.  I am so happy about it.,377,Have fun! It's raining where I live_comma_ so I'll be stuck indoors all day.,4|4|4_5|5|5,
hit:7602_conv:15205,1,annoyed,I was supposed to meet my mother for lunch yesterday_comma_ and she never showed up. I sat by myself for 30 minutes.,377,I was supposed to meet my mother for lunch yesterday_comma_ and she never showed up. What a pain!,5|5|5_4|4|4,
hit:7602_conv:15205,2,annoyed,I was supposed to meet my mother for lunch yesterday_comma_ and she never showed up. I sat by myself for 30 minutes.,322,I'm sorry that happened.,5|5|5_4|4|4,
hit:7602_conv:15205,3,annoyed,I was supposed to meet my mother for lunch yesterday_comma_ and she never showed up. I sat by myself for 30 minutes.,377,I sat by myself for 30 minutes. I was not very happy by the end.,5|5|5_4|4|4,
hit:7602_conv:15205,4,annoyed,I was supposed to meet my mother for lunch yesterday_comma_ and she never showed up. I sat by myself for 30 minutes.,322,I hope it goes better next time.,5|5|5_4|4|4,
hit:7603_conv:15206,1,ashamed,I just ate four donuts myself.,196,I just ate 4 donuts by myself,5|5|5_5|5|5,
hit:7603_conv:15206,2,ashamed,I just ate four donuts myself.,238,i bet those tasted mighty good!,5|5|5_5|5|5,
hit:7603_conv:15206,3,ashamed,I just ate four donuts myself.,196,They certainly did. And there's one left.,5|5|5_5|5|5,
hit:7603_conv:15206,4,ashamed,I just ate four donuts myself.,238,its all about number one.,5|5|5_5|5|5,
hit:7603_conv:15207,1,anxious,i cant wait for next week!,238,i cant wait for next week!,5|5|5_5|5|5,
hit:7603_conv:15207,2,anxious,i cant wait for next week!,196,What are you doing next week?,5|5|5_5|5|5,
hit:7603_conv:15207,3,anxious,i cant wait for next week!,238,well the kids are going to school.,5|5|5_5|5|5,
hit:7603_conv:15207,4,anxious,i cant wait for next week!,196,Ah - so you can relax. With a bottle of wine.,5|5|5_5|5|5,
hit:7604_conv:15208,1,devastated,My parrot doesn't talk anymore.  I am so sad thinking about it.,322,My parrot doesn't talk anymore.  ,5|5|5_5|5|5,
hit:7604_conv:15208,2,devastated,My parrot doesn't talk anymore.  I am so sad thinking about it.,196,Oh. Do you know why?,5|5|5_5|5|5,
hit:7604_conv:15208,3,devastated,My parrot doesn't talk anymore.  I am so sad thinking about it.,322,No_comma_ I am so sad thinking about it.,5|5|5_5|5|5,
hit:7604_conv:15208,4,devastated,My parrot doesn't talk anymore.  I am so sad thinking about it.,196,Maybe it needs some company. I read that parrots like to have another bird around.,5|5|5_5|5|5,
hit:7604_conv:15209,1,excited,I'm looking forward to the Spanish Supercopa tonight,196,I'm looking forward to teh Spanish Supercopa tonight,5|5|5_5|5|5,
hit:7604_conv:15209,2,excited,I'm looking forward to the Spanish Supercopa tonight,322,Oh_comma_ I love soccer!,5|5|5_5|5|5,
hit:7604_conv:15209,3,excited,I'm looking forward to the Spanish Supercopa tonight,196,Who do you support?,5|5|5_5|5|5,
hit:7604_conv:15209,4,excited,I'm looking forward to the Spanish Supercopa tonight,322,I like Manchester City!,5|5|5_5|5|5,
hit:7604_conv:15209,5,excited,I'm looking forward to the Spanish Supercopa tonight,196,They looked good last week at Wembley.,5|5|5_5|5|5,
hit:7605_conv:15210,1,content,The sun is shining and I have no plans,196,The sun is shining and I have the day to myself.,5|5|5_4|5|5,
hit:7605_conv:15210,2,content,The sun is shining and I have no plans,322,I am so envious of you.,5|5|5_4|5|5,
hit:7605_conv:15210,3,content,The sun is shining and I have no plans,196,Yup. I'm living the dream,5|5|5_4|5|5,
hit:7605_conv:15210,4,content,The sun is shining and I have no plans,322,What will you do with your free time?,5|5|5_4|5|5,
hit:7605_conv:15210,5,content,The sun is shining and I have no plans,196,I think I'll watch the soccer,5|5|5_4|5|5,
hit:7605_conv:15211,1,disgusted,I saw a guy put mustard on his ice cream.  It was really nasty.,322,I saw a guy put mustard on his ice cream.,4|5|5_5|5|5,
hit:7605_conv:15211,2,disgusted,I saw a guy put mustard on his ice cream.  It was really nasty.,196,Woah! Really?? That's weird.,4|5|5_5|5|5,
hit:7605_conv:15211,3,disgusted,I saw a guy put mustard on his ice cream.  It was really nasty.,322,Yeah_comma_ it was really nasty.,4|5|5_5|5|5,
hit:7605_conv:15211,4,disgusted,I saw a guy put mustard on his ice cream.  It was really nasty.,196,Maybe it tastes amazing though - we'll never know. They put vinegar on ice cream in italy sometimes.,4|5|5_5|5|5,
hit:7606_conv:15212,1,trusting,I told someone something that I didn't want repeated. I had to believe that they would keep what I said private.,548,I told someone something that I didn't want repeated. I had to believe that they would keep what I said private.,4|4|5_5|5|5,
hit:7606_conv:15212,2,trusting,I told someone something that I didn't want repeated. I had to believe that they would keep what I said private.,238,and... im guessing they didnt.,4|4|5_5|5|5,
hit:7606_conv:15212,3,trusting,I told someone something that I didn't want repeated. I had to believe that they would keep what I said private.,548,Actually they did. It's hard to put that kind of faith in someone_comma_ so I'm glad that I did. It gives me hope.,4|4|5_5|5|5,
hit:7606_conv:15212,4,trusting,I told someone something that I didn't want repeated. I had to believe that they would keep what I said private.,238,wow you dont see that everyday. thats a good friend to have by your side,4|4|5_5|5|5,
hit:7606_conv:15213,1,confident,i went into that interview knowing i was going to crush it,238,i went into that interview knowing i was going to crush it,5|5|5_4|4|5,
hit:7606_conv:15213,2,confident,i went into that interview knowing i was going to crush it,548,Did it go as well as you had hoped?,5|5|5_4|4|5,
hit:7606_conv:15213,3,confident,i went into that interview knowing i was going to crush it,238,yea i got a job offer just waiting on the background check to come back,5|5|5_4|4|5,
hit:7606_conv:15213,4,confident,i went into that interview knowing i was going to crush it,548,That's great news. Your confidence paid off.,5|5|5_4|4|5,
hit:7607_conv:15214,1,prepared,I have done lots of research for an interview,196,I have been researching for a job interview allweek,5|5|5_5|5|5,
hit:7607_conv:15214,2,prepared,I have done lots of research for an interview,322,What kind of job?,5|5|5_5|5|5,
hit:7607_conv:15214,3,prepared,I have done lots of research for an interview,196,It's in a call center,5|5|5_5|5|5,
hit:7607_conv:15214,4,prepared,I have done lots of research for an interview,322,I wish you the best of luck!,5|5|5_5|5|5,
hit:7607_conv:15214,5,prepared,I have done lots of research for an interview,196,Thanks_comma_ I hope I get it,5|5|5_5|5|5,
hit:7607_conv:15215,1,surprised,I bought a lottery ticket and won 20 dollars.  I am so shocked it happened.,322,I bought a lottery ticket and won 20 dollars!,5|5|5_5|5|5,
hit:7607_conv:15215,2,surprised,I bought a lottery ticket and won 20 dollars.  I am so shocked it happened.,196,Well done! What are you going to spend it on?,5|5|5_5|5|5,
hit:7607_conv:15215,3,surprised,I bought a lottery ticket and won 20 dollars.  I am so shocked it happened.,322,I bought a pizza_comma_ I am so shocked it happened!,5|5|5_5|5|5,
hit:7607_conv:15215,4,surprised,I bought a lottery ticket and won 20 dollars.  I am so shocked it happened.,196,Free pizza tastes good.,5|5|5_5|5|5,
hit:7608_conv:15216,1,anxious,Cinema's freak me out. The silence_comma_ the darkness.,196,I feel so anxious whenever I go to the movies.,5|5|5_5|5|5,
hit:7608_conv:15216,2,anxious,Cinema's freak me out. The silence_comma_ the darkness.,238,why is that,5|5|5_5|5|5,
hit:7608_conv:15216,3,anxious,Cinema's freak me out. The silence_comma_ the darkness.,196,I think because it's dark and quiet. I worry if I'm talking to much or breathing to loudly. Then I start holding my breath,5|5|5_5|5|5,
hit:7608_conv:15216,4,anxious,Cinema's freak me out. The silence_comma_ the darkness.,238,ah yea i can see that,5|5|5_5|5|5,
hit:7608_conv:15217,1,terrified,i was scared walking home alone last week,238,i was scared walking home last week,5|5|5_5|5|5,
hit:7608_conv:15217,2,terrified,i was scared walking home alone last week,196,Was it a bad area?,5|5|5_5|5|5,
hit:7608_conv:15217,3,terrified,i was scared walking home alone last week,238,it was a dark wooded area,5|5|5_5|5|5,
hit:7608_conv:15217,4,terrified,i was scared walking home alone last week,196,Did anything happen or did you get home safely?,5|5|5_5|5|5,
hit:7609_conv:15218,1,lonely,I went to the mall by myself again.  I hate being by myself.,322,I went to the mall by myself again.,4|4|5_5|5|5,
hit:7609_conv:15218,2,lonely,I went to the mall by myself again.  I hate being by myself.,196,Yeah - did you have a good time?,4|4|5_5|5|5,
hit:7609_conv:15218,3,lonely,I went to the mall by myself again.  I hate being by myself.,322,No_comma_ I hate being by myself.,4|4|5_5|5|5,
hit:7609_conv:15218,4,lonely,I went to the mall by myself again.  I hate being by myself.,196,OH dear. Maybe you can find some friends online,4|4|5_5|5|5,
hit:7609_conv:15219,1,ashamed,I just ate four donuts to myself,196,I just ate 4 donuts by myself,5|5|5_4|4|5,
hit:7609_conv:15219,2,ashamed,I just ate four donuts to myself,322,I'm sorry you don't have someone to eat donuts with.,5|5|5_4|4|5,
hit:7609_conv:15219,3,ashamed,I just ate four donuts to myself,196,I'm not - I just feel ashamed that I ate them,5|5|5_4|4|5,
hit:7609_conv:15219,4,ashamed,I just ate four donuts to myself,322,You can always have a salad today.,5|5|5_4|4|5,
hit:7609_conv:15219,5,ashamed,I just ate four donuts to myself,196,I suppose so ,5|5|5_4|4|5,
hit:7610_conv:15220,1,excited,I'm looking forward to the Spanish Supercopa tonight,196,I'm looking forward to the Spanish Supercopa tonight,5|5|5_5|5|5,
hit:7610_conv:15220,2,excited,I'm looking forward to the Spanish Supercopa tonight,238,what happens at the spanish supercopa ?,5|5|5_5|5|5,
hit:7610_conv:15220,3,excited,I'm looking forward to the Spanish Supercopa tonight,196,The winner of the Spanish league and the Spanish cup play against each other. Barcelona v Sevilla. It takes place tonight in Tangiers.,5|5|5_5|5|5,
hit:7610_conv:15220,4,excited,I'm looking forward to the Spanish Supercopa tonight,238,oh that sounds like fun,5|5|5_5|5|5,
hit:7610_conv:15220,5,excited,I'm looking forward to the Spanish Supercopa tonight,196,It should be a good match,5|5|5_5|5|5,
hit:7610_conv:15221,1,trusting,i knew i shouildnt have trusted my brother with my dog,238,i knew i shouldnt have trusted my brother with my dog,5|5|5_5|5|5,
hit:7610_conv:15221,2,trusting,i knew i shouildnt have trusted my brother with my dog,196,What happened?,5|5|5_5|5|5,
hit:7610_conv:15221,3,trusting,i knew i shouildnt have trusted my brother with my dog,238,he got out and is missing,5|5|5_5|5|5,
hit:7610_conv:15221,4,trusting,i knew i shouildnt have trusted my brother with my dog,196,Oh no. Shouldn't you go and look for him?,5|5|5_5|5|5,
hit:7611_conv:15222,1,afraid,We had a bad storm last night. The power was off until daylight. It was pretty scary being in the dark all night.,548,We had a bad storm last night. The power was off until daylight. It was pretty scary being in the dark all night.,4|4|5_5|5|5,
hit:7611_conv:15222,2,afraid,We had a bad storm last night. The power was off until daylight. It was pretty scary being in the dark all night.,322,That must have been annoying.,4|4|5_5|5|5,
hit:7611_conv:15222,3,afraid,We had a bad storm last night. The power was off until daylight. It was pretty scary being in the dark all night.,548,It was hot but the worse part was how spooky everything is during the night. I could not sleep at all.,4|4|5_5|5|5,
hit:7611_conv:15222,4,afraid,We had a bad storm last night. The power was off until daylight. It was pretty scary being in the dark all night.,322,I hope everything is okay now.,4|4|5_5|5|5,
hit:7611_conv:15223,1,surprised,My family threw a party for me.  I am so shocked they did that.,322,My family threw a party for me.,5|5|5_4|4|5,
hit:7611_conv:15223,2,surprised,My family threw a party for me.  I am so shocked they did that.,548,That was nice of them. Were you surprised?,5|5|5_4|4|5,
hit:7611_conv:15223,3,surprised,My family threw a party for me.  I am so shocked they did that.,322,Yes_comma_ I am so shocked they did that!,5|5|5_4|4|5,
hit:7611_conv:15223,4,surprised,My family threw a party for me.  I am so shocked they did that.,548,It has to make you feel good to know that they care so much.,5|5|5_4|4|5,
hit:7612_conv:15224,1,anxious,Cinemas freak me out. They're to dark and quiet.,196,I always feel anxious whenever  go to the movies.,5|5|5_5|5|5,
hit:7612_conv:15224,2,anxious,Cinemas freak me out. They're to dark and quiet.,408,Is it because of the shootings?,5|5|5_5|5|5,
hit:7612_conv:15224,3,anxious,Cinemas freak me out. They're to dark and quiet.,196,No - it's because it's dark and quiet. I always worry whether I'm breathing too loud and then start holding my breath,5|5|5_5|5|5,
hit:7612_conv:15224,4,anxious,Cinemas freak me out. They're to dark and quiet.,408,is it bevcause yo udon't want to bother the other people?,5|5|5_5|5|5,
hit:7612_conv:15224,5,anxious,Cinemas freak me out. They're to dark and quiet.,196,Exactly. That's why.,5|5|5_5|5|5,
hit:7612_conv:15225,1,surprised,I knew I was pretty smart and I had a good chance of getting into our local college. But when the acceptance letter came from Havard. I have never been more shocked.,408,I knew I was smart but I had no clue I was that smart.,5|5|5_5|5|5,
hit:7612_conv:15225,2,surprised,I knew I was pretty smart and I had a good chance of getting into our local college. But when the acceptance letter came from Havard. I have never been more shocked.,196,How smart?,5|5|5_5|5|5,
hit:7612_conv:15225,3,surprised,I knew I was pretty smart and I had a good chance of getting into our local college. But when the acceptance letter came from Havard. I have never been more shocked.,408,I got into our state college but never would have thought I could get into an Ivy league...I was completely shocked,5|5|5_5|5|5,
hit:7612_conv:15225,4,surprised,I knew I was pretty smart and I had a good chance of getting into our local college. But when the acceptance letter came from Havard. I have never been more shocked.,196,Well done!,5|5|5_5|5|5,
hit:7613_conv:15226,1,devastated,My girlfriend dumped me yesterday.  I am so sad over it.,322,My girlfriend dumped me yesterday.,4|5|5_5|5|5,
hit:7613_conv:15226,2,devastated,My girlfriend dumped me yesterday.  I am so sad over it.,238,sorry to hear that.,4|5|5_5|5|5,
hit:7613_conv:15226,3,devastated,My girlfriend dumped me yesterday.  I am so sad over it.,322,I am so sad over it.,4|5|5_5|5|5,
hit:7613_conv:15226,4,devastated,My girlfriend dumped me yesterday.  I am so sad over it.,238,i hope it gets better over time for you,4|5|5_5|5|5,
hit:7613_conv:15227,1,caring,i loved taking care of my sisters pet the other day,238,i loved taking care of my sisters pet the other day,5|5|5_4|5|5,
hit:7613_conv:15227,2,caring,i loved taking care of my sisters pet the other day,322,Oh_comma_ what kind of pet?,5|5|5_4|5|5,
hit:7613_conv:15227,3,caring,i loved taking care of my sisters pet the other day,238,a bearded dragon,5|5|5_4|5|5,
hit:7613_conv:15227,4,caring,i loved taking care of my sisters pet the other day,322,That is quite an exotic pet.,5|5|5_4|5|5,
hit:7614_conv:15228,1,proud,I made an amazing pizza the other day,196,I made an amazing pizza the other day,5|5|5_4|5|5,
hit:7614_conv:15228,2,proud,I made an amazing pizza the other day,322,Oh_comma_ what kind was it?,5|5|5_4|5|5,
hit:7614_conv:15228,3,proud,I made an amazing pizza the other day,196,It was a detroit style cheese pizza. Three types of chees and then sauce and garlic,5|5|5_4|5|5,
hit:7614_conv:15228,4,proud,I made an amazing pizza the other day,322,That sounds so good!,5|5|5_4|5|5,
hit:7614_conv:15228,5,proud,I made an amazing pizza the other day,196,It was delicious,5|5|5_4|5|5,
hit:7614_conv:15229,1,ashamed,I took some of my Mom's medicine.  I feel so guilty about it.,322,I took some of my Mom's medicine.,4|5|5_5|5|5,
hit:7614_conv:15229,2,ashamed,I took some of my Mom's medicine.  I feel so guilty about it.,196,Why did you do that?,4|5|5_5|5|5,
hit:7614_conv:15229,3,ashamed,I took some of my Mom's medicine.  I feel so guilty about it.,322,I thought I would feel better_comma_ I feel so guilty about it.,4|5|5_5|5|5,
hit:7614_conv:15229,4,ashamed,I took some of my Mom's medicine.  I feel so guilty about it.,196,Does she know?,4|5|5_5|5|5,
hit:7615_conv:15230,1,disgusted,I watched a tv show last night where a guy ate a live squid,196,I watched a TV show last night where the presenter ate a live squid,5|5|5_5|5|5,
hit:7615_conv:15230,2,disgusted,I watched a tv show last night where a guy ate a live squid,238,oh i heard about this earlier_comma_ its still gross and i dont know why they showed that on tv,5|5|5_5|5|5,
hit:7615_conv:15230,3,disgusted,I watched a tv show last night where a guy ate a live squid,196,I know. I've been telling everyone about it!,5|5|5_5|5|5,
hit:7615_conv:15230,4,disgusted,I watched a tv show last night where a guy ate a live squid,238,keep spreading the word!,5|5|5_5|5|5,
hit:7615_conv:15230,5,disgusted,I watched a tv show last night where a guy ate a live squid,196,Don't worry I will,5|5|5_5|5|5,
hit:7615_conv:15231,1,lonely,i felt all alone the the other day,238,i felt all alone the other day,5|5|5_5|5|5,
hit:7615_conv:15231,2,lonely,i felt all alone the the other day,196,I'm sorry to hear that. What did you do about it?,5|5|5_5|5|5,
hit:7615_conv:15231,3,lonely,i felt all alone the the other day,238,i went to walmart to walk around and spend some money,5|5|5_5|5|5,
hit:7615_conv:15231,4,lonely,i felt all alone the the other day,196,That always makes me fee better. ,5|5|5_5|5|5,
hit:7616_conv:15232,1,caring,i loved taking care of my sisters pet the other day,238,i loved taking care of my sisters pet the other day,5|5|5_4|4|5,
hit:7616_conv:15232,2,caring,i loved taking care of my sisters pet the other day,322,You must be a nice sibling.,5|5|5_4|4|5,
hit:7616_conv:15232,3,caring,i loved taking care of my sisters pet the other day,238,you better believe it.,5|5|5_4|4|5,
hit:7616_conv:15232,4,caring,i loved taking care of my sisters pet the other day,322,Was it a komodo dragon?,5|5|5_4|4|5,
hit:7616_conv:15233,1,confident,I have to make a quiche later.  I am sure it will be easy to make.,322,I have to make a quiche later.,4|4|5_5|5|5,
hit:7616_conv:15233,2,confident,I have to make a quiche later.  I am sure it will be easy to make.,238,how long does that take to make,4|4|5_5|5|5,
hit:7616_conv:15233,3,confident,I have to make a quiche later.  I am sure it will be easy to make.,322,30 Minutes_comma_ I am sure it will be easy to make.,4|4|5_5|5|5,
hit:7616_conv:15233,4,confident,I have to make a quiche later.  I am sure it will be easy to make.,238,ah i figured it would take longer,4|4|5_5|5|5,
hit:7617_conv:15234,1,disgusted,I accidentally fell in horse manure. I was so grossed out by it.,322,I accidentally fell in horse manure.,5|5|5_4|4|5,
hit:7617_conv:15234,2,disgusted,I accidentally fell in horse manure. I was so grossed out by it.,548,I cannot even imagine how horrible that must have been. What did you do?,5|5|5_4|4|5,
hit:7617_conv:15234,3,disgusted,I accidentally fell in horse manure. I was so grossed out by it.,322,I took a shower_comma_ I was so grossed out by it.,5|5|5_4|4|5,
hit:7617_conv:15234,4,disgusted,I accidentally fell in horse manure. I was so grossed out by it.,548,LOL....I think I would have still been showering.,5|5|5_4|4|5,
hit:7617_conv:15235,1,furious,I got pulled over for speeding but there were cars passing me that didn't get stopped. I was fit to be tied.,548,I got pulled over for speeding but there were cars passing me that didn't get stopped. I was fit to be tied.,4|4|5_5|5|5,
hit:7617_conv:15235,2,furious,I got pulled over for speeding but there were cars passing me that didn't get stopped. I was fit to be tied.,322,That doesn't sound fair.,4|4|5_5|5|5,
hit:7617_conv:15235,3,furious,I got pulled over for speeding but there were cars passing me that didn't get stopped. I was fit to be tied.,548,It wasn't. I think the reason I only got a warning was because of how mad I was.,4|4|5_5|5|5,
hit:7617_conv:15235,4,furious,I got pulled over for speeding but there were cars passing me that didn't get stopped. I was fit to be tied.,322,I'm glad you didn't get a ticket.,4|4|5_5|5|5,
hit:7618_conv:15236,1,trusting,I knew I was in a good relationship when I felt safe and secure. My last girlfriend I just had this feeling that she was the one I was searching for_comma_ forever.,408,Have you ever had a person who you knew you could trust and go to with anything?,5|5|5_5|5|5,
hit:7618_conv:15236,2,trusting,I knew I was in a good relationship when I felt safe and secure. My last girlfriend I just had this feeling that she was the one I was searching for_comma_ forever.,196,Not really - other than close family - you?,5|5|5_5|5|5,
hit:7618_conv:15236,3,trusting,I knew I was in a good relationship when I felt safe and secure. My last girlfriend I just had this feeling that she was the one I was searching for_comma_ forever.,408,Yeah_comma_ I've had crappy girlfriends in the past. But I've been dating one for over a year and its so different- Its one of those when you know you know.,5|5|5_5|5|5,
hit:7618_conv:15236,4,trusting,I knew I was in a good relationship when I felt safe and secure. My last girlfriend I just had this feeling that she was the one I was searching for_comma_ forever.,196,Good - does she feel the same way?,5|5|5_5|5|5,
hit:7618_conv:15237,1,content,The sun is shining and I have no plans,196,The sun is shining here and I have the day to myself,5|5|5_5|5|5,
hit:7618_conv:15237,2,content,The sun is shining and I have no plans,408,Do you have any big plans? OR just a day of relaxation and recovery for this coming week?,5|5|5_5|5|5,
hit:7618_conv:15237,3,content,The sun is shining and I have no plans,196,Probably just a day of chillaxation ,5|5|5_5|5|5,
hit:7618_conv:15237,4,content,The sun is shining and I have no plans,408,Ah. I'm jealous- Maybe you should go for a walk or something and get fresh air too.,5|5|5_5|5|5,
hit:7618_conv:15237,5,content,The sun is shining and I have no plans,196,I might do that too. ,5|5|5_5|5|5,
hit:7619_conv:15239,1,proud,I made an amazing pizza the other day,196,I made an amazing pizza the other day,5|5|5_5|5|5,
hit:7619_conv:15239,2,proud,I made an amazing pizza the other day,587,Pizza is my favorite food.   WHat toppings did you use?,5|5|5_5|5|5,
hit:7619_conv:15239,3,proud,I made an amazing pizza the other day,196,Three different cheeses and sauce,5|5|5_5|5|5,
hit:7619_conv:15239,4,proud,I made an amazing pizza the other day,587,That sounds so delicious.  If I had known you made a pizza I would definitely be over at your house for dinner.,5|5|5_5|5|5,
hit:7619_conv:15239,5,proud,I made an amazing pizza the other day,196,You know that you are welcome anytime,5|5|5_5|5|5,
hit:7620_conv:15240,1,prepared,i was fully prepared for my interview last week,238,i was fully prepared for my interview last week,5|5|5_5|5|5,
hit:7620_conv:15240,2,prepared,i was fully prepared for my interview last week,274,That is always best if you are really hoping to get the job.  Did you get it?,5|5|5_5|5|5,
hit:7620_conv:15240,3,prepared,i was fully prepared for my interview last week,238,yes i did get the job!,5|5|5_5|5|5,
hit:7620_conv:15240,4,prepared,i was fully prepared for my interview last week,274,Congrats!!,5|5|5_5|5|5,<UNIGRAM>
hit:7620_conv:15241,1,anticipating,There is a bad storm outside right now. My dog is curled up like a ball in the basement corner.,274,We have some bad storm weather out here right now.  My dog absolutely hates it.,5|5|5_5|5|5,
hit:7620_conv:15241,2,anticipating,There is a bad storm outside right now. My dog is curled up like a ball in the basement corner.,238,did you put him inside?,5|5|5_5|5|5,
hit:7620_conv:15241,3,anticipating,There is a bad storm outside right now. My dog is curled up like a ball in the basement corner.,274,No_comma_ he goes down to the basement and curls up at the first sound of thunder.,5|5|5_5|5|5,
hit:7620_conv:15241,4,anticipating,There is a bad storm outside right now. My dog is curled up like a ball in the basement corner.,238,ah well at least he is inside.,5|5|5_5|5|5,
hit:7621_conv:15242,1,excited,I am ordering a pizza later.  I can't wait to eat it.,322,I am ordering a pizza later.,5|5|5_4|4|5,
hit:7621_conv:15242,2,excited,I am ordering a pizza later.  I can't wait to eat it.,548,I love Pizza. What kind are you going to have?,5|5|5_4|4|5,
hit:7621_conv:15242,3,excited,I am ordering a pizza later.  I can't wait to eat it.,322,I have going to have a deep dish pepperoni.  I can't wait to eat it!,5|5|5_4|4|5,
hit:7621_conv:15242,4,excited,I am ordering a pizza later.  I can't wait to eat it.,548,Sounds great. What time is supper? I want to be on time.LOL,5|5|5_4|4|5,
hit:7621_conv:15243,1,sad,The anniversary of my sisters death was a few days ago. It has me really feeling down.,548,The anniversary of my sisters death was a few days ago. It has me really feeling down,4|4|5_5|5|5,
hit:7621_conv:15243,2,sad,The anniversary of my sisters death was a few days ago. It has me really feeling down.,322,I'm sorry you feel down.,4|4|5_5|5|5,
hit:7621_conv:15243,3,sad,The anniversary of my sisters death was a few days ago. It has me really feeling down.,548,It's hard to be happy during this time of the year.,4|4|5_5|5|5,
hit:7621_conv:15243,4,sad,The anniversary of my sisters death was a few days ago. It has me really feeling down.,322,I hope you feel better that must be tough.,4|4|5_5|5|5,
hit:7622_conv:15244,1,surprised,I received an unexpected parcel in the mail this week,196,I got a parcel in the mail this week,5|5|5_5|5|5,
hit:7622_conv:15244,2,surprised,I received an unexpected parcel in the mail this week,408,Was it from someone you knew?,5|5|5_5|5|5,
hit:7622_conv:15244,3,surprised,I received an unexpected parcel in the mail this week,196,Yes - it was from my Aunt.,5|5|5_5|5|5,
hit:7622_conv:15244,4,surprised,I received an unexpected parcel in the mail this week,408,Were you expecting it_comma_ or was it a surpise? What as in it?,5|5|5_5|5|5,
hit:7622_conv:15244,5,surprised,I received an unexpected parcel in the mail this week,196,It was a surprise. It was full of chocolate and cany,5|5|5_5|5|5,
hit:7622_conv:15245,1,grateful,When I choice to finally divorce my husband I had no place to go. Thats why I stayed so long until my best friend told me without a doubt I could stay with her for as long as I needed to.,408,Have you ever had that one go to person that no matter what_comma_ no matter what mess you get into they are always there for you?,5|5|5_5|5|5,
hit:7622_conv:15245,2,grateful,When I choice to finally divorce my husband I had no place to go. Thats why I stayed so long until my best friend told me without a doubt I could stay with her for as long as I needed to.,196,Yeah - my wife.,5|5|5_5|5|5,
hit:7622_conv:15245,3,grateful,When I choice to finally divorce my husband I had no place to go. Thats why I stayed so long until my best friend told me without a doubt I could stay with her for as long as I needed to.,408,Isn't it such a great feeling! Mine is my best friend_comma_ when I showed up at her door with my bag after my divorce- she let me stay with her without a doubt.,5|5|5_5|5|5,
hit:7622_conv:15245,4,grateful,When I choice to finally divorce my husband I had no place to go. Thats why I stayed so long until my best friend told me without a doubt I could stay with her for as long as I needed to.,196,That's nice of her. Make sure you let her know how you feel,5|5|5_5|5|5,
hit:7623_conv:15246,1,caring,i loved taking care of my sisters pet the other day,238,i loved taking care of my sisters pet the other day,5|5|5_5|5|5,
hit:7623_conv:15246,2,caring,i loved taking care of my sisters pet the other day,196,What kind of pet is it?,5|5|5_5|5|5,
hit:7623_conv:15246,3,caring,i loved taking care of my sisters pet the other day,238,a bearded dragon,5|5|5_5|5|5,
hit:7623_conv:15246,4,caring,i loved taking care of my sisters pet the other day,196,Is that a kind of lizard?,5|5|5_5|5|5,
hit:7623_conv:15247,1,guilty,I killed my family and fed them  to pigs. Then one of the pigs died from food poisoning.,196,I killed my family and fed them to pigs,5|5|5_5|5|5,
hit:7623_conv:15247,2,guilty,I killed my family and fed them  to pigs. Then one of the pigs died from food poisoning.,238,did it work ?,5|5|5_5|5|5,
hit:7623_conv:15247,3,guilty,I killed my family and fed them  to pigs. Then one of the pigs died from food poisoning.,196,Well_comma_ yes. The pigs chewed up the bodies and crunched through the bones leaving nothing but manure. But I feel guilty because a few days later one of the pigs died from food poisoning.,5|5|5_5|5|5,
hit:7623_conv:15247,4,guilty,I killed my family and fed them  to pigs. Then one of the pigs died from food poisoning.,238,oh well thats life. thats a good job on your part to have pigs available to do that for you.,5|5|5_5|5|5,
hit:7623_conv:15247,5,guilty,I killed my family and fed them  to pigs. Then one of the pigs died from food poisoning.,196,I love those pigs. So friendly. ,5|5|5_5|5|5,
hit:7624_conv:15248,1,caring,i loved taking care of my sisters pet last week,238,i loved taking care of my sisters pet last week,5|5|5_5|5|5,
hit:7624_conv:15248,2,caring,i loved taking care of my sisters pet last week,196,Is it her that's got a lizard - bearded dragon thing?,5|5|5_5|5|5,
hit:7624_conv:15248,3,caring,i loved taking care of my sisters pet last week,238,yes it is_comma_ his name is spoons!,5|5|5_5|5|5,
hit:7624_conv:15248,4,caring,i loved taking care of my sisters pet last week,196,What does he eat?,5|5|5_5|5|5,
hit:7624_conv:15248,5,caring,i loved taking care of my sisters pet last week,238,crickets and dubia roaches,5|5|5_5|5|5,
hit:7624_conv:15248,6,caring,i loved taking care of my sisters pet last week,196,Where do you get them?,5|5|5_5|5|5,


hit:7846_conv:15692,1,joyful,I just found out I am pregnant after years of trying! I've had so many disappointments but it's finally happened!,99,"I just found out I am pregnant after years of trying!,5|5|5_5|5|5,
hit:7846_conv:15692,2,joyful,I just found out I am pregnant after years of trying! I've had so many disappointments but it's finally happened!,478,That's really great! I'm so happy for you!,5|5|5_5|5|5,
hit:7846_conv:15692,3,joyful,I just found out I am pregnant after years of trying! I've had so many disappointments but it's finally happened!,99,Thank you! I've had so many disappointments but it's finally happened! I couldn't be happier.,5|5|5_5|5|5,
hit:7846_conv:15692,4,joyful,I just found out I am pregnant after years of trying! I've had so many disappointments but it's finally happened!,478,Well I think you will make a great parent. Congratulations!,5|5|5_5|5|5,
hit:7846_conv:15693,1,trusting,I'm really gulible. I will believe anything,478,I am so gulible. I will believe anything you say.,5|5|5_5|5|5,
hit:7846_conv:15693,2,trusting,I'm really gulible. I will believe anything,99,I have had problems with that as well. I wish I had some advice to give.,5|5|5_5|5|5,
hit:7846_conv:15693,3,trusting,I'm really gulible. I will believe anything,478,Me too. I just have to spend a lot of time trying to figure out if someone is lying to me or not. ,5|5|5_5|5|5,
hit:7846_conv:15693,4,trusting,I'm really gulible. I will believe anything,99,Same here. And that can be very difficult_comma_ as you know!,5|5|5_5|5|5,
hit:7847_conv:15694,1,terrified,I stay away from the dark.,9,I stay away from the dark.,5|5|5_1|1|1,
hit:7847_conv:15694,2,terrified,I stay away from the dark.,139,Why do you do that? Are you scared of the dark?,5|5|5_1|1|1,
hit:7847_conv:15694,3,terrified,I stay away from the dark.,9,It makes me think of bad things.,5|5|5_1|1|1,
hit:7847_conv:15694,4,terrified,I stay away from the dark.,139,Well that makes sense then. Good luck staying away from the dark.,5|5|5_1|1|1,
hit:7847_conv:15695,1,angry,Every time I clean up my kids room they mess it up again within a day. I don't know why they won't just keep their room clean. I get so upset when they don't listen to me!,139,Every time I clean my kids room they go in and mess it up again. I am so ticked off at them.,1|1|1_5|5|5,
hit:7847_conv:15695,2,angry,Every time I clean up my kids room they mess it up again within a day. I don't know why they won't just keep their room clean. I get so upset when they don't listen to me!,9,That is why you never have kids.,1|1|1_5|5|5,
hit:7847_conv:15695,3,angry,Every time I clean up my kids room they mess it up again within a day. I don't know why they won't just keep their room clean. I get so upset when they don't listen to me!,139,Well I don't understand why they can't keep their room clean. I tell them so many times!,1|1|1_5|5|5,
hit:7847_conv:15695,4,angry,Every time I clean up my kids room they mess it up again within a day. I don't know why they won't just keep their room clean. I get so upset when they don't listen to me!,9,I also keep my room dirty.,1|1|1_5|5|5,
hit:7848_conv:15696,1,annoyed,It's annoying how is always scorching hot on your days off_comma_ but nice when you have to work inside all day!,448,It's annoying how is always scorching hot on your days off_comma_ but nice when you have to work inside all day!,5|5|5_5|5|5,
hit:7848_conv:15696,2,annoyed,It's annoying how is always scorching hot on your days off_comma_ but nice when you have to work inside all day!,466,I know the feeling! What you have to do is call in sick on a day that you're schedule to go in.,5|5|5_5|5|5,
hit:7848_conv:15696,3,annoyed,It's annoying how is always scorching hot on your days off_comma_ but nice when you have to work inside all day!,448,Haha_comma_ that's a good  idea_comma_ might have to do that one day!,5|5|5_5|5|5,
hit:7848_conv:15696,4,annoyed,It's annoying how is always scorching hot on your days off_comma_ but nice when you have to work inside all day!,466,Let me know if it works for you!,5|5|5_5|5|5,
hit:7849_conv:15698,1,excited,Meeting characters in Disney world!,9,I'm going to meet characters in disney world!,5|5|5_5|5|5,
hit:7849_conv:15698,2,excited,Meeting characters in Disney world!,409,That sounds like so much fun! Is this your first time?,5|5|5_5|5|5,
hit:7849_conv:15698,3,excited,Meeting characters in Disney world!,9,This is my first time since I was much smaller as a kid.,5|5|5_5|5|5,
hit:7849_conv:15698,4,excited,Meeting characters in Disney world!,409,You will have a lot of fun. It really is nice there. ,5|5|5_5|5|5,
hit:7849_conv:15699,1,impressed,A friend of mine is a writer. She put out 22 books this year! I can't believe how many she did!,409,A friend of mine is a self-published writer. She put out 22 books this year! I can't believe how many she did. ,5|5|5_5|5|5,
hit:7849_conv:15699,2,impressed,A friend of mine is a writer. She put out 22 books this year! I can't believe how many she did!,9,That is wild_comma_ I wish i could write and make money.,5|5|5_5|5|5,
hit:7849_conv:15699,3,impressed,A friend of mine is a writer. She put out 22 books this year! I can't believe how many she did!,409,I am also a writer. I love it ,5|5|5_5|5|5,
hit:7849_conv:15699,4,impressed,A friend of mine is a writer. She put out 22 books this year! I can't believe how many she did!,9,Maybe i will look into it.,5|5|5_5|5|5,
hit:7850_conv:15700,1,surprised,My friends threw me a surprise party for my birthday last year! ,292,My friends organized a surprise birthday party for me last year! ,5|5|5_5|5|5,
hit:7850_conv:15700,2,surprised,My friends threw me a surprise party for my birthday last year! ,448,Awesome! were you surprised?,5|5|5_5|5|5,
hit:7850_conv:15700,3,surprised,My friends threw me a surprise party for my birthday last year! ,292,I really was_comma_ I had no idea! It was a lot of fun. ,5|5|5_5|5|5,
hit:7850_conv:15700,4,surprised,My friends threw me a surprise party for my birthday last year! ,448,That's really cool_comma_ I'm glad you were surprised and had a blast!,5|5|5_5|5|5,
hit:7850_conv:15701,1,sad,I'm sad the weekend is almost over and I have to go back to work tomorrow!,448,I'm sad the weekend is almost over and I have to go back to work tomorrow!,5|5|5_5|5|5,
hit:7850_conv:15701,2,sad,I'm sad the weekend is almost over and I have to go back to work tomorrow!,292,Ugh yeah same. Did you have a nice weekend though? ,5|5|5_5|5|5,
hit:7850_conv:15701,3,sad,I'm sad the weekend is almost over and I have to go back to work tomorrow!,448,Yeah_comma_ it was very relaxing_comma_ I chilled with my girlfriend and watched a lot of netflix.,5|5|5_5|5|5,
hit:7850_conv:15701,4,sad,I'm sad the weekend is almost over and I have to go back to work tomorrow!,292,Those kind of weekends are the best. ,5|5|5_5|5|5,
hit:7851_conv:15702,1,joyful,My dog did great at training!,9,My dog did great at training.,5|5|5_5|5|5,
hit:7851_conv:15702,2,joyful,My dog did great at training!,321,thats awesome _comma_ is your pet being trained at a store by chance ? ,5|5|5_5|5|5,
hit:7851_conv:15702,3,joyful,My dog did great at training!,9,No_comma_ we have a professional doing it that works with the sheriff department.,5|5|5_5|5|5,
hit:7851_conv:15702,4,joyful,My dog did great at training!,321,even more impressive _comma_ i hope your pet does great,5|5|5_5|5|5,
hit:7851_conv:15703,1,excited,i saw the new resident evil 2 trailer i was pumped,321,I am a gamer and i saw all the new game trailers _comma_ i saw the trailer for resident evil 2 last night it was so awesome !!!,5|5|5_5|5|5,
hit:7851_conv:15703,2,excited,i saw the new resident evil 2 trailer i was pumped,9,I play some games on my pc_comma_ at times.,5|5|5_5|5|5,
hit:7851_conv:15703,3,excited,i saw the new resident evil 2 trailer i was pumped,321,what sort of games ? I used to be really into diablo 2 when i was younger ,5|5|5_5|5|5,
hit:7851_conv:15703,4,excited,i saw the new resident evil 2 trailer i was pumped,9,I do love me some diablo 3,5|5|5_5|5|5,
hit:7852_conv:15704,1,excited,Next week I get to visit my sister and her family. I haven't seen them in a long time. I am so excited.,139,I can't wait until next week. I get to visit my sister and her family. I haven't seen them in a while.,5|5|5_5|5|5,
hit:7852_conv:15704,2,excited,Next week I get to visit my sister and her family. I haven't seen them in a long time. I am so excited.,478,That's awesome that you are getting to see your family again!,5|5|5_5|5|5,
hit:7852_conv:15704,3,excited,Next week I get to visit my sister and her family. I haven't seen them in a long time. I am so excited.,139,Yeah I am so looking forward to it. I will have a blast with them!,5|5|5_5|5|5,
hit:7852_conv:15704,4,excited,Next week I get to visit my sister and her family. I haven't seen them in a long time. I am so excited.,478,Well I hope that you have a great time!,5|5|5_5|5|5,
hit:7852_conv:15705,1,annoyed,I get so upset when ever a telemarkter calls. I can't stand those people. ,478,I hate telemarketers. They call me all the time and I can't stand it!,5|5|5_5|5|5,
hit:7852_conv:15705,2,annoyed,I get so upset when ever a telemarkter calls. I can't stand those people. ,139,Oh I know_comma_ they are so annoying. You should try the Do Not Call list.,5|5|5_5|5|5,
hit:7852_conv:15705,3,annoyed,I get so upset when ever a telemarkter calls. I can't stand those people. ,478,I do but they still call me. Frustrating!,5|5|5_5|5|5,
hit:7852_conv:15705,4,annoyed,I get so upset when ever a telemarkter calls. I can't stand those people. ,139,Well if they violate it they will have to pay you some money. May be good to contact a lawyer!,5|5|5_5|5|5,
hit:7853_conv:15706,1,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,463,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,2|4|5_5|5|4,
hit:7853_conv:15706,2,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,607,What is something that you have achieved by following your heart?,2|4|5_5|5|4,
hit:7853_conv:15706,3,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,463,being able to get our finances in order so we could buy our first home together,2|4|5_5|5|4,
hit:7853_conv:15706,4,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,607,That's a great achievement. Buying a house is not easy in this economy!,2|4|5_5|5|4,
hit:7853_conv:15706,5,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,463,it's not a mansion but it is home to us and we earned it. i'm very proud of that accomplishment.,2|4|5_5|5|4,
hit:7853_conv:15706,6,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,607,I would be too! I hope you enjoy your home for many years to come.,2|4|5_5|5|4,
hit:7853_conv:15706,7,faithful,i believe in my heart no matter what i want all i have to do is put my mind to it and i'll be able to achieve it,463,thank you very much. wish you the best also.,2|4|5_5|5|4,
hit:7854_conv:15708,1,impressed,I was really impressed with the school play my son's 6th grade class put on.,53,I was really impressed with the school play my 7th graders class put onm.,5|5|5_4|5|5,
hit:7854_conv:15708,2,impressed,I was really impressed with the school play my son's 6th grade class put on.,463,oh how fun. what was the play about?,5|5|5_4|5|5,
hit:7854_conv:15708,3,impressed,I was really impressed with the school play my son's 6th grade class put on.,53,It was about a really big sale going on at a department store and there were a lot of funny goings ons.,5|5|5_4|5|5,
hit:7854_conv:15708,4,impressed,I was really impressed with the school play my son's 6th grade class put on.,463,what does that have to do with the school play?,5|5|5_4|5|5,
hit:7854_conv:15708,5,impressed,I was really impressed with the school play my son's 6th grade class put on.,53,That was what the play was about.,5|5|5_4|5|5,
hit:7854_conv:15708,6,impressed,I was really impressed with the school play my son's 6th grade class put on.,463,oh. how interesting. i can see why you enjoyed it.,5|5|5_4|5|5,
hit:7854_conv:15708,7,impressed,I was really impressed with the school play my son's 6th grade class put on.,53,Yes it was very funny and entertaining.,5|5|5_4|5|5,
hit:7854_conv:15708,8,impressed,I was really impressed with the school play my son's 6th grade class put on.,463,that's always a bonus. it's great to be able to support your children's passions,5|5|5_4|5|5,
hit:7854_conv:15709,1,caring,i really enjoy having pets. they bring a new life into an empty feeling house,463,i really enjoy having pets. they bring a new life into an empty feeling house,4|5|5_5|5|5,
hit:7854_conv:15709,2,caring,i really enjoy having pets. they bring a new life into an empty feeling house,53,Yes I only have one dog but he's cool.,4|5|5_5|5|5,
hit:7854_conv:15709,3,caring,i really enjoy having pets. they bring a new life into an empty feeling house,463,we have a cat_comma_ a dog_comma_ a bunny_comma_ a snake_comma_ and a betta fish.,4|5|5_5|5|5,
hit:7854_conv:15709,4,caring,i really enjoy having pets. they bring a new life into an empty feeling house,53,Wow that's a lot. I've been thinking about getting a bunny.,4|5|5_5|5|5,
hit:7854_conv:15709,5,caring,i really enjoy having pets. they bring a new life into an empty feeling house,463,it doesn't seem so but when i mention them all together i can see how it sounds like we have a zoo,4|5|5_5|5|5,
hit:7854_conv:15709,6,caring,i really enjoy having pets. they bring a new life into an empty feeling house,53,It's a lot of work to have pets.,4|5|5_5|5|5,
hit:7854_conv:15709,7,caring,i really enjoy having pets. they bring a new life into an empty feeling house,463,it is. but their little faces are so worth it.,4|5|5_5|5|5,
hit:7854_conv:15709,8,caring,i really enjoy having pets. they bring a new life into an empty feeling house,53,They are cute!,4|5|5_5|5|5,
hit:7855_conv:15711,1,excited,I'll have my truck paid off next month.  I don't know why but I am so happy to finally have that debt off my hands.,525,I can't wait till next month_comma_ I'll finally have my truck all paid off!,5|5|5_4|4|4,
hit:7855_conv:15711,2,excited,I'll have my truck paid off next month.  I don't know why but I am so happy to finally have that debt off my hands.,466,That's awesome! Nothing like freeing up your budget with no car payment.,5|5|5_4|4|4,
hit:7855_conv:15711,3,excited,I'll have my truck paid off next month.  I don't know why but I am so happy to finally have that debt off my hands.,525,Yeah I feel like I can breath a little bit with our finances now!,5|5|5_4|4|4,
hit:7855_conv:15711,4,excited,I'll have my truck paid off next month.  I don't know why but I am so happy to finally have that debt off my hands.,466,Congratulations on getting it all paid off_comma_ your wallet is gonna thank you!,5|5|5_4|4|4,
hit:7856_conv:15712,1,surprised,I was so shocked when I cam home today. My wife had cleaned the entire house. ,478,I cam home form work and I was so shocked. My wife had cleaned the house. It was such a great presant!,5|5|5_5|5|5,
hit:7856_conv:15712,2,surprised,I was so shocked when I cam home today. My wife had cleaned the entire house. ,139,That is such a wonderful surprise. You may a great wife for sure.,5|5|5_5|5|5,
hit:7856_conv:15712,3,surprised,I was so shocked when I cam home today. My wife had cleaned the entire house. ,478,She is the best. I'm a lucky guy!,5|5|5_5|5|5,
hit:7856_conv:15712,4,surprised,I was so shocked when I cam home today. My wife had cleaned the entire house. ,139,You should tell her that if you didn't already. A happy wife is a happy life!,5|5|5_5|5|5,
hit:7856_conv:15712,5,surprised,I was so shocked when I cam home today. My wife had cleaned the entire house. ,478,Very true!,5|5|5_5|5|5,
hit:7856_conv:15713,1,hopeful,I think my dog is sick. He is moving around so slowly. I am positive it will go away soon though.,139,I'm worried my dog is sick. He has been moving around slow recently.,5|5|5_5|5|5,
hit:7856_conv:15713,2,hopeful,I think my dog is sick. He is moving around so slowly. I am positive it will go away soon though.,478,Oh no I hope that he pulls through!,5|5|5_5|5|5,
hit:7856_conv:15713,3,hopeful,I think my dog is sick. He is moving around so slowly. I am positive it will go away soon though.,139,Yeah I am sure he will get over it. It must have been that dead bird I saw him eating the other day.,5|5|5_5|5|5,
hit:7856_conv:15713,4,hopeful,I think my dog is sick. He is moving around so slowly. I am positive it will go away soon though.,478,Yeah probably. You should take him to the vet right away!,5|5|5_5|5|5,
hit:7857_conv:15714,1,guilty,I told someone about my best friend's secret and I feel terribly bad about it. I'm scared to tell her but I feel so bad.,99,I told someone about my best friend's secret and I feel terribly bad about it.,5|5|5_5|5|5,
hit:7857_conv:15714,2,guilty,I told someone about my best friend's secret and I feel terribly bad about it. I'm scared to tell her but I feel so bad.,147,oh no_comma_ was it a big secret? ,5|5|5_5|5|5,
hit:7857_conv:15714,3,guilty,I told someone about my best friend's secret and I feel terribly bad about it. I'm scared to tell her but I feel so bad.,99,It was. I'm scared to tell her I told someone_comma_ but I feel so bad.,5|5|5_5|5|5,
hit:7857_conv:15714,4,guilty,I told someone about my best friend's secret and I feel terribly bad about it. I'm scared to tell her but I feel so bad.,147,Do you think she will ever find out without you telling her? ,5|5|5_5|5|5,
hit:7857_conv:15714,5,guilty,I told someone about my best friend's secret and I feel terribly bad about it. I'm scared to tell her but I feel so bad.,99,Probably not_comma_ but my conscience won't let me not tell her.,5|5|5_5|5|5,
hit:7859_conv:15719,1,furious,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me! I've never been so mad!,99,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me!,5|5|5_5|5|5,
hit:7859_conv:15719,2,furious,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me! I've never been so mad!,147,Omg! Did you call the police?,5|5|5_5|5|5,
hit:7859_conv:15719,3,furious,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me! I've never been so mad!,99,Yes I did_comma_ because I was so mad that he was acting that way in front of my kid! ,5|5|5_5|5|5,
hit:7859_conv:15719,4,furious,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me! I've never been so mad!,147,Wow! Did the police arrest him or ticket him?,5|5|5_5|5|5,
hit:7859_conv:15719,5,furious,Some guy cut me off in traffic while I had my daughter in the backseat_comma_ then proceeded to get out of his car and curse at me! I've never been so mad!,99,He was arrested_comma_ turns out he was drunk!,5|5|5_5|5|5,
hit:7860_conv:15720,1,anticipating,I am taking a trip to my hometown soon! It has been so long since I've been there!,99,I am taking a trip to my hometown soon!,5|5|5_5|5|5,
hit:7860_conv:15720,2,anticipating,I am taking a trip to my hometown soon! It has been so long since I've been there!,385,Cool! Where are you from?,5|5|5_5|5|5,
hit:7860_conv:15720,3,anticipating,I am taking a trip to my hometown soon! It has been so long since I've been there!,99,Indiana_comma_ but I live in Florida now. It's been so long since I've visited Indiana!,5|5|5_5|5|5,
hit:7860_conv:15720,4,anticipating,I am taking a trip to my hometown soon! It has been so long since I've been there!,385,Haha_comma_ I live in Florida too_comma_ but grew up in Massachusetts. I swear_comma_ is ANYONE born here?,5|5|5_5|5|5,
hit:7860_conv:15720,5,anticipating,I am taking a trip to my hometown soon! It has been so long since I've been there!,99,I don't think so! Hahaha!,5|5|5_5|5|5,
hit:7860_conv:15721,1,surprised,Last Christmas_comma_ my wife got me a Switch.,385,Every year_comma_ my wife continues to surprise me with something really cool. Last time_comma_ she bought me a switch!,5|5|5_5|5|5,
hit:7860_conv:15721,2,surprised,Last Christmas_comma_ my wife got me a Switch.,99,Nintendo Switch? ,5|5|5_5|5|5,
hit:7860_conv:15721,3,surprised,Last Christmas_comma_ my wife got me a Switch.,385,That's the one. She always surpasses my expectations!,5|5|5_5|5|5,
hit:7860_conv:15721,4,surprised,Last Christmas_comma_ my wife got me a Switch.,99,My boyfriend just bought himself one. $300 was way too much for my budget. I'm glad your wife could do that for you.,5|5|5_5|5|5,
hit:7860_conv:15721,5,surprised,Last Christmas_comma_ my wife got me a Switch.,385,Try it out_comma_ it has some fantastic games!,5|5|5_5|5|5,
hit:7861_conv:15722,1,nostalgic,I really miss when my kids were younger. All they wanted to do was hang out with me. Now they want to be with their friends instead.,139,I miss the days when my kids were younger. I was their hero and they just wanted to spend time with me.,5|5|5_5|5|5,
hit:7861_conv:15722,2,nostalgic,I really miss when my kids were younger. All they wanted to do was hang out with me. Now they want to be with their friends instead.,282,It is natural that kids grow up and want a little independence.  I know it happened with me.,5|5|5_5|5|5,
hit:7861_conv:15722,3,nostalgic,I really miss when my kids were younger. All they wanted to do was hang out with me. Now they want to be with their friends instead.,139,Yeah now they rather hang out with their friends. I understand but still miss when they were young.,5|5|5_5|5|5,
hit:7861_conv:15722,4,nostalgic,I really miss when my kids were younger. All they wanted to do was hang out with me. Now they want to be with their friends instead.,282,At least you will always have the memories to look back on,5|5|5_5|5|5,
hit:7861_conv:15723,1,annoyed,My neighbors had a party last night and played loud music until 1AM.  I was not happy,282,My neighbors decided to have a party last night and they played loud music until 1 AM.  It really got on my nerves.,5|5|5_5|5|5,
hit:7861_conv:15723,2,annoyed,My neighbors had a party last night and played loud music until 1AM.  I was not happy,139,Oh I bet you wanted to over there and yell at them. Some people are so inconsiderate.,5|5|5_5|5|5,
hit:7861_conv:15723,3,annoyed,My neighbors had a party last night and played loud music until 1AM.  I was not happy,282,And this is not the first time either.  You are right_comma_ so people are very selfish and it is frusterating.  ,5|5|5_5|5|5,
hit:7861_conv:15723,4,annoyed,My neighbors had a party last night and played loud music until 1AM.  I was not happy,139,If it is too bad you should file a noise complaint with the police. Sometimes that is the only way to get the message across.,5|5|5_5|5|5,
hit:7862_conv:15725,1,caring,I love my mom. She is the best and always is looking out for me. ,478,I love my mom. She is always looking out for me and is the absolute best. ,5|5|5_5|3|4,
hit:7862_conv:15725,2,caring,I love my mom. She is the best and always is looking out for me. ,49,Mom's are the best. I miss mine so much so enjoy every single moment you have with her.,5|5|5_5|3|4,
hit:7862_conv:15725,3,caring,I love my mom. She is the best and always is looking out for me. ,478,I most certainly will. ,5|5|5_5|3|4,
hit:7862_conv:15725,4,caring,I love my mom. She is the best and always is looking out for me. ,49,I always try to be as good of a mom as my mom was to me. Hard shoes to fill.,5|5|5_5|3|4,
hit:7863_conv:15726,1,surprised,All of my colleagues at work got together and threw a party for me due to a big contract I managed to snag. I wasn't expecting it!,99,All of my colleagues at work got together and threw a party for me due to a big contract I managed to snag.,5|5|5_5|5|5,
hit:7863_conv:15726,2,surprised,All of my colleagues at work got together and threw a party for me due to a big contract I managed to snag. I wasn't expecting it!,139,That's pretty awesome. The recognition must feel good.,5|5|5_5|5|5,
hit:7863_conv:15726,3,surprised,All of my colleagues at work got together and threw a party for me due to a big contract I managed to snag. I wasn't expecting it!,99,I definitely wasn't expecting them to throw me a party but it did feel good!,5|5|5_5|5|5,
hit:7863_conv:15726,4,surprised,All of my colleagues at work got together and threw a party for me due to a big contract I managed to snag. I wasn't expecting it!,139,Well it is well deserved. You are an important part of your company.,5|5|5_5|5|5,
hit:7863_conv:15727,1,content,I really can't complain about my life. I have everything I could possibly need. Life is quite good for me.,139,I can't complain about my life. I have everything I have ever wanted. Life is good.,5|5|5_5|5|5,
hit:7863_conv:15727,2,content,I really can't complain about my life. I have everything I could possibly need. Life is quite good for me.,99,That is cool. You are very fortunate.,5|5|5_5|5|5,
hit:7863_conv:15727,3,content,I really can't complain about my life. I have everything I could possibly need. Life is quite good for me.,139,Yeah I am happy with what I got. I don't need new things to make me happy.,5|5|5_5|5|5,
hit:7863_conv:15727,4,content,I really can't complain about my life. I have everything I could possibly need. Life is quite good for me.,99,That's really great. I wish I felt that way.,5|5|5_5|5|5,
hit:7864_conv:15728,1,annoyed,Driving in traffic is such a frustration and in the city I live in there is always traffic. ,356,Driving in traffic is such a frustration and in the city I live in there is always traffic. ,5|5|5_5|5|5,
hit:7864_conv:15728,2,annoyed,Driving in traffic is such a frustration and in the city I live in there is always traffic. ,99,I live in the city too_comma_ I can't stand traffic either.,5|5|5_5|5|5,
hit:7864_conv:15728,3,annoyed,Driving in traffic is such a frustration and in the city I live in there is always traffic. ,356,It takes me 30 minutes to go a couple of miles! Sooo frustrating. ,5|5|5_5|5|5,
hit:7864_conv:15728,4,annoyed,Driving in traffic is such a frustration and in the city I live in there is always traffic. ,99,Oh that sounds even worse than my city! I'm sorry!,5|5|5_5|5|5,
hit:7864_conv:15729,1,prepared,When the hurricane came last year_comma_ I made sure to stock up on supplies and board all the windows. I didn't want anything bad to happen.,99,When the hurricane came last year_comma_ I made sure to stock up on supplies and board all the windows.,5|5|5_5|5|5,
hit:7864_conv:15729,2,prepared,When the hurricane came last year_comma_ I made sure to stock up on supplies and board all the windows. I didn't want anything bad to happen.,356,That's a smart thing to do! What hurricane was coming? I got hit by Irma pretty bad last year.,5|5|5_5|5|5,
hit:7864_conv:15729,3,prepared,When the hurricane came last year_comma_ I made sure to stock up on supplies and board all the windows. I didn't want anything bad to happen.,99,That is the one! I just didn't want to be caught up_comma_ it was my first hurricane and I wanted to think ahead.,5|5|5_5|5|5,
hit:7864_conv:15729,4,prepared,When the hurricane came last year_comma_ I made sure to stock up on supplies and board all the windows. I didn't want anything bad to happen.,356,I've lived in South Florida all my life_comma_ so I've been through quite a few. As long as you have your supplies you're good to go.,5|5|5_5|5|5,
hit:7865_conv:15730,1,joyful,finally day a day off i feel so good,321,been  a long week finally a day off i feel so happy ...... and just like that my sisters asking me to take her to the mall lol _comma_ o well,5|5|5_5|5|5,
hit:7865_conv:15730,2,joyful,finally day a day off i feel so good,525,No rest for the weary I guess_comma_ huh?,5|5|5_5|5|5,
hit:7865_conv:15730,3,joyful,finally day a day off i feel so good,321,im gonna at least go the this chinese buffet _comma_ that will be my highlight for the day,5|5|5_5|5|5,
hit:7865_conv:15730,4,joyful,finally day a day off i feel so good,525,Yeah treat yourself!,5|5|5_5|5|5,
hit:7865_conv:15731,1,grateful,My wife wanted something done to the house that required a special tool.  I asked my dad if he had the tool_comma_ and instead of loaning me the tool_comma_ he did the job while I was at work.,525,My wife wanted something done at the house that required a special tool_comma_ so I went and asked my dad if he had the tool.,5|5|5_5|5|5,
hit:7865_conv:15731,2,grateful,My wife wanted something done to the house that required a special tool.  I asked my dad if he had the tool_comma_ and instead of loaning me the tool_comma_ he did the job while I was at work.,321,I used to have a giant bag of tools but everything got stolen _comma_ what tool was it ,5|5|5_5|5|5,
hit:7865_conv:15731,3,grateful,My wife wanted something done to the house that required a special tool.  I asked my dad if he had the tool_comma_ and instead of loaning me the tool_comma_ he did the job while I was at work.,525,It was a belt sander_comma_ but instead of letting me borrow it_comma_ he just came over and did the job while I was at work!  What a guy...,5|5|5_5|5|5,
hit:7865_conv:15731,4,grateful,My wife wanted something done to the house that required a special tool.  I asked my dad if he had the tool_comma_ and instead of loaning me the tool_comma_ he did the job while I was at work.,321,No matter how old we get_comma_ we will always be children in there eyes sounds like a great man ,5|5|5_5|5|5,
hit:7866_conv:15732,1,proud,My kid just got a full ride to college. I'm so excited for him!,478,My oldest son just got a full ride to college. He is so smart and I'm so excited for him!,5|5|5_5|5|5,
hit:7866_conv:15732,2,proud,My kid just got a full ride to college. I'm so excited for him!,147,Is it his first year?,5|5|5_5|5|5,
hit:7866_conv:15732,3,proud,My kid just got a full ride to college. I'm so excited for him!,478,Yeah he just graduated high school. ,5|5|5_5|5|5,
hit:7866_conv:15732,4,proud,My kid just got a full ride to college. I'm so excited for him!,147,Oh wow I bet you are proud!,5|5|5_5|5|5,
hit:7867_conv:15735,1,lonely,My girlfriend left me last week. I miss her so much. I wish she would give me one more chance.,139,I was so sad my girlfriend left me last week. I miss her terribly.,5|5|5_5|5|5,
hit:7867_conv:15735,2,lonely,My girlfriend left me last week. I miss her so much. I wish she would give me one more chance.,607,That is awful to hear about. Were you guys together for a long time?,5|5|5_5|5|5,
hit:7867_conv:15735,3,lonely,My girlfriend left me last week. I miss her so much. I wish she would give me one more chance.,139,Yeah over a year. I wish she gives me one more chance. I can't live without her.,5|5|5_5|5|5,
hit:7867_conv:15735,4,lonely,My girlfriend left me last week. I miss her so much. I wish she would give me one more chance.,607,You may try talking to her or taking her out for a nice_comma_ quiet dinner. Some problems can be worked through with good communication.,5|5|5_5|5|5,
hit:7868_conv:15736,1,trusting,My boyfriend helped to calm me down as I went through a procedure at the doctor a few weeks ago. He made me feel much better as I believed when he said I would be fine.,99,My boyfriend helped to calm me down as I went through a procedure at the doctor a few weeks ago.,5|5|5_3|5|5,
hit:7868_conv:15736,2,trusting,My boyfriend helped to calm me down as I went through a procedure at the doctor a few weeks ago. He made me feel much better as I believed when he said I would be fine.,466,Sounds like a good boyfriend! Did everything turn out ok?,5|5|5_3|5|5,
hit:7868_conv:15736,3,trusting,My boyfriend helped to calm me down as I went through a procedure at the doctor a few weeks ago. He made me feel much better as I believed when he said I would be fine.,99,He made me feel much better as I believed when he said I would be fine. And yes_comma_ everything turned out great!,5|5|5_3|5|5,
hit:7868_conv:15736,4,trusting,My boyfriend helped to calm me down as I went through a procedure at the doctor a few weeks ago. He made me feel much better as I believed when he said I would be fine.,466,Awesome_comma_ I'm glad you have someone like that in your life to support you!,5|5|5_3|5|5,
hit:7869_conv:15739,1,impressed,I was watching a NFL preseason game the other day. I was so impressed at the athleticism of the players. They must train hard.,139,I was watching a preseason football game the other night. I was impressed at how athletic the players are.,5|5|5_5|5|5,
hit:7869_conv:15739,2,impressed,I was watching a NFL preseason game the other day. I was so impressed at the athleticism of the players. They must train hard.,147,Oh wow_comma_ What team was you watching ?,5|5|5_5|5|5,
hit:7869_conv:15739,3,impressed,I was watching a NFL preseason game the other day. I was so impressed at the athleticism of the players. They must train hard.,139,It was the Tampa Bay Bucs. They must train real hard. I will have to watch the regular season now.,5|5|5_5|5|5,
hit:7869_conv:15739,4,impressed,I was watching a NFL preseason game the other day. I was so impressed at the athleticism of the players. They must train hard.,147,Oh wow! Personally I am a fan of the Falcons.,5|5|5_5|5|5,
hit:8514_conv:17028,1,embarrassed,When I was a preteen_comma_ my parents and my brother and I went on a Disney cruise. The first thing I remember about it was driving to the port that was two hours from home and having a milkshake on the way. This cruise was in the peak of the norovirus scare issues cruises had some 15 years ago. We got to the front of the ticket line and I puked all over the front of the counter. We were quarantined after that.,227, "When I was a preteen_comma_ my parents and my brother and I went on a Disney cruise. The first thing I remember about it was driving to the port from home and having a milkshake on the way.ort that was two hou This cruise was in the peak of the norovirus scare issues cruises had some 15 years ago. We got to the front of the ticket line and I puked all over the front of the counter. We were quarantined after that,5|5|5_5|5|5,
hit:8514_conv:17028,2,embarrassed,When I was a preteen_comma_ my parents and my brother and I went on a Disney cruise. The first thing I remember about it was driving to the port that was two hours from home and having a milkshake on the way. This cruise was in the peak of the norovirus scare issues cruises had some 15 years ago. We got to the front of the ticket line and I puked all over the front of the counter. We were quarantined after that.,284,Sounds like it was a great trip and good memory to have_comma_ do you have a family of your own to take them on the same Disney cruise?,5|5|5_5|5|5,
hit:8514_conv:17028,3,embarrassed,When I was a preteen_comma_ my parents and my brother and I went on a Disney cruise. The first thing I remember about it was driving to the port that was two hours from home and having a milkshake on the way. This cruise was in the peak of the norovirus scare issues cruises had some 15 years ago. We got to the front of the ticket line and I puked all over the front of the counter. We were quarantined after that.,227,It really was bogged down by me being sick the whole time. But yes_comma_ I have a 2 year old son and would love to take him sometime_comma_ Im just not as rich as my parents were LOL,5|5|5_5|5|5,
hit:8514_conv:17028,4,embarrassed,When I was a preteen_comma_ my parents and my brother and I went on a Disney cruise. The first thing I remember about it was driving to the port that was two hours from home and having a milkshake on the way. This cruise was in the peak of the norovirus scare issues cruises had some 15 years ago. We got to the front of the ticket line and I puked all over the front of the counter. We were quarantined after that.,284,Lol maybe one day your son is still small you have lot's of time to save_comma_ I think I will look into it for me and my family for our next vacation. ,5|5|5_5|5|5,
hit:8515_conv:17030,1,anxious,i cant wait for the weekend to get here,238,i cant wait for the weekend to get here,5|5|5_5|5|5,
hit:8515_conv:17030,2,anxious,i cant wait for the weekend to get here,56,Do you have some awesome plans?,5|5|5_5|5|5,
hit:8515_conv:17030,3,anxious,i cant wait for the weekend to get here,238,i want to go fly fishing,5|5|5_5|5|5,
hit:8515_conv:17030,4,anxious,i cant wait for the weekend to get here,56,I've been regular rod and reel fishing and love it_comma_ but I've never been fly fishing.  Do you enjoy it?,5|5|5_5|5|5,
hit:8515_conv:17031,1,sentimental,I've been going through old photos and trying to figure out how to store them.  I found a lot of my kids when they were babies_comma_ I cannot believe they are 18 and 16 now!,56,I've been going through old photos and trying to figure out how to store them_comma_ but then I just spend half the time crying.,5|5|5_5|5|5,
hit:8515_conv:17031,2,sentimental,I've been going through old photos and trying to figure out how to store them.  I found a lot of my kids when they were babies_comma_ I cannot believe they are 18 and 16 now!,238,brought back good memories?,5|5|5_5|5|5,
hit:8515_conv:17031,3,sentimental,I've been going through old photos and trying to figure out how to store them.  I found a lot of my kids when they were babies_comma_ I cannot believe they are 18 and 16 now!,56,Yes_comma_ tons of photos of my girls when they were babies.  I cannot believe they are 16 and 18 now.,5|5|5_5|5|5,
hit:8515_conv:17031,4,sentimental,I've been going through old photos and trying to figure out how to store them.  I found a lot of my kids when they were babies_comma_ I cannot believe they are 18 and 16 now!,238,they grow up so fast.,5|5|5_5|5|5,
hit:8516_conv:17032,1,impressed,I just watched a show about stunt people in the movies.  Its amazing what they can do!,438,I just watched a show about stunt people in the movies. Its amazing what they can do!,3|5|5_5|5|5,
hit:8516_conv:17032,2,impressed,I just watched a show about stunt people in the movies.  Its amazing what they can do!,10,Yeah it is_comma_ I always thought stunt doubles were cool,3|5|5_5|5|5,
hit:8516_conv:17032,3,impressed,I just watched a show about stunt people in the movies.  Its amazing what they can do!,438,They really get bashed and smashed.  They should get paid more.,3|5|5_5|5|5,
hit:8516_conv:17032,4,impressed,I just watched a show about stunt people in the movies.  Its amazing what they can do!,10,I agree_comma_ it is tough work!,3|5|5_5|5|5,
hit:8516_conv:17033,1,jealous,I am jealous of my coworker_comma_ he is very athletic and smart,10,I am jealous of my coworker_comma_ he is very athletic and smart,5|5|5_3|5|5,
hit:8516_conv:17033,2,jealous,I am jealous of my coworker_comma_ he is very athletic and smart,438,Don't worry about him.  Keep working on you.  I'm sure you have a lot of great qualities.,5|5|5_3|5|5,
hit:8516_conv:17033,3,jealous,I am jealous of my coworker_comma_ he is very athletic and smart,10,Thank you_comma_ I like to think that sometimes_comma_ but it can be tough,5|5|5_3|5|5,
hit:8516_conv:17033,4,jealous,I am jealous of my coworker_comma_ he is very athletic and smart,438,I understand.  Its hard to not compare yourself with others but we're all made differently with our own strengths.  Try to shine with those!,5|5|5_3|5|5,
hit:8517_conv:17034,1,confident,I think I make a great cheesecake. My friends and family always enjoy it.,247,I think I make a great cheesecake. My friends and family always enjoy it.,5|5|5_5|5|5,
hit:8517_conv:17034,2,confident,I think I make a great cheesecake. My friends and family always enjoy it.,473,Ooh I love cheesecake! Is it your own recipe or one that was handed down to you or what?,5|5|5_5|5|5,
hit:8517_conv:17034,3,confident,I think I make a great cheesecake. My friends and family always enjoy it.,247,Oh I just looked for one online_comma_ but my own secret recipe is adding some orange zest. ,5|5|5_5|5|5,
hit:8517_conv:17034,4,confident,I think I make a great cheesecake. My friends and family always enjoy it.,473,I will definitely have to try that next time I make cheesecake_comma_ thanks for the tip!,5|5|5_5|5|5,
hit:8518_conv:17036,1,nostalgic,Sometimes I hum songs from the 90s to myself_comma_ they were such earworms,247,Sometimes I hum songs from the 90s to myself_comma_ they were such earworms,5|5|5_5|5|5,
hit:8518_conv:17036,2,nostalgic,Sometimes I hum songs from the 90s to myself_comma_ they were such earworms,437,Older songs definitely have more texture and quality to them compared to the manufactured stuff we have that all sounds the same,5|5|5_5|5|5,
hit:8518_conv:17036,3,nostalgic,Sometimes I hum songs from the 90s to myself_comma_ they were such earworms,247,Yeah_comma_ but maybe we'll feel the same way in another 20 years haha,5|5|5_5|5|5,
hit:8518_conv:17036,4,nostalgic,Sometimes I hum songs from the 90s to myself_comma_ they were such earworms,437,Yeah when robots are like walking around playing the latest hologram music or something haha,5|5|5_5|5|5,
hit:8519_conv:17038,1,content,i am finally good at where i am at work,238,i am finally good at where i am at work,5|5|5_5|5|5,
hit:8519_conv:17038,2,content,i am finally good at where i am at work,56,That's a good feeling I'm sure.  Stability means a lot these days.,5|5|5_5|5|5,
hit:8519_conv:17038,3,content,i am finally good at where i am at work,238,yea .. only 8 more years and i can retire,5|5|5_5|5|5,
hit:8519_conv:17038,4,content,i am finally good at where i am at work,56,8 more years?  I wish I could retire in 8 years!  I bet that's exciting to think about!!,5|5|5_5|5|5,
hit:8519_conv:17039,1,devastated,My dad had a stroke a few months ago.  At first it seemed that he'd recovered almost 100%.  But I went to see him today and his speech is definitely affected and he's depressed - I'm worried about him.,56,My dad had a stroke a few months ago.  It seemed like he'd pretty much recovered 100%.,5|5|5_5|5|5,
hit:8519_conv:17039,2,devastated,My dad had a stroke a few months ago.  At first it seemed that he'd recovered almost 100%.  But I went to see him today and his speech is definitely affected and he's depressed - I'm worried about him.,238,is he ok?,5|5|5_5|5|5,
hit:8519_conv:17039,3,devastated,My dad had a stroke a few months ago.  At first it seemed that he'd recovered almost 100%.  But I went to see him today and his speech is definitely affected and he's depressed - I'm worried about him.,56,Well_comma_ we thought so_comma_ but I went to visit him today and his speech is definitely affected and he's depressed.  I'm worried about him,5|5|5_5|5|5,
hit:8519_conv:17039,4,devastated,My dad had a stroke a few months ago.  At first it seemed that he'd recovered almost 100%.  But I went to see him today and his speech is definitely affected and he's depressed - I'm worried about him.,238,is he at the hospital?,5|5|5_5|5|5,
hit:8520_conv:17041,1,nostalgic,I found an old shirt of my dad's in a box that I had stored for years.  It really brought back some good memories.,294,I found an old shirt of my dad's in a box that I had stored for years.  It really brought back some good memories.,5|5|5_5|5|5,
hit:8520_conv:17041,2,nostalgic,I found an old shirt of my dad's in a box that I had stored for years.  It really brought back some good memories.,284,Something dear and sentimental you should wear it sometimes on special occasions. ,5|5|5_5|5|5,
hit:8520_conv:17041,3,nostalgic,I found an old shirt of my dad's in a box that I had stored for years.  It really brought back some good memories.,294,I think I will.  It still has a slight scent of the Gray Flannel cologne he used to wear.  I miss him.,5|5|5_5|5|5,
hit:8520_conv:17041,4,nostalgic,I found an old shirt of my dad's in a box that I had stored for years.  It really brought back some good memories.,284,That is even better I know how you feel I lost my grand pa and he was everything to me I miss him dearly. ,5|5|5_5|5|5,
hit:8521_conv:17042,1,angry,i was mad when i saw my gate open,238,i was mad when i saw my gate open,5|5|5_3|2|3,
hit:8521_conv:17042,2,angry,i was mad when i saw my gate open,375,What happened?,5|5|5_3|2|3,
hit:8521_conv:17042,3,angry,i was mad when i saw my gate open,238,my dogs got out and they are still missing,5|5|5_3|2|3,
hit:8521_conv:17042,4,angry,i was mad when i saw my gate open,375,That sucks_comma_ did you find them?,5|5|5_3|2|3,
hit:8521_conv:17043,1,faithful,A girl tried to kiss me at a party but I told her no. I have a girlfriend already.,375,She came on to me but I had to tell her no.,3|2|3_5|5|5,
hit:8521_conv:17043,2,faithful,A girl tried to kiss me at a party but I told her no. I have a girlfriend already.,238,who,3|2|3_5|5|5,<UNIGRAM>
hit:8521_conv:17043,3,faithful,A girl tried to kiss me at a party but I told her no. I have a girlfriend already.,375,A girl_comma_ but I'm already taken. She wanted a kiss.,3|2|3_5|5|5,
hit:8521_conv:17043,4,faithful,A girl tried to kiss me at a party but I told her no. I have a girlfriend already.,238,ah.. thats understandable then,3|2|3_5|5|5,
hit:8522_conv:17044,1,lonely,Sat at home all weekend long with nothing to do,389,I didn't do anything all weekend,5|5|5_5|5|5,
hit:8522_conv:17044,2,lonely,Sat at home all weekend long with nothing to do,27,Why not?,5|5|5_5|5|5,
hit:8522_conv:17044,3,lonely,Sat at home all weekend long with nothing to do,389,Nothing to do.  Just sat around at home playing video games.  ,5|5|5_5|5|5,
hit:8522_conv:17044,4,lonely,Sat at home all weekend long with nothing to do,27,I love video games. Thats not a bad weekend,5|5|5_5|5|5,
hit:8522_conv:17044,5,lonely,Sat at home all weekend long with nothing to do,389,Yeah.  Should have gotten out to do something though.  Felt like I wasted it.,5|5|5_5|5|5,
hit:8522_conv:17044,6,lonely,Sat at home all weekend long with nothing to do,27,Theres always next weekend,5|5|5_5|5|5,
hit:8522_conv:17044,7,lonely,Sat at home all weekend long with nothing to do,389,Yeah that's true!  Might turn out to be a better weekend :D,5|5|5_5|5|5,
hit:8522_conv:17045,1,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,27,I ran into a group of people  I went to high school wiith over the weekend,5|5|5_5|5|5,
hit:8522_conv:17045,2,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,389,Hey cool!  Did you all catch up on old times?,5|5|5_5|5|5,
hit:8522_conv:17045,3,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,27,I was headed somewhere but we are getting together at the casino this weekend,5|5|5_5|5|5,
hit:8522_conv:17045,4,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,389,Oh heck yes!  That sounds awesome!  bring me back some of your winnings!,5|5|5_5|5|5,
hit:8522_conv:17045,5,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,27,They wont get a dime of my money. I have a free buffet lol,5|5|5_5|5|5,
hit:8522_conv:17045,6,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,389,hahaha awesome!  Have fun this weekend mate!,5|5|5_5|5|5,
hit:8522_conv:17045,7,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,27,you too! get out of the house!,5|5|5_5|5|5,
hit:8522_conv:17045,8,anxious,I can't wait for tomorrow.  I am meeting with some people I went to school with,389,haha!  Copy that_comma_ will do!,5|5|5_5|5|5,
hit:8523_conv:17046,1,disgusted,I just found a dead squirrel in our basement.  Something was stinking and it was that.  I want to throw up because I have to clean it out.,438,I just found a dead squirrel in our basement. Something was stinking and it was that. I want to throw up because I have to clean it out.,5|5|5_5|5|5,
hit:8523_conv:17046,2,disgusted,I just found a dead squirrel in our basement.  Something was stinking and it was that.  I want to throw up because I have to clean it out.,437,Dang_comma_ I guess you should get on that_comma_ hopefully you drew straws or something_comma_ but at least you found the source!,5|5|5_5|5|5,
hit:8523_conv:17046,3,disgusted,I just found a dead squirrel in our basement.  Something was stinking and it was that.  I want to throw up because I have to clean it out.,438,Ha ha_comma_ yeah.  My husband is out of town so I'm stuck with it.  Might just pick it up with the broom and throw it out with the broom! True that.  Stank!,5|5|5_5|5|5,
hit:8523_conv:17046,4,disgusted,I just found a dead squirrel in our basement.  Something was stinking and it was that.  I want to throw up because I have to clean it out.,437,Or get the cat on that_comma_ all they do is sleep anyway,5|5|5_5|5|5,
hit:8524_conv:17048,1,angry,i am pissed that someone put a dent in my door,238,i am pissed that someone put a dent in my door,5|5|5_3|5|5,
hit:8524_conv:17048,2,angry,i am pissed that someone put a dent in my door,294,They didn't leave a note!?!?!?!?  That is the worst.  That is the kind of thing that makes you want to bring out forensics to take a paint sample and hunt the people down to bring them to justice.,5|5|5_3|5|5,
hit:8524_conv:17048,3,angry,i am pissed that someone put a dent in my door,238,no they didnt leave anything except for the dent :(,5|5|5_3|5|5,
hit:8524_conv:17048,4,angry,i am pissed that someone put a dent in my door,294,Ugh_comma_ that should be illegal...wait a minute...I think it is illegal.  So frustrating.,5|5|5_3|5|5,
hit:8524_conv:17049,1,excited,I bought my son a tiny kitten for his birthday tomorrow.,294,I bought my son a tiny kitten for his birthday tomorrow.,3|5|5_5|5|5,
hit:8524_conv:17049,2,excited,I bought my son a tiny kitten for his birthday tomorrow.,238,thats a nice birthday gift,3|5|5_5|5|5,
hit:8524_conv:17049,3,excited,I bought my son a tiny kitten for his birthday tomorrow.,294,He will be over the moon_comma_ and I am probably going to be happier to see his face than he will be about getting it.,3|5|5_5|5|5,
hit:8524_conv:17049,4,excited,I bought my son a tiny kitten for his birthday tomorrow.,238,take tons of pictures,3|5|5_5|5|5,
hit:8525_conv:17051,1,excited,When my mother got me a new puppy. I was not expecting it.,375,Just got a new puppy!,5|5|4_5|5|5,
hit:8525_conv:17051,2,excited,When my mother got me a new puppy. I was not expecting it.,284,I love puppies what kind did you get? ,5|5|4_5|5|5,
hit:8525_conv:17051,3,excited,When my mother got me a new puppy. I was not expecting it.,375,It's a small husky with huge blue eyes.,5|5|4_5|5|5,
hit:8525_conv:17051,4,excited,When my mother got me a new puppy. I was not expecting it.,284,I never seen a husky with blue eyes I know it is beautiful great addition to the family. ,5|5|4_5|5|5,
hit:8526_conv:17052,1,disgusted,I was disgusted when I saw someone get their skull cracked in a fight,10,I was disgusted when I saw someone get their skull cracked in a fight,5|5|5_5|5|5,
hit:8526_conv:17052,2,disgusted,I was disgusted when I saw someone get their skull cracked in a fight,27,you saw that?,5|5|5_5|5|5,
hit:8526_conv:17052,3,disgusted,I was disgusted when I saw someone get their skull cracked in a fight,10,Yes_comma_ in high school,5|5|5_5|5|5,
hit:8526_conv:17052,4,disgusted,I was disgusted when I saw someone get their skull cracked in a fight,27,Who did it?,5|5|5_5|5|5,
hit:8526_conv:17053,1,nostalgic,I still think about when my teenage daughter was baby. She was an angel,27,i still think about when my daughter was a babu.,5|5|5_5|5|5,
hit:8526_conv:17053,2,nostalgic,I still think about when my teenage daughter was baby. She was an angel,10,Is that a good memory or bad one?,5|5|5_5|5|5,
hit:8526_conv:17053,3,nostalgic,I still think about when my teenage daughter was baby. She was an angel,27,good. she is 13 now and the worst lol,5|5|5_5|5|5,
hit:8526_conv:17053,4,nostalgic,I still think about when my teenage daughter was baby. She was an angel,10,Oh yeah_comma_ that is a rough phase,5|5|5_5|5|5,
hit:8526_conv:17053,5,nostalgic,I still think about when my teenage daughter was baby. She was an angel,27,for me i know,5|5|5_5|5|5,
hit:8527_conv:17054,1,anticipating,I can't wait for game of thrones to come back. Tv is terrible without it,27,I cant wait for game of thrones to vome back,5|5|5_5|5|5,
hit:8527_conv:17054,2,anticipating,I can't wait for game of thrones to come back. Tv is terrible without it,238,when does it return?,5|5|5_5|5|5,
hit:8527_conv:17054,3,anticipating,I can't wait for game of thrones to come back. Tv is terrible without it,27,next summer. tv is so dull without it,5|5|5_5|5|5,
hit:8527_conv:17054,4,anticipating,I can't wait for game of thrones to come back. Tv is terrible without it,238,oh thats a long time away,5|5|5_5|5|5,
hit:8527_conv:17054,5,anticipating,I can't wait for game of thrones to come back. Tv is terrible without it,27,I know. and its the last season,5|5|5_5|5|5,
hit:8527_conv:17055,1,anticipating,i cant wait for the weekend,238,i cant wait for the weekend,5|5|5_5|5|5,
hit:8527_conv:17055,2,anticipating,i cant wait for the weekend,27,whats going on?,5|5|5_5|5|5,
hit:8527_conv:17055,3,anticipating,i cant wait for the weekend,238,wanna go fishing,5|5|5_5|5|5,
hit:8527_conv:17055,4,anticipating,i cant wait for the weekend,27,i love fishing,5|5|5_5|5|5,
hit:8528_conv:17057,1,sentimental,I'm totally missing my family today... can't believe it's been almost 6 months I moved away,397,I'm totally missing my family today... can't believe it's been almost 6 months I moved away,5|5|5_4|4|5,
hit:8528_conv:17057,2,sentimental,I'm totally missing my family today... can't believe it's been almost 6 months I moved away,117,I live far away from my family so I understand completely. Why did you move?,5|5|5_4|4|5,
hit:8528_conv:17057,3,sentimental,I'm totally missing my family today... can't believe it's been almost 6 months I moved away,397,for work... just one state over but I miss seeing them so often,5|5|5_4|4|5,
hit:8528_conv:17057,4,sentimental,I'm totally missing my family today... can't believe it's been almost 6 months I moved away,117,Of course_comma_ it's always better when family is close. I hope you can visit with them soon!,5|5|5_4|4|5,
hit:8529_conv:17058,1,angry,Someone stole my package off my porch! This is ridiculous.,530,I ordered a bunch of video games and someone stole the box off my porch!,5|5|5_4|4|5,
hit:8529_conv:17058,2,angry,Someone stole my package off my porch! This is ridiculous.,375,Wow that's awful_comma_ do you know who?,5|5|5_4|4|5,
hit:8529_conv:17058,3,angry,Someone stole my package off my porch! This is ridiculous.,530,Probably one of these kids around here. They were expensive too. All retro games from the 90s.,5|5|5_4|4|5,
hit:8529_conv:17058,4,angry,Someone stole my package off my porch! This is ridiculous.,375,I used to love the old starfox games.,5|5|5_4|4|5,
hit:8529_conv:17058,5,angry,Someone stole my package off my porch! This is ridiculous.,530,That was one of them I ordered! It just makes me livid!,5|5|5_4|4|5,
hit:8529_conv:17059,1,devastated,My aunt died of cancer. She was sick for 2 years.,375,I just lost my aunt.,4|4|5_5|5|5,
hit:8529_conv:17059,2,devastated,My aunt died of cancer. She was sick for 2 years.,530,I'm so sorry to hear that!,4|4|5_5|5|5,
hit:8529_conv:17059,3,devastated,My aunt died of cancer. She was sick for 2 years.,375,She had breast cancer_comma_ it was awful.,4|4|5_5|5|5,
hit:8529_conv:17059,4,devastated,My aunt died of cancer. She was sick for 2 years.,530,That is awful_comma_ I can't wait until they find a cure.,4|4|5_5|5|5,
hit:8530_conv:17060,1,caring,My friend has been taking care of his elderly mom_comma_ going to school and work full-time.,1,My friend has been taking care of his elderly mom_comma_ going to school and work full-time.,5|5|5_5|5|5,
hit:8530_conv:17060,2,caring,My friend has been taking care of his elderly mom_comma_ going to school and work full-time.,473,Wow_comma_ that's amazing of him_comma_ it must be really hard.,5|5|5_5|5|5,
hit:8530_conv:17060,3,caring,My friend has been taking care of his elderly mom_comma_ going to school and work full-time.,1,It is hard for him. I try to help_comma_ when I can.,5|5|5_5|5|5,
hit:8530_conv:17060,4,caring,My friend has been taking care of his elderly mom_comma_ going to school and work full-time.,473,That's really kind of you_comma_ I bet he appreciates all the help he can get big or small.,5|5|5_5|5|5,
hit:8531_conv:17062,1,caring,My girlfriend visited everyday while I was sick.,1,My girlfriend visited everyday while I was sick.,5|5|5_5|5|5,
hit:8531_conv:17062,2,caring,My girlfriend visited everyday while I was sick.,56,I'm sorry you were sick_comma_ but that's a good girlfriend taking care of you like that.,5|5|5_5|5|5,
hit:8531_conv:17062,3,caring,My girlfriend visited everyday while I was sick.,1,Yea. She had school all day and work_comma_ but she still came over to cook and clean for me.,5|5|5_5|5|5,
hit:8531_conv:17062,4,caring,My girlfriend visited everyday while I was sick.,56,That's pretty special_comma_ you should probably keep her!,5|5|5_5|5|5,
hit:8531_conv:17063,1,grateful,My daughter needed a computer for school.  My dad just gave her a surface pro.,56,My daughter needed a computer for school and we were trying to figure out which one to buy and how much we could spend.,5|5|5_5|5|5,
hit:8531_conv:17063,2,grateful,My daughter needed a computer for school.  My dad just gave her a surface pro.,1,Did you guys figure it out?,5|5|5_5|5|5,
hit:8531_conv:17063,3,grateful,My daughter needed a computer for school.  My dad just gave her a surface pro.,56,Well_comma_ in the midst of trying to figure it out_comma_ my dad called and said he had a Surface Pro that he didn't use.  So he gave it to her and it was a huge blessing to our family.,5|5|5_5|5|5,
hit:8531_conv:17063,4,grateful,My daughter needed a computer for school.  My dad just gave her a surface pro.,1,That is really good timing! I think dads have that instinct when their children need help.,5|5|5_5|5|5,
hit:8532_conv:17064,1,disappointed,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,294,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,5|5|5_5|5|5,
hit:8532_conv:17064,2,disappointed,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,438,Oh man!  That's sick!  So close!  That is torture.  ,5|5|5_5|5|5,
hit:8532_conv:17064,3,disappointed,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,294,I know_comma_ the excitement was building as every number came up_comma_ then.....so excited....NOTHING!,5|5|5_5|5|5,
hit:8532_conv:17064,4,disappointed,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,438,Horrible!!! I know you probably could've used it to go on a vacay_comma_ right? Outer Banks beach or something? Right? Sorry man.,5|5|5_5|5|5,
hit:8532_conv:17064,5,disappointed,I had 4 out of 5 lottery numbers right_comma_ but I didn't get any money for it.,294,You have never been so right.  It's like you took the words right out of my mouth right down to the Outer Banks.  I love that place.,5|5|5_5|5|5,
hit:8532_conv:17065,1,afraid,I keep seeing shadows outside my front windows and then hearing footsteps by the door but when I check_comma_ there's no one there.,438,I keep seeing shadows outside my front windows and then hearing footsteps by the door but when I check_comma_ there's no one there.,5|5|5_5|5|5,
hit:8532_conv:17065,2,afraid,I keep seeing shadows outside my front windows and then hearing footsteps by the door but when I check_comma_ there's no one there.,294,That is terrifying.  Do you think someone is messing with you_comma_ or do you think it is nothing?,5|5|5_5|5|5,
hit:8532_conv:17065,3,afraid,I keep seeing shadows outside my front windows and then hearing footsteps by the door but when I check_comma_ there's no one there.,438,I think my crazy nut neighbor might be trying to sneak a peek at me in my pjs.  I need to move from this duplex!  Move to the beach!,5|5|5_5|5|5,
hit:8532_conv:17065,4,afraid,I keep seeing shadows outside my front windows and then hearing footsteps by the door but when I check_comma_ there's no one there.,294,I k now exactly what you mean.  Your neighbor must be really weird.  I used to want to move to the beach_comma_ but then my wife got really weird about it.  Now she is obsessed and I'm kinda' cooling on the idea.  She can be really overbearing.  You know what I mean_comma_ man?,5|5|5_5|5|5,
hit:8533_conv:17066,1,disappointed,I missed the sale on wool socks. They are pretty pricey.,1,I missed the sale on wool socks. They are pretty pricey.,5|5|5_5|5|5,
hit:8533_conv:17066,2,disappointed,I missed the sale on wool socks. They are pretty pricey.,284,Are those the 20 dollar socks why are they so expensive? ,5|5|5_5|5|5,
hit:8533_conv:17066,3,disappointed,I missed the sale on wool socks. They are pretty pricey.,1,I have no idea. My feet get real cold_comma_ and I heard they're supposed to help.,5|5|5_5|5|5,
hit:8533_conv:17066,4,disappointed,I missed the sale on wool socks. They are pretty pricey.,284,I'll take the regular socks and just double them lol must be some magical socks. ,5|5|5_5|5|5,
hit:8534_conv:17068,1,anticipating,I am super excited for the new super smash bros!,10,I am super excited for the new super smash bros!,5|5|5_5|5|5,
hit:8534_conv:17068,2,anticipating,I am super excited for the new super smash bros!,1,Sweet. Will there be new characters?,5|5|5_5|5|5,
hit:8534_conv:17068,3,anticipating,I am super excited for the new super smash bros!,10,Oh yeah_comma_ all the most requested ones,5|5|5_5|5|5,
hit:8534_conv:17068,4,anticipating,I am super excited for the new super smash bros!,1,When did you start playing?,5|5|5_5|5|5,
hit:8534_conv:17069,1,confident,My friend has been a good dad for 2 decades. I'm sure his daughter will do well_comma_ when she goes off to college.,1,My friend has been a good dad for 2 decades. I'm sure his daughter will do well_comma_ when she goes off to college.,5|5|5_5|5|5,
hit:8534_conv:17069,2,confident,My friend has been a good dad for 2 decades. I'm sure his daughter will do well_comma_ when she goes off to college.,10,That is good_comma_ that is rare these days unfortunately,5|5|5_5|5|5,
hit:8534_conv:17069,3,confident,My friend has been a good dad for 2 decades. I'm sure his daughter will do well_comma_ when she goes off to college.,1,You're right. It is rare and quite sad.,5|5|5_5|5|5,
hit:8534_conv:17069,4,confident,My friend has been a good dad for 2 decades. I'm sure his daughter will do well_comma_ when she goes off to college.,10,That is sad but in our times_comma_ that should not be taken for granted,5|5|5_5|5|5,
hit:8535_conv:17070,1,hopeful,Ordered pizza from a new place. Should be coming soon.,1,Ordered pizza from a new place. Should be coming soon,5|5|5_5|5|5,
hit:8535_conv:17070,2,hopeful,Ordered pizza from a new place. Should be coming soon.,305,What style of pizza did you order?,5|5|5_5|5|5,
hit:8535_conv:17070,3,hopeful,Ordered pizza from a new place. Should be coming soon.,1,It's a regular large. Nothing fancy lol. I've just never tried this place before.,5|5|5_5|5|5,
hit:8535_conv:17070,4,hopeful,Ordered pizza from a new place. Should be coming soon.,305,Which toppings did you get?,5|5|5_5|5|5,
hit:8536_conv:17072,1,jealous,My friend got some sweet new jackets and I didn't get one. They were so nice.,375,So jealous of my friend.,4|5|4_5|5|5,
hit:8536_conv:17072,2,jealous,My friend got some sweet new jackets and I didn't get one. They were so nice.,1,Why? What's going on?,4|5|4_5|5|5,
hit:8536_conv:17072,3,jealous,My friend got some sweet new jackets and I didn't get one. They were so nice.,375,All my friends got amazing jackets and I didnt.,4|5|4_5|5|5,
hit:8536_conv:17072,4,jealous,My friend got some sweet new jackets and I didn't get one. They were so nice.,1,Did they buy it?,4|5|4_5|5|5,
hit:8536_conv:17073,1,surprised,I applied to a scholarship over a year ago. I actually got news that I won it today.,1,I applied to a scholarship over a year ago. I actually got news that I won it today.,5|5|5_4|5|4,
hit:8536_conv:17073,2,surprised,I applied to a scholarship over a year ago. I actually got news that I won it today.,375,You must be so proud of yourself.,5|5|5_4|5|4,
hit:8536_conv:17073,3,surprised,I applied to a scholarship over a year ago. I actually got news that I won it today.,1,Yea_comma_ I am_comma_ and I'm so thakful to the people who granted it to me.,5|5|5_4|5|4,
hit:8536_conv:17073,4,surprised,I applied to a scholarship over a year ago. I actually got news that I won it today.,375,What school did you pick?,5|5|5_4|5|4,
hit:8537_conv:17074,1,disappointed,We are going to have to cancel our Labor Day plans.  My husband was told he will have to work over the holiday instead and that he can't have the time off.,56,It looks like we are going to have to cancel our Labor Day plans.,5|5|5_5|5|5,
hit:8537_conv:17074,2,disappointed,We are going to have to cancel our Labor Day plans.  My husband was told he will have to work over the holiday instead and that he can't have the time off.,530,Oh no! Why?,5|5|5_5|5|5,
hit:8537_conv:17074,3,disappointed,We are going to have to cancel our Labor Day plans.  My husband was told he will have to work over the holiday instead and that he can't have the time off.,56,Well_comma_ his boss told him that he's going to have to work over the holiday instead and that he can't have the time off.  I'm kinda heartbroken over it_comma_ it was our only planned vacation this year.,5|5|5_5|5|5,
hit:8537_conv:17074,4,disappointed,We are going to have to cancel our Labor Day plans.  My husband was told he will have to work over the holiday instead and that he can't have the time off.,530,That sucks_comma_ maybe you guys can plan something for another weekend before it gets cold. I'm sure it will be cheaper too.,5|5|5_5|5|5,
hit:8537_conv:17075,1,afraid,I keep hearing noises outside my window. It sounds like scratching.,530,I keep hearing noises out my window and I'm home alone. It's very concerning.,5|5|5_5|5|5,
hit:8537_conv:17075,2,afraid,I keep hearing noises outside my window. It sounds like scratching.,56,Should you call the police or something?  I hate being home alone for this exact reason.,5|5|5_5|5|5,
hit:8537_conv:17075,3,afraid,I keep hearing noises outside my window. It sounds like scratching.,530,I don't know_comma_ it's just scratching so maybe it's just a raccoon.,5|5|5_5|5|5,
hit:8537_conv:17075,4,afraid,I keep hearing noises outside my window. It sounds like scratching.,56,Or a cat maybe?  Maybe you should look into installing those motion flood lights or something so that you feel safer when you are home alone.,5|5|5_5|5|5,


hit:8754_conv:17508,1,confident,I got a new haircut the other day after not cutting it for years. It feels so good to have some type of change_comma_  and I feel so good about myself just from such a small change!,227, "I got a new haircut the other day after not cutting it for years. It feels so good to have some type of change_comma_ and I feel so good about myself just from such a small change!,5|5|5_5|5|5,
hit:8754_conv:17508,2,confident,I got a new haircut the other day after not cutting it for years. It feels so good to have some type of change_comma_  and I feel so good about myself just from such a small change!,10,How long did you let your hair get?,5|5|5_5|5|5,
hit:8754_conv:17508,3,confident,I got a new haircut the other day after not cutting it for years. It feels so good to have some type of change_comma_  and I feel so good about myself just from such a small change!,227,It only got to midway down my arms_comma_ but I never got it cut for years because I just have a fear of cutting my hair like it will never grow back or something. It was only 2 inches but looks so much better,5|5|5_5|5|5,
hit:8754_conv:17508,4,confident,I got a new haircut the other day after not cutting it for years. It feels so good to have some type of change_comma_  and I feel so good about myself just from such a small change!,10,Good_comma_ I am glad you like the new change!,5|5|5_5|5|5,
hit:8754_conv:17509,1,sentimental,I get emotional about my brothers_comma_ they are very important to me,10,I get emotional about my brothers_comma_ they are very important to me,5|5|5_5|5|5,
hit:8754_conv:17509,2,sentimental,I get emotional about my brothers_comma_ they are very important to me,227,I understand_comma_ I have a brother as well. Do they live with you?,5|5|5_5|5|5,
hit:8754_conv:17509,3,sentimental,I get emotional about my brothers_comma_ they are very important to me,10,Yes_comma_ they both live in the same house as me,5|5|5_5|5|5,
hit:8754_conv:17509,4,sentimental,I get emotional about my brothers_comma_ they are very important to me,227,Oh okay thats good! Mine went across the country to college last year_comma_ so I get emotional about that. ,5|5|5_5|5|5,
hit:8755_conv:17511,1,terrified,The other night I was walking alone and a guy kept following me. I turned to go in another direction and he still followed. I ended up getting OFF the walkway and walking quickly so I could lose him.,296,The other night I was walking alone and a guy kept following me. ,5|5|5_4|5|5,
hit:8755_conv:17511,2,terrified,The other night I was walking alone and a guy kept following me. I turned to go in another direction and he still followed. I ended up getting OFF the walkway and walking quickly so I could lose him.,346,I hope you had something to protect you? Like  a whistle or pepper spray? ,5|5|5_4|5|5,
hit:8755_conv:17511,3,terrified,The other night I was walking alone and a guy kept following me. I turned to go in another direction and he still followed. I ended up getting OFF the walkway and walking quickly so I could lose him.,296,I did not that night. You just can't let men or anyone get close to you at night...It's too much of a risk. I turned to go in another direction and he still followed. I ended up getting OFF the walkway and walking quickly so I could lose him. I have a small pepper spray now but I had forgotten it.,5|5|5_4|5|5,
hit:8755_conv:17511,4,terrified,The other night I was walking alone and a guy kept following me. I turned to go in another direction and he still followed. I ended up getting OFF the walkway and walking quickly so I could lose him.,346,Thank goodness the outcome was good. Everyone needs something on them to deter someone that might be wanting to harm you. ,5|5|5_4|5|5,
hit:8756_conv:17512,1,content,Over the weekend instead of going out_comma_ I decided to stay home and work in my garden. It was very relaxing and I wouldn't trade it for anything,302,I am getting started on my work week. I didn't go out much this past weekend and instead worked in my garden. It was very tranquil.,4|5|5_5|5|5,
hit:8756_conv:17512,2,content,Over the weekend instead of going out_comma_ I decided to stay home and work in my garden. It was very relaxing and I wouldn't trade it for anything,100,That sounds very relaxing_comma_ what do you grow?,4|5|5_5|5|5,
hit:8756_conv:17512,3,content,Over the weekend instead of going out_comma_ I decided to stay home and work in my garden. It was very relaxing and I wouldn't trade it for anything,302,I have tomatoes_comma_ peppers_comma_ corn_comma_ and summer squash coming up now. I just put down some winter squash so I'm good for when it's cold.,4|5|5_5|5|5,
hit:8756_conv:17512,4,content,Over the weekend instead of going out_comma_ I decided to stay home and work in my garden. It was very relaxing and I wouldn't trade it for anything,100,Wow that's quite the collection_comma_ you must be pretty experienced.,4|5|5_5|5|5,
hit:8756_conv:17512,5,content,Over the weekend instead of going out_comma_ I decided to stay home and work in my garden. It was very relaxing and I wouldn't trade it for anything,302,I think so_comma_ I have been doing it for awhile now.,4|5|5_5|5|5,
hit:8756_conv:17513,1,faithful,I asked my friend to get some stuff for a party_comma_ I knew I could count on him though.,100,I asked my friend to get some stuff for a party_comma_ I knew I could count on him though.,5|5|5_4|5|5,
hit:8756_conv:17513,2,faithful,I asked my friend to get some stuff for a party_comma_ I knew I could count on him though.,302,How many people are you expecting? I hope he pulls through.,5|5|5_4|5|5,
hit:8756_conv:17513,3,faithful,I asked my friend to get some stuff for a party_comma_ I knew I could count on him though.,100,About 50 people roughly_comma_ I have no doubt he will pull through though.,5|5|5_4|5|5,
hit:8756_conv:17513,4,faithful,I asked my friend to get some stuff for a party_comma_ I knew I could count on him though.,302,Wow that sounds like a big shindig! I bet everyone is going to have a great time_comma_ good for you!,5|5|5_4|5|5,
hit:8757_conv:17515,1,apprehensive,I got a job offer yesterday. I am not sure if I should take this new position or not.,297,I just received a job offer but I am not sure if I should take it or not.,5|5|5_5|5|5,
hit:8757_conv:17515,2,apprehensive,I got a job offer yesterday. I am not sure if I should take this new position or not.,607,Is it a new field? How's the pay and benifits?,5|5|5_5|5|5,
hit:8757_conv:17515,3,apprehensive,I got a job offer yesterday. I am not sure if I should take this new position or not.,297,The pay and benefits are great but the location is not ideal. So I dont know.,5|5|5_5|5|5,
hit:8757_conv:17515,4,apprehensive,I got a job offer yesterday. I am not sure if I should take this new position or not.,607,You could always give it a shot for a few weeks. If you don't like it_comma_ you can try for new opportunities.,5|5|5_5|5|5,
hit:8758_conv:17516,1,guilty,I took my best friend's sister out on a date secretly.  I feel so ashamed.,322,I took my best friend's sister out on a date secretly.,5|5|5_5|5|5,
hit:8758_conv:17516,2,guilty,I took my best friend's sister out on a date secretly.  I feel so ashamed.,4,Why didn't you tell your best friend?,5|5|5_5|5|5,
hit:8758_conv:17516,3,guilty,I took my best friend's sister out on a date secretly.  I feel so ashamed.,322,He would be upset.  I feel so ashamed.,5|5|5_5|5|5,
hit:8758_conv:17516,4,guilty,I took my best friend's sister out on a date secretly.  I feel so ashamed.,4,Are you going to go out with his sister again?,5|5|5_5|5|5,
hit:8758_conv:17516,5,guilty,I took my best friend's sister out on a date secretly.  I feel so ashamed.,322,I don't think so.,5|5|5_5|5|5,
hit:8758_conv:17517,1,angry,I am so mad that I lost my wallet last night.,4,I am so mad that I lost my wallet last night.,5|5|5_5|5|5,
hit:8758_conv:17517,2,angry,I am so mad that I lost my wallet last night.,322,I'm sorry.  I hope you find it.,5|5|5_5|5|5,
hit:8758_conv:17517,3,angry,I am so mad that I lost my wallet last night.,4,If not I am in big trouble due to it having all my Id's.,5|5|5_5|5|5,
hit:8758_conv:17517,4,angry,I am so mad that I lost my wallet last night.,322,You also need to cancel any credit cards.,5|5|5_5|5|5,
hit:8759_conv:17518,1,annoyed,Really not looking forward to having to pay student loans once I graduate. Why does it cost so much to get an education? How do young people survive right out of college when they get slammed with all of this debt? It seems kind of rigged,273,Really not looking forward to having to pay student loans once I graduate. Why does it cost so much to get an education? How do young people survive right out of college when they get slammed with all of this debt? It seems kind of rigged,5|5|5_5|5|5,
hit:8759_conv:17518,2,annoyed,Really not looking forward to having to pay student loans once I graduate. Why does it cost so much to get an education? How do young people survive right out of college when they get slammed with all of this debt? It seems kind of rigged,220,I have no idea. As someone with loans myself_comma_ it makes me so angry thinking about how much money I've been robbed. I wasn't thinking at 18 years old_comma_ I didn't understand the amount.,5|5|5_5|5|5,
hit:8759_conv:17518,3,annoyed,Really not looking forward to having to pay student loans once I graduate. Why does it cost so much to get an education? How do young people survive right out of college when they get slammed with all of this debt? It seems kind of rigged,273,Well it's not necessarily that it was dumb to take out loans_comma_ it's just the amount you have to take out_comma_ how expensive college is_comma_ how much interest is paid on them_comma_ how you can escape them from bankruptcy_comma_ and how you have almost no time at all before you have to start paying them back before you've even gotten on your feet after graduating... Sorry_comma_ just stressful,5|5|5_5|5|5,
hit:8759_conv:17518,4,annoyed,Really not looking forward to having to pay student loans once I graduate. Why does it cost so much to get an education? How do young people survive right out of college when they get slammed with all of this debt? It seems kind of rigged,220,No_comma_ I understand. It's really stressful and someone needs to do something about it.,5|5|5_5|5|5,
hit:8759_conv:17519,1,joyful,My birthday is soon!,220,My birthday is soon_comma_ it's always a wonderful time to reflect. It's a great time of year for me!,5|5|5_5|5|5,
hit:8759_conv:17519,2,joyful,My birthday is soon!,273,Well hey_comma_ happy early birthday. I'm happy you take a positive spin on it.. I wish I could do that myself. I hate mine_comma_ always dread when it comes. Makes me feel like I haven't achieved what I would have liked to have achieved at the age I'm turning to_comma_ so I just get self-conscious about it.,5|5|5_5|5|5,
hit:8759_conv:17519,3,joyful,My birthday is soon!,220,Thank you. I think you need to be a little easier on yourself. No one's life is perfect or just as they want it but we need to make the most out of what we have.,5|5|5_5|5|5,
hit:8759_conv:17519,4,joyful,My birthday is soon!,273,That's good advice! I will try to. In general I don't like talking about myself_comma_ or having attention put on me_comma_ so that's partly the reason birthdays aren't my thing either. But it's good to have people who like theirs,5|5|5_5|5|5,
hit:8760_conv:17521,1,hopeful,Feeling positive about this week. Hoping my partner grows by leaps and bounds after a month in therapy.,296,Feeling positive about this week. ,5|5|5_5|5|5,
hit:8760_conv:17521,2,hopeful,Feeling positive about this week. Hoping my partner grows by leaps and bounds after a month in therapy.,310,I'm glad!  Anything good happening?,5|5|5_5|5|5,
hit:8760_conv:17521,3,hopeful,Feeling positive about this week. Hoping my partner grows by leaps and bounds after a month in therapy.,296,Well sure! I am alive_comma_ I have some of my health. A new baby puppy_comma_ money etc etc! Good food. Lots of positives :) Also Hoping my partner grows by leaps and bounds after a month in therapy.,5|5|5_5|5|5,
hit:8760_conv:17521,4,hopeful,Feeling positive about this week. Hoping my partner grows by leaps and bounds after a month in therapy.,310,Lots of positives!  I'm sending good vibes your way!,5|5|5_5|5|5,
hit:8761_conv:17522,1,surprised,My brother is having a baby_comma_ I'm shocked.,4,My brother is having a baby_comma_ I'm shocked.,5|5|5_4|5|5,
hit:8761_conv:17522,2,surprised,My brother is having a baby_comma_ I'm shocked.,302,Congratulations! Why are you so surprised?,5|5|5_4|5|5,
hit:8761_conv:17522,3,surprised,My brother is having a baby_comma_ I'm shocked.,4,Cause I didn't even know he talked to girls.,5|5|5_4|5|5,
hit:8761_conv:17522,4,surprised,My brother is having a baby_comma_ I'm shocked.,302,Oh wow_comma_ well I'm sure he's going to be a great father! That is such amazing news!,5|5|5_4|5|5,
hit:8761_conv:17523,1,anticipating,I am slow cooking some pork shoulder at the moment_comma_ it's going to take awhile but I bet it's going to be very tasty. I'm cooking a lot of it.,302,I'm in the middle of breaking down some pork shoulder. I have to slow cook for an eternity haha_comma_ it's going to be so tasty.,4|5|5_5|5|5,
hit:8761_conv:17523,2,anticipating,I am slow cooking some pork shoulder at the moment_comma_ it's going to take awhile but I bet it's going to be very tasty. I'm cooking a lot of it.,4,I love pork shoulder.,4|5|5_5|5|5,
hit:8761_conv:17523,3,anticipating,I am slow cooking some pork shoulder at the moment_comma_ it's going to take awhile but I bet it's going to be very tasty. I'm cooking a lot of it.,302,I will mail you some_comma_ haha! I'm cooking a lot of it_comma_ I will be set for a very long time.,4|5|5_5|5|5,
hit:8761_conv:17523,4,anticipating,I am slow cooking some pork shoulder at the moment_comma_ it's going to take awhile but I bet it's going to be very tasty. I'm cooking a lot of it.,4,I will take that offer_comma_ I could eat pork shoulder every day.,4|5|5_5|5|5,
hit:8762_conv:17524,1,excited,I get to go to Greece in a few weeks! I've been wanting to go there forever,234,I get to go to Greece in a few weeks! ,5|5|5_5|5|5,
hit:8762_conv:17524,2,excited,I get to go to Greece in a few weeks! I've been wanting to go there forever,10,Wow_comma_ how did you afford that?,5|5|5_5|5|5,
hit:8762_conv:17524,3,excited,I get to go to Greece in a few weeks! I've been wanting to go there forever,234,I budgeted for a year to make sure I could afford it. Well worth it!,5|5|5_5|5|5,
hit:8762_conv:17524,4,excited,I get to go to Greece in a few weeks! I've been wanting to go there forever,10,Awesome_comma_ I hope it is all worth it,5|5|5_5|5|5,
hit:8762_conv:17525,1,excited,I am hyped for the new super smash bros game coming out for the switch soon,10,I am hyped for the new super smash bros game coming out for the switch soon,5|5|5_5|5|5,
hit:8762_conv:17525,2,excited,I am hyped for the new super smash bros game coming out for the switch soon,234,Haha every character ever_comma_ right? I bet that will be fun,5|5|5_5|5|5,
hit:8762_conv:17525,3,excited,I am hyped for the new super smash bros game coming out for the switch soon,10,It sure will be_comma_ I am super excited for it_comma_ plus they will add more characters I am sure,5|5|5_5|5|5,
hit:8762_conv:17525,4,excited,I am hyped for the new super smash bros game coming out for the switch soon,234,Yup. I heard it's all of them and it will be the biggest smash ever!,5|5|5_5|5|5,
hit:8763_conv:17526,1,nostalgic,I enjoy working on and driving my old truck.,225,I have an old truck that I enjoy working on and driving around.,5|5|5_5|5|5,
hit:8763_conv:17526,2,nostalgic,I enjoy working on and driving my old truck.,343,What great fun! My husband had a 1960 Ford F150 he completely redid. It was quite an attraction around town!,5|5|5_5|5|5,
hit:8763_conv:17526,3,nostalgic,I enjoy working on and driving my old truck.,225,That's cool. Mine is a 68 chevy c10,5|5|5_5|5|5,
hit:8763_conv:17526,4,nostalgic,I enjoy working on and driving my old truck.,343,Nice! Where did you find it?,5|5|5_5|5|5,
hit:8763_conv:17526,5,nostalgic,I enjoy working on and driving my old truck.,225,It's a family heirloom.,5|5|5_5|5|5,
hit:8763_conv:17526,6,nostalgic,I enjoy working on and driving my old truck.,343,Even better. Are you doing all the work yourself or do you have a restoration person helping you?,5|5|5_5|5|5,
hit:8763_conv:17526,7,nostalgic,I enjoy working on and driving my old truck.,225,I've done it all myself. It takes a long time_comma_ but that's part of the fun.,5|5|5_5|5|5,
hit:8763_conv:17526,8,nostalgic,I enjoy working on and driving my old truck.,343,That's true. You must be a very good mechanic to do all that - I know the amount of work it takes!,5|5|5_5|5|5,
hit:8764_conv:17528,1,confident,My class is finally over and now I just have to wait for my grade. I know I did well.,243,My class is finally over and now I just wait for my grade to be posted.,5|5|5_5|5|5,
hit:8764_conv:17528,2,confident,My class is finally over and now I just have to wait for my grade. I know I did well.,262,How do you think you did?,5|5|5_5|5|5,
hit:8764_conv:17528,3,confident,My class is finally over and now I just have to wait for my grade. I know I did well.,243,I know I did very well.,5|5|5_5|5|5,
hit:8764_conv:17528,4,confident,My class is finally over and now I just have to wait for my grade. I know I did well.,262,Well no worries then,5|5|5_5|5|5,
hit:8764_conv:17529,1,nostalgic,I remeber the good old days taking road trips with friends. No fun now with a car full of kids,262,I miss taking road trips with my friends,5|5|5_5|5|5,
hit:8764_conv:17529,2,nostalgic,I remeber the good old days taking road trips with friends. No fun now with a car full of kids,243,Maybe you could take one soon.,5|5|5_5|5|5,
hit:8764_conv:17529,3,nostalgic,I remeber the good old days taking road trips with friends. No fun now with a car full of kids,262,No fun riding fours hours with a bunch of kids. I think those days are over,5|5|5_5|5|5,
hit:8764_conv:17529,4,nostalgic,I remeber the good old days taking road trips with friends. No fun now with a car full of kids,243,Oh that is true_comma_ but maybe you could leave the kids home with babysitters. That does sound fun.,5|5|5_5|5|5,
hit:8764_conv:17529,5,nostalgic,I remeber the good old days taking road trips with friends. No fun now with a car full of kids,262,Im am going to try to get a group together to do something soon. I think it will be fun,5|5|5_5|5|5,
hit:8765_conv:17530,1,disgusted,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,273,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,5|5|5_5|5|5,
hit:8765_conv:17530,2,disgusted,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,340,I would be mad as heck. Did you confront them about this behavior?,5|5|5_5|5|5,
hit:8765_conv:17530,3,disgusted,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,273,I'm not really a confrontational person... I asked my roommate to say something since he knows him better obviously. He said he brought it up_comma_ but didn't seem to change much. Then I kind of said something too_comma_ but the guy just kind of laughed it off_comma_ like he thinks we're just joking. I dunno what goes through his head,5|5|5_5|5|5,
hit:8765_conv:17530,4,disgusted,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,340,Well_comma_ I might leave him a surprise in the fridge for the next time he comes over. Some Ex-Lax chocolate pudding or something.,5|5|5_5|5|5,
hit:8765_conv:17530,5,disgusted,My roommate's friend comes over and just eats all of the food in our fridge without really asking. I don't get how people can have a total lack of self-awareness and can be so selfish to just barge in and eat other people's food. I cringe at thinking about being that rude to someone,273,Haha... maybe I'll take that into consideration,5|5|5_5|5|5,
hit:8766_conv:17532,1,sentimental,I get emotional about my brothers_comma_ they are very important to me,10,I get emotional about my brothers_comma_ they are very important to me,5|5|5_5|5|5,
hit:8766_conv:17532,2,sentimental,I get emotional about my brothers_comma_ they are very important to me,220,Is there a reason you get so emotional besides them being improtant to you?,5|5|5_5|5|5,
hit:8766_conv:17532,3,sentimental,I get emotional about my brothers_comma_ they are very important to me,10,I guess not_comma_ I just do not want them to move away or leave,5|5|5_5|5|5,
hit:8766_conv:17532,4,sentimental,I get emotional about my brothers_comma_ they are very important to me,220,Tell them that!,5|5|5_5|5|5,
hit:8766_conv:17533,1,jealous,I hate seeing my friends take fancy vacations.,220,I hate seeing my friends take fancy vacations all the time.,5|5|5_5|5|5,
hit:8766_conv:17533,2,jealous,I hate seeing my friends take fancy vacations.,10,I understand that_comma_ it is not fun to not make money the way others do,5|5|5_5|5|5,
hit:8766_conv:17533,3,jealous,I hate seeing my friends take fancy vacations.,220,It is_comma_ and with social media it's like we are bombarded by the rich and famous.,5|5|5_5|5|5,
hit:8766_conv:17533,4,jealous,I hate seeing my friends take fancy vacations.,10,Exactly_comma_ it makes being happy tough sometimes,5|5|5_5|5|5,
hit:8768_conv:17536,1,hopeful,I've been feeling horrible lately and went to the doctor. The doctor determined I have a thyroid problem_comma_ so I'm optimistic that I'll be feeling better soon.,104,I've been feeling so horrible lately and I went to the doctor.,5|5|5_5|5|5,
hit:8768_conv:17536,2,hopeful,I've been feeling horrible lately and went to the doctor. The doctor determined I have a thyroid problem_comma_ so I'm optimistic that I'll be feeling better soon.,302,I'm so sorry to hear that. Any news back yet on what it could be?,5|5|5_5|5|5,
hit:8768_conv:17536,3,hopeful,I've been feeling horrible lately and went to the doctor. The doctor determined I have a thyroid problem_comma_ so I'm optimistic that I'll be feeling better soon.,104,Yes! The doctor determined that I have a thyroid problem and started me on medication. I'm optimistic that I'll be feeling better soon!,5|5|5_5|5|5,
hit:8768_conv:17536,4,hopeful,I've been feeling horrible lately and went to the doctor. The doctor determined I have a thyroid problem_comma_ so I'm optimistic that I'll be feeling better soon.,302,Excellent! Thyroid issues are easily treatable though they can be a burden to deal with. I hope you feel better as soon as possible!,5|5|5_5|5|5,
hit:8768_conv:17537,1,terrified,I heard a group of coyotes howling the other night. I had not heard them all summer long. I abhor running into them.,302,I live out in the middle of nowhere and after not hearing them at all this summer_comma_ I heard a pack of coyotes the other night.,5|5|5_5|5|5,
hit:8768_conv:17537,2,terrified,I heard a group of coyotes howling the other night. I had not heard them all summer long. I abhor running into them.,104,That would be really scary! What do you do if they get close to your home?,5|5|5_5|5|5,
hit:8768_conv:17537,3,terrified,I heard a group of coyotes howling the other night. I had not heard them all summer long. I abhor running into them.,302,I try to set up natural barriers to deter them. I'm sure they are scare of me more so than I am of them but they can be quite the nuisance and scary to see on sight.,5|5|5_5|5|5,
hit:8768_conv:17537,4,terrified,I heard a group of coyotes howling the other night. I had not heard them all summer long. I abhor running into them.,104,I would be terrified! It sounds like you have a good system to deal with them!,5|5|5_5|5|5,
hit:8769_conv:17538,1,annoyed,My sister insisted on popping a pimple on my back. I really didn't want her to do it.,548,My sister insisted on popping a pimple on my back. I really didn't want her to do it.,5|5|5_5|5|5,
hit:8769_conv:17538,2,annoyed,My sister insisted on popping a pimple on my back. I really didn't want her to do it.,475,lol I don't blame you. Did you end up letting her?,5|5|5_5|5|5,
hit:8769_conv:17538,3,annoyed,My sister insisted on popping a pimple on my back. I really didn't want her to do it.,548,She was sitting behind me and helped herself to it.,5|5|5_5|5|5,
hit:8769_conv:17538,4,annoyed,My sister insisted on popping a pimple on my back. I really didn't want her to do it.,475,My mother used to do that when I was young. I hated it. hurt so bad with her long nails.,5|5|5_5|5|5,
hit:8770_conv:17540,1,hopeful,I am hoping the new super smash bros will be excellent_comma_ it looks the part so far!,10,I am hoping the new super smash bros will be excellent_comma_ it looks the part so far!,5|5|5_5|5|5,
hit:8770_conv:17540,2,hopeful,I am hoping the new super smash bros will be excellent_comma_ it looks the part so far!,243,When does it come out? Sounds cool!,5|5|5_5|5|5,
hit:8770_conv:17540,3,hopeful,I am hoping the new super smash bros will be excellent_comma_ it looks the part so far!,10,In december_comma_ still far away to me!,5|5|5_5|5|5,
hit:8770_conv:17540,4,hopeful,I am hoping the new super smash bros will be excellent_comma_ it looks the part so far!,243,Oh that is a ways away! Can you preorder?,5|5|5_5|5|5,
hit:8770_conv:17541,1,jealous,My boyfriend is starting to get on my nerves. He is always talking to other girls.,243,My boyfriend is starting to get on my nerves.,5|5|5_5|5|5,
hit:8770_conv:17541,2,jealous,My boyfriend is starting to get on my nerves. He is always talking to other girls.,10,Why is that?,5|5|5_5|5|5,
hit:8770_conv:17541,3,jealous,My boyfriend is starting to get on my nerves. He is always talking to other girls.,243,He is always talking to other girls. ,5|5|5_5|5|5,
hit:8770_conv:17541,4,jealous,My boyfriend is starting to get on my nerves. He is always talking to other girls.,10,Oh_comma_ well if he is flirting_comma_ I would dump him,5|5|5_5|5|5,
hit:8771_conv:17542,1,surprised,This year seems to have been a bit on the slow side for me.,225,This has been a long year.,5|5|5_5|5|5,
hit:8771_conv:17542,2,surprised,This year seems to have been a bit on the slow side for me.,340,What has made it such a long  year for you?,5|5|5_5|5|5,
hit:8771_conv:17542,3,surprised,This year seems to have been a bit on the slow side for me.,225,Financial troubles mostly. I've been working my tail off to solve that.,5|5|5_5|5|5,
hit:8771_conv:17542,4,surprised,This year seems to have been a bit on the slow side for me.,340,Well_comma_ I hope you can get that stuff taken care of_comma_ I have had lots of trouble with money in my life. Takes a lot of hard work to fix it.,5|5|5_5|5|5,
hit:8772_conv:17544,1,furious,I walked my kid all the way to school and then realized I had forgetten her back pack. I had to make a second trip,262,I walked my kids all the way to school and then realized I had forgotten her backpack,5|5|5_5|5|5,
hit:8772_conv:17544,2,furious,I walked my kid all the way to school and then realized I had forgetten her back pack. I had to make a second trip,4,What did you do after you realized that?,5|5|5_5|5|5,
hit:8772_conv:17544,3,furious,I walked my kid all the way to school and then realized I had forgetten her back pack. I had to make a second trip,262,I had to make a second trip. Its a long walk. I was so mad at myself,5|5|5_5|5|5,
hit:8772_conv:17544,4,furious,I walked my kid all the way to school and then realized I had forgetten her back pack. I had to make a second trip,4,Exercise is a good thing at least.,5|5|5_5|5|5,
hit:8772_conv:17544,5,furious,I walked my kid all the way to school and then realized I had forgetten her back pack. I had to make a second trip,262,Yeah I may have lost a couple of pounds already,5|5|5_5|5|5,
hit:8772_conv:17545,1,faithful,I have never lied to my wife.,4,I have never lied to my wife.,5|5|5_5|5|5,
hit:8772_conv:17545,2,faithful,I have never lied to my wife.,262,thats great,5|5|5_5|5|5,
hit:8772_conv:17545,3,faithful,I have never lied to my wife.,4,I have been tempted to lie_comma_ but I never can bring myself to do it.,5|5|5_5|5|5,
hit:8772_conv:17545,4,faithful,I have never lied to my wife.,262,well you are a rare breed.,5|5|5_5|5|5,
hit:8773_conv:17546,1,apprehensive,I'm feeling very concerned. I lent my brother my car and he's a horrible driver. I'm really hoping he brings it back in one piece.,104,I'm feeling so concerned this morning. I lent my brother my car and he's a horrible driver.,5|5|5_5|5|5,
hit:8773_conv:17546,2,apprehensive,I'm feeling very concerned. I lent my brother my car and he's a horrible driver. I'm really hoping he brings it back in one piece.,10,Oh_comma_ I hope it turns out ookay for you!,5|5|5_5|5|5,
hit:8773_conv:17546,3,apprehensive,I'm feeling very concerned. I lent my brother my car and he's a horrible driver. I'm really hoping he brings it back in one piece.,104,Thanks_comma_ me too! He was stuck and needed it and it's only a car_comma_ right? Still_comma_ I know I'll be on the edge of my seat all day.,5|5|5_5|5|5,
hit:8773_conv:17546,4,apprehensive,I'm feeling very concerned. I lent my brother my car and he's a horrible driver. I'm really hoping he brings it back in one piece.,10,That was kind of you_comma_ so hopefully it does not backfire,5|5|5_5|5|5,
hit:8773_conv:17547,1,lonely,Before I met my girlfriend and friends I was very alone_comma_ that was not a fun time,10,Before I met my girlfriend and friends I was very alone_comma_ that was not a fun time,5|5|5_5|5|5,
hit:8773_conv:17547,2,lonely,Before I met my girlfriend and friends I was very alone_comma_ that was not a fun time,104,Aw_comma_ I bet it wasn't! You must have felt terribly lonely! I'm glad you found some new friends!,5|5|5_5|5|5,
hit:8773_conv:17547,3,lonely,Before I met my girlfriend and friends I was very alone_comma_ that was not a fun time,10,Thank you_comma_ I am glad as well_comma_ life has improved a lot,5|5|5_5|5|5,
hit:8773_conv:17547,4,lonely,Before I met my girlfriend and friends I was very alone_comma_ that was not a fun time,104,That's great! Congrats to you on your new life!,5|5|5_5|5|5,
hit:8775_conv:17550,1,angry,I am mad at my job recently_comma_ they are overworking us a lot,10,I am mad at my job recently_comma_ they are overworking us a lot,5|5|5_5|5|5,
hit:8775_conv:17550,2,angry,I am mad at my job recently_comma_ they are overworking us a lot,302,That's terrible. Are you working more hours or are they cramming it into the hours you already have?,5|5|5_5|5|5,
hit:8775_conv:17550,3,angry,I am mad at my job recently_comma_ they are overworking us a lot,10,More hours and also more work crammed in_comma_ so bad on both sides,5|5|5_5|5|5,
hit:8775_conv:17550,4,angry,I am mad at my job recently_comma_ they are overworking us a lot,302,I hate when management does that. I would ask you if they are compensating you for overtime but if they're already treating you like this_comma_ then I doubt it.,5|5|5_5|5|5,
hit:8775_conv:17551,1,proud,My wife graduated from a masters program over the summer. She worked incredibly hard at it for and it paid off!,302,It took a long time to do it due to a lot of hardships in life_comma_ but my wife finally graduated from a masters program over the Summer.,5|5|5_5|5|5,
hit:8775_conv:17551,2,proud,My wife graduated from a masters program over the summer. She worked incredibly hard at it for and it paid off!,10,Wow_comma_ wat is her degree in?,5|5|5_5|5|5,
hit:8775_conv:17551,3,proud,My wife graduated from a masters program over the summer. She worked incredibly hard at it for and it paid off!,302,Her degree is in forensic chemistry. She already landed a job right after graduating too_comma_ we are both ecstatic.,5|5|5_5|5|5,
hit:8775_conv:17551,4,proud,My wife graduated from a masters program over the summer. She worked incredibly hard at it for and it paid off!,10,Wow_comma_ that is fantastic_comma_ good for you guys!,5|5|5_5|5|5,
hit:9486_conv:18972,1,jealous,My roomate just bought a brand new car. Its the exact same model that I wanted.,244,"My roomate just bought a brand new car. Its the exact same model that I wanted.,5|5|5_5|5|5,
hit:9486_conv:18972,2,jealous,My roomate just bought a brand new car. Its the exact same model that I wanted.,639,That would make me jealous. Do they let you ride in it or anythign?,5|5|5_5|5|5,
hit:9486_conv:18972,3,jealous,My roomate just bought a brand new car. Its the exact same model that I wanted.,244,Yeah I got to take it for a spin.  I don't think that helped though.,5|5|5_5|5|5,
hit:9486_conv:18972,4,jealous,My roomate just bought a brand new car. Its the exact same model that I wanted.,639,I understand. That sounds frustrating,5|5|5_5|5|5,
hit:9486_conv:18973,1,afraid,A few years ago a SWAT team broke down the door to our apartment complex. I went and hid in the bathroom.,639,A few years ago a SWAT team broke down the door to our apartment complex.,5|5|5_5|5|5,
hit:9486_conv:18973,2,afraid,A few years ago a SWAT team broke down the door to our apartment complex. I went and hid in the bathroom.,244,Why did they do that?,5|5|5_5|5|5,
hit:9486_conv:18973,3,afraid,A few years ago a SWAT team broke down the door to our apartment complex. I went and hid in the bathroom.,639,They came to arrest our neighbors_comma_ we think it was for drug dealing.,5|5|5_5|5|5,
hit:9486_conv:18973,4,afraid,A few years ago a SWAT team broke down the door to our apartment complex. I went and hid in the bathroom.,244,Crappy neighbors are crappy. That must have been pretty scary.,5|5|5_5|5|5,
hit:9487_conv:18974,1,disappointed,I got a hotdog lunch the other day with my last 5.95 and I dropped it while I was getting into my car.,294,I got a hotdog lunch the other day with my last 5.95 and I dropped it while I was getting into my car.,5|5|5_5|5|5,
hit:9487_conv:18974,2,disappointed,I got a hotdog lunch the other day with my last 5.95 and I dropped it while I was getting into my car.,244,That's the worst.  No money and now hungry.,5|5|5_5|5|5,
hit:9487_conv:18974,3,disappointed,I got a hotdog lunch the other day with my last 5.95 and I dropped it while I was getting into my car.,294,I wanted to cry_comma_ but then I just toughened up and wondered why I was so broke_comma_ so I got a second job.,5|5|5_5|5|5,
hit:9487_conv:18974,4,disappointed,I got a hotdog lunch the other day with my last 5.95 and I dropped it while I was getting into my car.,244,I know how that goes. I got a side job so I could help pay off debt to get a house.,5|5|5_5|5|5,
hit:9487_conv:18975,1,lonely,I haven't had a girlfriend for a year.,244,I haven't had a girlfriend for a year.,5|5|5_5|5|5,
hit:9487_conv:18975,2,lonely,I haven't had a girlfriend for a year.,294,Are you looking for one_comma_ or are you enjoying your singlehood?,5|5|5_5|5|5,
hit:9487_conv:18975,3,lonely,I haven't had a girlfriend for a year.,244,I'm lookin'  Good women are hard to come by.,5|5|5_5|5|5,
hit:9487_conv:18975,4,lonely,I haven't had a girlfriend for a year.,294,That is true_comma_ but they are out there.  Keep looking and you will find one.  Where are you trying to find them?,5|5|5_5|5|5,
hit:9487_conv:18975,5,lonely,I haven't had a girlfriend for a year.,244,Anywhere I can.. ha.,5|5|5_5|5|5,
hit:9488_conv:18977,1,confident,I knew as soon as I left the interview room that the job was mine. i was very secure with the answers i gave.,658,I knew as soon as I left the interview room that the job was mine. i was very secure with the answers i gave,5|5|5_5|5|5,
hit:9488_conv:18977,2,confident,I knew as soon as I left the interview room that the job was mine. i was very secure with the answers i gave.,340,It's great when you can be so confident after a interview. So I am guessing you got the job?,5|5|5_5|5|5,
hit:9488_conv:18977,3,confident,I knew as soon as I left the interview room that the job was mine. i was very secure with the answers i gave.,658,i did get the job_comma_ and i love the work i'm doing!,5|5|5_5|5|5,
hit:9488_conv:18977,4,confident,I knew as soon as I left the interview room that the job was mine. i was very secure with the answers i gave.,340,I am so glad to hear that for you. I hope this job goes really great for you for many years to come.,5|5|5_5|5|5,
hit:9489_conv:18978,1,apprehensive,I made a big mistake at work. I am afraid of the consequences.,7,I made a big mistake at work. I am afraid of the consequences.,5|5|5_5|5|5,
hit:9489_conv:18978,2,apprehensive,I made a big mistake at work. I am afraid of the consequences.,147,Oh no. What happened? Hopefully they will understand it was a mistake.,5|5|5_5|5|5,
hit:9489_conv:18978,3,apprehensive,I made a big mistake at work. I am afraid of the consequences.,7,I made a mistake in one of our client account. It will cost the company thousands of dollars.,5|5|5_5|5|5,
hit:9489_conv:18978,4,apprehensive,I made a big mistake at work. I am afraid of the consequences.,147,Oh I am sorry that happened. Hopefully they understand it was a simple mistake. ,5|5|5_5|5|5,
hit:9490_conv:18981,1,afraid,I went to take the trash out last night and there was a stray dog in my driveway.,294,I went to take the trash out last night and there was a stray dog in my driveway.,5|5|5_5|5|5,
hit:9490_conv:18981,2,afraid,I went to take the trash out last night and there was a stray dog in my driveway.,420,Oh no.  Was the dog friendly?,5|5|5_5|5|5,
hit:9490_conv:18981,3,afraid,I went to take the trash out last night and there was a stray dog in my driveway.,294,No_comma_ it was growling at me_comma_ so I dropped the bag and ran back into the house.  I went out this morning and their was trash all over the place.,5|5|5_5|5|5,
hit:9490_conv:18981,4,afraid,I went to take the trash out last night and there was a stray dog in my driveway.,420,Oh wow.  Are you sure it was not a coyote?  They are around a lot as well.,5|5|5_5|5|5,
hit:9491_conv:18983,1,caring,I adopted an 8 week old kitten last week from the shelter. He will be apart of the family for a long time,506,I adopted an 8 week old kitten last week from the shelter. ,5|5|5_5|5|5,
hit:9491_conv:18983,2,caring,I adopted an 8 week old kitten last week from the shelter. He will be apart of the family for a long time,147,Awe. Have you named her already?,5|5|5_5|5|5,
hit:9491_conv:18983,3,caring,I adopted an 8 week old kitten last week from the shelter. He will be apart of the family for a long time,506,His name is Bosco_comma_ he will be apart of the family for a long time,5|5|5_5|5|5,
hit:9491_conv:18983,4,caring,I adopted an 8 week old kitten last week from the shelter. He will be apart of the family for a long time,147,Thats such a sweet name_comma_ and very kind of you.,5|5|5_5|5|5,
hit:9492_conv:18984,1,disappointed,I went to bed and found cat puke on my rug and stepped in it,659,I was getting ready to go to sleep and i walked near the bed when I realized I stepped in cat puke. ugh,5|5|5_5|5|5,
hit:9492_conv:18984,2,disappointed,I went to bed and found cat puke on my rug and stepped in it,284,OMG so gross I would have been so mad where was your cat at the time.,5|5|5_5|5|5,
hit:9492_conv:18984,3,disappointed,I went to bed and found cat puke on my rug and stepped in it,659,She was laying on the couch like nothing was wrong!,5|5|5_5|5|5,
hit:9492_conv:18984,4,disappointed,I went to bed and found cat puke on my rug and stepped in it,284,Lol that is to funny have to be careful next time.,5|5|5_5|5|5,
hit:9492_conv:18984,5,disappointed,I went to bed and found cat puke on my rug and stepped in it,659,Yes_comma_ going to steam clean the rug today. so gross,5|5|5_5|5|5,
hit:9493_conv:18986,1,proud,I submitted a big budget proposal that was approved yesterday. I put in a lot of hard work and am happy with the outcome,506,I  submitted a big budget proposal that was approved yesterday.,5|5|5_5|5|5,
hit:9493_conv:18986,2,proud,I submitted a big budget proposal that was approved yesterday. I put in a lot of hard work and am happy with the outcome,507,that is awesome_comma_ are you planning on doing anything to celebrate?,5|5|5_5|5|5,
hit:9493_conv:18986,3,proud,I submitted a big budget proposal that was approved yesterday. I put in a lot of hard work and am happy with the outcome,506,Maybe celebrate with a nice dinner_comma_ I am really happy with the outcome,5|5|5_5|5|5,
hit:9493_conv:18986,4,proud,I submitted a big budget proposal that was approved yesterday. I put in a lot of hard work and am happy with the outcome,507,that is awesome_comma_ are you planning on doing anything to celebrate?,5|5|5_5|5|5,
hit:9494_conv:18988,1,apprehensive,I got offered another job today... Not sure If I should take it. The pay is about the same.,244,I got offered another job today... Not sure If I should take it. The pay is about the same.,5|5|5_5|5|5,
hit:9494_conv:18988,2,apprehensive,I got offered another job today... Not sure If I should take it. The pay is about the same.,616,What kind of job is it?,5|5|5_5|5|5,
hit:9494_conv:18988,3,apprehensive,I got offered another job today... Not sure If I should take it. The pay is about the same.,244,IT,5|5|5_5|5|5,<UNIGRAM>
hit:9494_conv:18988,4,apprehensive,I got offered another job today... Not sure If I should take it. The pay is about the same.,616,Is that similar to what you do now? Sometimes change is good.,5|5|5_5|5|5,
hit:9494_conv:18988,5,apprehensive,I got offered another job today... Not sure If I should take it. The pay is about the same.,244,Yeah but_comma_ the only quesiton is _comma_ is this place better_comma_ worse or the same?,5|5|5_5|5|5,
hit:9495_conv:18990,1,content,I'm happy with my new diet_comma_ I don't feel hungry and have cravings like I usually do,247,I'm happy with my new diet_comma_ I don't feel hungry and have cravings like I usually do,5|5|5_5|5|5,
hit:9495_conv:18990,2,content,I'm happy with my new diet_comma_ I don't feel hungry and have cravings like I usually do,310,That's great!  I've had trouble with changing my diet before.  What kind are you doing?,5|5|5_5|5|5,
hit:9495_conv:18990,3,content,I'm happy with my new diet_comma_ I don't feel hungry and have cravings like I usually do,247,the ketogenic diet_comma_ I feel a lot more energetic now that I've cut carbs out. It's like my body was slowed down from processing all the sugars,5|5|5_5|5|5,
hit:9495_conv:18990,4,content,I'm happy with my new diet_comma_ I don't feel hungry and have cravings like I usually do,310,I've heard that about keto.  I might just try it sometime!,5|5|5_5|5|5,
hit:9496_conv:18993,1,jealous,My friend got an advance_comma_ signed copy of a book. That book was written by my favorite author.,639,My friend got an advance signed copy of a book by my favorite author.,5|5|5_5|5|5,
hit:9496_conv:18993,2,jealous,My friend got an advance_comma_ signed copy of a book. That book was written by my favorite author.,284,That is awesome who is your favorite author hopefully you can meet that person now.,5|5|5_5|5|5,
hit:9496_conv:18993,3,jealous,My friend got an advance_comma_ signed copy of a book. That book was written by my favorite author.,639,Jasper Fforde. His books are really nerdy and awesome.,5|5|5_5|5|5,
hit:9496_conv:18993,4,jealous,My friend got an advance_comma_ signed copy of a book. That book was written by my favorite author.,284,I never heard of him what is a name of the book your friend plans on writing with this author?,5|5|5_5|5|5,
hit:9497_conv:18994,1,surprised,I could not believe I wont a raffle at my company last week. I usually never win!,506,I could not believe I wont a raffle at my company last week.,5|5|5_5|5|5,
hit:9497_conv:18994,2,surprised,I could not believe I wont a raffle at my company last week. I usually never win!,294,That is amazing_comma_ I never win anything.  What was the prize?,5|5|5_5|5|5,
hit:9497_conv:18994,3,surprised,I could not believe I wont a raffle at my company last week. I usually never win!,506,It was 2 plan tickets for anywhere in the world,5|5|5_5|5|5,
hit:9497_conv:18994,4,surprised,I could not believe I wont a raffle at my company last week. I usually never win!,294,Holy cow_comma_ what a great prize.  Congratulations!,5|5|5_5|5|5,
hit:9497_conv:18995,1,proud,My son is learning to overcome his speech impediment.  He is working so hard at it.,294,My son is learning to overcome his speech impediment.  He is working so hard at it.,5|5|5_5|5|5,
hit:9497_conv:18995,2,proud,My son is learning to overcome his speech impediment.  He is working so hard at it.,506,What age is he now?,5|5|5_5|5|5,
hit:9497_conv:18995,3,proud,My son is learning to overcome his speech impediment.  He is working so hard at it.,294,He is 5 and has trouble with his R's.  I know it is best to get rid of it now_comma_ but it is so cute.  But he is amazing for working at it so hard.,5|5|5_5|5|5,
hit:9497_conv:18995,4,proud,My son is learning to overcome his speech impediment.  He is working so hard at it.,506,I hope he can get over it soon!,5|5|5_5|5|5,
hit:9498_conv:18996,1,surprised,My friend planned a party for me without telling me.,59,My friend planned a party for me when I returned from college without telling me!,5|5|5_5|5|5,
hit:9498_conv:18996,2,surprised,My friend planned a party for me without telling me.,313,Oh man! I would be so surprised! Where you happy for it?,5|5|5_5|5|5,
hit:9498_conv:18996,3,surprised,My friend planned a party for me without telling me.,59,I was! They made cookies for me and I ate them all the same day.,5|5|5_5|5|5,
hit:9498_conv:18996,4,surprised,My friend planned a party for me without telling me.,313,I'm so excited! I want some cookies now! ,5|5|5_5|5|5,
hit:9499_conv:18998,1,hopeful,Just bought a lottery ticket because the pot is getting pretty big. I love buying these things because you get to go home and think about all the stuff you'd do with the winnings,273,Just bought a lottery ticket because the pot is getting pretty big. I love buying these things because you get to go home and think about all the stuff you'd do with the winnings,3|5|5_5|5|5,
hit:9499_conv:18998,2,hopeful,Just bought a lottery ticket because the pot is getting pretty big. I love buying these things because you get to go home and think about all the stuff you'd do with the winnings,147,I do the same! I will be crossing my fingers for tonight's lottery! ,3|5|5_5|5|5,
hit:9499_conv:18998,3,hopeful,Just bought a lottery ticket because the pot is getting pretty big. I love buying these things because you get to go home and think about all the stuff you'd do with the winnings,273,I used to give my mom crap all the time about it_comma_ because she'd buy one every week. Then she'd talk about all the nice things she'd do and get for people when she won. But now I can see it was just her way of relieving stress and having fun,3|5|5_5|5|5,
hit:9499_conv:18998,4,hopeful,Just bought a lottery ticket because the pot is getting pretty big. I love buying these things because you get to go home and think about all the stuff you'd do with the winnings,147,Oh that is awesome you are able to understand your mom now_comma_,3|5|5_5|5|5,
hit:9500_conv:19000,1,grateful,Scratched someone's car in the parking lot today_comma_ bumped it a little with mine as I was pulling in. They were right there when it happened_comma_ too. There was a little damage... but they said to not worry about it_comma_ that they didn't want to get people involved. That was very nice of them,273,Scratched someone's car in the parking lot today_comma_ bumped it a little with mine as I was pulling in. They were right there when it happened_comma_ too. There was a little damage... but they said to not worry about it_comma_ that they didn't want to get people involved. That was very nice of them,3|5|5_4|4|4,
hit:9500_conv:19000,2,grateful,Scratched someone's car in the parking lot today_comma_ bumped it a little with mine as I was pulling in. They were right there when it happened_comma_ too. There was a little damage... but they said to not worry about it_comma_ that they didn't want to get people involved. That was very nice of them,317,That was very nice of them. So you didn't have to pay anything?,3|5|5_4|4|4,
hit:9500_conv:19000,3,grateful,Scratched someone's car in the parking lot today_comma_ bumped it a little with mine as I was pulling in. They were right there when it happened_comma_ too. There was a little damage... but they said to not worry about it_comma_ that they didn't want to get people involved. That was very nice of them,273,Nope_comma_ nothing. I asked multiple times. I offered anything and everything. I think they could see that I didn't have a lot (that my car was crappy and that I was a young college-age student) and that it was an accident_comma_ so they didn't want to rake me over the coals for it,3|5|5_4|4|4,
hit:9500_conv:19000,4,grateful,Scratched someone's car in the parking lot today_comma_ bumped it a little with mine as I was pulling in. They were right there when it happened_comma_ too. There was a little damage... but they said to not worry about it_comma_ that they didn't want to get people involved. That was very nice of them,317,Wow! That is truly awesome. Congratulations,3|5|5_4|4|4,
hit:9501_conv:19002,1,apprehensive,I start my new job on Monday. I almost feel like I am having second thoughts about it though.,297,I start my new job on Monday. I almost feel like I am having second thoughts about it though.,5|5|5_5|5|5,
hit:9501_conv:19002,2,apprehensive,I start my new job on Monday. I almost feel like I am having second thoughts about it though.,639,Why are you having second thoughts? I have been there.,5|5|5_5|5|5,
hit:9501_conv:19002,3,apprehensive,I start my new job on Monday. I almost feel like I am having second thoughts about it though.,297,I am just always a little hesitant about starting something new.,5|5|5_5|5|5,
hit:9501_conv:19002,4,apprehensive,I start my new job on Monday. I almost feel like I am having second thoughts about it though.,639,I'm sure you will be great!,5|5|5_5|5|5,
hit:9501_conv:19003,1,nostalgic,I recently watched Matilda again. It was my favorite movie as a kid.,639,I recently watched Matilda again. It was one of mmy favorites when I was a kid.,5|5|5_5|5|5,
hit:9501_conv:19003,2,nostalgic,I recently watched Matilda again. It was my favorite movie as a kid.,297,Oh I remember that movie. I used to watch it all the time. What was it like watching it again now?,5|5|5_5|5|5,
hit:9501_conv:19003,3,nostalgic,I recently watched Matilda again. It was my favorite movie as a kid.,639,It really holds up. It is a little funnier but overall still cute and happy.,5|5|5_5|5|5,
hit:9501_conv:19003,4,nostalgic,I recently watched Matilda again. It was my favorite movie as a kid.,297,I may have to go back and check it out.,5|5|5_5|5|5,
hit:9502_conv:19004,1,nostalgic,I took a stroll down memory lane with my best childhood friend last night.,294,I took a stroll down memory lane with my best childhood friend last night.,5|5|5_5|5|5,
hit:9502_conv:19004,2,nostalgic,I took a stroll down memory lane with my best childhood friend last night.,243,I bet that was fun!,5|5|5_5|5|5,
hit:9502_conv:19004,3,nostalgic,I took a stroll down memory lane with my best childhood friend last night.,294,It was_comma_ it brought back a lot of good memories of when I was younger_comma_ and reminded me of how good life is.,5|5|5_5|5|5,
hit:9502_conv:19004,4,nostalgic,I took a stroll down memory lane with my best childhood friend last night.,243,That's something we should all do more often!,5|5|5_5|5|5,
hit:9502_conv:19005,1,sad,My old cat is very sick. We will have to put her down next week if she is not better.,243,My old cat is very sick.,5|5|5_5|5|5,
hit:9502_conv:19005,2,sad,My old cat is very sick. We will have to put her down next week if she is not better.,294,I'm sorry to hear that.  How old is the cat?,5|5|5_5|5|5,
hit:9502_conv:19005,3,sad,My old cat is very sick. We will have to put her down next week if she is not better.,243,She is 23. If she isn't better in a few days_comma_ we will have to put her down.,5|5|5_5|5|5,
hit:9502_conv:19005,4,sad,My old cat is very sick. We will have to put her down next week if she is not better.,294,Ohh no.  That is a long life for a cat_comma_ and I know that makes it even harder.  I hope she gets better!,5|5|5_5|5|5,
hit:9503_conv:19006,1,disgusted,I went to get an apple this morning and it was rotting. Completely ruined my breakfast.,639,I went to get an apple this morning. It was rotting.,5|5|5_5|5|5,
hit:9503_conv:19006,2,disgusted,I went to get an apple this morning and it was rotting. Completely ruined my breakfast.,244,That's disappointing. I know how that goes. Its happened to be before,5|5|5_5|5|5,
hit:9503_conv:19006,3,disgusted,I went to get an apple this morning and it was rotting. Completely ruined my breakfast.,639,It was_comma_ it completely ruined my breakfast.,5|5|5_5|5|5,
hit:9503_conv:19006,4,disgusted,I went to get an apple this morning and it was rotting. Completely ruined my breakfast.,244,Sorry :(,5|5|5_5|5|5,<IRREGULAR_COLON_FORMAT>
hit:9503_conv:19007,1,terrified,I got home for lunch and found a bat outside on my front porch.    ,244,I got home for lunch and found a bat outside on my front porch. It probably has rabies. Bats shouldn't be out during the day.,5|5|5_5|5|5,
hit:9503_conv:19007,2,terrified,I got home for lunch and found a bat outside on my front porch.    ,639,Doesn't rabies cause sensativity to light? Either way I would freak out...,5|5|5_5|5|5,
hit:9503_conv:19007,3,terrified,I got home for lunch and found a bat outside on my front porch.    ,244,It can but_comma_ it also causes anmails to behave erratically... like bats wadering around in the middle of the day.,5|5|5_5|5|5,
hit:9503_conv:19007,4,terrified,I got home for lunch and found a bat outside on my front porch.    ,639,Oh yeah_comma_ gotcha. I really don't like animals that are small and move quickly,5|5|5_5|5|5,
hit:9503_conv:19007,5,terrified,I got home for lunch and found a bat outside on my front porch.    ,244,Generally yes.,5|5|5_5|5|5,
hit:9504_conv:19008,1,trusting,i knew i shouldnt have trusted my brother with my dog,238,i knew i shouldnt have trusted my brother with my dog,5|5|5_5|5|5,
hit:9504_conv:19008,2,trusting,i knew i shouldnt have trusted my brother with my dog,284,What happened did he hurt your dog? ,5|5|5_5|5|5,
hit:9504_conv:19008,3,trusting,i knew i shouldnt have trusted my brother with my dog,238,he got out of the back yard and ran away,5|5|5_5|5|5,
hit:9504_conv:19008,4,trusting,i knew i shouldnt have trusted my brother with my dog,284,Wow ave you put up signs for lost dog hopefully someone will bring him back home to you.,5|5|5_5|5|5,
hit:9505_conv:19011,1,proud,when i repared my bicycle on my own ,11,I repaired my bike all on my own_comma_ pretty neat what we can teach ourselves on the internet. ,5|5|5_5|5|5,
hit:9505_conv:19011,2,proud,when i repared my bicycle on my own ,147,Wow! Maybe you have a hidden talent lol.,5|5|5_5|5|5,
hit:9505_conv:19011,3,proud,when i repared my bicycle on my own ,11,Maybe I should open up a bike repair shop and live the dream! ,5|5|5_5|5|5,
hit:9505_conv:19011,4,proud,when i repared my bicycle on my own ,147,I don't see no reason you shouldn't try!,5|5|5_5|5|5,
hit:9506_conv:19012,1,lonely,I live really far from my family. I have been missing them a lot lately.,639,I live really far from my family. I have been missing them a lot lately.,5|5|5_5|5|5,
hit:9506_conv:19012,2,lonely,I live really far from my family. I have been missing them a lot lately.,313,I would be too! Try and find a time you can go back and see them?,5|5|5_5|5|5,
hit:9506_conv:19012,3,lonely,I live really far from my family. I have been missing them a lot lately.,639,We're working on it. It is more about their schedule than mine. I work from home_comma_ I can go anywhere any time.,5|5|5_5|5|5,
hit:9506_conv:19012,4,lonely,I live really far from my family. I have been missing them a lot lately.,313,Oh man! That sounds great but bad at the same time! ,5|5|5_5|5|5,
hit:9507_conv:19015,1,caring,I showed my wife how to use a gun,52,So much stuff going on in our neighbourhood i just don't feel safe anymore. I taught my wife how to use my gun so she could protect herself when i'm at work,5|5|5_5|5|4,
hit:9507_conv:19015,2,caring,I showed my wife how to use a gun,507,that is really sad to hear_comma_ i remember when i was a kid there was no problem even leaving the door unlock,5|5|5_5|5|4,
hit:9507_conv:19015,3,caring,I showed my wife how to use a gun,52,yes_comma_ me too! and when we moved in it looked like a very safe neighborhood until recently. so much gang activity,5|5|5_5|5|4,
hit:9507_conv:19015,4,caring,I showed my wife how to use a gun,507,have you contacted local police to warned them?,5|5|5_5|5|4,
hit:9508_conv:19016,1,annoyed,Two weeks ago my mom signed me up for a therapy session without telling me in advance. She never respects my time and assumes I sit in a room and stare into the empty void all day. This made me angry_comma_ and I almost impulsively refused to go but after a few hours on the advice of a friend agreed to let her help me.,660,So my mom signed me up for therapy tomorrow and only just now decided to tell me about it today.,4|5|5_5|5|5,
hit:9508_conv:19016,2,annoyed,Two weeks ago my mom signed me up for a therapy session without telling me in advance. She never respects my time and assumes I sit in a room and stare into the empty void all day. This made me angry_comma_ and I almost impulsively refused to go but after a few hours on the advice of a friend agreed to let her help me.,273,That seems a little odd.. It sounds like you guys don't have the best communication. Did she say why she didn't ask you?,4|5|5_5|5|5,
hit:9508_conv:19016,3,annoyed,Two weeks ago my mom signed me up for a therapy session without telling me in advance. She never respects my time and assumes I sit in a room and stare into the empty void all day. This made me angry_comma_ and I almost impulsively refused to go but after a few hours on the advice of a friend agreed to let her help me.,660,Nope_comma_ she didn't say why.,4|5|5_5|5|5,
hit:9508_conv:19016,4,annoyed,Two weeks ago my mom signed me up for a therapy session without telling me in advance. She never respects my time and assumes I sit in a room and stare into the empty void all day. This made me angry_comma_ and I almost impulsively refused to go but after a few hours on the advice of a friend agreed to let her help me.,273,I see. I'm not sure that's healthy_comma_ but it's also not my business. I hope that for both your guys' sake you are able to communicate a little more properly in the future. I don't think therapy on its own can work unless everyone is willing to listen to each other.,4|5|5_5|5|5,
hit:9508_conv:19017,1,proud,My son cheated on an exam in high school last year. I was disappointed when he told me_comma_ and I gave him the choice to come clean. I told him I wouldn't turn him in_comma_ that only he could decide that. He decided to tell the teacher what he did,273,My son cheated on an exam in high school last year. I was disappointed when he told me_comma_ and I gave him the choice to come clean. I told him I wouldn't turn him in_comma_ that only he could decide that. He decided to tell the teacher what he did,5|5|5_4|5|5,
hit:9508_conv:19017,2,proud,My son cheated on an exam in high school last year. I was disappointed when he told me_comma_ and I gave him the choice to come clean. I told him I wouldn't turn him in_comma_ that only he could decide that. He decided to tell the teacher what he did,660,He did the right thing_comma_ how did the teacher react?,5|5|5_4|5|5,
hit:9508_conv:19017,3,proud,My son cheated on an exam in high school last year. I was disappointed when he told me_comma_ and I gave him the choice to come clean. I told him I wouldn't turn him in_comma_ that only he could decide that. He decided to tell the teacher what he did,273,He got the punishment he deserved. Failed the exam_comma_ failed the class. Had to make it up over the summer. Learned his lesson I'm sure. But I'd rather he do that then cover his tracks and get rewarded for dishonesty.,5|5|5_4|5|5,
hit:9508_conv:19017,4,proud,My son cheated on an exam in high school last year. I was disappointed when he told me_comma_ and I gave him the choice to come clean. I told him I wouldn't turn him in_comma_ that only he could decide that. He decided to tell the teacher what he did,660,Well_comma_ you're doing a good job as a parent. There's no justification for what he's done but he had the strength of character to confess to the teacher when confronted. He's still a good kid even if he does bad things occasionally. I hope you guys can put this past you quickly.,5|5|5_4|5|5,
hit:9509_conv:19018,1,grateful,My husband and I recently decided to get a puppy but the nearest place we could find one available is 15 hours away! We decided that we would like to make the drive for him. When my husband told our neighbor about it he offered to drive with him since I have to stay home with our kids. We were so thankful for such a helpful neighbor!,435,My husband and I recently decided we are going to get another puppy! He has to go pick him up and the drive is 15 hours!,5|5|5_5|5|5,
hit:9509_conv:19018,2,grateful,My husband and I recently decided to get a puppy but the nearest place we could find one available is 15 hours away! We decided that we would like to make the drive for him. When my husband told our neighbor about it he offered to drive with him since I have to stay home with our kids. We were so thankful for such a helpful neighbor!,11,That must be one special puppy for that drive! How adorable... are you going with him to pick it up? ,5|5|5_5|5|5,
hit:9509_conv:19018,3,grateful,My husband and I recently decided to get a puppy but the nearest place we could find one available is 15 hours away! We decided that we would like to make the drive for him. When my husband told our neighbor about it he offered to drive with him since I have to stay home with our kids. We were so thankful for such a helpful neighbor!,435,I will have to stay home because we have two kids_comma_ but our neighbor actually offered to make the drive with him!,5|5|5_5|5|5,
hit:9509_conv:19018,4,grateful,My husband and I recently decided to get a puppy but the nearest place we could find one available is 15 hours away! We decided that we would like to make the drive for him. When my husband told our neighbor about it he offered to drive with him since I have to stay home with our kids. We were so thankful for such a helpful neighbor!,11,What a nice neighbor you have_comma_ awh well happy and congrats to you and the fam. ,5|5|5_5|5|5,
hit:9509_conv:19019,1,trusting,when i let my neighbor borrow money. ,11,My neighbor really needed money for food_comma_ and so I let her borrow some. ,5|5|5_5|5|5,
hit:9509_conv:19019,2,trusting,when i let my neighbor borrow money. ,435,That was very kind of you. It's nice to have good neighbors.,5|5|5_5|5|5,
hit:9509_conv:19019,3,trusting,when i let my neighbor borrow money. ,11,Yeah we have to stick together_comma_ and I know she will pay me back. ,5|5|5_5|5|5,
hit:9509_conv:19019,4,trusting,when i let my neighbor borrow money. ,435,Well I hope you know that you're appreciated and that was a very nice thing to do.,5|5|5_5|5|5,
hit:9510_conv:19020,1,impressed,i was impressed when i saw this guy eat a 3 lbs burger and 1 lbs fries for a free t-shirt,238,i was impressed when i saw this guy eat a 3 lbs burger and 1 lbs fries for a free t-shirt,5|5|5_5|5|5,
hit:9510_conv:19020,2,impressed,i was impressed when i saw this guy eat a 3 lbs burger and 1 lbs fries for a free t-shirt,50,That's a lot of food lol i bet he felt sick afterwards. I think i would try to do the same_comma_ just curious if i could eat that much,5|5|5_5|5|5,
hit:9510_conv:19020,3,impressed,i was impressed when i saw this guy eat a 3 lbs burger and 1 lbs fries for a free t-shirt,238,yea i think he threw it up after the 15 time limit was up,5|5|5_5|5|5,
hit:9510_conv:19020,4,impressed,i was impressed when i saw this guy eat a 3 lbs burger and 1 lbs fries for a free t-shirt,50,ewww. well_comma_ at least he got a shirt. right? lol,5|5|5_5|5|5,
hit:9511_conv:19023,1,excited,I just won a bet against my friend while watching the horse races.,128,I just won a bet against my friend while watching the horse races.,5|5|5_4|4|4,
hit:9511_conv:19023,2,excited,I just won a bet against my friend while watching the horse races.,317,That's great! What did you win?,5|5|5_4|4|4,
hit:9511_conv:19023,3,excited,I just won a bet against my friend while watching the horse races.,128,$400 thanks to Red. I opted not to bet with a big group because I don't know much about horses.,5|5|5_4|4|4,
hit:9511_conv:19023,4,excited,I just won a bet against my friend while watching the horse races.,317,I don't blame you. I don't know anything about Horses_comma_ but congrats on winning the bet.,5|5|5_4|4|4,


hit:10015_conv:20030,1,content,I got all of my housework done. For the rest of the day I am satisfied to just relax and do whatever I want.,548,"I got all of my housework done. For the rest of the day I am satisfied to just relax and do whatever I want.,5|5|5_5|5|5,
hit:10015_conv:20030,2,content,I got all of my housework done. For the rest of the day I am satisfied to just relax and do whatever I want.,43,Good that you done! Have a relaxing day!,5|5|5_5|5|5,
hit:10015_conv:20030,3,content,I got all of my housework done. For the rest of the day I am satisfied to just relax and do whatever I want.,548,I intend to. I am going to binge watch one of my favorite shows and snack eat all day.,5|5|5_5|5|5,
hit:10015_conv:20030,4,content,I got all of my housework done. For the rest of the day I am satisfied to just relax and do whatever I want.,43,Awesome! enjoy your own time!,5|5|5_5|5|5,
hit:10015_conv:20031,1,impressed,My little child is dancing now. I'm amazing about her talent.,43,My little child is dancing now.,5|5|5_5|5|5,
hit:10015_conv:20031,2,impressed,My little child is dancing now. I'm amazing about her talent.,548,How sweet that is. Enjoy all these little moments. Snap a picture if you can to remember it.,5|5|5_5|5|5,
hit:10015_conv:20031,3,impressed,My little child is dancing now. I'm amazing about her talent.,43,Yeah_comma_ I'm doing. I just amazed with her dance.,5|5|5_5|5|5,
hit:10015_conv:20031,4,impressed,My little child is dancing now. I'm amazing about her talent.,548,I wish I could see her doing it. Who knows maybe you have a future ballerina on your hands.,5|5|5_5|5|5,
hit:10016_conv:20032,1,impressed,I really like the new Nike shoes.,4,I really like the new Nike shoes.,5|5|5_5|5|5,
hit:10016_conv:20032,2,impressed,I really like the new Nike shoes.,243,Nikes are usually pretty cool. What do you like about the new ones?,5|5|5_5|5|5,
hit:10016_conv:20032,3,impressed,I really like the new Nike shoes.,4,I really like the new comfort fit soles.,5|5|5_5|5|5,
hit:10016_conv:20032,4,impressed,I really like the new Nike shoes.,243,Did you buy some yet? ,5|5|5_5|5|5,
hit:10016_conv:20033,1,content,I have everything I have ever wanted in life right now. I don't want anything to change.,243,I have everything I have ever wanted.,5|5|5_5|5|5,
hit:10016_conv:20033,2,content,I have everything I have ever wanted in life right now. I don't want anything to change.,4,That must be a great feeling.,5|5|5_5|5|5,
hit:10016_conv:20033,3,content,I have everything I have ever wanted in life right now. I don't want anything to change.,243,It is and I don't want anything to change.,5|5|5_5|5|5,
hit:10016_conv:20033,4,content,I have everything I have ever wanted in life right now. I don't want anything to change.,4,Hopefully you can keep it that way.,5|5|5_5|5|5,
hit:10017_conv:20034,1,lonely,My husband went to abroad for a month. I'm badly missing him.,43,I badly miss my hubby.,5|5|5_4|4|3,
hit:10017_conv:20034,2,lonely,My husband went to abroad for a month. I'm badly missing him.,516,Why did he left?,5|5|5_4|4|3,
hit:10017_conv:20034,3,lonely,My husband went to abroad for a month. I'm badly missing him.,43,He went to London for our business work.,5|5|5_4|4|3,
hit:10017_conv:20034,4,lonely,My husband went to abroad for a month. I'm badly missing him.,516,Oh_comma_ for how long he will be away?,5|5|5_4|4|3,
hit:10017_conv:20035,1,impressed,Yesterday when my son got the highest approving grade for a test he didn't studied to,516,Yesterday my son came home with thehighest grade for his test,4|4|3_5|5|5,
hit:10017_conv:20035,2,impressed,Yesterday when my son got the highest approving grade for a test he didn't studied to,43,Wow. It's really a super moment to cherish.,4|4|3_5|5|5,
hit:10017_conv:20035,3,impressed,Yesterday when my son got the highest approving grade for a test he didn't studied to,516,It was_comma_ specially because he told me didn't studied for the test,4|4|3_5|5|5,
hit:10017_conv:20035,4,impressed,Yesterday when my son got the highest approving grade for a test he didn't studied to,43,Oh_comma_ It's great that he scored good grade though he didn't study well.,4|4|3_5|5|5,
hit:10017_conv:20035,5,impressed,Yesterday when my son got the highest approving grade for a test he didn't studied to,516,Yes_comma_ it as the highest passing grade of his class,4|4|3_5|5|5,
hit:10018_conv:20036,1,jealous,My brother has the coolest girlfriend. I wish she was mine.,4,My brother has the coolest girlfriend. I wish she was mine.,5|5|5_5|5|5,
hit:10018_conv:20036,2,jealous,My brother has the coolest girlfriend. I wish she was mine.,389,Is his name Jessie by chance?,5|5|5_5|5|5,
hit:10018_conv:20036,3,jealous,My brother has the coolest girlfriend. I wish she was mine.,4,No_comma_ why do you ask?,5|5|5_5|5|5,
hit:10018_conv:20036,4,jealous,My brother has the coolest girlfriend. I wish she was mine.,389,Because of that popular song years ago.  ,5|5|5_5|5|5,
hit:10018_conv:20037,1,devastated,My dog died today,389,I came home from work today and found my dog dead,5|5|5_5|5|5,
hit:10018_conv:20037,2,devastated,My dog died today,4,That is terrible_comma_ did you cry?,5|5|5_5|5|5,
hit:10018_conv:20037,3,devastated,My dog died today,389,Definitely!  He was my best friend for 11 years!,5|5|5_5|5|5,
hit:10018_conv:20037,4,devastated,My dog died today,4,Will you get a new dog?,5|5|5_5|5|5,
hit:10018_conv:20037,5,devastated,My dog died today,389,No I don't think so.  Not right now :(,5|5|5_5|5|5,
hit:10019_conv:20038,1,furious,I am so mad that my window broke.,4,I am so mad that my window broke.,5|5|5_4|5|5,
hit:10019_conv:20038,2,furious,I am so mad that my window broke.,513,That's horrible_comma_ what happened?,5|5|5_4|5|5,
hit:10019_conv:20038,3,furious,I am so mad that my window broke.,4,A bird flew into it.,5|5|5_4|5|5,
hit:10019_conv:20038,4,furious,I am so mad that my window broke.,513,That sounds like an awful thing to see. Will it be hard to replace?,5|5|5_4|5|5,
hit:10020_conv:20040,1,anticipating,I cannot wait to receive this package that I ordered. It should be getting here tomorrow. It is a gift for my nephew.,366,I am really excited to get this package that I had ordered a few days ago.,4|4|4_5|5|5,
hit:10020_conv:20040,2,anticipating,I cannot wait to receive this package that I ordered. It should be getting here tomorrow. It is a gift for my nephew.,4,What did you order?,4|4|4_5|5|5,
hit:10020_conv:20040,3,anticipating,I cannot wait to receive this package that I ordered. It should be getting here tomorrow. It is a gift for my nephew.,366,I had ordered this gift for my nephew_comma_ and it should be here tomorrow. It is those magnet tiles that are popular now.,4|4|4_5|5|5,
hit:10020_conv:20040,4,anticipating,I cannot wait to receive this package that I ordered. It should be getting here tomorrow. It is a gift for my nephew.,4,How much did those cost?,4|4|4_5|5|5,
hit:10020_conv:20040,5,anticipating,I cannot wait to receive this package that I ordered. It should be getting here tomorrow. It is a gift for my nephew.,366,Anywhere from as low as $15 to about $30 for a good set.,4|4|4_5|5|5,
hit:10020_conv:20041,1,hopeful,I really wish I get into law school.,4,I really wish I get into law school.,5|5|5_4|4|4,
hit:10020_conv:20041,2,hopeful,I really wish I get into law school.,366,It takes a lot of discipline to get prepared for the LSAT exam and apply to schools.,5|5|5_4|4|4,
hit:10020_conv:20041,3,hopeful,I really wish I get into law school.,4,Yes but I know I can do it.,5|5|5_4|4|4,
hit:10020_conv:20041,4,hopeful,I really wish I get into law school.,366,Good luck! I'm sure you can do it!,5|5|5_4|4|4,
hit:10021_conv:20042,1,sad,I am so upset my girlfriend dumped me.,4,I am so upset my girlfriend dumped me.,5|5|5_5|5|5,
hit:10021_conv:20042,2,sad,I am so upset my girlfriend dumped me.,43,Sorry to hear_comma_ Why did she do this to you?,5|5|5_5|5|5,
hit:10021_conv:20042,3,sad,I am so upset my girlfriend dumped me.,4,She said she needed a break.,5|5|5_5|5|5,
hit:10021_conv:20042,4,sad,I am so upset my girlfriend dumped me.,43,Oh_comma_ then you too take break!,5|5|5_5|5|5,
hit:10021_conv:20043,1,afraid,I lied to my girl friend. Now I become nervous to meet her.,43,I lied to my lady love.,5|5|5_5|5|5,
hit:10021_conv:20043,2,afraid,I lied to my girl friend. Now I become nervous to meet her.,4,What did you lie about?,5|5|5_5|5|5,
hit:10021_conv:20043,3,afraid,I lied to my girl friend. Now I become nervous to meet her.,43,I told her am busy studying and cant meet her but i was watching football with friends. Somehow she found out,5|5|5_5|5|5,
hit:10021_conv:20043,4,afraid,I lied to my girl friend. Now I become nervous to meet her.,4,She must be very mad at you.,5|5|5_5|5|5,
hit:10022_conv:20044,1,sad,My dog had a surgery and even if it goes well he wont be able to walk again after it,516,My dog will have a surgery soon,5|5|5_3|3|3,
hit:10022_conv:20044,2,sad,My dog had a surgery and even if it goes well he wont be able to walk again after it,545,Oh no_comma_ what's wrong with your dog?,5|5|5_3|3|3,
hit:10022_conv:20044,3,sad,My dog had a surgery and even if it goes well he wont be able to walk again after it,516,Someone ran over him,5|5|5_3|3|3,
hit:10022_conv:20044,4,sad,My dog had a surgery and even if it goes well he wont be able to walk again after it,545,That's terrible! I hope the operation goes smoothly.,5|5|5_3|3|3,
hit:10022_conv:20044,5,sad,My dog had a surgery and even if it goes well he wont be able to walk again after it,516,Even if it does_comma_ the vet said he wont walk again,5|5|5_3|3|3,
hit:10022_conv:20045,1,excited,I finally was able to afford a used car! It took a lot of saving but I can finally drive myself around. IT's such a relief.,545,I finally saved enough money for a used car. It was a lot of effort but I'm glad I finally can drive again.,3|3|3_5|5|5,
hit:10022_conv:20045,2,excited,I finally was able to afford a used car! It took a lot of saving but I can finally drive myself around. IT's such a relief.,516,Oh_comma_ that's pretty cool_comma_ how long did it took?,3|3|3_5|5|5,
hit:10022_conv:20045,3,excited,I finally was able to afford a used car! It took a lot of saving but I can finally drive myself around. IT's such a relief.,545,It took quite a while! About four months of extra cash.,3|3|3_5|5|5,
hit:10022_conv:20045,4,excited,I finally was able to afford a used car! It took a lot of saving but I can finally drive myself around. IT's such a relief.,516,I'm glad to hear it made you very happy_comma_ which car are you going to get?,3|3|3_5|5|5,
hit:10022_conv:20045,5,excited,I finally was able to afford a used car! It took a lot of saving but I can finally drive myself around. IT's such a relief.,545,A Honda accord. I really like Hondas for used cars. They seem to be built to last.,3|3|3_5|5|5,
hit:10023_conv:20046,1,sad,My grandma was transferred to hospice care. I have always been so close to her.,243,My grandma was just transferred to hospice care.,5|5|5_5|5|5,
hit:10023_conv:20046,2,sad,My grandma was transferred to hospice care. I have always been so close to her.,4,That is bad news_comma_ hope you go visit her.,5|5|5_5|5|5,
hit:10023_conv:20046,3,sad,My grandma was transferred to hospice care. I have always been so close to her.,243,Her and I have always been so close. I am devastated. ,5|5|5_5|5|5,
hit:10023_conv:20046,4,sad,My grandma was transferred to hospice care. I have always been so close to her.,4,Make sure you visit her every day.,5|5|5_5|5|5,
hit:10023_conv:20047,1,afraid,I really do not like pigs_comma_ they scare me.,4,I really do not like pigs_comma_ they scare me.,5|5|5_5|5|5,
hit:10023_conv:20047,2,afraid,I really do not like pigs_comma_ they scare me.,243,I can understand that. Have you ever been up close to one?,5|5|5_5|5|5,
hit:10023_conv:20047,3,afraid,I really do not like pigs_comma_ they scare me.,4,No_comma_ I do not even want to be within a mile of one.,5|5|5_5|5|5,
hit:10023_conv:20047,4,afraid,I really do not like pigs_comma_ they scare me.,243,That is how I feel about mice,5|5|5_5|5|5,
hit:10024_conv:20048,1,anticipating,I can't wait for the summer to be gone.,4,I can't wait for the summer to be gone.,5|5|5_5|5|5,
hit:10024_conv:20048,2,anticipating,I can't wait for the summer to be gone.,389,Why?  Don't like the hot weather?,5|5|5_5|5|5,
hit:10024_conv:20048,3,anticipating,I can't wait for the summer to be gone.,4,I can't stand the heat.,5|5|5_5|5|5,
hit:10024_conv:20048,4,anticipating,I can't wait for the summer to be gone.,389,I hear ya.  That's why I stay inside with the AC!!,5|5|5_5|5|5,
hit:10024_conv:20049,1,angry,Ugghhh I hate this printer!!,389,Man.  You know what really grinds my gears??,5|5|5_5|5|5,
hit:10024_conv:20049,2,angry,Ugghhh I hate this printer!!,4,What is that?,5|5|5_5|5|5,
hit:10024_conv:20049,3,angry,Ugghhh I hate this printer!!,389,This STUPID printer we have at work!!!!  I hate it hate it HATE it!!! ,5|5|5_5|5|5,
hit:10024_conv:20049,4,angry,Ugghhh I hate this printer!!,4,You should throw it out the window.,5|5|5_5|5|5,
hit:10024_conv:20049,5,angry,Ugghhh I hate this printer!!,389,I might!!! I might also take it out in the field with a baseball bat!!,5|5|5_5|5|5,
hit:10025_conv:20051,1,disappointed,My weekend plan is cancelled since the weather is not okay to go out.,43,My weekend plan is cancelled due to the weather. It sucks,5|5|5_5|5|5,
hit:10025_conv:20051,2,disappointed,My weekend plan is cancelled since the weather is not okay to go out.,278,That's never fun. hope you can find something else to do.,5|5|5_5|5|5,
hit:10025_conv:20051,3,disappointed,My weekend plan is cancelled since the weather is not okay to go out.,43,Yeah i got to do that. but  it was so frustrating it got cancellaed,5|5|5_5|5|5,
hit:10025_conv:20051,4,disappointed,My weekend plan is cancelled since the weather is not okay to go out.,278,Yea_comma_ good luck for a reschedule.,5|5|5_5|5|5,
hit:10026_conv:20053,1,lonely,My wife is going to be gone for the weekend. I'll be all alone and with nothing to do.,366,I'm feeling a little down since my wife will be gone for the weekend.,5|5|5_4|5|5,
hit:10026_conv:20053,2,lonely,My wife is going to be gone for the weekend. I'll be all alone and with nothing to do.,513,That's unfortunate. What are you going to do to keep yourself busy all weekend?,5|5|5_4|5|5,
hit:10026_conv:20053,3,lonely,My wife is going to be gone for the weekend. I'll be all alone and with nothing to do.,366,I don't really have anything to do! I will be all alone here_comma_ so I probably will sleep in.,5|5|5_4|5|5,
hit:10026_conv:20053,4,lonely,My wife is going to be gone for the weekend. I'll be all alone and with nothing to do.,513,Sleeping in should be a positive thing! Take this as your time for yourself and relax.,5|5|5_4|5|5,
hit:10027_conv:20054,1,anticipating,Me and my friends are going on a road trip. I think it will go smooth,43,Me and my friends are going on a road trip. I think it will go well,5|5|5_5|5|5,
hit:10027_conv:20054,2,anticipating,Me and my friends are going on a road trip. I think it will go smooth,4,Where will you be heading?,5|5|5_5|5|5,
hit:10027_conv:20054,3,anticipating,Me and my friends are going on a road trip. I think it will go smooth,43,Gold coast. I don't anticipate any issues though,5|5|5_5|5|5,
hit:10027_conv:20054,4,anticipating,Me and my friends are going on a road trip. I think it will go smooth,4,Where is that located.,5|5|5_5|5|5,
hit:10027_conv:20055,1,excited,I am so ready for the football season.,4,I am so ready for the football season.,5|5|5_5|5|5,
hit:10027_conv:20055,2,excited,I am so ready for the football season.,43,Wow_comma_ which team you support,5|5|5_5|5|5,
hit:10027_conv:20055,3,excited,I am so ready for the football season.,4,I am a Falcons fan.,5|5|5_5|5|5,
hit:10027_conv:20055,4,excited,I am so ready for the football season.,43,nice _comma_ good luck for your new season!!,5|5|5_5|5|5,
hit:10028_conv:20056,1,sad,I really wish my fish was still alive.,4,I really wish my fish was still alive.,5|5|5_5|5|5,
hit:10028_conv:20056,2,sad,I really wish my fish was still alive.,516,That's sad_comma_ when did it died?,5|5|5_5|5|5,
hit:10028_conv:20056,3,sad,I really wish my fish was still alive.,4,He died two weeks ago.,5|5|5_5|5|5,
hit:10028_conv:20056,4,sad,I really wish my fish was still alive.,516,Of natural causes?,5|5|5_5|5|5,
hit:10028_conv:20057,1,disappointed,When my wife didn't used the money I gave her to pay the bills we had and we almost lost our house,516,We almost lose our house last month,5|5|5_5|5|5,
hit:10028_conv:20057,2,disappointed,When my wife didn't used the money I gave her to pay the bills we had and we almost lost our house,4,What happened to make that almost happen?,5|5|5_5|5|5,
hit:10028_conv:20057,3,disappointed,When my wife didn't used the money I gave her to pay the bills we had and we almost lost our house,516,My wife didn't paid the bills with the money I gave her to,5|5|5_5|5|5,
hit:10028_conv:20057,4,disappointed,When my wife didn't used the money I gave her to pay the bills we had and we almost lost our house,4,What did she do with the money?,5|5|5_5|5|5,
hit:10028_conv:20057,5,disappointed,When my wife didn't used the money I gave her to pay the bills we had and we almost lost our house,516,She bought a purse,5|5|5_5|5|5,
hit:10029_conv:20058,1,grateful,I really appropriate all the friends I have.,4,I really appreciate all my friends I have.,5|5|5_5|5|5,
hit:10029_conv:20058,2,grateful,I really appropriate all the friends I have.,669,Friends are so useful and nice to have on happy and sad days.,5|5|5_5|5|5,
hit:10029_conv:20058,3,grateful,I really appropriate all the friends I have.,4,Yes_comma_ I prefer them on the happy days thou,5|5|5_5|5|5,
hit:10029_conv:20058,4,grateful,I really appropriate all the friends I have.,669,Same here for sure but they make gloomy days a lot better.,5|5|5_5|5|5,
hit:10029_conv:20059,1,annoyed,I get super upset whenever my partner sleeps all day.,669,Sometimes my partner can sleep for over 24 hours_comma_ it is so crazy.,5|5|5_5|5|5,
hit:10029_conv:20059,2,annoyed,I get super upset whenever my partner sleeps all day.,4,That is a long time_comma_ has he gone to a doctor?,5|5|5_5|5|5,
hit:10029_conv:20059,3,annoyed,I get super upset whenever my partner sleeps all day.,669,No_comma_ he just does that like once or twice a month.  I can sleep up to 16 hours straight if I tried.,5|5|5_5|5|5,
hit:10029_conv:20059,4,annoyed,I get super upset whenever my partner sleeps all day.,4,That is a long time also.,5|5|5_5|5|5,
hit:10029_conv:20059,5,annoyed,I get super upset whenever my partner sleeps all day.,669,It is for sure but I guess normal for us.,5|5|5_5|5|5,
hit:10030_conv:20060,1,prepared,My exams are next week. I think i will do well,43,My exams are next week and i have prepared very well. i think it will go smooth,5|5|5_5|5|5,
hit:10030_conv:20060,2,prepared,My exams are next week. I think i will do well,548,All you can do is prepare the best you can. After that there's not much else you can do.,5|5|5_5|5|5,
hit:10030_conv:20060,3,prepared,My exams are next week. I think i will do well,43,Yeah i know. It just the intuition i will do well. Exam's are scary though,5|5|5_5|5|5,
hit:10030_conv:20060,4,prepared,My exams are next week. I think i will do well,548,That they are. Just don't psych yourself out. Relax and you will do well.,5|5|5_5|5|5,
hit:10030_conv:20061,1,sentimental,I was watching an old movie and when it came to the sad part it made me cry.,548,I was watching an old movie and when it came to the sad part it made me cry.,5|5|5_5|5|5,
hit:10030_conv:20061,2,sentimental,I was watching an old movie and when it came to the sad part it made me cry.,43,Wow_comma_ you are sentimental i guess!!,5|5|5_5|5|5,
hit:10030_conv:20061,3,sentimental,I was watching an old movie and when it came to the sad part it made me cry.,548,Yes I am. My family always makes fun of me. I can cry at the drop of a hat.,5|5|5_5|5|5,
hit:10030_conv:20061,4,sentimental,I was watching an old movie and when it came to the sad part it made me cry.,43,you are who you are!!. Sentimental people are so kind hearted!!,5|5|5_5|5|5,
hit:10031_conv:20062,1,apprehensive,I am trying to decide if I want to move in with my fiancee before the wedding. I probably will_comma_ bit I dont know if its the best idea.,243,I am trying to decide if I want to move in with my fiancee before the wedding.,5|5|5_5|5|5,
hit:10031_conv:20062,2,apprehensive,I am trying to decide if I want to move in with my fiancee before the wedding. I probably will_comma_ bit I dont know if its the best idea.,332,Oh yeah that is a really big step. Have you guys talked about it yet/,5|5|5_5|5|5,
hit:10031_conv:20062,3,apprehensive,I am trying to decide if I want to move in with my fiancee before the wedding. I probably will_comma_ bit I dont know if its the best idea.,243,We have talked and I probably will so we can save money_comma_ but I am not sureof it is the right choice.,5|5|5_5|5|5,
hit:10031_conv:20062,4,apprehensive,I am trying to decide if I want to move in with my fiancee before the wedding. I probably will_comma_ bit I dont know if its the best idea.,332,It's a hard decision. I think you guys should weigh everything together before making a final choice_comma_ to be sure.,5|5|5_5|5|5,
hit:10031_conv:20063,1,disgusted,I went to my friends house and his place is just a pig sty. ,332,I went to my friend's house recently and it was horrific. ,5|5|5_5|5|5,
hit:10031_conv:20063,2,disgusted,I went to my friends house and his place is just a pig sty. ,243,That sounds terrible! What happened? ,5|5|5_5|5|5,
hit:10031_conv:20063,3,disgusted,I went to my friends house and his place is just a pig sty. ,332,It was just so dirty inside. Like he never cleaned it at all.,5|5|5_5|5|5,
hit:10031_conv:20063,4,disgusted,I went to my friends house and his place is just a pig sty. ,243,That is so gross! ,5|5|5_5|5|5,
hit:10032_conv:20064,1,surprised,I just got a check for 100_comma_000 dollars from a relative,328,So I just got a check from a family member who just passed. For 100_comma_000 dollars,5|5|5_5|5|5,
hit:10032_conv:20064,2,surprised,I just got a check for 100_comma_000 dollars from a relative,366,Wow_comma_ that is a big inheritance. What do you plan to do with it?,5|5|5_5|5|5,
hit:10032_conv:20064,3,surprised,I just got a check for 100_comma_000 dollars from a relative,328,Payoff my credit cards_comma_ invest some of it. I don't know how taxes work on this stuff so I don't want to spend too much,5|5|5_5|5|5,
hit:10032_conv:20064,4,surprised,I just got a check for 100_comma_000 dollars from a relative,366,Yeah_comma_ it's better to make some good choices with that amount of money.,5|5|5_5|5|5,
hit:10032_conv:20064,5,surprised,I just got a check for 100_comma_000 dollars from a relative,328,Better safe then sorry aye?,5|5|5_5|5|5,
hit:10032_conv:20065,1,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,366,I was so upset after I discovered that half of my take out order was completely missing from the bag!,5|5|5_5|5|5,
hit:10032_conv:20065,2,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,328,That sucks! Did you have to go back for it?,5|5|5_5|5|5,
hit:10032_conv:20065,3,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,366,Yep_comma_ and it was so frustrating because I had to drive back twenty minutes to the place.,5|5|5_5|5|5,
hit:10032_conv:20065,4,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,328,Did they at least throw in something extra for the trouble?,5|5|5_5|5|5,
hit:10032_conv:20065,5,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,366,No_comma_ they just were apologetic about it. Very frustrating.,5|5|5_5|5|5,
hit:10032_conv:20065,6,angry,I had ordered a take out order_comma_ and by the time I got home I realized it was missing half of the food! So I got upset and drove back to fix it.,328,Man sounds like they have 'great' costume service.,5|5|5_5|5|5,
hit:10033_conv:20067,1,ashamed,I didn't return my wife's phone to the phone company in time and she had to go through an ordeal having them not charge her for it and I never owned up to it.,302,The phone company almost charged my wife 500 dollars for a broken phone she had to return.,4|5|5_5|5|5,
hit:10033_conv:20067,2,ashamed,I didn't return my wife's phone to the phone company in time and she had to go through an ordeal having them not charge her for it and I never owned up to it.,547,Wow! Ask for a refund or some extra stuff!,4|5|5_5|5|5,
hit:10033_conv:20067,3,ashamed,I didn't return my wife's phone to the phone company in time and she had to go through an ordeal having them not charge her for it and I never owned up to it.,302,They fortunately worked it out and there was no charge_comma_ but the worst part was that she tasked me with returning it on time and I totally didn't. I really screwed up.,4|5|5_5|5|5,
hit:10033_conv:20067,4,ashamed,I didn't return my wife's phone to the phone company in time and she had to go through an ordeal having them not charge her for it and I never owned up to it.,547,Dang_comma_ well at least you got your money,4|5|5_5|5|5,
hit:10033_conv:20067,5,ashamed,I didn't return my wife's phone to the phone company in time and she had to go through an ordeal having them not charge her for it and I never owned up to it.,302,Yes indeed_comma_ that would've been quite the financial ding!,4|5|5_5|5|5,
hit:10034_conv:20068,1,ashamed,When I was younger my family didn't have a lot of money. One day I went into a guitar shop and stole some guitar strings. I still feel bad about that.,640,When I was about 12 years old_comma_ I went into a guitar shop and stole some strings. I always felt bad about it. They were cheap_comma_ but still_comma_ I hate that I did it.,5|5|5_5|5|5,
hit:10034_conv:20068,2,ashamed,When I was younger my family didn't have a lot of money. One day I went into a guitar shop and stole some guitar strings. I still feel bad about that.,639,I feel you. I once stole drumsticks. I swear it is something about being a teenager.,5|5|5_5|5|5,
hit:10034_conv:20068,3,ashamed,When I was younger my family didn't have a lot of money. One day I went into a guitar shop and stole some guitar strings. I still feel bad about that.,640,Haha that makes me feel better_comma_ thanks!,5|5|5_5|5|5,
hit:10034_conv:20068,4,ashamed,When I was younger my family didn't have a lot of money. One day I went into a guitar shop and stole some guitar strings. I still feel bad about that.,639,No problem!,5|5|5_5|5|5,
hit:10034_conv:20069,1,ashamed,I had to cancel plans for the weekend because I got sick to my stomach. It was really embarassing.,639,I had to cancel plans for this weekend. I got really sick last night. I feel bad.,5|5|5_5|5|5,
hit:10034_conv:20069,2,ashamed,I had to cancel plans for the weekend because I got sick to my stomach. It was really embarassing.,640,Oh that sucks_comma_ getting sick is the worst. Hope you feel better soon. Take vitamin C_comma_ it really does help some people. ,5|5|5_5|5|5,
hit:10034_conv:20069,3,ashamed,I had to cancel plans for the weekend because I got sick to my stomach. It was really embarassing.,639,I feel okay now_comma_ with a lot of sleep. But I dont want to risk going on a roadtrip.,5|5|5_5|5|5,
hit:10034_conv:20069,4,ashamed,I had to cancel plans for the weekend because I got sick to my stomach. It was really embarassing.,640,Yea better play it safe so that it doesn't get worse.,5|5|5_5|5|5,
hit:10035_conv:20070,1,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,671,I don't drink anymore and I'm careful about new medications.,4|5|5_4|5|4,
hit:10035_conv:20070,2,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,609,Congratulations! That's probably a health choice that you've made.,4|5|5_4|5|4,
hit:10035_conv:20070,3,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,671,I wish it was for health reasons.,4|5|5_4|5|4,
hit:10035_conv:20070,4,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,609,Well_comma_ I don't want to pry_comma_ but I hope everything is going well for you now and wish you all the best.,4|5|5_4|5|4,
hit:10035_conv:20070,5,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,671,The doctor warned me but I ended up making a fool of myself. ,4|5|5_4|5|4,
hit:10035_conv:20070,6,ashamed,I was on new medications. The doctor warned me I might have problems with balance. I went out anyway. I slipped on a step and fell in front of a crowd of people. ,609,Well_comma_ the past is in the past. All you can do now is try your best moving forward.,4|5|5_4|5|4,
hit:10035_conv:20071,1,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,609,A couple of years ago_comma_ when I was in college_comma_ I needed to move several heavy things from my apartment but I didn't have a car.,4|5|4_4|5|5,
hit:10035_conv:20071,2,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,671,Did you ask a friend for help?,4|5|4_4|5|5,
hit:10035_conv:20071,3,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,609,I did_comma_ actually. I didn't know him very well_comma_ but he was in my class and very generously offered to help.  I wasn't sure he would come through_comma_ but fortunately he did!,4|5|4_4|5|5,
hit:10035_conv:20071,4,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,671,That's great. So everything went well?,4|5|4_4|5|5,
hit:10035_conv:20071,5,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,609,Yep_comma_ we got everything into storage without a problem.  I was super appreciative_comma_ since I definitely couldn't afford to hire anybody on a college student's budget.,4|5|4_4|5|5,
hit:10035_conv:20071,6,trusting,A couple of years ago_comma_ I needed to move some heavy things from my apartment into storage. I was in college at the time_comma_ and didn't have a car.  A friend from a class offered to help_comma_ and even though I didn't know him very well_comma_ he came through and helped me with his truck.,671,I was worried you were going to say something bad happened. I'm relived. ,4|5|4_4|5|5,
hit:10037_conv:20074,1,ashamed,I was caught looking through old photos of an attractive friend on facebook. It was pretty bad to be called out for being a bit creepy.,315,You ever browse through old photos on a friend's Facebook page? Kinda weird right?,4|4|5_5|5|5,
hit:10037_conv:20074,2,ashamed,I was caught looking through old photos of an attractive friend on facebook. It was pretty bad to be called out for being a bit creepy.,666,I have. Sometimes it's nice to see what old friends have been up to_comma_ and what new friends have done. I don't think it's too strange.,4|4|5_5|5|5,
hit:10037_conv:20074,3,ashamed,I was caught looking through old photos of an attractive friend on facebook. It was pretty bad to be called out for being a bit creepy.,315,It is when it's an attractive friend...and it's obvious you're creeping on them. Man that's embarrassing.,4|4|5_5|5|5,
hit:10037_conv:20074,4,ashamed,I was caught looking through old photos of an attractive friend on facebook. It was pretty bad to be called out for being a bit creepy.,666,That definitely changes things. I could understand how that might be considered weird_comma_ I agree.,4|4|5_5|5|5,
hit:10037_conv:20074,5,ashamed,I was caught looking through old photos of an attractive friend on facebook. It was pretty bad to be called out for being a bit creepy.,315,We do dumb stuff when we're young. I don't think I'mma live that one down tho,4|4|5_5|5|5,


hit:10381_conv:20763,1,grateful,I really needed some help to get ready for my Moms birthday. I was so thankful that my nieces came to the rescue.,548, "I really needed some help to get ready for my Moms birthday. I was so thankful that my nieces came to the rescue.,2|2|4_4|4|4,
hit:10381_conv:20763,2,grateful,I really needed some help to get ready for my Moms birthday. I was so thankful that my nieces came to the rescue.,181,Thats so good of them,2|2|4_4|4|4,
hit:10381_conv:20763,3,grateful,I really needed some help to get ready for my Moms birthday. I was so thankful that my nieces came to the rescue.,548,They are good kids. I didn't even have to ask_comma_ they just showed up ready to help.,2|2|4_4|4|4,
hit:10381_conv:20763,4,grateful,I really needed some help to get ready for my Moms birthday. I was so thankful that my nieces came to the rescue.,181,Thats nice of them,2|2|4_4|4|4,
hit:10382_conv:20764,1,furious,I spent a while shopping and getting everything I needed. Then when I went to the car_comma_ I dropped the groceries. My cookies went everywhere.,409,I spent hours shopping and getting everything I needed. I was finally done and went to the car and dropped my cookies all over the floor. I was so mad at myself!,5|5|5_4|5|5,
hit:10382_conv:20764,2,furious,I spent a while shopping and getting everything I needed. Then when I went to the car_comma_ I dropped the groceries. My cookies went everywhere.,715,That sounds frustrating. Did you drop anything else?,5|5|5_4|5|5,
hit:10382_conv:20764,3,furious,I spent a while shopping and getting everything I needed. Then when I went to the car_comma_ I dropped the groceries. My cookies went everywhere.,409,Thankfully no. And next time I went_comma_ I bought more cookies!,5|5|5_4|5|5,
hit:10382_conv:20764,4,furious,I spent a while shopping and getting everything I needed. Then when I went to the car_comma_ I dropped the groceries. My cookies went everywhere.,715,Well_comma_ that's good that it was just the cookies? It could have been much worse.,5|5|5_4|5|5,
hit:10382_conv:20765,1,ashamed,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people.,715,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people. ,4|5|5_5|5|5,
hit:10382_conv:20765,2,ashamed,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people.,409,Oh no_comma_ I'm sorry to hear that. With five kids_comma_ that has happened to me multiple times. I can commiserate!,4|5|5_5|5|5,
hit:10382_conv:20765,3,ashamed,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people.,715,I was picking my oldest up from school. I really thought he'd be fine for 5 minutes.,4|5|5_5|5|5,
hit:10382_conv:20765,4,ashamed,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people.,409,It's okay. It happens to all of us!,4|5|5_5|5|5,
hit:10382_conv:20765,5,ashamed,I put off changing my kids diaper for 5 minutes. Poop ended up leaking out in front of many other people.,715,Thanks. It felt like the other parents were judging. That made it worse.,4|5|5_5|5|5,
hit:10383_conv:20766,1,faithful,I have recently left my job with no new position to go into. I am worried about the future_comma_ but also confident that I will find something new.,716,Hi there_comma_ I have just left my current job in education due to a variety of unpleasant circumstances_comma_ with no other job to go into! It's both frightening and liberating!,4|5|5_5|5|5,<HI>
hit:10383_conv:20766,2,faithful,I have recently left my job with no new position to go into. I am worried about the future_comma_ but also confident that I will find something new.,409,If you were unhappy_comma_ sometimes you just have to make that leap. Now you can explore your options!,4|5|5_5|5|5,
hit:10383_conv:20766,3,faithful,I have recently left my job with no new position to go into. I am worried about the future_comma_ but also confident that I will find something new.,716,I know - it's getting a bit difficult for money_comma_ and I'm struggling to find positions that match my skills_comma_ but I'm hopeful that something will come up soon.,4|5|5_5|5|5,
hit:10383_conv:20766,4,faithful,I have recently left my job with no new position to go into. I am worried about the future_comma_ but also confident that I will find something new.,409,Just keep looking. My sister started out with a temp job that has since turned into a well paying permanent job that is even paying for her education. ,4|5|5_5|5|5,
hit:10383_conv:20766,5,faithful,I have recently left my job with no new position to go into. I am worried about the future_comma_ but also confident that I will find something new.,716,Ah_comma_ that's brilliant and exciting_comma_ good for her! Hopefully something like that can happen for me too!,4|5|5_5|5|5,
hit:10383_conv:20767,1,nostalgic,I took my children to a play area I used to go to as a child. As I was there_comma_ I was remembering all the fun I used to have. It was very sweet to remember. ,409,I took my kids to a play area I used to go to as a kid. As I was there_comma_ I remembered all the good times that I had there. It was very sweet to remember. ,5|5|5_4|5|5,
hit:10383_conv:20767,2,nostalgic,I took my children to a play area I used to go to as a child. As I was there_comma_ I was remembering all the fun I used to have. It was very sweet to remember. ,716,Ah_comma_ that's lovely! I have such fond memories of running around in parks as a child. I don't think young people today really understand the value of just running around and playing.,5|5|5_4|5|5,
hit:10383_conv:20767,3,nostalgic,I took my children to a play area I used to go to as a child. As I was there_comma_ I was remembering all the fun I used to have. It was very sweet to remember. ,409,No_comma_ they are always on their screens. There are definitely too many electronics right now!,5|5|5_4|5|5,
hit:10383_conv:20767,4,nostalgic,I took my children to a play area I used to go to as a child. As I was there_comma_ I was remembering all the fun I used to have. It was very sweet to remember. ,716,Absolutely! I used to love computer games too as a kid_comma_ but I always knew I had to disconnect with them. And it's so easy for children to access anything_comma_ at any time now. Some of the content young people can access is really worrying. It's good that you took your kids to the park_comma_ I hope they enjoyed it!,5|5|5_4|5|5,
hit:10385_conv:20770,1,faithful,I recently had a big fight with my boyfriend. I went out with some friends_comma_ and I saw a guy I work with. We were talking and he said he liked me_comma_ and I always liked him. He tried to kiss me_comma_ but I told him I was taken. ,577,After having a big fight with my boyfriend_comma_ I came very close to cheating on him recently with a colleague of mine. But didn't. I couldn't bring myself to.,5|5|5_5|5|5,
hit:10385_conv:20770,2,faithful,I recently had a big fight with my boyfriend. I went out with some friends_comma_ and I saw a guy I work with. We were talking and he said he liked me_comma_ and I always liked him. He tried to kiss me_comma_ but I told him I was taken. ,717,i am glad you didn't no one deserves to be cheated on_comma_ it's the most distusting and stupid thing anyone can do,5|5|5_5|5|5,
hit:10385_conv:20770,3,faithful,I recently had a big fight with my boyfriend. I went out with some friends_comma_ and I saw a guy I work with. We were talking and he said he liked me_comma_ and I always liked him. He tried to kiss me_comma_ but I told him I was taken. ,577,True. I realized that as soon as the alcohol wore off_comma_ and I went back home to my boyfriend_comma_ then made up with him.,5|5|5_5|5|5,
hit:10385_conv:20770,4,faithful,I recently had a big fight with my boyfriend. I went out with some friends_comma_ and I saw a guy I work with. We were talking and he said he liked me_comma_ and I always liked him. He tried to kiss me_comma_ but I told him I was taken. ,717,see? and word of advice never drink without your boyfriend around_comma_ trust me it will save you and make him happy_comma_ and in return he will make you happy,5|5|5_5|5|5,
hit:10386_conv:20772,1,hopeful,I was promised a car by  my mum,686,My mum promised me a car last month if I pass my exam ,4|4|4_5|5|5,
hit:10386_conv:20772,2,hopeful,I was promised a car by  my mum,706,and did you pass your exam?,4|4|4_5|5|5,
hit:10386_conv:20772,3,hopeful,I was promised a car by  my mum,686,Oh yes I did_comma_  but yet to see the car_comma_  still expecting ,4|4|4_5|5|5,
hit:10386_conv:20772,4,hopeful,I was promised a car by  my mum,706,i hope she gets you something you like!,4|4|4_5|5|5,
hit:10386_conv:20773,1,disgusted, recently talked to new people and they were so mean,706,i recently talked to new people and they were really mean,5|5|5_4|4|4,
hit:10386_conv:20773,2,disgusted, recently talked to new people and they were so mean,686,Wow_comma_  what happened? ,5|5|5_4|4|4,
hit:10386_conv:20773,3,disgusted, recently talked to new people and they were so mean,706,i was just talking about how i miss old days and they said that i was too positive.,5|5|5_4|4|4,
hit:10386_conv:20773,4,disgusted, recently talked to new people and they were so mean,686,That is their problem _comma_ please do not let it affect you,5|5|5_4|4|4,
hit:10387_conv:20775,1,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,563,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,2|5|4_1|3|5,
hit:10387_conv:20775,2,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,717,that's nice_comma_ what did she say tho,2|5|4_1|3|5,
hit:10387_conv:20775,3,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,563,who cares! who cares if she said go away! ill do it again_comma_ the world is my oyster,2|5|4_1|3|5,
hit:10387_conv:20775,4,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,717,don't force people to do anything_comma_ you need to show respect ,2|5|4_1|3|5,
hit:10387_conv:20775,5,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,563,they need to give me respect because im so wonderful,2|5|4_1|3|5,
hit:10387_conv:20775,6,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,717,well i don't know how wonderful you are_comma_ but you sure are narcissistic_comma_ which isn't a good trade to have_comma_ people are all equal there is no difference in us,2|5|4_1|3|5,
hit:10387_conv:20775,7,confident,i walked right up to the girl i like today and told her how i felt. i didn't even care what she was going to say_comma_ i just love to tell people what i feel,563,dude i just know what im good at and i don't care what people think,2|5|4_1|3|5,
hit:10388_conv:20777,1,faithful,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave. ,620,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave.,5|5|5_5|5|5,
hit:10388_conv:20777,2,faithful,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave. ,630,Being able to enjoy your workplace is worth it. You're lucky to be in that position,5|5|5_5|5|5,
hit:10388_conv:20777,3,faithful,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave. ,620,Yeah_comma_ it's a rare thing sometimes. I plan to do this until I finish school_comma_ so for a little more than a year.,5|5|5_5|5|5,
hit:10388_conv:20777,4,faithful,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave. ,630,Do you think you'll keep the job after school? ,5|5|5_5|5|5,
hit:10388_conv:20777,5,faithful,I started a new job recently at a pretty great restaurant owned by some pretty great people. I then was offered a higher pay by my previous employer to try and persuade me to come back. I had to turn it down_comma_ though. My new place is so good_comma_ that it would take a lot to make me leave. ,620,Probably not_comma_ since getting a specific job is the whole reason I'm going to school.,5|5|5_5|5|5,
hit:10389_conv:20778,1,apprehensive,I tried a new recipe the other day. It did not look good as it was cooking.,715,I tried cooking a new recipe. But it really didn't look good while it was cooking.,2|4|2_4|4|4,
hit:10389_conv:20778,2,apprehensive,I tried a new recipe the other day. It did not look good as it was cooking.,181,Wait untill its done,2|4|2_4|4|4,
hit:10389_conv:20778,3,apprehensive,I tried a new recipe the other day. It did not look good as it was cooking.,715,I did_comma_ although I put some more seasonings in before it was finished.,2|4|2_4|4|4,
hit:10389_conv:20778,4,apprehensive,I tried a new recipe the other day. It did not look good as it was cooking.,181,It will be fine,2|4|2_4|4|4,
hit:10389_conv:20779,1,impressed,I came back home and saw that every where was clean_comma_my baby sister did the magic_comma_and i was excited about that,181,She always make me proud,4|4|4_2|4|2,
hit:10389_conv:20779,2,impressed,I came back home and saw that every where was clean_comma_my baby sister did the magic_comma_and i was excited about that,715,Who always made you proud?,4|4|4_2|4|2,
hit:10389_conv:20779,3,impressed,I came back home and saw that every where was clean_comma_my baby sister did the magic_comma_and i was excited about that,181,My baby sister that came for holidays,4|4|4_2|4|2,
hit:10389_conv:20779,4,impressed,I came back home and saw that every where was clean_comma_my baby sister did the magic_comma_and i was excited about that,715,That's great that she makes you proud.,4|4|4_2|4|2,
hit:10390_conv:20780,1,apprehensive,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,620,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,5|5|5_5|5|4,
hit:10390_conv:20780,2,apprehensive,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,211,Change is always difficult. How do you feel about it now that you have made the transition?,5|5|5_5|5|4,
hit:10390_conv:20780,3,apprehensive,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,620,I',5|5|5_5|5|4,<UNIGRAM>
hit:10390_conv:20780,4,apprehensive,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,211,Your message got cut off - but I hope it has been a good switch for you!,5|5|5_5|5|4,
hit:10390_conv:20780,5,apprehensive,I changed jobs recently. I was a little unsure whether or not I was making the right choice_comma_ as the pay was about the same. I ended up changing_comma_ though.,620,Sorry about that. It is a good one!,5|5|5_5|5|4,
hit:10391_conv:20782,1,disgusted,i hate this society,706,i hate the society we live in.,5|5|5_5|5|5,
hit:10391_conv:20782,2,disgusted,i hate this society,717,me too,5|5|5_5|5|5,
hit:10391_conv:20782,3,disgusted,i hate this society,706,why do you hate it?,5|5|5_5|5|5,
hit:10391_conv:20782,4,disgusted,i hate this society,717,can you not see it? how everyone is just chasing after their desires_comma_ living in this delusional reality thinking they are free but in reality they are just sheep living in a cettle_comma_ they don't even see that because of their desires_comma_ as long sa they can chase after women_comma_ money and drugs they are fine_comma_ but they can't see how they are nothing more than just sheep... if only there was a way to wake them up to show them what being a human is,5|5|5_5|5|5,
hit:10392_conv:20784,1,hopeful,I entered a raffle to win these amazing homemade things. I really wanted this big beautiful quilt. It was so pretty. ,409,I entered a raffle to win these amazing homemade things. I really wanted this big beautiful quilt. It was so pretty. ,5|5|5_5|5|5,
hit:10392_conv:20784,2,hopeful,I entered a raffle to win these amazing homemade things. I really wanted this big beautiful quilt. It was so pretty. ,706,and what were the results? did you win?,5|5|5_5|5|5,
hit:10392_conv:20784,3,hopeful,I entered a raffle to win these amazing homemade things. I really wanted this big beautiful quilt. It was so pretty. ,409,I did! I won the quilt!,5|5|5_5|5|5,
hit:10392_conv:20784,4,hopeful,I entered a raffle to win these amazing homemade things. I really wanted this big beautiful quilt. It was so pretty. ,706,that's some good news!,5|5|5_5|5|5,
hit:10392_conv:20785,1,anxious,i feel very stressed because of my job,706,i feel stressed out because of work,5|5|5_5|5|5,
hit:10392_conv:20785,2,anxious,i feel very stressed because of my job,409,Oh I'm sorry. Try to relax if you can. ,5|5|5_5|5|5,
hit:10392_conv:20785,3,anxious,i feel very stressed because of my job,706,i am i een started doing yoga to relax. but nothing helps!,5|5|5_5|5|5,
hit:10392_conv:20785,4,anxious,i feel very stressed because of my job,409,Try meditation. It worked well for me!,5|5|5_5|5|5,
hit:10393_conv:20786,1,terrified,the unknown is so scary,706,the future can be so scary_comma_ the unknown terrifies me.,5|5|5_2|2|5,
hit:10393_conv:20786,2,terrified,the unknown is so scary,630,Yeah it scares me to think about that sometimes_comma_ too. What in particular do you fear could happen?,5|5|5_2|2|5,
hit:10393_conv:20786,3,terrified,the unknown is so scary,706,i don't know_comma_ i guess just not being able to control life.,5|5|5_2|2|5,
hit:10393_conv:20786,4,terrified,the unknown is so scary,630,Yeah that's true_comma_ you can't always be in control of everything.  ,5|5|5_2|2|5,
hit:10394_conv:20788,1,apprehensive,I was invited to go to Turkey recently by a friend_comma_ but I still haven't gotten back to him because I've heard such bad things about the political situation there right now.,577,My friend invited me to visit him in the new country he moved to_comma_ but I've heard such bad things about it that I'm afraid to go_comma_ and I keep putting it off.,3|3|3_5|5|5,
hit:10394_conv:20788,2,apprehensive,I was invited to go to Turkey recently by a friend_comma_ but I still haven't gotten back to him because I've heard such bad things about the political situation there right now.,409,I would be careful. There is a lot of unrest in the world right now. ,3|3|3_5|5|5,
hit:10394_conv:20788,3,apprehensive,I was invited to go to Turkey recently by a friend_comma_ but I still haven't gotten back to him because I've heard such bad things about the political situation there right now.,577,That's what I'm thinking. But how do I say that to him without coming off ignorant and offensive?,3|3|3_5|5|5,
hit:10394_conv:20788,4,apprehensive,I was invited to go to Turkey recently by a friend_comma_ but I still haven't gotten back to him because I've heard such bad things about the political situation there right now.,409,I would just have a heart to heart with him. I guess I would be honest. You want to please him but you have to take care of yourself_comma_ too. ,3|3|3_5|5|5,
hit:10394_conv:20789,1,disappointed,I bought some beautiful jewelry online. It looked so nice in the picture. When I got it_comma_ I was very unhappy.,409,I bought some beautiful jewelry online. It looked so nice in the picture. When I got it_comma_ I was very unhappy,5|5|5_3|3|3,
hit:10394_conv:20789,2,disappointed,I bought some beautiful jewelry online. It looked so nice in the picture. When I got it_comma_ I was very unhappy.,577,That's annoying when that happens. Did you ask for your money back?,5|5|5_3|3|3,
hit:10394_conv:20789,3,disappointed,I bought some beautiful jewelry online. It looked so nice in the picture. When I got it_comma_ I was very unhappy.,409,Yes_comma_ I sent it back,5|5|5_3|3|3,
hit:10394_conv:20789,4,disappointed,I bought some beautiful jewelry online. It looked so nice in the picture. When I got it_comma_ I was very unhappy.,577,Good. You were probably not the only one who felt that way about the jewelry,5|5|5_3|3|3,
hit:10395_conv:20790,1,hopeful,My family said they were going to be coming in for a visit. I really wanted to see them and thought there was a good chance they would come!,409,My family said they were going to be coming in for a visit. I really wanted to see them and thought there was a good chance they would come!,5|5|5_5|5|5,
hit:10395_conv:20790,2,hopeful,My family said they were going to be coming in for a visit. I really wanted to see them and thought there was a good chance they would come!,449,That is exciting!  Did they come?,5|5|5_5|5|5,
hit:10395_conv:20790,3,hopeful,My family said they were going to be coming in for a visit. I really wanted to see them and thought there was a good chance they would come!,409,Yes! It was so wonderful to see them because we have a small family_comma_ and I just love visiting.  ,5|5|5_5|5|5,
hit:10395_conv:20790,4,hopeful,My family said they were going to be coming in for a visit. I really wanted to see them and thought there was a good chance they would come!,449,I enjoy visiting with my family too.  Unfortunately_comma_ I don't get to see them that often.  But when we do_comma_ I have a lot of fun.  Did you do anything special?,5|5|5_5|5|5,
hit:10395_conv:20791,1,ashamed,I made brownies_comma_ my favorite food.  And then I ate way too many and felt horrible.  ,449,I baked my favorite food last night -- brownies!,5|5|5_5|5|5,
hit:10395_conv:20791,2,ashamed,I made brownies_comma_ my favorite food.  And then I ate way too many and felt horrible.  ,409,That sounds good. I wish I could have some!,5|5|5_5|5|5,
hit:10395_conv:20791,3,ashamed,I made brownies_comma_ my favorite food.  And then I ate way too many and felt horrible.  ,449,I try not to make them because I eat way too many.  Now I am paying for it with a stomach ache and feel horribly ashamed.,5|5|5_5|5|5,
hit:10395_conv:20791,4,ashamed,I made brownies_comma_ my favorite food.  And then I ate way too many and felt horrible.  ,409,Oh that's okay. Everyone overindulges once in a while,5|5|5_5|5|5,
hit:10395_conv:20791,5,ashamed,I made brownies_comma_ my favorite food.  And then I ate way too many and felt horrible.  ,449,But now I have enabled myself to eat them even more_comma_ I learned how to make them in the microwave! lol,5|5|5_5|5|5,
hit:10396_conv:20792,1,disgusted,I went to throw out the trash_comma_  I opened it and saw maggots _comma_ I almost threw up,686,I almost threw up when I went to discard the trash_comma_  I saw maggots inside ,4|4|5_1|4|5,
hit:10396_conv:20792,2,disgusted,I went to throw out the trash_comma_  I opened it and saw maggots _comma_ I almost threw up,715,Maggots are so disgusting! Had the trash been sitting for awhile?,4|4|5_1|4|5,
hit:10396_conv:20792,3,disgusted,I went to throw out the trash_comma_  I opened it and saw maggots _comma_ I almost threw up,686,Oh yeah _comma_ but never knew it could be that much . The sight alone was horrible ,4|4|5_1|4|5,
hit:10396_conv:20792,4,disgusted,I went to throw out the trash_comma_  I opened it and saw maggots _comma_ I almost threw up,715,Yeah_comma_ they can multiply pretty quickly.,4|4|5_1|4|5,
hit:10396_conv:20793,1,sentimental,I recently found my oldest kids coming home outfit. I forgot how small he used to be.,715,I found my oldest kids coming home outfit. It was so tiny!,1|4|5_4|4|5,
hit:10396_conv:20793,2,sentimental,I recently found my oldest kids coming home outfit. I forgot how small he used to be.,686,Lol_comma_  exactly_comma_  because they are children ,1|4|5_4|4|5,
hit:10396_conv:20793,3,sentimental,I recently found my oldest kids coming home outfit. I forgot how small he used to be.,715,They're just so little when they're first born. It's easy to forget.,1|4|5_4|4|5,
hit:10396_conv:20793,4,sentimental,I recently found my oldest kids coming home outfit. I forgot how small he used to be.,686,Yeah.  So what did you do then,1|4|5_4|4|5,
hit:10396_conv:20793,5,sentimental,I recently found my oldest kids coming home outfit. I forgot how small he used to be.,715,I held it up to him and showed him how little he used to be. He didn't care very much.,1|4|5_4|4|5,
hit:10397_conv:20794,1,faithful,when my boss decided i should be incharge of his job when he is not ard,718,i was very happy to be incharge,5|5|5_5|5|5,
hit:10397_conv:20794,2,faithful,when my boss decided i should be incharge of his job when he is not ard,719,That's always a nice thing.  In charge of what?,5|5|5_5|5|5,
hit:10397_conv:20794,3,faithful,when my boss decided i should be incharge of his job when he is not ard,718,incharge of the office,5|5|5_5|5|5,
hit:10397_conv:20794,4,faithful,when my boss decided i should be incharge of his job when he is not ard,719,Sounds like you have a fulfilling job. That's very lucky.,5|5|5_5|5|5,
hit:10397_conv:20795,1,trusting,My trainer at my current job was supposed to train me on an experiment.  I believed her when she told me I was learning too slowly.  Turns out she was deliberately ignoring my training because she didn't like me. ,719,I didn't get a promotion I was up for because the person training me disliked me intensely. ,5|5|5_5|5|5,
hit:10397_conv:20795,2,trusting,My trainer at my current job was supposed to train me on an experiment.  I believed her when she told me I was learning too slowly.  Turns out she was deliberately ignoring my training because she didn't like me. ,718,oooh no  thats was so bad ,5|5|5_5|5|5,
hit:10397_conv:20795,3,trusting,My trainer at my current job was supposed to train me on an experiment.  I believed her when she told me I was learning too slowly.  Turns out she was deliberately ignoring my training because she didn't like me. ,719,Thanks.  It was pretty awful when I found out she did it on purpose.  I thought I was just bad at the job.  ,5|5|5_5|5|5,
hit:10397_conv:20795,4,trusting,My trainer at my current job was supposed to train me on an experiment.  I believed her when she told me I was learning too slowly.  Turns out she was deliberately ignoring my training because she didn't like me. ,718,don't worry there's always another chance,5|5|5_5|5|5,
hit:10397_conv:20795,5,trusting,My trainer at my current job was supposed to train me on an experiment.  I believed her when she told me I was learning too slowly.  Turns out she was deliberately ignoring my training because she didn't like me. ,719,Yeah_comma_ I'm training with the doctoral students now and it's going better. ,5|5|5_5|5|5,
hit:10398_conv:20796,1,disgusted,I was eating some cake and then I saw some mold in it. It was at a restaurant we loved. It was not fun!,409,I was eating some cake and then I saw some mold in it. It was at a restaurant we loved. It was not fun!,5|5|5_3|4|5,
hit:10398_conv:20796,2,disgusted,I was eating some cake and then I saw some mold in it. It was at a restaurant we loved. It was not fun!,211,That's so disgusting! I hope you didn't have to pay your bill!,5|5|5_3|4|5,
hit:10398_conv:20796,3,disgusted,I was eating some cake and then I saw some mold in it. It was at a restaurant we loved. It was not fun!,409,Yeah_comma_ she did something. The manager was pretty upset too. ,5|5|5_3|4|5,
hit:10398_conv:20796,4,disgusted,I was eating some cake and then I saw some mold in it. It was at a restaurant we loved. It was not fun!,211,I bet. That's so gross. I would have been sick!,5|5|5_3|4|5,
hit:10399_conv:20799,1,hopeful,There was a boy that I really liked. I was hoping he would ask me out on a date!,409,There was a boy that I really liked. I was hoping he would ask me out on a date!,5|5|5_4|5|4,
hit:10399_conv:20799,2,hopeful,There was a boy that I really liked. I was hoping he would ask me out on a date!,403,Oh_comma_ that's cute!  How did that work out for you?,5|5|5_4|5|4,
hit:10399_conv:20799,3,hopeful,There was a boy that I really liked. I was hoping he would ask me out on a date!,409,He did ask me out. Now we are married with 5 kids!,5|5|5_4|5|4,
hit:10399_conv:20799,4,hopeful,There was a boy that I really liked. I was hoping he would ask me out on a date!,403,Wow that's awesome!  That's a very long lasting relationship_comma_ I'm happy for you.,5|5|5_4|5|4,
hit:10400_conv:20801,1,furious,i had worked for 3 months on an oil painting and then my little brother tripped over it and put his foot through it,563,oh my GOD! I CAN'T BELIEVE IT! I HATE MY BROTHER!!!,5|3|3_5|4|5,
hit:10400_conv:20801,2,furious,i had worked for 3 months on an oil painting and then my little brother tripped over it and put his foot through it,717,WHAT HAPPEND ?,5|3|3_5|4|5,
hit:10400_conv:20801,3,furious,i had worked for 3 months on an oil painting and then my little brother tripped over it and put his foot through it,563,i worked for WEEKS AND WEEKS on an oil painting for my class....i spent my LIFE ON IT!,5|3|3_5|4|5,
hit:10400_conv:20801,4,furious,i had worked for 3 months on an oil painting and then my little brother tripped over it and put his foot through it,717,What did he do? what a mean person,5|3|3_5|4|5,
hit:10400_conv:20801,5,furious,i had worked for 3 months on an oil painting and then my little brother tripped over it and put his foot through it,563,my brother put his foot through my oil painting by accident and IM GOING TO KILL HIM!!!,5|3|3_5|4|5,
hit:10401_conv:20802,1,trusting,Yesterday my car broke down on the side of the road.  I got out and was trying to figure out what the problem was. Someone finally pulled over to help who thankfully knew cars_comma_ said they needed to go grab a few things to fix it and left me there waiting.  ,720,My car broke down recently and I got out to figure out what the problem was.  Someone finally pulled over to look_comma_ said he needed to get a few things and left me there waiting,5|5|5_5|5|5,
hit:10401_conv:20802,2,trusting,Yesterday my car broke down on the side of the road.  I got out and was trying to figure out what the problem was. Someone finally pulled over to help who thankfully knew cars_comma_ said they needed to go grab a few things to fix it and left me there waiting.  ,706,oh god_comma_ what did you end up doing?,5|5|5_5|5|5,
hit:10401_conv:20802,3,trusting,Yesterday my car broke down on the side of the road.  I got out and was trying to figure out what the problem was. Someone finally pulled over to help who thankfully knew cars_comma_ said they needed to go grab a few things to fix it and left me there waiting.  ,720,I was very hopeful that he would come back_comma_ and he did!  He fixed it and wouldn't accept any type of payment,5|5|5_5|5|5,
hit:10401_conv:20802,4,trusting,Yesterday my car broke down on the side of the road.  I got out and was trying to figure out what the problem was. Someone finally pulled over to help who thankfully knew cars_comma_ said they needed to go grab a few things to fix it and left me there waiting.  ,706,the world still have some people in it .,|5|5_5|5|5,



hit:10424_conv:20849,1,angry,I was backing out of the driveway and I ran over a bike that I had told the kids to put away. I was so mad.,548, "I was backing out of the driveway and I ran over a bike that I had told the kids to put away. I was so mad.,4|4|5_5|5|5,
hit:10424_conv:20849,2,angry,I was backing out of the driveway and I ran over a bike that I had told the kids to put away. I was so mad.,686,Wow_comma_  that was not cool at all,4|4|5_5|5|5,
hit:10424_conv:20849,3,angry,I was backing out of the driveway and I ran over a bike that I had told the kids to put away. I was so mad.,548,The bike was totaled and I had damage to the car. Needless to say somebody was in big trouble.,4|4|5_5|5|5,
hit:10424_conv:20849,4,angry,I was backing out of the driveway and I ran over a bike that I had told the kids to put away. I was so mad.,686,Ohhh kids.  Please just take it easy on them ok,4|4|5_5|5|5,
hit:10425_conv:20850,1,proud,I was in a class when my name was mentioned for an award_comma_i came out feeling high,181,Everyone applauded me,4|4|4_5|2|2,
hit:10425_conv:20850,2,proud,I was in a class when my name was mentioned for an award_comma_i came out feeling high,381,That must feel amazing! What did you do to deserve the recognition? ,4|4|4_5|2|2,
hit:10425_conv:20850,3,proud,I was in a class when my name was mentioned for an award_comma_i came out feeling high,181,I cleard my papers ,4|4|4_5|2|2,
hit:10425_conv:20850,4,proud,I was in a class when my name was mentioned for an award_comma_i came out feeling high,381,Was this at the office? Or perhaps at school?,4|4|4_5|2|2,
hit:10425_conv:20851,1,nostalgic,I was thinking about my childhood the other day and feel so lucky. I was raised in a good family and love many old memories I have.,381,I was thinking about my childhood the other day. I came from a pretty good family and life was so nice and simple when you're young.,5|2|2_4|4|4,
hit:10425_conv:20851,2,nostalgic,I was thinking about my childhood the other day and feel so lucky. I was raised in a good family and love many old memories I have.,181,Thats good_comma_it feels good,5|2|2_4|4|4,
hit:10425_conv:20851,3,nostalgic,I was thinking about my childhood the other day and feel so lucky. I was raised in a good family and love many old memories I have.,381,Yeah. I was going through some old VHS tapes and found some of when I was a kid. I used to love this old dog I had.,5|2|2_4|4|4,
hit:10425_conv:20851,4,nostalgic,I was thinking about my childhood the other day and feel so lucky. I was raised in a good family and love many old memories I have.,181,Whats the name of the dog?,5|2|2_4|4|4,
hit:10426_conv:20852,1,afraid,I ran out of the house thinking I saw a ghost,686,I thought I saw a ghost in my room_comma_  so I ran out of the house ,5|4|5_4|4|5,
hit:10426_conv:20852,2,afraid,I ran out of the house thinking I saw a ghost,548,I'll bet that was scary. I would have run too.,5|4|5_4|4|5,
hit:10426_conv:20852,3,afraid,I ran out of the house thinking I saw a ghost,686,Lol_comma_  but do you know at the end_comma_  it was just a white cloth I hanged there,5|4|5_4|4|5,
hit:10426_conv:20852,4,afraid,I ran out of the house thinking I saw a ghost,548,LOL....White cloths can be scary too.,5|4|5_4|4|5,
hit:10426_conv:20853,1,proud,My nephew signed up to join the Marines. I was overwhelmed with happy emotions that he would choose to serve our country.,548,My nephew signed up to join the Marines. I was overwhelmed with happy emotions that he would choose to serve our country.,4|4|5_5|4|5,
hit:10426_conv:20853,2,proud,My nephew signed up to join the Marines. I was overwhelmed with happy emotions that he would choose to serve our country.,686,Wow_comma_  that's great news.  I love that ,4|4|5_5|4|5,
hit:10426_conv:20853,3,proud,My nephew signed up to join the Marines. I was overwhelmed with happy emotions that he would choose to serve our country.,548,I think it is such an honorable thing to do. ,4|4|5_5|4|5,
hit:10426_conv:20853,4,proud,My nephew signed up to join the Marines. I was overwhelmed with happy emotions that he would choose to serve our country.,686,Exactly _comma_ it is quite cool,4|4|5_5|4|5,
hit:10427_conv:20854,1,apprehensive,I was scared to go on the new roller coaster at Six Flags_comma_ because it is so high and fast. I felt sick to my stomach while I was waiting in line.,724,I felt sick to my stomach waiting in line for a new roller coaster.,4|5|5_5|5|5,
hit:10427_conv:20854,2,apprehensive,I was scared to go on the new roller coaster at Six Flags_comma_ because it is so high and fast. I felt sick to my stomach while I was waiting in line.,49,I would have felt the same way. I am really afraid to get on any roller coaster. How was it?,4|5|5_5|5|5,
hit:10427_conv:20854,3,apprehensive,I was scared to go on the new roller coaster at Six Flags_comma_ because it is so high and fast. I felt sick to my stomach while I was waiting in line.,724,Well_comma_ my fear didn't get any better as it started moving. But by the end I loved it!,4|5|5_5|5|5,
hit:10427_conv:20854,4,apprehensive,I was scared to go on the new roller coaster at Six Flags_comma_ because it is so high and fast. I felt sick to my stomach while I was waiting in line.,49,That's awesome. Glad you had fun! That's what's important,4|5|5_5|5|5,
hit:10427_conv:20854,5,apprehensive,I was scared to go on the new roller coaster at Six Flags_comma_ because it is so high and fast. I felt sick to my stomach while I was waiting in line.,724,Somehow I remember them being less scary when I was a kid ,4|5|5_5|5|5,
hit:10428_conv:20856,1,sentimental,I hung out with one of my old friends yesterday. We had a good time talking about the things we used to get into_comma_ back in the day.,620,I hung out with one of my old friends yesterday. We had a good time talking about the things we used to get into_comma_ back in the day.,5|5|5_5|5|5,
hit:10428_conv:20856,2,sentimental,I hung out with one of my old friends yesterday. We had a good time talking about the things we used to get into_comma_ back in the day.,381,What kind of things did you all like to do? Sounds like it was fun to catch up.,5|5|5_5|5|5,
hit:10428_conv:20856,3,sentimental,I hung out with one of my old friends yesterday. We had a good time talking about the things we used to get into_comma_ back in the day.,620,Just general teenage tomfoolery. Nothing too serious. Silly things that kids do.,5|5|5_5|5|5,
hit:10428_conv:20856,4,sentimental,I hung out with one of my old friends yesterday. We had a good time talking about the things we used to get into_comma_ back in the day.,381,Was he one of your best friends? Sounds like you have some more catching up yo do!,5|5|5_5|5|5,
hit:10428_conv:20857,1,disgusted,Having kids is great but some of the things they do are gross. My girlfriend's kid is always making a mess!,381,Do you have kids? Some of the things they do are nasty! My girlfriends kid is such a messy eater!,5|5|5_5|5|5,
hit:10428_conv:20857,2,disgusted,Having kids is great but some of the things they do are gross. My girlfriend's kid is always making a mess!,620,I do. I have one and another one on the way. They can be gross little critters!,5|5|5_5|5|5,
hit:10428_conv:20857,3,disgusted,Having kids is great but some of the things they do are gross. My girlfriend's kid is always making a mess!,381,You're not kidding! When he eats spaghetti it's the worst. Pasta sauce gets everywhere!,5|5|5_5|5|5,
hit:10428_conv:20857,4,disgusted,Having kids is great but some of the things they do are gross. My girlfriend's kid is always making a mess!,620,Oh_comma_ that's rough. That can be a lot of clean up.,5|5|5_5|5|5,
hit:10429_conv:20859,1,caring,My friend is from work is pregnant and she's so excited.  I'm excited to_comma_ but also worried since we work in a lab and we have to be very careful of the chemicals. ,719,I just found out a friend at work is pregnant!,5|5|5_5|5|5,
hit:10429_conv:20859,2,caring,My friend is from work is pregnant and she's so excited.  I'm excited to_comma_ but also worried since we work in a lab and we have to be very careful of the chemicals. ,630,Wow_comma_ exciting! Is this going to be your friend's first child?,5|5|5_5|5|5,
hit:10429_conv:20859,3,caring,My friend is from work is pregnant and she's so excited.  I'm excited to_comma_ but also worried since we work in a lab and we have to be very careful of the chemicals. ,719,Second actually. have to be a bit careful though.  We work around some dangerous chemicals.  ,5|5|5_5|5|5,
hit:10429_conv:20859,4,caring,My friend is from work is pregnant and she's so excited.  I'm excited to_comma_ but also worried since we work in a lab and we have to be very careful of the chemicals. ,630,That sounds scary. How often do children of the employees at your work end up harmed because of the chemicals?,5|5|5_5|5|5,
hit:10429_conv:20859,5,caring,My friend is from work is pregnant and she's so excited.  I'm excited to_comma_ but also worried since we work in a lab and we have to be very careful of the chemicals. ,719,Not often.  We have a really good Health and Safety program.  ,5|5|5_5|5|5,
hit:10430_conv:20860,1,impressed,I came back home and noticed my younger bro kept everywhere clean,686,I came back home and noticed my younger brother kept everywhere clean_comma_ It pleased me,4|4|4_5|4|4,
hit:10430_conv:20860,2,impressed,I came back home and noticed my younger bro kept everywhere clean,724,That's a pleasant surprise! Is this out of the ordinary?,4|4|4_5|4|4,
hit:10430_conv:20860,3,impressed,I came back home and noticed my younger bro kept everywhere clean,686,Oh yeah.  Because he hardly sweep,4|4|4_5|4|4,
hit:10430_conv:20860,4,impressed,I came back home and noticed my younger bro kept everywhere clean,724,Let's hope he keeps it up!,4|4|4_5|4|4,
hit:10430_conv:20861,1,sentimental,I was just looking at some old pictures of my Dad. I miss him.,724,Found some old pictures of my Dad fishing. I miss fishing with him.,5|4|4_4|4|4,
hit:10430_conv:20861,2,sentimental,I was just looking at some old pictures of my Dad. I miss him.,686,Wow_comma_ I've never done fishing_comma_  so how is it like ,5|4|4_4|4|4,
hit:10430_conv:20861,3,sentimental,I was just looking at some old pictures of my Dad. I miss him.,724,I used to love going with Dad_comma_ but haven't really gone much since he passed.,5|4|4_4|4|4,
hit:10430_conv:20861,4,sentimental,I was just looking at some old pictures of my Dad. I miss him.,686,It's well.  Please accept my condolence ,5|4|4_4|4|4,
hit:10430_conv:20861,5,sentimental,I was just looking at some old pictures of my Dad. I miss him.,724,Thank you. It was quite some time ago_comma_ I was just being nostalgic.,5|4|4_4|4|4,
hit:10431_conv:20862,1,excited,My dad bought me a golden wrist watch last week and i was happy,181,I felt on top of the world when i recieved the present,4|4|4_4|4|4,
hit:10431_conv:20862,2,excited,My dad bought me a golden wrist watch last week and i was happy,577,What type of present was it? It must have been pretty special.,4|4|4_4|4|4,
hit:10431_conv:20862,3,excited,My dad bought me a golden wrist watch last week and i was happy,181,Yes_comma_it was a golden wrist watch from my dad,4|4|4_4|4|4,
hit:10431_conv:20862,4,excited,My dad bought me a golden wrist watch last week and i was happy,577,Cool. There's nothing like receiving family heirlooms.,4|4|4_4|4|4,
hit:10431_conv:20863,1,caring,I stumbled upon a couple of abandoned kittens recently. I took them home.,577,I found some kittens in an abandoned house recently. They were so malnourished_comma_ I took them home_comma_ fed them_comma_ and now they're part of the family.,4|4|4_4|4|4,
hit:10431_conv:20863,2,caring,I stumbled upon a couple of abandoned kittens recently. I took them home.,181,You did so well_comma_have you given them names?,4|4|4_4|4|4,
hit:10431_conv:20863,3,caring,I stumbled upon a couple of abandoned kittens recently. I took them home.,577,Yes_comma_ Foster and Buster. Don't ask me why I chose those names_comma_ lol. That's what they looked like_comma_ I guess,4|4|4_4|4|4,
hit:10431_conv:20863,4,caring,I stumbled upon a couple of abandoned kittens recently. I took them home.,181,Am so happy for you_comma_you have gotten a pet,4|4|4_4|4|4,
hit:10432_conv:20864,1,anxious,Recently I found out I may have a hernia. It's causing so much anxiety that it keeps me up at night.,381,I was diagnosed with a inguinal hernia yesterday. The amount of stress and anxiety is through the roof for me right now.,5|5|5_3|4|5,
hit:10432_conv:20864,2,anxious,Recently I found out I may have a hernia. It's causing so much anxiety that it keeps me up at night.,368,Holy crap! Can it be remedied?,5|5|5_3|4|5,
hit:10432_conv:20864,3,anxious,Recently I found out I may have a hernia. It's causing so much anxiety that it keeps me up at night.,381,Yes with surgery. This is going to require some time off from work which may be difficult.,5|5|5_3|4|5,
hit:10432_conv:20864,4,anxious,Recently I found out I may have a hernia. It's causing so much anxiety that it keeps me up at night.,368,I really hope it all works out for you. Work is important but your health is way more important!,5|5|5_3|4|5,
hit:10432_conv:20865,1,faithful,My marriage,368,I've been married nearly eight years and someone hit on me last week. Luckily we have always been happy together or else that could have taken a weird turn.,3|4|5_5|5|5,
hit:10432_conv:20865,2,faithful,My marriage,381,I bet it felt good though? That's good that you're happily married!,3|4|5_5|5|5,
hit:10432_conv:20865,3,faithful,My marriage,368,It did! I guess I've 'still got it!' Ha!,3|4|5_5|5|5,
hit:10432_conv:20865,4,faithful,My marriage,381,That's the spirit! It's a positive thing so long as you remain faithful. Did you tell your significant other?,3|4|5_5|5|5,
hit:10432_conv:20865,5,faithful,My marriage,368,Of course!,3|4|5_5|5|5,
hit:10433_conv:20866,1,sentimental,I wanted to leave my boyfriend but I felt pity on him,686,I felt pity on my boyfriend last week_comma_  but I wanted to leave him,4|4|4_5|4|5,
hit:10433_conv:20866,2,sentimental,I wanted to leave my boyfriend but I felt pity on him,724,Oh yeah? Did you get it worked out?,4|4|4_5|4|5,
hit:10433_conv:20866,3,sentimental,I wanted to leave my boyfriend but I felt pity on him,686,No_comma_  he convinced me_comma_  so I stayed.  But don't like his attitude ,4|4|4_5|4|5,
hit:10433_conv:20866,4,sentimental,I wanted to leave my boyfriend but I felt pity on him,724,Have you considered couples therapy to try and work things out?,4|4|4_5|4|5,
hit:10433_conv:20867,1,devastated,I bought a new jeep and someone crashed into it in a parking lot. They didn't even leave a note.,724,I bought a brand new jeep_comma_ and it's damaged already. I'm upset.,5|4|5_4|4|4,
hit:10433_conv:20867,2,devastated,I bought a new jeep and someone crashed into it in a parking lot. They didn't even leave a note.,686,Oh no_comma_  that is not cool.  What happened? ,5|4|5_4|4|4,
hit:10433_conv:20867,3,devastated,I bought a new jeep and someone crashed into it in a parking lot. They didn't even leave a note.,724,Someone crashed into it in a parking lot. Didn't even leave a note.,5|4|5_4|4|4,
hit:10433_conv:20867,4,devastated,I bought a new jeep and someone crashed into it in a parking lot. They didn't even leave a note.,686,That person is heartless though ,5|4|5_4|4|4,
hit:10433_conv:20867,5,devastated,I bought a new jeep and someone crashed into it in a parking lot. They didn't even leave a note.,724,I agree. People can be such jerks sometimes.,5|4|5_4|4|4,
hit:10434_conv:20868,1,trusting,it is really a happy feeling that you get a good cooperation from some unexpected person,2,i got permission to begin a start up company by my own.. i am so excited,4|4|4_5|5|5,
hit:10434_conv:20868,2,trusting,it is really a happy feeling that you get a good cooperation from some unexpected person,725,Wow_comma_ that sounds exciting! I hope everything works out for you!,4|4|4_5|5|5,
hit:10434_conv:20868,3,trusting,it is really a happy feeling that you get a good cooperation from some unexpected person,2,yes.. it is unexpected indeed.. i thought i would be struggling to get support.. ,4|4|4_5|5|5,
hit:10434_conv:20868,4,trusting,it is really a happy feeling that you get a good cooperation from some unexpected person,725,Seems like it is your fate then. I am also starting a new venture and it is taking a lot of time and hard work but I think it is finally paying off soon. ,4|4|4_5|5|5,
hit:10435_conv:20870,1,terrified,Lately I've been having horrible nightmares. It feels like having sleep paralysis!,381,Lately at night when I go to sleep I've been having sleep paralysis. It feels terrifying! Like someone is in the room with me.,5|4|5_3|3|4,
hit:10435_conv:20870,2,terrified,Lately I've been having horrible nightmares. It feels like having sleep paralysis!,577,That's horrible. Have you any idea what might have caused it?,5|4|5_3|3|4,
hit:10435_conv:20870,3,terrified,Lately I've been having horrible nightmares. It feels like having sleep paralysis!,381,I've watched some Youtube videos on it and it seems to be hereditary or sometimes sleep deprived. I guess I am usually sleep deprived!,5|4|5_3|3|4,
hit:10435_conv:20870,4,terrified,Lately I've been having horrible nightmares. It feels like having sleep paralysis!,577,You should go see a doctor about that. Sleep deprivation is very unhealthy,5|4|5_3|3|4,
hit:10435_conv:20871,1,content,I moved into a new house recently. And while it isn't a mansion_comma_ it fits my needs perfectly.,577,I moved house recently_comma_ from an absolute dump. The new place isn't huge_comma_ but it's perfect for me.,3|3|4_5|4|5,
hit:10435_conv:20871,2,content,I moved into a new house recently. And while it isn't a mansion_comma_ it fits my needs perfectly.,381,That's one of the most responsible things you can do. I'm sure your family is proud of you.,3|3|4_5|4|5,
hit:10435_conv:20871,3,content,I moved into a new house recently. And while it isn't a mansion_comma_ it fits my needs perfectly.,577,Well_comma_ I live alone_comma_ but I'm certainly proud of myself for finally moving. Now I can focus on work.,3|3|4_5|4|5,
hit:10435_conv:20871,4,content,I moved into a new house recently. And while it isn't a mansion_comma_ it fits my needs perfectly.,381,You're taking steps in the right direction I get a feeling you're doing alright for yourself out there. ,3|3|4_5|4|5,
hit:10436_conv:20873,1,devastated,A few years ago_comma_ my employer of 13 years shut their doors and we all were laid off. I would probably have stayed at that place my entire career if it hadn't closed.,620,A few years ago_comma_ my employer of 13 years shut their doors and we all were laid off. I would probably have stayed at that place my entire career if it hadn't closed.,5|5|5_4|5|5,
hit:10436_conv:20873,2,devastated,A few years ago_comma_ my employer of 13 years shut their doors and we all were laid off. I would probably have stayed at that place my entire career if it hadn't closed.,211,That must have been devastating. Did it take you a long time to find a new position?,5|5|5_4|5|5,
hit:10436_conv:20873,3,devastated,A few years ago_comma_ my employer of 13 years shut their doors and we all were laid off. I would probably have stayed at that place my entire career if it hadn't closed.,620,No_comma_ I worked at another place for a year or so_comma_ but it was terrible_comma_ so I left there and am now going to school for something better. So_comma_ it all has worked out well.,5|5|5_4|5|5,
hit:10436_conv:20873,4,devastated,A few years ago_comma_ my employer of 13 years shut their doors and we all were laid off. I would probably have stayed at that place my entire career if it hadn't closed.,211,I'm glad things have worked out in the end for you. Sometimes bad things happen for a reason. ,5|5|5_4|5|5,
hit:10437_conv:20874,1,sentimental,3,449,My aunt died last year and I was thinking about her yesterday.,5|5|5_5|5|5,
hit:10437_conv:20874,2,sentimental,3,719,That's very hard. I lost my uncle in january and it's so hard to think about.,5|5|5_5|5|5,
hit:10437_conv:20874,3,sentimental,3,449,She was like a mom to be and raised me in my younger years.,5|5|5_5|5|5,
hit:10437_conv:20874,4,sentimental,3,719,I'm so sorry.  Are you doing all right?,5|5|5_5|5|5,
hit:10437_conv:20874,5,sentimental,3,449,I am_comma_ it's just that the fleeting memories make me sad.,5|5|5_5|5|5,
hit:10437_conv:20874,6,sentimental,3,719,They always do. It'll get easier to think about with time,5|5|5_5|5|5,
hit:10437_conv:20874,7,sentimental,3,449,I hope.  I keep her close to by heart.,5|5|5_5|5|5,
hit:10437_conv:20875,1,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,719,My cat is such a comfort.  He likes to sleep under the covers with me.  ,5|5|5_5|5|5,
hit:10437_conv:20875,2,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,449,That is awesome!  I have 5 cats.  They are pretty snobbish and only 2 like to cuddle.,5|5|5_5|5|5,
hit:10437_conv:20875,3,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,719,Aww!  I've only the one right now_comma_ but he is the cuddliest kitty.  Demands to be under the blankets with me most nights.  ,5|5|5_5|5|5,
hit:10437_conv:20875,4,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,449,That is sweet.  My cats have human names_comma_ lol.  What about yours?,5|5|5_5|5|5,
hit:10437_conv:20875,5,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,719,Same.  I got his out of an old detective novel.  ,5|5|5_5|5|5,
hit:10437_conv:20875,6,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,449,I recently had to surrender 1_comma_ we had six before.  But he was being bullied by the others.,5|5|5_5|5|5,
hit:10437_conv:20875,7,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,719,We do the best for them_comma_ even if it's not what we want.  ,5|5|5_5|5|5,
hit:10437_conv:20875,8,content,My cat likes to sleep under the covers with me a night.  He comes up and pulls at the blankets until I let him in and then curls up and sleeps by my side. ,449,I'm glad you have a cuddly fur-baby!,5|5|5_5|5|5,
hit:10438_conv:20876,1,surprised,I just came into the house_comma_on the light and heard _comma_'happy birthday',181,I was given a beffitting birthday by my sibblings,4|4|4_5|5|5,
hit:10438_conv:20876,2,surprised,I just came into the house_comma_on the light and heard _comma_'happy birthday',381,How old did you turn? I hope you guys had some cake too!,4|4|4_5|5|5,
hit:10438_conv:20876,3,surprised,I just came into the house_comma_on the light and heard _comma_'happy birthday',181,I was a year older_comma_lol,4|4|4_5|5|5,
hit:10438_conv:20876,4,surprised,I just came into the house_comma_on the light and heard _comma_'happy birthday',381,That's awesome! Well I want to say happy birthday to you as well!,4|4|4_5|5|5,
hit:10438_conv:20877,1,disappointed,I was dog sitting my moms dog and she got out. It's been over 24 hours now since she's gone missing.,381,I lost my mom's dog yesterday and it's been about 24 hours since she went missing! I'm trying to find her but had to take a break.,5|5|5_4|4|4,
hit:10438_conv:20877,2,disappointed,I was dog sitting my moms dog and she got out. It's been over 24 hours now since she's gone missing.,181,So sorry about that_comma_she will be found,5|5|5_4|4|4,
hit:10438_conv:20877,3,disappointed,I was dog sitting my moms dog and she got out. It's been over 24 hours now since she's gone missing.,381,Thank you so much that means a lot. I've posted on a few dog watching apps and people are supportive.,5|5|5_4|4|4,
hit:10438_conv:20877,4,disappointed,I was dog sitting my moms dog and she got out. It's been over 24 hours now since she's gone missing.,181,You are most welcomed_comma_just keep trying,5|5|5_4|4|4,
hit:10439_conv:20878,1,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,726,I am broke and cannot afford to pay my bills. I am afraid that I have hurt my family because of my mental issues. I am embarrassed to tell anyone. ,5|5|5_5|4|5,
hit:10439_conv:20878,2,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,724,I'm so sorry to hear that. What can I do to help?,5|5|5_5|4|5,
hit:10439_conv:20878,3,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,726,I don't think there is anything you can do to help. This is something_comma_ I most likely will need to resolve on my own. Unfortunately. ,5|5|5_5|4|5,
hit:10439_conv:20878,4,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,724,Perhaps you could see a counselor or therapist_comma_ that might help.,5|5|5_5|4|5,
hit:10439_conv:20878,5,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,726,My insurance will not cover this type of expenses. I am medicated_comma_ but I do not feel that it helps. ,5|5|5_5|4|5,
hit:10439_conv:20878,6,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,724,Oh that's too bad. There are many free programs that you might be able to access.,5|5|5_5|4|5,
hit:10439_conv:20878,7,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,726,Maybe so_comma_ I should look into that. I feel like just speaking to someone about this would help. ,5|5|5_5|4|5,
hit:10439_conv:20878,8,ashamed,I have severe social phobia_comma_ anxiety and depression. I often miss work. Recently_comma_ due to this issue_comma_ I missed quite a bit of work and am now in trouble financially. ,724,That's a great idea. I'm always here if you need someone to talk to.,5|5|5_5|4|5,
hit:10439_conv:20879,1,caring,My sister had a baby. It's undescribeable how much love I feel for it.,724,I feel such love and joy towards my sister's new baby. It's overwhelming.,5|4|5_5|5|5,
hit:10439_conv:20879,2,caring,My sister had a baby. It's undescribeable how much love I feel for it.,726,Oh wow_comma_ that is wonderful. Is it a girl or boy?,5|4|5_5|5|5,
hit:10439_conv:20879,3,caring,My sister had a baby. It's undescribeable how much love I feel for it.,724,It's a girl. My first niece_comma_ and she's beautiful.,5|4|5_5|5|5,
hit:10439_conv:20879,4,caring,My sister had a baby. It's undescribeable how much love I feel for it.,726,Congratulations! Babies are amazing. I have two of my own. Do you have any of your own?,5|4|5_5|5|5,
hit:10439_conv:20879,5,caring,My sister had a baby. It's undescribeable how much love I feel for it.,724,Unfortunately no_comma_ not yet. My wife and I are hoping to have kids someday.,5|4|5_5|5|5,
hit:10439_conv:20879,6,caring,My sister had a baby. It's undescribeable how much love I feel for it.,726,It is a wonderful feeling_comma_ to be a parent. Fingers crossed_comma_ you'll be next. ,5|4|5_5|5|5,
hit:10439_conv:20879,7,caring,My sister had a baby. It's undescribeable how much love I feel for it.,724,Thanks! Right now I'm happy to be spoiling my new niece.,5|4|5_5|5|5,
hit:10439_conv:20879,8,caring,My sister had a baby. It's undescribeable how much love I feel for it.,726,Well_comma_ keep up the good work. Give that baby all the love in the world. ,5|4|5_5|5|5,
hit:10440_conv:20881,1,caring,I have a great niece who was born on Halloween. I would do anything to help take care of her.,349,I have a 9 month old great niece_comma_ she's the sweetest thing ever.,5|5|5_5|5|5,
hit:10440_conv:20881,2,caring,I have a great niece who was born on Halloween. I would do anything to help take care of her.,211,Aww! Babies are so much fun! I have a new granddaughter_comma_ so I can relate!,5|5|5_5|5|5,
hit:10440_conv:20881,3,caring,I have a great niece who was born on Halloween. I would do anything to help take care of her.,349,I love being able to help take care of her.,5|5|5_5|5|5,
hit:10440_conv:20881,4,caring,I have a great niece who was born on Halloween. I would do anything to help take care of her.,211,I bet you are amazing with her- and such a blessing in her life.,5|5|5_5|5|5,
hit:10441_conv:20883,1,joyful,I got to see my baby born yesterday. It was like witnessing a miracle!,381,I got to see my baby born on Friday. My first son was born and is in healthy shape.,5|5|5_5|5|5,
hit:10441_conv:20883,2,joyful,I got to see my baby born yesterday. It was like witnessing a miracle!,717,wow that must have been so hard to do! i am so proud of you worker_2 how do you feel?,5|5|5_5|5|5,
hit:10441_conv:20883,3,joyful,I got to see my baby born yesterday. It was like witnessing a miracle!,381,Like a brand new person. I want to do everything I can to show him how to love a good_comma_ moral life!,5|5|5_5|5|5,
hit:10441_conv:20883,4,joyful,I got to see my baby born yesterday. It was like witnessing a miracle!,717,YES! that is exactly what i would say keep doing that and i hope everything goes well,5|5|5_5|5|5,
hit:10443_conv:20886,1,trusting,I know that my partner can never cheat on me no matter what,181,We are so close and always believe in each other,4|4|4_3|3|4,
hit:10443_conv:20886,2,trusting,I know that my partner can never cheat on me no matter what,548,Are you talking about me or someone else?,4|4|4_3|3|4,
hit:10443_conv:20886,3,trusting,I know that my partner can never cheat on me no matter what,181,Am taling about my partner,4|4|4_3|3|4,
hit:10443_conv:20886,4,trusting,I know that my partner can never cheat on me no matter what,548,It's good that you have someone who is there for you no matter what.,4|4|4_3|3|4,
hit:10443_conv:20887,1,excited,I got tickets to see Pink in concert. I can't wait_comma_ she is one of my favorite singers.,548,I got tickets to see Pink in concert. I can't wait_comma_ she is one of my favorite singers.,3|3|4_4|4|4,
hit:10443_conv:20887,2,excited,I got tickets to see Pink in concert. I can't wait_comma_ she is one of my favorite singers.,181,She sings so well,3|3|4_4|4|4,
hit:10443_conv:20887,3,excited,I got tickets to see Pink in concert. I can't wait_comma_ she is one of my favorite singers.,548,She also puts on a great show. Well worth the price of the tickets.,3|3|4_4|4|4,
hit:10443_conv:20887,4,excited,I got tickets to see Pink in concert. I can't wait_comma_ she is one of my favorite singers.,181,Thats so good_comma_enough enjoyment,3|3|4_4|4|4,
hit:10444_conv:20888,1,trusting,When I met a new friend who offered to help_comma_ I assumed she was an honest person.,727,She had a genuine personality that made you feel comfortable almost immediately.,3|5|5_4|2|5,
hit:10444_conv:20888,2,trusting,When I met a new friend who offered to help_comma_ I assumed she was an honest person.,381,Who was she because she sounds special? Someone close to you?,3|5|5_4|2|5,
hit:10444_conv:20888,3,trusting,When I met a new friend who offered to help_comma_ I assumed she was an honest person.,727,She became one of my closest friends and confidant.,3|5|5_4|2|5,
hit:10444_conv:20888,4,trusting,When I met a new friend who offered to help_comma_ I assumed she was an honest person.,381,Did she pass away recently? I hope everything is ok.,3|5|5_4|2|5,
hit:10444_conv:20888,5,trusting,When I met a new friend who offered to help_comma_ I assumed she was an honest person.,727,She sadly moved far away.,3|5|5_4|2|5,

hit:10444_conv:20889,1,afraid,I'm scared my car may be on it's last legs. It keeps overheating on me and I don't understand anything about cars.,381,My car keeps overheating and it's definitely not a good sign. I'm afraid I may have to find a new car soon.,4|2|5_3|5|5,
hit:10444_conv:20889,2,afraid,I'm scared my car may be on it's last legs. It keeps overheating on me and I don't understand anything about cars.,727,Do you know what kind of car you want to get?,4|2|5_3|5|5,
hit:10444_conv:20889,3,afraid,I'm scared my car may be on it's last legs. It keeps overheating on me and I don't understand anything about cars.,381,Possibly a truck this time. I've never had the pleasure of owning one.,4|2|5_3|5|5,
hit:10444_conv:20889,4,afraid,I'm scared my car may be on it's last legs. It keeps overheating on me and I don't understand anything about cars.,727,What kind of things would you use the truck for?,4|2|5_3|5|5,

hit:11574_conv:23148,1,embarrassed,Last weekend_comma_ I slipped on some wet grass and slid_comma_ hand first_comma_ into a pile of dog poop.  In front of everyone at a BBQ.,438, "Last weekend_comma_ I slipped on some wet grass and slid_comma_ hand first_comma_ into a pile of dog poop. In front of everyone at a BBQ.,5|5|5_5|5|5,
hit:11574_conv:23148,2,embarrassed,Last weekend_comma_ I slipped on some wet grass and slid_comma_ hand first_comma_ into a pile of dog poop.  In front of everyone at a BBQ.,725,Oh no! How gross! That must have been so embarrassing! Were you able to clean yourself up?,5|5|5_5|5|5,
hit:11574_conv:23148,3,embarrassed,Last weekend_comma_ I slipped on some wet grass and slid_comma_ hand first_comma_ into a pile of dog poop.  In front of everyone at a BBQ.,438,It was so awful and disgusting.  I actually just left because I was so grossed out that I wanted to go home and take a shower.,5|5|5_5|5|5,
hit:11574_conv:23148,4,embarrassed,Last weekend_comma_ I slipped on some wet grass and slid_comma_ hand first_comma_ into a pile of dog poop.  In front of everyone at a BBQ.,725,I can imagine. I would have lost my appetite for sure. That is a crazy story! ,5|5|5_5|5|5,
hit:11575_conv:23150,1,faithful,I don't have much more money to keep living as a college student. But I just have one more year left. After that_comma_ hopefully things stabilize.,273,I don't have much more money to keep living as a college student. But I just have one more year left. After that_comma_ hopefully things stabilize.,5|5|5_5|5|5,
hit:11575_conv:23150,2,faithful,I don't have much more money to keep living as a college student. But I just have one more year left. After that_comma_ hopefully things stabilize.,805,It's great that you only have one more year left! What are you getting a degree in?,5|5|5_5|5|5,
hit:11575_conv:23150,3,faithful,I don't have much more money to keep living as a college student. But I just have one more year left. After that_comma_ hopefully things stabilize.,273,Thanks! I appreciate that. I actually am getting a degree in social work. At the moment_comma_ I'm just trying not to become the needy/helpless demographic I seek to serve in a career after I graduate,5|5|5_5|5|5,
hit:11575_conv:23150,4,faithful,I don't have much more money to keep living as a college student. But I just have one more year left. After that_comma_ hopefully things stabilize.,805,I totally get that! I am still trying to pay off my student loans. ,5|5|5_5|5|5,
hit:11575_conv:23151,1,jealous,My friend just sold her book to a publisher and I am a bit jealous.,805,Is it wrong to be jealous of your best friend?,5|5|5_5|5|5,
hit:11575_conv:23151,2,jealous,My friend just sold her book to a publisher and I am a bit jealous.,273,I think it's just human nature. Even if it were wrong_comma_ there may be nothing you can do about it. But I think there are ways you can take your negative emotions and put a positive spin on them_comma_ in order to motivate you in a more healthy_comma_ conducive way.,5|5|5_5|5|5,
hit:11575_conv:23151,3,jealous,My friend just sold her book to a publisher and I am a bit jealous.,805,That's true. She just sold her book to a publisher and it is just a little weird. ,5|5|5_5|5|5,
hit:11575_conv:23151,4,jealous,My friend just sold her book to a publisher and I am a bit jealous.,273,I could see that... Yeah. Well_comma_ I hope you are able to re-invest that energy into your own passions. I think you could do a lot for yourself.,5|5|5_5|5|5,
hit:11575_conv:23151,5,jealous,My friend just sold her book to a publisher and I am a bit jealous.,805,Thank you for saying that. It really makes me feel better.,5|5|5_5|5|5,
hit:11576_conv:23152,1,trusting,Going in for surgery_comma_ I've been trusting of the doctor.,578,So I've been having a problem lately.,5|5|5_5|5|5,
hit:11576_conv:23152,2,trusting,Going in for surgery_comma_ I've been trusting of the doctor.,59,Ok_comma_ and do you want to share this problem?,5|5|5_5|5|5,
hit:11576_conv:23152,3,trusting,Going in for surgery_comma_ I've been trusting of the doctor.,578,I've been seeing a doctor for it.  I've had to go through a lot of testing lately.,5|5|5_5|5|5,
hit:11576_conv:23152,4,trusting,Going in for surgery_comma_ I've been trusting of the doctor.,59,I see_comma_ at least you are getting professional help with your problem.,5|5|5_5|5|5,
hit:11576_conv:23152,5,trusting,Going in for surgery_comma_ I've been trusting of the doctor.,578,Kind of have to.  I have pelvic floor damage from childbirth.  I'm hoping the doctor knows what they're doing.,5|5|5_5|5|5,
hit:11576_conv:23153,1,jealous,My dad got my brother a watch but not me.,59,My dad got my brother a watch but not me.,5|5|5_5|5|5,
hit:11576_conv:23153,2,jealous,My dad got my brother a watch but not me.,578,Did you want a watch?,5|5|5_5|5|5,
hit:11576_conv:23153,3,jealous,My dad got my brother a watch but not me.,59,Yes_comma_ and I feel kind of envious of my brother now.,5|5|5_5|5|5,
hit:11576_conv:23153,4,jealous,My dad got my brother a watch but not me.,578,That stinks.  It's hard when a parent favors one child over another.,5|5|5_5|5|5,
hit:11577_conv:23154,1,content,It's friday. I'm going to go home and grill a steak and have a beer.,244,It's friday. I'm going to go home and grill a steak and have a beer.,5|5|5_5|5|5,
hit:11577_conv:23154,2,content,It's friday. I'm going to go home and grill a steak and have a beer.,525,It's your right as an American citizen.,5|5|5_5|5|5,
hit:11577_conv:23154,3,content,It's friday. I'm going to go home and grill a steak and have a beer.,244,I know right.  It's a free country.... murica',5|5|5_5|5|5,
hit:11577_conv:23154,4,content,It's friday. I'm going to go home and grill a steak and have a beer.,525,Have a beer for me too.,5|5|5_5|5|5,
hit:11577_conv:23154,5,content,It's friday. I'm going to go home and grill a steak and have a beer.,244,I'll have two for you.,5|5|5_5|5|5,
hit:11577_conv:23155,1,sentimental,I got into my 1.3 million dollar Ferrari F40 yesterday.  I can still remember watching my loving father stroke the 1.3 million dollar check for it.,525,I stepped into my Ferrari F-40 the other day_comma_ and I shed a tear.,5|5|5_5|5|5,
hit:11577_conv:23155,2,sentimental,I got into my 1.3 million dollar Ferrari F40 yesterday.  I can still remember watching my loving father stroke the 1.3 million dollar check for it.,244,Is it your first ferrari?,5|5|5_5|5|5,
hit:11577_conv:23155,3,sentimental,I got into my 1.3 million dollar Ferrari F40 yesterday.  I can still remember watching my loving father stroke the 1.3 million dollar check for it.,525,LOL_comma_ no.  But I can still remember watching my beloved late grandfather stroking the 1.3 million dollar check.  I was very fond of him.,5|5|5_5|5|5,
hit:11577_conv:23155,4,sentimental,I got into my 1.3 million dollar Ferrari F40 yesterday.  I can still remember watching my loving father stroke the 1.3 million dollar check for it.,244,LOL man that's... I don't even know what to say.,5|5|5_5|5|5,
hit:11579_conv:23158,1,faithful,My neighbor_comma_ asked me out on a date_comma_ even though he knows I am married. I told him no,653,My neighbor asked me out on a date_comma_ even  though he knows I am married. I told him no.,3|4|5_5|5|5,
hit:11579_conv:23158,2,faithful,My neighbor_comma_ asked me out on a date_comma_ even though he knows I am married. I told him no,375,What did he say in response?,3|4|5_5|5|5,
hit:11579_conv:23158,3,faithful,My neighbor_comma_ asked me out on a date_comma_ even though he knows I am married. I told him no,653,He was annoyed and was insisting.,3|4|5_5|5|5,
hit:11579_conv:23158,4,faithful,My neighbor_comma_ asked me out on a date_comma_ even though he knows I am married. I told him no,375,Did you tell your husband?,3|4|5_5|5|5,
hit:11579_conv:23158,5,faithful,My neighbor_comma_ asked me out on a date_comma_ even though he knows I am married. I told him no,653,Yes I did_comma_ My husband was not pleased.,3|4|5_5|5|5,
hit:11579_conv:23159,1,trusting,I went on my first flight_comma_ it was a scary experience but I did it. I flew to Montreal.,375,It was quite an experience flying for the  first time.,5|5|5_3|4|5,
hit:11579_conv:23159,2,trusting,I went on my first flight_comma_ it was a scary experience but I did it. I flew to Montreal.,653,I can just imagine_comma_ it must have been exciting for you.,5|5|5_3|4|5,
hit:11579_conv:23159,3,trusting,I went on my first flight_comma_ it was a scary experience but I did it. I flew to Montreal.,375,I was kind of scared but I put my trust in to the pilot to get me safely to Montreal for a festival.,5|5|5_3|4|5,
hit:11579_conv:23159,4,trusting,I went on my first flight_comma_ it was a scary experience but I did it. I flew to Montreal.,653,That is so nice_comma_ the good thing is you arrived safely.,5|5|5_3|4|5,
hit:11580_conv:23160,1,jealous,Every single person I know got to go on a vacation this year except for me. ,438,Every single person I know got to go on a vacation this year except for me. ,5|5|5_5|5|5,
hit:11580_conv:23160,2,jealous,Every single person I know got to go on a vacation this year except for me. ,580,I'm so sorry to hear that. Is there a reason why you couldn't?,5|5|5_5|5|5,
hit:11580_conv:23160,3,jealous,Every single person I know got to go on a vacation this year except for me. ,438,I just don't have the funds or time away from work. Even my sister and her husband_comma_ who are as broke as me_comma_ went on one.,5|5|5_5|5|5,
hit:11580_conv:23160,4,jealous,Every single person I know got to go on a vacation this year except for me. ,580,To be fair_comma_ travelling is expensive and probably overrated to a certain extent. I hope you do find the time to take some time away from work so you can enjoy yourself though.,5|5|5_5|5|5,
hit:11581_conv:23162,1,devastated,My garbage burned to the ground one night. I had to help my family escape.,375,Seeing my garage burn was insane.,4|5|5_5|5|5,
hit:11581_conv:23162,2,devastated,My garbage burned to the ground one night. I had to help my family escape.,799,Oh no that is terrible! I hope you and your family evacuated safely.,4|5|5_5|5|5,
hit:11581_conv:23162,3,devastated,My garbage burned to the ground one night. I had to help my family escape.,375,We did but it was some scary_comma_ the flames were massive.,4|5|5_5|5|5,
hit:11581_conv:23162,4,devastated,My garbage burned to the ground one night. I had to help my family escape.,799,Wow I'm glad you are safe though. Do you know what started it?,4|5|5_5|5|5,
hit:11582_conv:23164,1,guilty,I accidentally left the screen door cracked and my beagle got out.  He's really bad with strangers.  Luckily the neighbors found him before he got too far.,527,I accidentally left the screen door cracked open and my beagle got out. ,5|3|5_5|5|5,
hit:11582_conv:23164,2,guilty,I accidentally left the screen door cracked and my beagle got out.  He's really bad with strangers.  Luckily the neighbors found him before he got too far.,748,Oh no! Were you able to find him/her?,5|3|5_5|5|5,
hit:11582_conv:23164,3,guilty,I accidentally left the screen door cracked and my beagle got out.  He's really bad with strangers.  Luckily the neighbors found him before he got too far.,527,Luckily the neighbor we know caught him before he got too far away.  Thank goodness_comma_ he's not great with strangers.,5|3|5_5|5|5,
hit:11582_conv:23164,4,guilty,I accidentally left the screen door cracked and my beagle got out.  He's really bad with strangers.  Luckily the neighbors found him before he got too far.,748,Thank God! It can be heart wrenching when your pet is in danger.,5|3|5_5|5|5,
hit:11582_conv:23165,1,hopeful,I saw a politician give up his salary to charity.,748,I read an article the other day about a noble politician. he gave his salary to charity!,5|5|5_5|3|5,
hit:11582_conv:23165,2,hopeful,I saw a politician give up his salary to charity.,527,Oh wow_comma_ that's cool.  And surprising haha,5|5|5_5|3|5,
hit:11582_conv:23165,3,hopeful,I saw a politician give up his salary to charity.,748,I know right! I expect others to take notice and follow on this honorable path.,5|5|5_5|3|5,
hit:11582_conv:23165,4,hopeful,I saw a politician give up his salary to charity.,527,Let's hope this politician is honorable in other ways as well!,5|5|5_5|3|5,
hit:11582_conv:23165,5,hopeful,I saw a politician give up his salary to charity.,748,I believe he is. He doesn't take capmaign contributions from corporations.,5|5|5_5|3|5,
hit:11583_conv:23166,1,apprehensive,I want to try sushi some time.  I'm afraid of getting sick though.,322,I want to try sushi at some point soon.,4|3|5_3|4|4,
hit:11583_conv:23166,2,apprehensive,I want to try sushi some time.  I'm afraid of getting sick though.,224,I love sushi its delicious,4|3|5_3|4|4,
hit:11583_conv:23166,3,apprehensive,I want to try sushi some time.  I'm afraid of getting sick though.,322,I'm afraid of getting sick though.,4|3|5_3|4|4,
hit:11583_conv:23166,4,apprehensive,I want to try sushi some time.  I'm afraid of getting sick though.,224,my favorite is california roll,4|3|5_3|4|4,
hit:11583_conv:23167,1,sentimental,I love my mom she is very sweet to me,224,I love my mom shes really nice and helpful,3|4|4_4|3|5,
hit:11583_conv:23167,2,sentimental,I love my mom she is very sweet to me,322,I'm sure she is a lovely woman.,3|4|4_4|3|5,
hit:11583_conv:23167,3,sentimental,I love my mom she is very sweet to me,224,when you're sad nobody beats ma!,3|4|4_4|3|5,
hit:11583_conv:23167,4,sentimental,I love my mom she is very sweet to me,322,I love my mom very much too!,3|4|4_4|3|5,
hit:11584_conv:23168,1,impressed,A friend of mine got first place in her age category in a triathlon!  She is amazing!,203,A friend of mine got first place in her age category in a triathlon!,5|5|5_5|5|5,
hit:11584_conv:23168,2,impressed,A friend of mine got first place in her age category in a triathlon!  She is amazing!,725,That is exciting news! You must be so happy for her! Are you going out to celebrate?,5|5|5_5|5|5,
hit:11584_conv:23168,3,impressed,A friend of mine got first place in her age category in a triathlon!  She is amazing!,203,Yes_comma_ I think so_comma_ she is amazing!,5|5|5_5|5|5,
hit:11584_conv:23168,4,impressed,A friend of mine got first place in her age category in a triathlon!  She is amazing!,725,Well cheers to her! That is a great accomplishment!,5|5|5_5|5|5,
hit:11585_conv:23170,1,confident,I just got a fresh haircut and a new suit for my date.,59,I just got a fresh haircut_comma_ shaved my beard_comma_ and a new suit for my date!,5|5|5_5|5|5,
hit:11585_conv:23170,2,confident,I just got a fresh haircut and a new suit for my date.,322,I hope you have a good time!,5|5|5_5|5|5,
hit:11585_conv:23170,3,confident,I just got a fresh haircut and a new suit for my date.,59,Thank you. I feel confident that this date will go well!,5|5|5_5|5|5,
hit:11585_conv:23170,4,confident,I just got a fresh haircut and a new suit for my date.,322,Just remember to be a gentleman.,5|5|5_5|5|5,
hit:11585_conv:23170,5,confident,I just got a fresh haircut and a new suit for my date.,59,As long as she is willing to pay half the bill!,5|5|5_5|5|5,
hit:11585_conv:23171,1,content,I had a Cobb salad for lunch earlier.  I was quite satisfied with it.,322,I had a Cobb salad for lunch earlier.,5|5|5_5|5|5,
hit:11585_conv:23171,2,content,I had a Cobb salad for lunch earlier.  I was quite satisfied with it.,59,Hm sounds like a healthy choice! Do you feel good about yourself now?,5|5|5_5|5|5,
hit:11585_conv:23171,3,content,I had a Cobb salad for lunch earlier.  I was quite satisfied with it.,322,Yes_comma_ I do.  I was quite satisfied with the meal too.,5|5|5_5|5|5,
hit:11585_conv:23171,4,content,I had a Cobb salad for lunch earlier.  I was quite satisfied with it.,59,Good job! Healthy food is always a good option and is actually cheaper to make than fast food!,5|5|5_5|5|5,
hit:11586_conv:23172,1,content,I found a new job that pays very well. I have money for everything I need.,842,I found a job that pays very well. I have money for everything that I need.,3|4|3_5|5|5,
hit:11586_conv:23172,2,content,I found a new job that pays very well. I have money for everything I need.,525,Good.  Money_comma_ while important_comma_ is certainly not everything.,3|4|3_5|5|5,
hit:11586_conv:23172,3,content,I found a new job that pays very well. I have money for everything I need.,842,It's not but it is what I need right now.,3|4|3_5|5|5,
hit:11586_conv:23172,4,content,I found a new job that pays very well. I have money for everything I need.,525,Keep grindin' bro!,3|4|3_5|5|5,
hit:11586_conv:23172,5,content,I found a new job that pays very well. I have money for everything I need.,842,Thanks! I will! I'm on to the next level.,3|4|3_5|5|5,
hit:11586_conv:23173,1,devastated,I forgot to put the e-brake on in my car the other day_comma_ it was awful watching that 1.3 million dollar ferrari f40 roll into the lake.,525,I forgot to put on my e-brake in my car the other day.,5|5|5_3|4|3,
hit:11586_conv:23173,2,devastated,I forgot to put the e-brake on in my car the other day_comma_ it was awful watching that 1.3 million dollar ferrari f40 roll into the lake.,842,Wow! How did that work out for you?,5|5|5_3|4|3,
hit:11586_conv:23173,3,devastated,I forgot to put the e-brake on in my car the other day_comma_ it was awful watching that 1.3 million dollar ferrari f40 roll into the lake.,525,Not well_comma_ it was pretty awful watching my 1.3 million dollar ferrari f40 roll into the lake.,5|5|5_3|4|3,
hit:11586_conv:23173,4,devastated,I forgot to put the e-brake on in my car the other day_comma_ it was awful watching that 1.3 million dollar ferrari f40 roll into the lake.,842,I'm sorry to hear that. You have to be more careful.,5|5|5_3|4|3,
hit:11587_conv:23174,1,prepared,I have been practicing driving all week for my driver exam.,59,I have been practicing driving all week for my driver exam. I think I am ready!,5|5|5_5|5|5,
hit:11587_conv:23174,2,prepared,I have been practicing driving all week for my driver exam.,659,I'm sure you will ace it! what kind of car are you taking the exam in.,5|5|5_5|5|5,
hit:11587_conv:23174,3,prepared,I have been practicing driving all week for my driver exam.,59,In a Honda Fit. Its a tiny car so it will make it easier to ace the parking part.,5|5|5_5|5|5,
hit:11587_conv:23174,4,prepared,I have been practicing driving all week for my driver exam.,659,Yeah_comma_ that sounds like an easy car to drive. I will send out some good vibes for you. good luck!,5|5|5_5|5|5,
hit:11587_conv:23175,1,prepared,Looked up a recipe_comma_ bought all the things I need. I'm ready to cook a special dinner tonight,659,Looked up the recipe and went shopping and have everything I need!,5|5|5_5|5|5,
hit:11587_conv:23175,2,prepared,Looked up a recipe_comma_ bought all the things I need. I'm ready to cook a special dinner tonight,59,Nice_comma_ what will you be cooking?,5|5|5_5|5|5,
hit:11587_conv:23175,3,prepared,Looked up a recipe_comma_ bought all the things I need. I'm ready to cook a special dinner tonight,659,I'm making chicken marsala by emeril. Bam!,5|5|5_5|5|5,
hit:11587_conv:23175,4,prepared,Looked up a recipe_comma_ bought all the things I need. I'm ready to cook a special dinner tonight,59,Sounds like you have quite the job on your hands! Hopefully it tastes great and you have a great meal.,5|5|5_5|5|5,
hit:11587_conv:23175,5,prepared,Looked up a recipe_comma_ bought all the things I need. I'm ready to cook a special dinner tonight,659,I am ready to cook. been studying the recipe. I think it will be a hit.,5|5|5_5|5|5,
hit:11588_conv:23177,1,confident,I have been taking guitar lessons for almost a year now and will be playing in my first recital.  I've got my song down perfectly so I know I'll do great.,438,5,5|5|5_5|5|5,<UNIGRAM> <NUMERAL>
hit:11588_conv:23177,2,confident,I have been taking guitar lessons for almost a year now and will be playing in my first recital.  I've got my song down perfectly so I know I'll do great.,580,Five is a great number_comma_ but six is even better. It's my favorite number_comma_ don't know why either!,5|5|5_5|5|5,
hit:11588_conv:23177,3,confident,I have been taking guitar lessons for almost a year now and will be playing in my first recital.  I've got my song down perfectly so I know I'll do great.,438,oops!  sorry about that.  I want to tell you that I have been taking guitar lessons for almost a year now and will be playing in my first recital. I've got my song down perfectly so I know I'll do great.,5|5|5_5|5|5,
hit:11588_conv:23177,4,confident,I have been taking guitar lessons for almost a year now and will be playing in my first recital.  I've got my song down perfectly so I know I'll do great.,580,No worries! That's so awesome. Are you taking lessons from a musician in a class_comma_ one on one_comma_ or online?,5|5|5_5|5|5,
hit:11588_conv:23177,5,confident,I have been taking guitar lessons for almost a year now and will be playing in my first recital.  I've got my song down perfectly so I know I'll do great.,438,I am taking them from a musician; private lessons.  Its been great!  I've always wanted to learn.,5|5|5_5|5|5,
hit:11589_conv:23178,1,grateful,My mother was sick for a time_comma_ but she is now doing great_comma_ thank go!,203,My mother was sick for a time_comma_ but she is now doing great_comma_,5|5|5_5|5|5,
hit:11589_conv:23178,2,grateful,My mother was sick for a time_comma_ but she is now doing great_comma_ thank go!,322,I'm glad to hear she is doing well.,5|5|5_5|5|5,
hit:11589_conv:23178,3,grateful,My mother was sick for a time_comma_ but she is now doing great_comma_ thank go!,203,Yes_comma_ thank god!  ,5|5|5_5|5|5,
hit:11589_conv:23178,4,grateful,My mother was sick for a time_comma_ but she is now doing great_comma_ thank go!,322,I hope you both stay healthy!,5|5|5_5|5|5,
hit:11589_conv:23179,1,embarrassed,I accidentally spilled soup all over myself at the restaurant.  I feel so humiliated for doing that.,322,I accidentally spilled soup all over myself at the restaurant.,5|5|5_5|5|5,
hit:11589_conv:23179,2,embarrassed,I accidentally spilled soup all over myself at the restaurant.  I feel so humiliated for doing that.,203,Oh no!  what a bother_comma_  were you able to clean it up?,5|5|5_5|5|5,
hit:11589_conv:23179,3,embarrassed,I accidentally spilled soup all over myself at the restaurant.  I feel so humiliated for doing that.,322,No_comma_ it was a mess.  I feel so humiliated for doing that.,5|5|5_5|5|5,
hit:11589_conv:23179,4,embarrassed,I accidentally spilled soup all over myself at the restaurant.  I feel so humiliated for doing that.,203,You shouldn't that could happen to the best of us.   ,5|5|5_5|5|5,
hit:11590_conv:23180,1,caring,I comforted my friend when she broke up with her boyfriend. Brought her food.,375,It was rough comforting my friend when she left her boyfriend.,4|5|5_5|5|5,
hit:11590_conv:23180,2,caring,I comforted my friend when she broke up with her boyfriend. Brought her food.,104,Oh_comma_ that would be tough! Were they together for a long time?,4|5|5_5|5|5,
hit:11590_conv:23180,3,caring,I comforted my friend when she broke up with her boyfriend. Brought her food.,375,They were for 5 years_comma_ I brought her food to calm her down_comma_,4|5|5_5|5|5,
hit:11590_conv:23180,4,caring,I comforted my friend when she broke up with her boyfriend. Brought her food.,104,That was really nice of you. Your friend is lucky to have you!,4|5|5_5|5|5,
hit:11590_conv:23180,5,caring,I comforted my friend when she broke up with her boyfriend. Brought her food.,375,Thank you I do my best_comma_ have you had a similar experience?,4|5|5_5|5|5,
hit:11590_conv:23181,1,disappointed,I feel so let down. I applied for a job_comma_ but didn't get it. I need to find something soon.,104,I feel so let down. I applied for a job_comma_ but didn't get it.,5|5|5_4|5|5,
hit:11590_conv:23181,2,disappointed,I feel so let down. I applied for a job_comma_ but didn't get it. I need to find something soon.,375,What happened? How come you didnt get it?,5|5|5_4|5|5,
hit:11590_conv:23181,3,disappointed,I feel so let down. I applied for a job_comma_ but didn't get it. I need to find something soon.,104,I guess I didn't do well on the interview. I need to find something soon though!,5|5|5_4|5|5,
hit:11590_conv:23181,4,disappointed,I feel so let down. I applied for a job_comma_ but didn't get it. I need to find something soon.,375,Do you have any major bills coming up?,5|5|5_4|5|5,
hit:11591_conv:23182,1,apprehensive,My roommate wants to get a cat. But he's already irresponsible enough as it is_comma_ and can't clean up after himself. How can he clean up after a cat_comma_ too? But I don't want to be the person to say no...,273,My roommate wants to get a cat. But he's already irresponsible enough as it is_comma_ and can't clean up after himself. How can he clean up after a cat_comma_ too? But I don't want to be the person to say no..,5|5|5_5|5|5,
hit:11591_conv:23182,2,apprehensive,My roommate wants to get a cat. But he's already irresponsible enough as it is_comma_ and can't clean up after himself. How can he clean up after a cat_comma_ too? But I don't want to be the person to say no...,59,I think you will need to tell him how you feel about this. Otherwise you may end up taking more care of the cat than him.,5|5|5_5|5|5,
hit:11591_conv:23182,3,apprehensive,My roommate wants to get a cat. But he's already irresponsible enough as it is_comma_ and can't clean up after himself. How can he clean up after a cat_comma_ too? But I don't want to be the person to say no...,273,Yeah_comma_ you're right. I'm not good at confrontation_comma_ however. But yeah_comma_ I've already decided I don't want to take care of the cat myself_comma_ so if he can't do it_comma_ then it shouldn't be here. I just think he's kind of childish in the sense that he'll assume he can do it just so he can have the cat in the beginning_comma_ but later on he will get lazy.,5|5|5_5|5|5,
hit:11591_conv:23182,4,apprehensive,My roommate wants to get a cat. But he's already irresponsible enough as it is_comma_ and can't clean up after himself. How can he clean up after a cat_comma_ too? But I don't want to be the person to say no...,59,Yea it will be unfair for the cat too who may feel lonely and unloved. You should tell him to be more responsible before he can get a pet.,5|5|5_5|5|5,
hit:11591_conv:23183,1,anticipating,My company got a huge order. I think it will be a busy few months.,59,My company just got a huge order from a Fortune 500!,5|5|5_5|5|5,
hit:11591_conv:23183,2,anticipating,My company got a huge order. I think it will be a busy few months.,273,I'm not 100% sure what that means_comma_ but that sounds exciting_comma_ especially if you're excited about it. Congratulations! Hopefully that means you guys will just keep getting bigger and bigger. ,5|5|5_5|5|5,
hit:11591_conv:23183,3,anticipating,My company got a huge order. I think it will be a busy few months.,59,Yep. It means we are anticipating a lot of work and a lot of money. So pay raises for everyone!,5|5|5_5|5|5,
hit:11591_conv:23183,4,anticipating,My company got a huge order. I think it will be a busy few months.,273,That sounds awesome. I hope everyone is prepared for the next step so that you guys can keep getting better.,5|5|5_5|5|5,
hit:11592_conv:23184,1,lonely,Today I am alone in the house_comma_ since everybody had something to do.  I think I will call a friend.,203,Today I am alone in the house_comma_ since everybody had something to do.,5|5|5_5|5|5,
hit:11592_conv:23184,2,lonely,Today I am alone in the house_comma_ since everybody had something to do.  I think I will call a friend.,799,Do you enjoy your alone time or do you wish you had people around?,5|5|5_5|5|5,
hit:11592_conv:23184,3,lonely,Today I am alone in the house_comma_ since everybody had something to do.  I think I will call a friend.,203,I usually enjoy it_comma_ but today I don't feel like been alone_comma_  I guess I should call a friend.,5|5|5_5|5|5,
hit:11592_conv:23184,4,lonely,Today I am alone in the house_comma_ since everybody had something to do.  I think I will call a friend.,799,You totally should call a friend. I usually enjoy my alone time also but it is nice to have company around.,5|5|5_5|5|5,
hit:11593_conv:23186,1,prepared,I had an exam this past week and I got every question right because I was super prepared.,805,I just found out that I got every question on my last exam correct! I am super happy!,5|5|5_5|5|5,
hit:11593_conv:23186,2,prepared,I had an exam this past week and I got every question right because I was super prepared.,248,hey that's awesome man I'm proud of you_comma_ was it hard?,5|5|5_5|5|5,
hit:11593_conv:23186,3,prepared,I had an exam this past week and I got every question right because I was super prepared.,805,It was really difficult_comma_ but I studied really hard for this one!,5|5|5_5|5|5,
hit:11593_conv:23186,4,prepared,I had an exam this past week and I got every question right because I was super prepared.,248,that's really good keep doing a good job okay,5|5|5_5|5|5,
hit:11593_conv:23186,5,prepared,I had an exam this past week and I got every question right because I was super prepared.,805,Okay!,5|5|5_5|5|5,<UNIGRAM>
hit:11593_conv:23187,1,sad,i missed my period_comma_ took a pregnancy test_comma_ and it's positive. I am not doing okay,248,i missed my period_comma_ took a pregnancy test_comma_ and it's positive. I am not doing okay,5|5|5_5|5|5,
hit:11593_conv:23187,2,sad,i missed my period_comma_ took a pregnancy test_comma_ and it's positive. I am not doing okay,805,How far along do you think you are?,5|5|5_5|5|5,
hit:11593_conv:23187,3,sad,i missed my period_comma_ took a pregnancy test_comma_ and it's positive. I am not doing okay,248,a month and a half I would say..,5|5|5_5|5|5,
hit:11593_conv:23187,4,sad,i missed my period_comma_ took a pregnancy test_comma_ and it's positive. I am not doing okay,805,Well_comma_ I want you to know that it WILL be okay. I was scared to death when I found out I was pregnant with my first one_comma_ and it was really hard_comma_ but everything worked out and he is over 18 now and moved out. Also_comma_ if you decide that you don't want to keep the baby there are lots of options available too.,5|5|5_5|5|5,
hit:11594_conv:23188,1,embarrassed,I was trying to cross a busy street the other day. As I was crossing_comma_ I tripped and fell down to the ground.,842,I was crossing the street the other day. As I was crossing_comma_ I tripped and fell to the ground.,5|5|5_5|5|4,
hit:11594_conv:23188,2,embarrassed,I was trying to cross a busy street the other day. As I was crossing_comma_ I tripped and fell down to the ground.,375,That must have hurt_comma_ did you get injured?,5|5|5_5|5|4,
hit:11594_conv:23188,3,embarrassed,I was trying to cross a busy street the other day. As I was crossing_comma_ I tripped and fell down to the ground.,842,I scraped my knee and dropped my food.,5|5|5_5|5|4,
hit:11594_conv:23188,4,embarrassed,I was trying to cross a busy street the other day. As I was crossing_comma_ I tripped and fell down to the ground.,375,Tragic_comma_ I'd be more annoyed at losing my food I think.,5|5|5_5|5|4,
hit:11594_conv:23189,1,content,Living with my family_comma_ I have no rushes to move out. Love my parents a lot.,375,I'm in no hurry to move out of my parents house.,5|5|4_5|5|5,
hit:11594_conv:23189,2,content,Living with my family_comma_ I have no rushes to move out. Love my parents a lot.,842,I don't blame you. I would be trying to avoid bills too.,5|5|4_5|5|5,
hit:11594_conv:23189,3,content,Living with my family_comma_ I have no rushes to move out. Love my parents a lot.,375,Plus my moms my bestfriend so its a win win situation.,5|5|4_5|5|5,
hit:11594_conv:23189,4,content,Living with my family_comma_ I have no rushes to move out. Love my parents a lot.,842,It is always good to have a pleasant relationship with your parents.,5|5|4_5|5|5,
hit:11595_conv:23190,1,anticipating,I finally get to go on vacation for the first time in 5 years!  Only 7 more days away!,438,I finally get to go on vacation for the first time in 5 years! Only 7 more days away!,5|5|5_5|5|5,
hit:11595_conv:23190,2,anticipating,I finally get to go on vacation for the first time in 5 years!  Only 7 more days away!,44,im excited for you! where are you goin?,5|5|5_5|5|5,
hit:11595_conv:23190,3,anticipating,I finally get to go on vacation for the first time in 5 years!  Only 7 more days away!,438,Going to the beach with just my little family of 4.  Looks like it's going to be beautiful weather!,5|5|5_5|5|5,
hit:11595_conv:23190,4,anticipating,I finally get to go on vacation for the first time in 5 years!  Only 7 more days away!,44,i hope you all have a wonderful time,5|5|5_5|5|5,
hit:11596_conv:23192,1,sad,A good friend of mine is moving to another city in the other side of the country_comma_  I guess we won't see each other much in the future, 203, A good friend of mine is moving to another city in the other side of the country., 5|5|5_5|5|5,
hit:11596_conv:23192,2,sad,A good friend of mine is moving to another city in the other side of the country_comma_  I guess we won't see each other much in the future.,10,What city are they moving to?,5|5|5_5|5|5,
hit:11596_conv:23192,3,sad,A good friend of mine is moving to another city in the other side of the country_comma_  I guess we won't see each other much in the future.,203,To Seattle_comma_ I guess we won't see each other much in the future since I live  in Florida.,5|5|5_5|5|5,
hit:11596_conv:23192,4,sad,A good friend of mine is moving to another city in the other side of the country_comma_  I guess we won't see each other much in the future.,10,That is about as far away as you can move_comma_ I am sorry to hear that_comma_ hope everything works out!,5|5|5_5|5|5,


hit:11626_conv:23253,1,apprehensive,I was thinking about going camping_comma_ but I'm worried about the weather ruining it.,100,"I was thinking about going camping_comma_ but I'm worried about the weather ruining it.,5|5|5_5|5|5,
hit:11626_conv:23253,2,apprehensive,I was thinking about going camping_comma_ but I'm worried about the weather ruining it.,59,Is it possible to re-arrange the date to when the weather would be more favorable?,5|5|5_5|5|5,
hit:11626_conv:23253,3,apprehensive,I was thinking about going camping_comma_ but I'm worried about the weather ruining it.,100,Unfortunately not_comma_ this time is the only time I can get off work until a few months later.,5|5|5_5|5|5,
hit:11626_conv:23253,4,apprehensive,I was thinking about going camping_comma_ but I'm worried about the weather ruining it.,59,I see. You shouldn't let it spoil your plans. Go and have fun but take a rain coat with you!,5|5|5_5|5|5,
hit:11627_conv:23254,1,proud,I have been working on training my puppy. He just started to grasp sitting.,639,I have been working on training my puppy. He just started to grasp sitting.,5|5|5_5|5|5,
hit:11627_conv:23254,2,proud,I have been working on training my puppy. He just started to grasp sitting.,725,Oh nice!! What kind of treats are you using?,5|5|5_5|5|5,
hit:11627_conv:23254,3,proud,I have been working on training my puppy. He just started to grasp sitting.,639,We use his food and some bananas. He loves them and is allergic to a lot of different treats.,5|5|5_5|5|5,
hit:11627_conv:23254,4,proud,I have been working on training my puppy. He just started to grasp sitting.,725,Oh_comma_ I see. I give my dogs popcorn and they love it!,5|5|5_5|5|5,
hit:11628_conv:23256,1,sad,My cat died the other day.,748,Man.....my cat died:( I feel horrible.,5|5|5_4|5|5,
hit:11628_conv:23256,2,sad,My cat died the other day.,375,That's awful_comma_ how did your cat die?,5|5|5_4|5|5,
hit:11628_conv:23256,3,sad,My cat died the other day.,748,Old age. she had a good life but it's still tearing me up.,5|5|5_4|5|5,
hit:11628_conv:23256,4,sad,My cat died the other day.,375,I understand_comma_ I have 4 cats myself. The oldest is 14.,5|5|5_4|5|5,
hit:11628_conv:23256,5,sad,My cat died the other day.,748,Holy Moly thats an old kitty.,5|5|5_4|5|5,
hit:11628_conv:23257,1,confident,Prior to my final math exam. I studied extremely hard up to that point.,375,Feeling pretty good about this upcoming math exam.,4|5|5_5|5|5,
hit:11628_conv:23257,2,confident,Prior to my final math exam. I studied extremely hard up to that point.,748,That's good! Do you usually like math?,4|5|5_5|5|5,
hit:11628_conv:23257,3,confident,Prior to my final math exam. I studied extremely hard up to that point.,375,i don't but this is calculus and I went hard on it.,4|5|5_5|5|5,
hit:11628_conv:23257,4,confident,Prior to my final math exam. I studied extremely hard up to that point.,748,haha good job man. Keep up the good work!,4|5|5_5|5|5,
hit:11629_conv:23258,1,embarrassed,Someone had to walk into the bathroom after I had used it and made it smell really badly. As I was leaving_comma_ they made a comment about how bad it smelled_comma_ and I couldn't help but cringe.,273,Someone had to walk into the bathroom after I had used it and made it smell really badly. As I was leaving_comma_ they made a comment about how bad it smelled_comma_ and I couldn't help but cringe,3|4|4_5|5|5,
hit:11629_conv:23258,2,embarrassed,Someone had to walk into the bathroom after I had used it and made it smell really badly. As I was leaving_comma_ they made a comment about how bad it smelled_comma_ and I couldn't help but cringe.,10,I am sorry to hear that_comma_ it is pretty embarassing to be in situations like that,3|4|4_5|5|5,
hit:11629_conv:23258,3,embarrassed,Someone had to walk into the bathroom after I had used it and made it smell really badly. As I was leaving_comma_ they made a comment about how bad it smelled_comma_ and I couldn't help but cringe.,273,I know... nothing I could have avoided. Either that_comma_ or soil myself in public. Yeah_comma_ easy choice. I guess we all have those experiences from time to time. I will never eat spicy Indian food again_comma_ unless it's at home.,3|4|4_5|5|5,
hit:11629_conv:23258,4,embarrassed,Someone had to walk into the bathroom after I had used it and made it smell really badly. As I was leaving_comma_ they made a comment about how bad it smelled_comma_ and I couldn't help but cringe.,10,Haha I hear you there_comma_ that stuff is potent,3|4|4_5|5|5,
hit:11629_conv:23259,1,content,I am happy with my life right now_comma_ everything is going well,10,I am happy with my life right now_comma_ everything is going well,5|5|5_3|4|4,
hit:11629_conv:23259,2,content,I am happy with my life right now_comma_ everything is going well,273,It's good to hear someone doing well for once. Most people are usually only up for complaining and seeing the negatives in things. I don't want to be too nosey_comma_ but can you tell me why you're feeling good about things lately?,5|5|5_3|4|4,
hit:11629_conv:23259,3,content,I am happy with my life right now_comma_ everything is going well,10,I have a good relationship_comma_ good friends_comma_ a decent job_comma_ and no debt! not much to complain about I guess!,5|5|5_3|4|4,
hit:11629_conv:23259,4,content,I am happy with my life right now_comma_ everything is going well,273,Man.. I wish I could join you in the no debt part. That sounds like it took a lot of work. You're reaping the rewards. I hope things continue to stay stable for you_comma_ and you can keep it going until setting up a good retirement.,5|5|5_3|4|4,
hit:11630_conv:23260,1,terrified,I saw a strange shadow in the kitchen at night. ,59,I saw a strange shadow in the kitchen at night and froze in fear! ,5|5|5_5|5|5,
hit:11630_conv:23260,2,terrified,I saw a strange shadow in the kitchen at night. ,799,Wow I bet you were terrified! Did you see again after or just that once.,5|5|5_5|5|5,
hit:11630_conv:23260,3,terrified,I saw a strange shadow in the kitchen at night. ,59,Just the once. I couldnt even sleep the whole night thinking it was a ghost! ,5|5|5_5|5|5,
hit:11630_conv:23260,4,terrified,I saw a strange shadow in the kitchen at night. ,799,I bet_comma_ I wouldn't be able to sleep after either.,5|5|5_5|5|5,
hit:11631_conv:23262,1,annoyed,My dogs bark is horrible and he won't stop.,748,My dog is getting on my nerves. he never shuts up.,5|5|5_5|5|5,
hit:11631_conv:23262,2,annoyed,My dogs bark is horrible and he won't stop.,139,Sorry to hear that. What is he barking at?,5|5|5_5|5|5,
hit:11631_conv:23262,3,annoyed,My dogs bark is horrible and he won't stop.,748,The freaking wall from waht i can tell.,5|5|5_5|5|5,
hit:11631_conv:23262,4,annoyed,My dogs bark is horrible and he won't stop.,139,Maybe turn on the TV and find a show about dogs. He might be too distracted to bark.,5|5|5_5|5|5,
hit:11631_conv:23262,5,annoyed,My dogs bark is horrible and he won't stop.,748,That might work. I haven't thought of that.,5|5|5_5|5|5,
hit:11631_conv:23263,1,ashamed,I baked my new neighbors a nice meatloaf as a welcome gift. It went well until I got so hungry I had to eat a piece. Then I ate another piece. Next think I knew it was half eaten. I felt so bad.,139,I got new neighbors recently. I wanted to do something nice so I baked a meatloaf for them. The problem is it smelled so good I had to try it first.,5|5|5_5|5|5,
hit:11631_conv:23263,2,ashamed,I baked my new neighbors a nice meatloaf as a welcome gift. It went well until I got so hungry I had to eat a piece. Then I ate another piece. Next think I knew it was half eaten. I felt so bad.,748,Haha that's not a problem! You must be a good cook!,5|5|5_5|5|5,
hit:11631_conv:23263,3,ashamed,I baked my new neighbors a nice meatloaf as a welcome gift. It went well until I got so hungry I had to eat a piece. Then I ate another piece. Next think I knew it was half eaten. I felt so bad.,139,Yeah well it becomes a problem when I keep eating a little more until it is half gone. Now I have to bake another one! I felt so bad.,5|5|5_5|5|5,
hit:11631_conv:23263,4,ashamed,I baked my new neighbors a nice meatloaf as a welcome gift. It went well until I got so hungry I had to eat a piece. Then I ate another piece. Next think I knew it was half eaten. I felt so bad.,748,Hahaha that's funny. Sorry but i find that hilarious.,5|5|5_5|5|5,
hit:11632_conv:23264,1,devastated,I didn't get a promotion at work. I had worked really hard to get.,123,I worked really hard for a promotion at work and I ended up not getting it.,5|5|5_5|5|5,
hit:11632_conv:23264,2,devastated,I didn't get a promotion at work. I had worked really hard to get.,725,That is a shame! I'm sorry to hear that. If you really like you job then just keep working hard. I am sure they will recognize your efforts pretty soon.,5|5|5_5|5|5,
hit:11632_conv:23264,3,devastated,I didn't get a promotion at work. I had worked really hard to get.,123,Thank you! I will keep trying and work harder next time.,5|5|5_5|5|5,
hit:11632_conv:23264,4,devastated,I didn't get a promotion at work. I had worked really hard to get.,725,Go get them my friend!! I know you can do it!,5|5|5_5|5|5,
hit:11633_conv:23267,1,caring,I felt this way while working with patients in the ER.,123,I really enjoyed caring for patients in the Emergency Room while working at my previous job.,5|5|5_5|5|5,
hit:11633_conv:23267,2,caring,I felt this way while working with patients in the ER.,64,That's great. Are you a nurse?,5|5|5_5|5|5,
hit:11633_conv:23267,3,caring,I felt this way while working with patients in the ER.,123,I was a patient access representative. I gathered information for the patients.,5|5|5_5|5|5,
hit:11633_conv:23267,4,caring,I felt this way while working with patients in the ER.,64,That sounds like a really cool profession,5|5|5_5|5|5,
hit:11634_conv:23268,1,faithful,One time I was hit on by a guy I liked at work. I was really interested in him_comma_ and I said no because I had a boyfriend.,843,I was so into that guy. I thought he was really attractive. ,2|5|4_5|5|5,
hit:11634_conv:23268,2,faithful,One time I was hit on by a guy I liked at work. I was really interested in him_comma_ and I said no because I had a boyfriend.,273,Is there any chance for something to happen? Seems like you developed a connection from afar_comma_ if he's still on your mind. Why do you think there's no chance?,2|5|4_5|5|5,
hit:11634_conv:23268,3,faithful,One time I was hit on by a guy I liked at work. I was really interested in him_comma_ and I said no because I had a boyfriend.,843,I really shouldn't because I have a serious boyfriend.,2|5|4_5|5|5,
hit:11634_conv:23268,4,faithful,One time I was hit on by a guy I liked at work. I was really interested in him_comma_ and I said no because I had a boyfriend.,273,Oh.. well that complicates things. Yeah_comma_ maybe that will make you seriously examine things. I think you're okay as long as you don't do anything while still in a relationship. But if you feel your current one is lacking something_comma_ and your eye is wandering_comma_ you owe it to yourself and your current boyfriend to reflect on why that is_comma_ and if there can be improvements_comma_ or if you should both move on.,2|5|4_5|5|5,
hit:11634_conv:23269,1,trusting,My girlfriend has been acting suspicious lately. Guarding her phone_comma_ being absent here and there when she didn't used to be. Having weird excuses for her weird behavior. She tells me nothing is going on_comma_ and I try to believe her. I don't know how much longer that will last.,273,My girlfriend has been acting suspicious lately. Guarding her phone_comma_ being absent here and there when she didn't used to be. Having weird excuses for her weird behavior. She tells me nothing is going on_comma_ and I try to believe her. I don't know how much longer that will last.,5|5|5_2|5|4,
hit:11634_conv:23269,2,trusting,My girlfriend has been acting suspicious lately. Guarding her phone_comma_ being absent here and there when she didn't used to be. Having weird excuses for her weird behavior. She tells me nothing is going on_comma_ and I try to believe her. I don't know how much longer that will last.,843,I'm sorry. It can be hard to be in a situation where you think the other person is cheating. Is there any other reason she might be acting suspicious? Have you talked to her about it?,5|5|5_2|5|4,
hit:11634_conv:23269,3,trusting,My girlfriend has been acting suspicious lately. Guarding her phone_comma_ being absent here and there when she didn't used to be. Having weird excuses for her weird behavior. She tells me nothing is going on_comma_ and I try to believe her. I don't know how much longer that will last.,273,I've talked to her about it a lot. At this point_comma_ I start to feel whiney. But I don't know what else to do. Avoid it when there's still concern? I'm still not getting the answers I want_comma_ and I still have suspicions.,5|5|5_2|5|4,
hit:11634_conv:23269,4,trusting,My girlfriend has been acting suspicious lately. Guarding her phone_comma_ being absent here and there when she didn't used to be. Having weird excuses for her weird behavior. She tells me nothing is going on_comma_ and I try to believe her. I don't know how much longer that will last.,843,Maybe you just need to give her some space. Women tend to pull back when they feel like they are being smothered. Either it works or she moves on. No matter which happens you know.,5|5|5_2|5|4,
hit:11635_conv:23271,1,embarrassed,I was sad as a kid when my pants fell down in class_comma_ everyone laughed at me,10,I was sad as a kid when my pants fell down in class_comma_ everyone laughed at me,5|5|5_5|5|5,
hit:11635_conv:23271,2,embarrassed,I was sad as a kid when my pants fell down in class_comma_ everyone laughed at me,725,Oh yeah_comma_ that is pretty embarrassing! But just imagine if you watched that happen to someone else. You would probably be laughing too. Lol,5|5|5_5|5|5,
hit:11635_conv:23271,3,embarrassed,I was sad as a kid when my pants fell down in class_comma_ everyone laughed at me,10,Yeah I definitely would have at that age haha,5|5|5_5|5|5,
hit:11635_conv:23271,4,embarrassed,I was sad as a kid when my pants fell down in class_comma_ everyone laughed at me,725,It would be pretty embarrassing too if it happened as an adult..... and I guess pretty funny watching it too.,5|5|5_5|5|5,
hit:11636_conv:23272,1,trusting,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship.,463,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship,5|5|5_4|5|5,
hit:11636_conv:23272,2,trusting,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship.,273,I agree! I'm in a long distance relationship_comma_ but we've developed so much trust over the 7 years we've been together that I never worry about what she's doing or who she's with_comma_ and neither does she worry about me. We've slowly built up our communication over the years. I'm glad to hear that you have the same thing in yours.,5|5|5_4|5|5,
hit:11636_conv:23272,3,trusting,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship.,463,we started off long distance and i moved here shortly after. now we've been together going on 6 and a half years,5|5|5_4|5|5,
hit:11636_conv:23272,4,trusting,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship.,273,You guys sound a lot like us. Glad to see that long distance can work for a change. It takes strong_comma_ honest_comma_ communicative people. It gets a bad reputation sometimes_comma_ when it shouldn't. I hope we end up like you guys and can end the long distance thing soon_comma_ though. Gets a little stale.,5|5|5_4|5|5,
hit:11636_conv:23272,5,trusting,i am so appreciative to be able to be open and honest with my husband and to be able to have an amazing relationship.,463,it will be alright if you both put in your efforts and just stay open and honest. it's the most important part.,5|5|5_4|5|5,
hit:11636_conv:23273,1,anticipating,Mentioned my company on social media the other day. Now I'm getting told I have to meet with management after the weekend for something they can't talk about until then. I'm not sure how they could know about what I said_comma_ but I have no idea what else it could be_comma_ and I'm nervous.,273,Mentioned my company on social media the other day. Now I'm getting told I have to meet with management after the weekend for something they can't talk about until then. I'm not sure how they could know about what I said_comma_ but I have no idea what else it could be_comma_ and I'm nervous.,4|5|5_5|5|5,
hit:11636_conv:23273,2,anticipating,Mentioned my company on social media the other day. Now I'm getting told I have to meet with management after the weekend for something they can't talk about until then. I'm not sure how they could know about what I said_comma_ but I have no idea what else it could be_comma_ and I'm nervous.,463,try not to stress out about it all weekend. if you didn't do anything wrong nothing bad will happen.,4|5|5_5|5|5,
hit:11636_conv:23273,3,anticipating,Mentioned my company on social media the other day. Now I'm getting told I have to meet with management after the weekend for something they can't talk about until then. I'm not sure how they could know about what I said_comma_ but I have no idea what else it could be_comma_ and I'm nervous.,273,Yeah_comma_ I tell myself I shouldn't_comma_ but I know I will. Like_comma_ there's no way they could tie my social media to me. But I never get called in to meet with these people. And it happens right after I made a negative comment about the company online? Can't just be a coincidence.,4|5|5_5|5|5,
hit:11636_conv:23273,4,anticipating,Mentioned my company on social media the other day. Now I'm getting told I have to meet with management after the weekend for something they can't talk about until then. I'm not sure how they could know about what I said_comma_ but I have no idea what else it could be_comma_ and I'm nervous.,463,maybe the suspect something and want to talk to you about it. or maybe it's nothing. try not to let it drive you crazy. i know it's easier said than done.,4|5|5_5|5|5,
hit:11637_conv:23274,1,furious,My brother tried fighting me after our hockey game. He said I wasn't passing him the puck enough.,375,So upset at my brother for trying to fight me,5|5|5_5|5|5,
hit:11637_conv:23274,2,furious,My brother tried fighting me after our hockey game. He said I wasn't passing him the puck enough.,286,Brothers always seem to want to get into fights with one another_comma_ but at the end of the day_comma_ I'm sure you guys love each other very much!,5|5|5_5|5|5,
hit:11637_conv:23274,3,furious,My brother tried fighting me after our hockey game. He said I wasn't passing him the puck enough.,375,We were quite furious_comma_ he was mad I wouldn't pass him the puck.,5|5|5_5|5|5,
hit:11637_conv:23274,4,furious,My brother tried fighting me after our hockey game. He said I wasn't passing him the puck enough.,286,Why wouldn't you pass it to'em? Is he a bad shot and you don't want to tell'em?,5|5|5_5|5|5,
hit:11637_conv:23275,1,sentimental,I thought I had given away all of my adult kids old toys_comma_ but I recently stumbled upon one of my daughters old dolls. I think I'll keep it for the memories it brings me.,286,When my kids became adults_comma_ I gave away all their old toys. Or so I thought... Today I found one of my daughter's old dolls. It brought back so many memories I don't think I can get rid of it.,5|5|5_5|5|5,
hit:11637_conv:23275,2,sentimental,I thought I had given away all of my adult kids old toys_comma_ but I recently stumbled upon one of my daughters old dolls. I think I'll keep it for the memories it brings me.,375,What made you give them all away?,5|5|5_5|5|5,
hit:11637_conv:23275,3,sentimental,I thought I had given away all of my adult kids old toys_comma_ but I recently stumbled upon one of my daughters old dolls. I think I'll keep it for the memories it brings me.,286,Well_comma_ I donated them to charity for kids who don't have much. ,5|5|5_5|5|5,
hit:11637_conv:23275,4,sentimental,I thought I had given away all of my adult kids old toys_comma_ but I recently stumbled upon one of my daughters old dolls. I think I'll keep it for the memories it brings me.,375,That is very kind of you_comma_ how old is your daughter now?,5|5|5_5|5|5,
hit:11637_conv:23275,5,sentimental,I thought I had given away all of my adult kids old toys_comma_ but I recently stumbled upon one of my daughters old dolls. I think I'll keep it for the memories it brings me.,286,She's 30 now. So it was quite some time ago that she played with this doll.,5|5|5_5|5|5,
hit:11638_conv:23276,1,faithful,I am a die hard braves fan. I stick with them regardless if they are winning or losing.,524,Luckily the braves are doing good this season_comma_ the past few years have been tough.,5|5|5_5|5|5,
hit:11638_conv:23276,2,faithful,I am a die hard braves fan. I stick with them regardless if they are winning or losing.,123,They are looking pretty good this year.,5|5|5_5|5|5,
hit:11638_conv:23276,3,faithful,I am a die hard braves fan. I stick with them regardless if they are winning or losing.,524,you are looking pretty good this year!,5|5|5_5|5|5,
hit:11638_conv:23276,4,faithful,I am a die hard braves fan. I stick with them regardless if they are winning or losing.,123,I hope the braves make it all the way!,5|5|5_5|5|5,
hit:11638_conv:23277,1,content,I am feeling this way in my life right now. I am so happy.,123,I am so happy with my life right now!,5|5|5_5|5|5,
hit:11638_conv:23277,2,content,I am feeling this way in my life right now. I am so happy.,524,thats great! how come?,5|5|5_5|5|5,
hit:11638_conv:23277,3,content,I am feeling this way in my life right now. I am so happy.,123,I have the best husband and children around!,5|5|5_5|5|5,
hit:11638_conv:23277,4,content,I am feeling this way in my life right now. I am so happy.,524,sounds like you do have a great life!,5|5|5_5|5|5,
hit:11639_conv:23279,1,jealous,In university_comma_ for my major of social work_comma_ we all get mandatory internships where they place us in certain ones and we have no control over it. Some other students received cool internship placements_comma_ and I feel like mine doesn't compare. Kind of disappointed I didn't get something more interesting.,273,In university_comma_ for my major of social work_comma_ we all get mandatory internships where they place us in certain ones and we have no control over it. Some other students received cool internship placements_comma_ and I feel like mine doesn't compare. Kind of disappointed I didn't get something more interesting,4|5|5_5|5|5,
hit:11639_conv:23279,2,jealous,In university_comma_ for my major of social work_comma_ we all get mandatory internships where they place us in certain ones and we have no control over it. Some other students received cool internship placements_comma_ and I feel like mine doesn't compare. Kind of disappointed I didn't get something more interesting.,64,That's unfortunate. At least you can use the experience with your current internship to get you to another place you want to go!,4|5|5_5|5|5,
hit:11639_conv:23279,3,jealous,In university_comma_ for my major of social work_comma_ we all get mandatory internships where they place us in certain ones and we have no control over it. Some other students received cool internship placements_comma_ and I feel like mine doesn't compare. Kind of disappointed I didn't get something more interesting.,273,Yeah_comma_ that's kind of true. We can't change them while in school. They have to place so many people_comma_ so they can't really give people the flexibility of choosing where they want to go_comma_ or else that would get too complex and hectic. So_comma_ I understand that. But still.. some of these places don't match up to some others. I'll be twiddling my thumbs and not learning much_comma_ while others will get a good experience. I guess I just have to make the most of it.,4|5|5_5|5|5,
hit:11639_conv:23279,4,jealous,In university_comma_ for my major of social work_comma_ we all get mandatory internships where they place us in certain ones and we have no control over it. Some other students received cool internship placements_comma_ and I feel like mine doesn't compare. Kind of disappointed I didn't get something more interesting.,64,That's a great attitude. You will be fine_comma_ and can use this as a learning experience,4|5|5_5|5|5,
hit:11640_conv:23280,1,joyful,My son is returning from his tour,59,My son is returning from his tour after ten years on Sunday!,2|2|5_5|5|5,
hit:11640_conv:23280,2,joyful,My son is returning from his tour,524,thats awesome! where has he been?,2|2|5_5|5|5,
hit:11640_conv:23280,3,joyful,My son is returning from his tour,59,He was stationed in Bulgaria. I am so full of joy that I will just give him a big_comma_ tight hug when I see him!,2|2|5_5|5|5,
hit:11640_conv:23280,4,joyful,My son is returning from his tour,524,thank you for his sacrifice and service! it is greatly appreciated!,2|2|5_5|5|5,
hit:11640_conv:23281,1,trusting,i am hoping our government continues to lead the country in the right direction.,524,i am excited about the direction our country is finally going_comma_ hopefully it continues this path,5|5|5_2|2|5,
hit:11640_conv:23281,2,trusting,i am hoping our government continues to lead the country in the right direction.,59,Nice! And what country might that be?,5|5|5_2|2|5,
hit:11640_conv:23281,3,trusting,i am hoping our government continues to lead the country in the right direction.,524,the greatest country in the world! USA!!!,5|5|5_2|2|5,
hit:11640_conv:23281,4,trusting,i am hoping our government continues to lead the country in the right direction.,59,Yes_comma_ things are looking quite good and will only get better from here.,5|5|5_2|2|5,
hit:11641_conv:23282,1,nostalgic,I saw my best friend from high school for the first time in 15 years.  We were able to catch up and reminisce about high school.  Even though we both hated high school.,527,I met up with my best friend in high school for the first time in 15 years.,5|5|5_5|5|5,
hit:11641_conv:23282,2,nostalgic,I saw my best friend from high school for the first time in 15 years.  We were able to catch up and reminisce about high school.  Even though we both hated high school.,725,That's cool! That is a long time! Was he the same as he used to be?,5|5|5_5|5|5,
hit:11641_conv:23282,3,nostalgic,I saw my best friend from high school for the first time in 15 years.  We were able to catch up and reminisce about high school.  Even though we both hated high school.,527,Yeah_comma_ we're both way different_comma_ haha!  But it was nice to catch up and reminisce.,5|5|5_5|5|5,
hit:11641_conv:23282,4,nostalgic,I saw my best friend from high school for the first time in 15 years.  We were able to catch up and reminisce about high school.  Even though we both hated high school.,725,Yeah_comma_ remembering old times with a good friends is always fun!,5|5|5_5|5|5,
hit:11642_conv:23284,1,content,I always wanted a ferrari for everyday of the week.  When my dad bought me my murcialago_comma_ I finally felt a sense of happiness roll over me.,525,I always wanted a ferrari for everyday of the week.,5|5|5_2|4|5,
hit:11642_conv:23284,2,content,I always wanted a ferrari for everyday of the week.  When my dad bought me my murcialago_comma_ I finally felt a sense of happiness roll over me.,100,That's a lofty goal_comma_ but if you work hard maybe you can achieve it.,5|5|5_2|4|5,
hit:11642_conv:23284,3,content,I always wanted a ferrari for everyday of the week.  When my dad bought me my murcialago_comma_ I finally felt a sense of happiness roll over me.,525,Well my dad bought me a murcialago the other day_comma_ so I made it monday murcialago.  I finally can sleep easy at night.,5|5|5_2|4|5,
hit:11642_conv:23284,4,content,I always wanted a ferrari for everyday of the week.  When my dad bought me my murcialago_comma_ I finally felt a sense of happiness roll over me.,100,Wow you must come from a very rich family_comma_ that must be nice.,5|5|5_2|4|5,
hit:11642_conv:23285,6-1,sentimental,I saw some kids playing with airsoft guns_comma_ it reminded me of my childhood.,100,I saw some kids playing with airsoft guns_comma_ it reminded me of my childhood.,2|4|5_5|5|5,
hit:11642_conv:23285,2,sentimental,I saw some kids playing with airsoft guns_comma_ it reminded me of my childhood.,525,Did you go buy a thousand dollar rig and show em who's boss?,2|4|5_5|5|5,
hit:11642_conv:23285,3,sentimental,I saw some kids playing with airsoft guns_comma_ it reminded me of my childhood.,100,No I just watched from the distance and reminisced.,2|4|5_5|5|5,
hit:11642_conv:23285,4,sentimental,I saw some kids playing with airsoft guns_comma_ it reminded me of my childhood.,525,You could have reminisced with an airsoft sniper from a safe distance.,2|4|5_5|5|5,
hit:11643_conv:23286,1,hopeful,I applied to university for a math degree. I get my letter soon.,375,I'm pretty excited to get my letter.,5|5|5_5|5|5,
hit:11643_conv:23286,2,hopeful,I applied to university for a math degree. I get my letter soon.,100,What kind of letter?,5|5|5_5|5|5,
hit:11643_conv:23286,3,hopeful,I applied to university for a math degree. I get my letter soon.,375,Acceptance to a university_comma_ I'm kind of nervous though.,5|5|5_5|5|5,
hit:11643_conv:23286,4,hopeful,I applied to university for a math degree. I get my letter soon.,100,I can understand the feeling_comma_ I think you'll get in though!,5|5|5_5|5|5,
hit:11643_conv:23287,1,devastated,I was building a sculpture and some neighborhood kid threw a baseball and destroyed it.,100,I was building a sculpture outside and some neighborhood kid threw a baseball and destroyed it.,5|5|5_5|5|5,
hit:11643_conv:23287,2,devastated,I was building a sculpture and some neighborhood kid threw a baseball and destroyed it.,375,You must have been devestated!,5|5|5_5|5|5,
hit:11643_conv:23287,3,devastated,I was building a sculpture and some neighborhood kid threw a baseball and destroyed it.,100,I was_comma_ I tried to chase him_comma_ but he was too fast.,5|5|5_5|5|5,
hit:11643_conv:23287,4,devastated,I was building a sculpture and some neighborhood kid threw a baseball and destroyed it.,375,Did you call the police or just let it go?,5|5|5_5|5|5,
hit:11644_conv:23288,1,furious,My roommate promised he'd be good with cleaning up after his cat when he asked if he could get one. Now he's hardly ever cleaning the litter box. It's getting old really fast.,273,My roommate promised he'd be good with cleaning up after his cat when he asked if he could get one. Now he's hardly ever cleaning the litter box. It's getting old really fast.,4|5|5_5|5|5,
hit:11644_conv:23288,2,furious,My roommate promised he'd be good with cleaning up after his cat when he asked if he could get one. Now he's hardly ever cleaning the litter box. It's getting old really fast.,274,Uh oh.  I've heard that plenty of times only to see very similar results.  Time for the cat to move out I think.,4|5|5_5|5|5,
hit:11644_conv:23288,3,furious,My roommate promised he'd be good with cleaning up after his cat when he asked if he could get one. Now he's hardly ever cleaning the litter box. It's getting old really fast.,273,Yeah.. no kidding. But I'm not a confrontational person.. I don't want to be bossy. But it's like_comma_ you knew what you were getting into. If you didn't want to deal with cat poop_comma_ why get the cat? He's 25. He should be mature enough to know these things and deal with them. It's annoying to deal with this mess.,4|5|5_5|5|5,
hit:11644_conv:23288,4,furious,My roommate promised he'd be good with cleaning up after his cat when he asked if he could get one. Now he's hardly ever cleaning the litter box. It's getting old really fast.,274,Maybe you can keep the cat and have him move out.,4|5|5_5|5|5,
hit:11644_conv:23289,1,caring,It is my friends birthday today.  I have put together a surprise party for him tonight.,274,I have a big surprise party planned for my friend tonight.,5|5|5_4|5|5,
hit:11644_conv:23289,2,caring,It is my friends birthday today.  I have put together a surprise party for him tonight.,273,You sound like a good friend_comma_ going through the effort and trouble to fix that up. I'm not into surprises_comma_ but I'm sure you know your friend well enough to know that he'd like it if you gave him one.,5|5|5_4|5|5,
hit:11644_conv:23289,3,caring,It is my friends birthday today.  I have put together a surprise party for him tonight.,274,Yeah I just wanted to show w him that we are thinking of him and that he is a very special person.,5|5|5_4|5|5,
hit:11644_conv:23289,4,caring,It is my friends birthday today.  I have put together a surprise party for him tonight.,273,It's going to be really easy to see that he is special to you guys after tonight. I wish I had a friend like you_comma_ even though I don't like surprises. You should be proud of yourself. I hope it goes smoothly!,5|5|5_4|5|5,





"""

In [60]:
split_lines = issues.split(",")
cleaned_split_lines = []

In [61]:
for line in split_lines:
    cleaned_split_lines.append(line.strip())

In [62]:
conversations = []
for i in range(0,len(cleaned_split_lines),7):
    conversations.append(cleaned_split_lines[i:i+7])

In [63]:
conversations

[['hit:3484_conv:6969',
  '1',
  'caring',
  'I found a stray dog last night and took him home with me.',
  '244',
  'I found a stray dog last night and took him home with me.',
  '5|5|5_5|5|5'],
 ['hit:3484_conv:6969',
  '2',
  'caring',
  'I found a stray dog last night and took him home with me.',
  '447',
  'What a nice thing to do.  Do you know what breed?',
  '5|5|5_5|5|5'],
 ['hit:3484_conv:6969',
  '3',
  'caring',
  'I found a stray dog last night and took him home with me.',
  '244',
  'some kind of heeler mix. Not sure.',
  '5|5|5_5|5|5'],
 ['hit:3484_conv:6969',
  '4',
  'caring',
  'I found a stray dog last night and took him home with me.',
  '447',
  'Is he a young dog or old?',
  '5|5|5_5|5|5'],
 ['hit:3484_conv:6969',
  '5',
  'caring',
  'I found a stray dog last night and took him home with me.',
  '244',
  'He looks young. A year or two old.',
  '5|5|5_5|5|5'],
 ['hit:3485_conv:6970',
  '1',
  'excited',
  'Me and my college friends have planned a trip to Miami this

In [64]:
pd.DataFrame.from_records(conversations).to_csv("cleaned_up_conv.csv",index=False)

In [72]:
valid_issues= """
hit:5003_conv:10007,1,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,60,"I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,5|5|5_5|5|5,
hit:5003_conv:10007,2,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,286,That's something that takes a lot of contemplation_comma_ because it's such a big decision to make. Will the extra money be worth the extra work?,5|5|5_5|5|5,
hit:5003_conv:10007,3,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,60,An extra 40k a year but it would be highly stressful. Currently I have the highest ratings in the USA_comma_ I'm sure any new employer would want me to bring them up to that stage as well. ,5|5|5_5|5|5,
hit:5003_conv:10007,4,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,286,Oh wow_comma_ an additional 40K is a lot! I'd be willing to put up with quite a bit for a pay increase that large.,5|5|5_5|5|5,
hit:5003_conv:10007,5,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,60,I currently work 50 hours a week. I'd be working around 75-80 a week then.,5|5|5_5|5|5,
hit:5003_conv:10007,6,apprehensive,I've been at my current job 5 years. I'm at the point I can go into upper management but I'm afraid. Maybe I'm just comfortable now. I don't know. ,286,Oh wow. I'll be honest and say it wouldn't be worth it for me! What's an extra 40K if you're never not at work to enjoy it?,5|5|5_5|5|5,
hit:5008_conv:10016,1,impressed,I took my wife surfing for the first time today. within an hour she was riding waves_comma_ Color me impressed!,60,I took my wife surfing for the first time today. within an hour she was riding waves_comma_ Color me impressed!,5|5|5_5|5|5,
hit:5008_conv:10016,2,impressed,I took my wife surfing for the first time today. within an hour she was riding waves_comma_ Color me impressed!,286,Oh wow_comma_ I think it'd take me much longer to not bust my butt! That is impressive!,5|5|5_5|5|5,
hit:5008_conv:10016,3,impressed,I took my wife surfing for the first time today. within an hour she was riding waves_comma_ Color me impressed!,60,Yes_comma_  I was surprised. I've been at it my entire life and never seen anyone pick it up so quickly. She is not even athletic.,5|5|5_5|5|5,
hit:5008_conv:10016,4,impressed,I took my wife surfing for the first time today. within an hour she was riding waves_comma_ Color me impressed!,286,Maybe she's more athletic than you think! She might be better than you soon. Better watch out!,5|5|5_5|5|5,
hit:5009_conv:10019,1,jealous,A friend of mine bought a new car.  It was a model I had my eye on for some time,111,A friend of mine bought a new car,4|4|4_5|5|5,
hit:5009_conv:10019,2,jealous,A friend of mine bought a new car.  It was a model I had my eye on for some time,524,awesome! what kind?,4|4|4_5|5|5,
hit:5009_conv:10019,3,jealous,A friend of mine bought a new car.  It was a model I had my eye on for some time,111,Tesla_comma_ I've had my eye on the model for some time,4|4|4_5|5|5,
hit:5009_conv:10019,4,jealous,A friend of mine bought a new car.  It was a model I had my eye on for some time,524,oh man those are super nice! all electric is pretty neat,4|4|4_5|5|5,
hit:5015_conv:10030,1,impressed,A friend of mine got a new high paying job.  I didn't think he could do it but he somehow pulled it off,111,A friend of mine got a new high paying job,4|4|4_5|5|5,
hit:5015_conv:10030,2,impressed,A friend of mine got a new high paying job.  I didn't think he could do it but he somehow pulled it off,448,That's good on his part_comma_ how do you feel about it?,4|4|4_5|5|5,
hit:5015_conv:10030,3,impressed,A friend of mine got a new high paying job.  I didn't think he could do it but he somehow pulled it off,111,I thought it was amazing he got it_comma_ seemed like a tough job to land,4|4|4_5|5|5,
hit:5015_conv:10030,4,impressed,A friend of mine got a new high paying job.  I didn't think he could do it but he somehow pulled it off,448,That's awesome_comma_ and I agree_comma_ the job market is tough now a days.  High paying or not.,4|4|4_5|5|5,
hit:5021_conv:10043,1,annoyed,A co-worker of mine was late delivering materials for a project.  I wish he'd care more,111,A co-worker of mine was late delivering materials for a project,4|4|4_5|5|5,
hit:5021_conv:10043,2,annoyed,A co-worker of mine was late delivering materials for a project.  I wish he'd care more,332,Oh that's no good. Any special reason why they were late?,4|4|4_5|5|5,
hit:5021_conv:10043,3,annoyed,A co-worker of mine was late delivering materials for a project.  I wish he'd care more,111,The guy just doesn't seem to care,4|4|4_5|5|5,
hit:5021_conv:10043,4,annoyed,A co-worker of mine was late delivering materials for a project.  I wish he'd care more,332,That's a bad attitude to have. I hope he can straighten up a bit if we wants to keep working there.,4|4|4_5|5|5,
hit:5030_conv:10061,1,ashamed,Going to the doctor for the first time since 2001 tomorrow. I'm dreading telling the doctor how long it has been since my last visit. ,60,Going to the doctor for the first time since 2001 tomorrow. I'm dreading telling the doctor how long it has been since my last visit. ,5|5|5_5|5|5,
hit:5030_conv:10061,2,ashamed,Going to the doctor for the first time since 2001 tomorrow. I'm dreading telling the doctor how long it has been since my last visit. ,366,It will be okay. It's always hard making that step_comma_ but it's good for your health to do it.,5|5|5_5|5|5,
hit:5030_conv:10061,3,ashamed,Going to the doctor for the first time since 2001 tomorrow. I'm dreading telling the doctor how long it has been since my last visit. ,60,I'm only going because my insurance gives me a gift card for going. I'm in perfect health. But still ashamed.,5|5|5_5|5|5,
hit:5030_conv:10061,4,ashamed,Going to the doctor for the first time since 2001 tomorrow. I'm dreading telling the doctor how long it has been since my last visit. ,366,I wish my insurance would give me something like that! It's good to go anyways.,5|5|5_5|5|5,
hit:5035_conv:10071,1,afraid,im scared of this big test tomorrow,44,im scared of this big test tomorrow,5|5|5_3|4|5,
hit:5035_conv:10071,2,afraid,im scared of this big test tomorrow,448,Have you studied for it?  Always helps to prepare!,5|5|5_3|4|5,
hit:5035_conv:10071,3,afraid,im scared of this big test tomorrow,44,yes i have_comma_ still nervous though,5|5|5_3|4|5,
hit:5035_conv:10071,4,afraid,im scared of this big test tomorrow,448,Good luck_comma_ I'm sure you will do fine!,5|5|5_3|4|5,
hit:5037_conv:10074,1,guilty,My wife and I are on a diet together for the last month. We have been doing great but today on my way home from work I stopped for Ice cream and did not tell her.,60,My wife and I are on a diet together for the last month. We have been doing great but today on my way home from work I stopped for Ice cream and did not tell her.,5|5|5_5|5|5,
hit:5037_conv:10074,2,guilty,My wife and I are on a diet together for the last month. We have been doing great but today on my way home from work I stopped for Ice cream and did not tell her.,478,Hey everyone trips up a little. What you need to remember is that you and your wife made a commitment and you should follow through on that. ,5|5|5_5|5|5,
hit:5037_conv:10074,3,guilty,My wife and I are on a diet together for the last month. We have been doing great but today on my way home from work I stopped for Ice cream and did not tell her.,60,Yea I know. The real issues is that I want to stop again from more tomorrow. ,5|5|5_5|5|5,
hit:5037_conv:10074,4,guilty,My wife and I are on a diet together for the last month. We have been doing great but today on my way home from work I stopped for Ice cream and did not tell her.,478,It's ok to have a cheat day everyone once in a while_comma_ but you can't give in. Be strong!,5|5|5_5|5|5,
hit:5043_conv:10087,1,afraid,I just turned in my dissertation. I'm scared it will not pass and all my work toward my masters will be for nothing.,60,I just turned in my dissertation. I'm scared it will not pass and all my work toward my masters will be for nothing.,5|5|5_5|5|5,
hit:5043_conv:10087,2,afraid,I just turned in my dissertation. I'm scared it will not pass and all my work toward my masters will be for nothing.,27,You only get to do it once?,5|5|5_5|5|5,
hit:5043_conv:10087,3,afraid,I just turned in my dissertation. I'm scared it will not pass and all my work toward my masters will be for nothing.,60,Yes_comma_ you turn it in then go in front of a board of people. In my case 10 people. If they don't like it I can't go one to work on my PhD and will need to write another. This one has taken 2 years to write.,5|5|5_5|5|5,
hit:5043_conv:10087,4,afraid,I just turned in my dissertation. I'm scared it will not pass and all my work toward my masters will be for nothing.,27,Oh no. I hope it goes well for you. That's a lot of hard work.,5|5|5_5|5|5,
hit:5045_conv:10091,1,joyful,I finally got a new raise at my job.  I've never been happier honestly,111,I finally got a raise at my job,4|4|4_5|5|5,
hit:5045_conv:10091,2,joyful,I finally got a new raise at my job.  I've never been happier honestly,27,Congrats!,4|4|4_5|5|5,<UNIGRAM>
hit:5045_conv:10091,3,joyful,I finally got a new raise at my job.  I've never been happier honestly,111,I was pretty happy_comma_ it was long overdue,4|4|4_5|5|5,
hit:5045_conv:10091,4,joyful,I finally got a new raise at my job.  I've never been happier honestly,27,Was it more than expected?,4|4|4_5|5|5,
hit:5048_conv:10096,1,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,473,The other day I was drinking and sent an email I shouldn't have. I felt so stupid when I woke up and realized what I'd done.,5|5|5_5|5|5,
hit:5048_conv:10096,2,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,27,That has happened to me_comma_ but with texts though lol,5|5|5_5|5|5,
hit:5048_conv:10096,3,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,473,Yeah I've done that_comma_ too. The funny thing is I'm usually pretty eloquent even when I'm drunk so I don't think the recipient even realized I was drunk when I wrote it.,5|5|5_5|5|5,
hit:5048_conv:10096,4,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,27,Did you apologize?,5|5|5_5|5|5,
hit:5048_conv:10096,5,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,473,No_comma_ when they responded I just played it off like I meant to send it. Everything turned out fine!,5|5|5_5|5|5,
hit:5048_conv:10096,6,embarrassed,I recently sent an email I shouldn't have after a night of drinking. I woke up in a panic after and hoped it was a dream but nah_comma_ I really did send the email. I feel so foolish.,27,LOL,5|5|5_5|5|5,<UNIGRAM>
hit:5049_conv:10099,1,apprehensive,My car has been having problems.  I'm worried it won't make it in the future,111,My car has been having issues lately,4|4|4_1|3|3,
hit:5049_conv:10099,2,apprehensive,My car has been having problems.  I'm worried it won't make it in the future,463,what's wrong with it?,4|4|4_1|3|3,
hit:5049_conv:10099,3,apprehensive,My car has been having problems.  I'm worried it won't make it in the future,111,Engine issues I think_comma_ not sure if the car will make it,4|4|4_1|3|3,
hit:5049_conv:10099,4,apprehensive,My car has been having problems.  I'm worried it won't make it in the future,463,can you fix it?,4|4|4_1|3|3,
hit:5051_conv:10103,1,surprised,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,60,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,5|5|5_5|5|5,
hit:5051_conv:10103,2,surprised,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,4,That is great news. Have you also been working out?,5|5|5_5|5|5,
hit:5051_conv:10103,3,surprised,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,60,Not really. I average 10 miles a day walking at work but did that when I was fat as well. I cut out all processed foods and drinks.,5|5|5_5|5|5,
hit:5051_conv:10103,4,surprised,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,4,Has it been easy or hard to do that?,5|5|5_5|5|5,
hit:5051_conv:10103,5,surprised,Got on the scale today. I've only been on a diet 2 weeks and I'm down 17 pounds.,60,Rough. I used to like Iced tea and now only drink black coffee or water. I miss flavor. Plus it's a lot of work cooking everything.,5|5|5_5|5|5,
hit:5053_conv:10106,1,guilty,A friend left some candy out and I took some.  I feel like paying him back for what I took,111,I took some candy a friend left out,4|4|4_5|5|5,
hit:5053_conv:10106,2,guilty,A friend left some candy out and I took some.  I feel like paying him back for what I took,514,did they notice it was missing?,4|4|4_5|5|5,
hit:5053_conv:10106,3,guilty,A friend left some candy out and I took some.  I feel like paying him back for what I took,111,Not yet_comma_ I plan on compensating my friend for what I took,4|4|4_5|5|5,
hit:5053_conv:10106,4,guilty,A friend left some candy out and I took some.  I feel like paying him back for what I took,514,well at least you are attempting to be honest about it.,4|4|4_5|5|5,
hit:5056_conv:10112,1,sad,I just realized today I have not been home to where my family is from in  a little over 10 years.,60,I just realized today I have not been home to where my family is from in a little over 10 years.,5|5|5_4|4|4,
hit:5056_conv:10112,2,sad,I just realized today I have not been home to where my family is from in  a little over 10 years.,317,Ohh wow! Are you excited?,5|5|5_4|4|4,
hit:5056_conv:10112,3,sad,I just realized today I have not been home to where my family is from in  a little over 10 years.,60,No I'm not going. I don't have time and my family dose not talk to me. I just miss home.,5|5|5_4|4|4,
hit:5056_conv:10112,4,sad,I just realized today I have not been home to where my family is from in  a little over 10 years.,317,Ohh that stinks. I dont blame you though,5|5|5_4|4|4,
hit:5057_conv:10115,1,terrified,Recently this strange man was following me around the store_comma_ and then followed me out to the parking lot. He kept asking me dumb questions_comma_ I couldn't tell if he was just awkward or sizing me up to rob me or what.,473,The other day this creepy guy followed me around the store and chased me out to the parking lot. He just wouldn't take a hint.,5|5|5_5|5|5,
hit:5057_conv:10115,2,terrified,Recently this strange man was following me around the store_comma_ and then followed me out to the parking lot. He kept asking me dumb questions_comma_ I couldn't tell if he was just awkward or sizing me up to rob me or what.,448,That's very creepy_comma_ did he say anything to you?,5|5|5_5|5|5,
hit:5057_conv:10115,3,terrified,Recently this strange man was following me around the store_comma_ and then followed me out to the parking lot. He kept asking me dumb questions_comma_ I couldn't tell if he was just awkward or sizing me up to rob me or what.,473,Yeah that's what made it weird_comma_ it was kind of like he was trying to hit on me but just awkward. But he was really pushy about it.,5|5|5_5|5|5,
hit:5057_conv:10115,4,terrified,Recently this strange man was following me around the store_comma_ and then followed me out to the parking lot. He kept asking me dumb questions_comma_ I couldn't tell if he was just awkward or sizing me up to rob me or what.,448,Those are definitely the scary types_comma_ they don't recognize social cues at all.,5|5|5_5|5|5,
hit:5057_conv:10115,5,terrified,Recently this strange man was following me around the store_comma_ and then followed me out to the parking lot. He kept asking me dumb questions_comma_ I couldn't tell if he was just awkward or sizing me up to rob me or what.,473,I hope that's all it was and not that he had some bad motive.,5|5|5_5|5|5,
hit:5058_conv:10117,1,anxious,A close friend said she was having problems with her knee.  I hope she goes to a doctor soon,111,A close friend said she was having problems with her knee,4|4|4_2|5|5,
hit:5058_conv:10117,2,anxious,A close friend said she was having problems with her knee.  I hope she goes to a doctor soon,438,Ouch!  Did she hurt it or is it a degenerative thing? I hope she can find help with physical therapy.,4|4|4_2|5|5,
hit:5058_conv:10117,3,anxious,A close friend said she was having problems with her knee.  I hope she goes to a doctor soon,111,Hopefully she will see a doctor soon about it,4|4|4_2|5|5,
hit:5058_conv:10117,4,anxious,A close friend said she was having problems with her knee.  I hope she goes to a doctor soon,438,Oh good.  Hopefully she'll try to fix it with physical therapy first.  Some doctors love to do surgery right away when its not necessary.  Recommend that she get a second opinion.  Hope she resolves it!,4|4|4_2|5|5,
hit:5061_conv:10122,1,furious,A co-worker stole my lunch I believe.  I ended up having to skip a meal,111,A co-worker stole my lunch,4|4|4_5|5|5,
hit:5061_conv:10122,2,furious,A co-worker stole my lunch I believe.  I ended up having to skip a meal,473,I've never understood that_comma_ it's not like they can't afford to eat since they have a job! Do you know which coworker it was?,4|4|4_5|5|5,
hit:5061_conv:10122,3,furious,A co-worker stole my lunch I believe.  I ended up having to skip a meal,111,Not entirely sure_comma_ I marked my lunch so they knew it wasn't theirs,4|4|4_5|5|5,
hit:5061_conv:10122,4,furious,A co-worker stole my lunch I believe.  I ended up having to skip a meal,473,Next time bring your lunch in a biohazard bag_comma_ maybe they will leave it alone.,4|4|4_5|5|5,
hit:5061_conv:10123,1,furious,A now ex-friend of mine revealed a very private secret I told her in confidence not only to our mutual friends but the entire internet! I couldn't believe she did that to me.,473,A friend of mine revealed a deep dark secret I told her in confidence to everyone. ,5|5|5_4|4|4,
hit:5061_conv:10123,2,furious,A now ex-friend of mine revealed a very private secret I told her in confidence not only to our mutual friends but the entire internet! I couldn't believe she did that to me.,111,What did you do?,5|5|5_4|4|4,
hit:5061_conv:10123,3,furious,A now ex-friend of mine revealed a very private secret I told her in confidence not only to our mutual friends but the entire internet! I couldn't believe she did that to me.,473,I blocked her and sent her a long text explaining how angry and disappointed I was. I didn't know what else to do.,5|5|5_4|4|4,
hit:5061_conv:10123,4,furious,A now ex-friend of mine revealed a very private secret I told her in confidence not only to our mutual friends but the entire internet! I couldn't believe she did that to me.,111,You definitely can't be too careful these days,5|5|5_4|4|4,
hit:5061_conv:10123,5,furious,A now ex-friend of mine revealed a very private secret I told her in confidence not only to our mutual friends but the entire internet! I couldn't believe she did that to me.,473,You're right. It makes me less likely to trust others in the future.,5|5|5_4|4|4,
hit:5062_conv:10124,1,angry,Every single time I go in the office bathroom someone comes in and needs something. If I don't go to the bathroom no one comes in all day.,60,Every single time I go in the office bathroom someone comes in and needs something. If I don't go to the bathroom no one comes in all day.,5|3|3_5|5|5,
hit:5062_conv:10124,2,angry,Every single time I go in the office bathroom someone comes in and needs something. If I don't go to the bathroom no one comes in all day.,238,how do you know no one goes in if you arent in there?,5|3|3_5|5|5,
hit:5062_conv:10124,3,angry,Every single time I go in the office bathroom someone comes in and needs something. If I don't go to the bathroom no one comes in all day.,60,Because I work in the same little office alone all day. Today I did not use the bathroom no one came in. yesterday I went twice and each time I went I had someone waiting on me when I got back to my desk.,5|3|3_5|5|5,
hit:5062_conv:10124,4,angry,Every single time I go in the office bathroom someone comes in and needs something. If I don't go to the bathroom no one comes in all day.,238,ok,5|3|3_5|5|5,<UNIGRAM>
hit:5073_conv:10147,1,disgusted,My cat peed on my bath mat yesterday. I didn't realize it until I stepped out of the shower today and it was already wet.,473,I discovered my cat peed on my bath mat when I got out of the shower today and the mat was wet.,5|5|5_5|5|5,
hit:5073_conv:10147,2,disgusted,My cat peed on my bath mat yesterday. I didn't realize it until I stepped out of the shower today and it was already wet.,273,Oh no! That stinks. I've never had a cat specifically because I've never been good with the idea of being able to train them. Plus it seems like you just get the one personality_comma_ and they never grow to like you. But I'm biased.,5|5|5_5|5|5,
hit:5073_conv:10147,3,disgusted,My cat peed on my bath mat yesterday. I didn't realize it until I stepped out of the shower today and it was already wet.,473,It's funny you say that_comma_ I actually like that about cats. They are independent and not too needy_comma_ which sort of describes me! I love my cats... the peeing on my bath mat aside.,5|5|5_5|5|5,
hit:5073_conv:10147,4,disgusted,My cat peed on my bath mat yesterday. I didn't realize it until I stepped out of the shower today and it was already wet.,273,Yeah_comma_ I know they definitely suit some people. I actually now technically have one... my roommate wanted to get one_comma_ and I felt too bad to say no. And he actually is a pretty nice guy_comma_ mostly because he's been affectionate from the start. So_comma_ we'll see how it goes! Sorry again about your mat.,5|5|5_5|5|5,
hit:5073_conv:10147,5,disgusted,My cat peed on my bath mat yesterday. I didn't realize it until I stepped out of the shower today and it was already wet.,473,Here's hoping your cat grows on you_comma_ they have a funny ability to do so.,5|5|5_5|5|5,
hit:5077_conv:10154,1,confident,I recently got the courage to approach a potential client and set a rate for work that is way higher than what I normally charge. I guess I was just feeling good and knew I do good work!,473,I got the courage to contact a potential client the other day and set a rate for my work that's higher than I usually charge.,5|5|5_5|5|5,
hit:5077_conv:10154,2,confident,I recently got the courage to approach a potential client and set a rate for work that is way higher than what I normally charge. I guess I was just feeling good and knew I do good work!,478,Wow good for you! I hope that it works out for you!,5|5|5_5|5|5,
hit:5077_conv:10154,3,confident,I recently got the courage to approach a potential client and set a rate for work that is way higher than what I normally charge. I guess I was just feeling good and knew I do good work!,473,Thanks! The work isn't terribly exciting but it felt good not having to negotiate knowing the client wanted me to do work for them.,5|5|5_5|5|5,
hit:5077_conv:10154,4,confident,I recently got the courage to approach a potential client and set a rate for work that is way higher than what I normally charge. I guess I was just feeling good and knew I do good work!,478,Yeah and I bet the higher paycheck will make the work worth it. ,5|5|5_5|5|5,
hit:5077_conv:10154,5,confident,I recently got the courage to approach a potential client and set a rate for work that is way higher than what I normally charge. I guess I was just feeling good and knew I do good work!,473,That always makes it worth it to be honest!,5|5|5_5|5|5,
hit:5081_conv:10162,1,furious,Someone hit my car and fled the scene.  I wish I could find the person,111,Someone hit my car and fled the scene,4|4|4_5|5|5,
hit:5081_conv:10162,2,furious,Someone hit my car and fled the scene.  I wish I could find the person,220,What! First of all_comma_ are you alright?,4|4|4_5|5|5,
hit:5081_conv:10162,3,furious,Someone hit my car and fled the scene.  I wish I could find the person,111,Yeah_comma_ I'm fine_comma_ the car was parked,4|4|4_5|5|5,
hit:5081_conv:10162,4,furious,Someone hit my car and fled the scene.  I wish I could find the person,220,Oh_comma_ well that's something_comma_ at least. I can't believe people are such cowards_comma_ though.,4|4|4_5|5|5,
hit:5082_conv:10165,1,caring,I'm promoting two staff members on Sunday. They work hard and are great people.,60,'m promoting two staff members on Sunday. They work hard and are great people.,5|5|5_5|5|5,
hit:5082_conv:10165,2,caring,I'm promoting two staff members on Sunday. They work hard and are great people.,27,That's great. Do they know?,5|5|5_5|5|5,
hit:5082_conv:10165,3,caring,I'm promoting two staff members on Sunday. They work hard and are great people.,60,They have no clue. They know I ask them to do more then others but are clueless. I'm excited for them and the also the stress load they will take off of me. ,5|5|5_5|5|5,
hit:5082_conv:10165,4,caring,I'm promoting two staff members on Sunday. They work hard and are great people.,27,Well that is awesome. I bet they won't let you down in the new positions either,5|5|5_5|5|5,
hit:5091_conv:10182,1,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,60,'m running my first Marathon Sunday. I'm scared that maybe I did not train hard enough.,5|5|5_5|5|5,
hit:5091_conv:10182,2,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,27,I have never run a marathon,5|5|5_5|5|5,
hit:5091_conv:10182,3,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,60,No better time to try then now.,5|5|5_5|5|5,
hit:5091_conv:10182,4,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,27,You can try your best and if you don't finish there is always the next one,5|5|5_5|5|5,
hit:5091_conv:10182,5,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,60,I will not run If I can't finish. I'm stubborn I'll finish or kill over.,5|5|5_5|5|5,
hit:5091_conv:10182,6,anxious,I'm running my first Marathon Sunday. I'm scared that maybe I did not train hard enough. ,27,Well that's some sort of motivation lol,5|5|5_5|5|5,
hit:5100_conv:10201,1,proud,Just finished my Master's degree now time to take on a PhD,60,Just finished my Master's degree now time to take on a PhD,5|5|5_4|5|5,
hit:5100_conv:10201,2,proud,Just finished my Master's degree now time to take on a PhD,294,Congratulations!  What are you getting your PhD in?,5|5|5_4|5|5,
hit:5100_conv:10201,3,proud,Just finished my Master's degree now time to take on a PhD,60,Astrophysics,5|5|5_4|5|5,<UNIGRAM>
hit:5100_conv:10201,4,proud,Just finished my Master's degree now time to take on a PhD,294,Oh geeze_comma_ I didn't know I was talking to a genius.  How long is the program?,5|5|5_4|5|5,
hit:5100_conv:10201,5,proud,Just finished my Master's degree now time to take on a PhD,60,I have another 3-4 years then I'll go into teaching.,5|5|5_4|5|5,
hit:5112_conv:10224,1,anticipating,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.  ,60,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.,5|5|5_5|5|5,
hit:5112_conv:10224,2,anticipating,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.  ,4,Have you tried to talk to your teacher?,5|5|5_5|5|5,
hit:5112_conv:10224,3,anticipating,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.  ,60,No it's mostly I feel behind on my thesis. When I started they told us keep up or drop out. I should have known_comma_ it's one of the hardest schools to get into.,5|5|5_5|5|5,
hit:5112_conv:10224,4,anticipating,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.  ,4,What school is this you are talking about?,5|5|5_5|5|5,
hit:5112_conv:10224,5,anticipating,I start my last year of my Masters program Friday. I'm scared I feel like I'm all ready behind.  ,60,Cornell University_comma_ after this year I transfer to Princeton for my PhD,5|5|5_5|5|5,
hit:5124_conv:10248,1,prepared,I'm spent all summer getting ready to write my thesis. I can't wait to get time to start.,60,I spent all summer getting ready to write my thesis. I can't wait to get time to start.,5|5|5_3|5|5,
hit:5124_conv:10248,2,prepared,I'm spent all summer getting ready to write my thesis. I can't wait to get time to start.,273,That's pretty cool that you're motivated and excited to start it. Most people would be scared and procrastinating. You'll do great.,5|5|5_3|5|5,
hit:5124_conv:10248,3,prepared,I'm spent all summer getting ready to write my thesis. I can't wait to get time to start.,60,I need to get it over. I need to start paying back student loans. It's the last thing I need to do before I can get hired as a Ad Professor. ,5|5|5_3|5|5,
hit:5124_conv:10248,4,prepared,I'm spent all summer getting ready to write my thesis. I can't wait to get time to start.,273,Sounds like you have a plan laid out. It also sounds like you've already accomplished so much.. You should be very proud. I'm with you_comma_ I have over $40_comma_000 in student loans. Such a crazy pressure on someone right when they graduate. Good luck!,5|5|5_3|5|5,
hit:5124_conv:10248,5,prepared,I'm spent all summer getting ready to write my thesis. I can't wait to get time to start.,60,I've been paying on mine and have 75 still to pay. Then I want a PhD so that will be another 50k,5|5|5_3|5|5,
hit:5211_conv:10423,1,jealous,I have to get all the attention from my boyfriend I get a little envious when he spend more time with others.,284,I love to be snuggled up with attention and time with my boyfriend when he doesn't I get sad and jealous. ,4|3|5_5|5|5,
hit:5211_conv:10423,2,jealous,I have to get all the attention from my boyfriend I get a little envious when he spend more time with others.,322,I can understand that.,4|3|5_5|5|5,
hit:5211_conv:10423,3,jealous,I have to get all the attention from my boyfriend I get a little envious when he spend more time with others.,284,I know he has to spend time with others but I get envious I want every minute spent together I'm spoiled lol.,4|3|5_5|5|5,
hit:5211_conv:10423,4,jealous,I have to get all the attention from my boyfriend I get a little envious when he spend more time with others.,322,You must love him very much_comma_ that's good.,4|3|5_5|5|5,
hit:5211_conv:10423,5,jealous,I have to get all the attention from my boyfriend I get a little envious when he spend more time with others.,284,I do best feeling ever.,4|3|5_5|5|5,
hit:5291_conv:10582,1,lonely,I don't have many friends at my new school,509,I'm at a new school and I won't have any friends,5|5|5_5|5|5,
hit:5291_conv:10582,2,lonely,I don't have many friends at my new school,234,Aw_comma_ it takes time! Making friends isn't always so easy so give it a little bit of time,5|5|5_5|5|5,
hit:5291_conv:10582,3,lonely,I don't have many friends at my new school,509,Yes_comma_ but how do I even talk to people I don't know?,5|5|5_5|5|5,
hit:5291_conv:10582,4,lonely,I don't have many friends at my new school,234,The best thing to do is join clubs and get involved. You generally make friends by association!,5|5|5_5|5|5,
hit:5292_conv:10584,1,hopeful,I felt very greata few weeks ago when i helped an elderly women with bagging her groceries ,12,I felt very great a few weeks ago when i helped an elderly women bag her groceries,5|5|5_5|5|4,
hit:5292_conv:10584,2,hopeful,I felt very greata few weeks ago when i helped an elderly women with bagging her groceries ,507,that is really nice of you to do_comma_ not a lot of people help others like that anymore,5|5|5_5|5|4,
hit:5292_conv:10584,3,hopeful,I felt very greata few weeks ago when i helped an elderly women with bagging her groceries ,12,Yeah thats true most people don't.  I love helping those in need it's a great feeling,5|5|5_5|5|4,
hit:5292_conv:10584,4,hopeful,I felt very greata few weeks ago when i helped an elderly women with bagging her groceries ,507,same with me if i ever see someone that could use my help_comma_ i will most likely bring it to you,5|5|5_5|5|4,





"""

In [74]:
valid_issues ="""
hit:4299_conv:8599,1,devastated,I'm devastated by my mother's death,509,"I'm devastated by my mother's death,5|5|5_5|5|5,
hit:4299_conv:8599,2,devastated,I'm devastated by my mother's death,294,I am so sorry to hear that she has passed.  How long has she been gone?,5|5|5_5|5|5,
hit:4299_conv:8599,3,devastated,I'm devastated by my mother's death,509,21 years_comma_ but I miss her every single day,5|5|5_5|5|5,
hit:4299_conv:8599,4,devastated,I'm devastated by my mother's death,294,I know how you feel. ,5|5|5_5|5|5,
hit:4302_conv:8605,1,jealous,A friend of mine bought a lottery ticket right after I bought one. Mine was a loser_comma_ his was a $15_comma_000 winning scratch off.,340,I have no luck when it comes to lottery tickets_comma_ I constantly buy losers.,5|5|5_5|5|5,
hit:4302_conv:8605,2,jealous,A friend of mine bought a lottery ticket right after I bought one. Mine was a loser_comma_ his was a $15_comma_000 winning scratch off.,296,I have that same problem!,5|5|5_5|5|5,
hit:4302_conv:8605,3,jealous,A friend of mine bought a lottery ticket right after I bought one. Mine was a loser_comma_ his was a $15_comma_000 winning scratch off.,340,A friend of mine bought one right after I did one day_comma_ mine was a loser_comma_ his was a $15_comma_000 winner.,5|5|5_5|5|5,
hit:4302_conv:8605,4,jealous,A friend of mine bought a lottery ticket right after I bought one. Mine was a loser_comma_ his was a $15_comma_000 winning scratch off.,296,Holy goodness!!! I would be a bit jealous :(,5|5|5_5|5|5,
hit:4327_conv:8654,1,sad,A friend of mine was in a car accident.  Unfortunately_comma_ he did not make it,111,A friend of mine was in a car accident,4|4|4_5|5|5,
hit:4327_conv:8654,2,sad,A friend of mine was in a car accident.  Unfortunately_comma_ he did not make it,4,Is your friend ok? What kind of car was it?,4|4|4_5|5|5,
hit:4327_conv:8654,3,sad,A friend of mine was in a car accident.  Unfortunately_comma_ he did not make it,111,He was driving a compact and a truck hit him.  He didn't make it unfortunately,4|4|4_5|5|5,
hit:4327_conv:8654,4,sad,A friend of mine was in a car accident.  Unfortunately_comma_ he did not make it,4,That is horrible news_comma_ I am so sorry for your loss.,4|4|4_5|5|5,
hit:4331_conv:8663,1,apprehensive,A co-worker mentioned that we were going to have a lot of work coming up.  I hope it isn't too bad,111,A co-worker told me we were going to have a lot of work coming up,4|4|4_5|5|5,
hit:4331_conv:8663,2,apprehensive,A co-worker mentioned that we were going to have a lot of work coming up.  I hope it isn't too bad,294,Is that good or bad?  Sometimes extra work can be a blessing_comma_ unless you are salaried_comma_ then it's just more work.,4|4|4_5|5|5,
hit:4331_conv:8663,3,apprehensive,A co-worker mentioned that we were going to have a lot of work coming up.  I hope it isn't too bad,111,Bad if anything_comma_ I just hope it isn't overwhelming ,4|4|4_5|5|5,
hit:4331_conv:8663,4,apprehensive,A co-worker mentioned that we were going to have a lot of work coming up.  I hope it isn't too bad,294,I hope so too.  What kind of work is it?,4|4|4_5|5|5,
hit:4339_conv:8679,1,sad,my father recently passed suddenly_comma_ i was devastated.,327,my father recently passed suddenly_comma_ i was devastated,5|5|5_5|5|5,
hit:4339_conv:8679,2,sad,my father recently passed suddenly_comma_ i was devastated.,261,Oh_comma_ I am sorry for your loss,5|5|5_5|5|5,
hit:4339_conv:8679,3,sad,my father recently passed suddenly_comma_ i was devastated.,327,thank you. i am having trouble getting over it.,5|5|5_5|5|5,
hit:4339_conv:8679,4,sad,my father recently passed suddenly_comma_ i was devastated.,261,Don't feel bad now. What's done is done,5|5|5_5|5|5,
hit:4339_conv:8679,5,sad,my father recently passed suddenly_comma_ i was devastated.,327,Yes_comma_ but I really miss him.,5|5|5_5|5|5,
hit:4339_conv:8679,6,sad,my father recently passed suddenly_comma_ i was devastated.,261,I can understand. I am really sorry,5|5|5_5|5|5,
hit:4345_conv:8690,1,annoyed,There was a bad crash this morning and I got so irritated it had me late for work.,284,There was a 2 car accident this morning a driver going the wrong way it had traffic backed up.,5|5|5_5|5|5,
hit:4345_conv:8690,2,annoyed,There was a bad crash this morning and I got so irritated it had me late for work.,243,Oh no! Was anyone hurt?,5|5|5_5|5|5,
hit:4345_conv:8690,3,annoyed,There was a bad crash this morning and I got so irritated it had me late for work.,284,Yes one driver was in critical condition and car ran into the side of the express way it was unfortunate but I was over a hour late for work. ,5|5|5_5|5|5,
hit:4345_conv:8690,4,annoyed,There was a bad crash this morning and I got so irritated it had me late for work.,243,That is so sad. I hope they recover okay. Hopefully you weren't in trouble for being late either.,5|5|5_5|5|5,
hit:4348_conv:8697,1,anticipating,i am excited to see my son and his fiance!,327,i am excited to see my son and his fiance!,5|5|5_5|5|5,
hit:4348_conv:8697,2,anticipating,i am excited to see my son and his fiance!,448,That is awesome_comma_ are they coming to visit you?,5|5|5_5|5|5,
hit:4348_conv:8697,3,anticipating,i am excited to see my son and his fiance!,327,Yes! In October. I haven't seen him in a few months.,5|5|5_5|5|5,
hit:4348_conv:8697,4,anticipating,i am excited to see my son and his fiance!,448,Awesome! I'm not a father yet_comma_ but I can only imagine your kids growing up and leaving to live their own lives.,5|5|5_5|5|5,
hit:4348_conv:8697,5,anticipating,i am excited to see my son and his fiance!,327,Yes_comma_ I do miss them.  But i am excited to see them get out there and leave their mark on the world.,5|5|5_5|5|5,
hit:4349_conv:8699,1,ashamed,i didn't ask my husband if i could buy something_comma_ now he is mad and i feel bad.,327,i didn't ask my husband if i could buy something_comma_ now he is mad and i feel bad.,5|5|5_5|5|5,
hit:4349_conv:8699,2,ashamed,i didn't ask my husband if i could buy something_comma_ now he is mad and i feel bad.,448,Aw_comma_ I'm sorry_comma_ did you just forget or you were thinking about other things?,5|5|5_5|5|5,
hit:4349_conv:8699,3,ashamed,i didn't ask my husband if i could buy something_comma_ now he is mad and i feel bad.,327,I didn't think he would mind.  But it was expensive_comma_ so maybe i should have asked.,5|5|5_5|5|5,
hit:4349_conv:8699,4,ashamed,i didn't ask my husband if i could buy something_comma_ now he is mad and i feel bad.,448,Yeah_comma_ had a similar scuffle with my wife_comma_ but being married sometimes you just have to forgive and forget.,5|5|5_5|5|5,
hit:4353_conv:8706,1,terrified,Over the weekend there was a large spider crawling in my car I was so petrify.,284,The bugs that creep me out the most is those eight legged spiders I seen a huge one in my car this weekend.,5|5|5_5|5|5,
hit:4353_conv:8706,2,terrified,Over the weekend there was a large spider crawling in my car I was so petrify.,4,I have never liked spiders.,5|5|5_5|5|5,
hit:4353_conv:8706,3,terrified,Over the weekend there was a large spider crawling in my car I was so petrify.,284,Me either I screamed so loud lol I be afraid thy may crawl on me I would freak out. ,5|5|5_5|5|5,
hit:4353_conv:8706,4,terrified,Over the weekend there was a large spider crawling in my car I was so petrify.,4,You should always have bug spray on you at all times.,5|5|5_5|5|5,
hit:4353_conv:8706,5,terrified,Over the weekend there was a large spider crawling in my car I was so petrify.,284,For now on I will.,5|5|5_5|5|5,
hit:4357_conv:8714,1,terrified,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,327,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,5|5|5_5|5|5,
hit:4357_conv:8714,2,terrified,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,228,That is really scary. Are you home alone?,5|5|5_5|5|5,
hit:4357_conv:8714,3,terrified,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,327,Yes! I don't have a gun or anything!,5|5|5_5|5|5,
hit:4357_conv:8714,4,terrified,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,228,I'm sure you are fine. I bet it is just a tree branch or something like that. Maybe a neighbor could go look for you.,5|5|5_5|5|5,
hit:4357_conv:8714,5,terrified,I hear something tapping outside my bathroom door. I don't know what it is and I am really scared.,327,I don't have any neighbors_comma_ and there are no trees there. ,5|5|5_5|5|5,
hit:4361_conv:8723,1,grateful,I received a promotion at my job I am so thankful for this opportunity. ,284,I got a new position at my job more money and also happy to be recognized for my hard work. ,5|5|5_5|5|5,
hit:4361_conv:8723,2,grateful,I received a promotion at my job I am so thankful for this opportunity. ,445,YAY_comma_ a promotion is always amazing.  when do you start?,5|5|5_5|5|5,
hit:4361_conv:8723,3,grateful,I received a promotion at my job I am so thankful for this opportunity. ,284,I actually started today it was a lot of information to take in but it was such a joy. ,5|5|5_5|5|5,
hit:4361_conv:8723,4,grateful,I received a promotion at my job I am so thankful for this opportunity. ,445,Thats amazing_comma_ financial stability can be a life changer.,5|5|5_5|5|5,
hit:4364_conv:8729,1,annoyed,I hate going to walmart.  People just walk right out in front of you and don't even say excuse me!,327,I hate going to walmart. People just walk right out in front of you and don't even say excuse me!,5|5|5_5|5|5,
hit:4364_conv:8729,2,annoyed,I hate going to walmart.  People just walk right out in front of you and don't even say excuse me!,511,Maybe they're very eager to go a certain item.,5|5|5_5|5|5,
hit:4364_conv:8729,3,annoyed,I hate going to walmart.  People just walk right out in front of you and don't even say excuse me!,327,Maybe. It's that hunter gatherer mentality.  Get it before everyone else.,5|5|5_5|5|5,
hit:4364_conv:8729,4,annoyed,I hate going to walmart.  People just walk right out in front of you and don't even say excuse me!,511,True_comma_ because regretfully I act that way sometimes too.,5|5|5_5|5|5,
hit:4364_conv:8729,5,annoyed,I hate going to walmart.  People just walk right out in front of you and don't even say excuse me!,327,I guess I might too.,5|5|5_5|5|5,
hit:4370_conv:8741,1,content,I like to relax on the weekends. It's nice not having anything to worry about.,513,I love being lazy on weekends. It's nice to just relax and have nothing to worry about.,3|5|5_5|5|5,
hit:4370_conv:8741,2,content,I like to relax on the weekends. It's nice not having anything to worry about.,469,Yeah I hear you! Do you like to read or watch TV?,3|5|5_5|5|5,
hit:4370_conv:8741,3,content,I like to relax on the weekends. It's nice not having anything to worry about.,513,Both! I spend more time watching TV though. How about you?,3|5|5_5|5|5,
hit:4370_conv:8741,4,content,I like to relax on the weekends. It's nice not having anything to worry about.,469,I like to do both as well. TV if I'm feeling lazy_comma_ reading if I feel more ambitious. ,3|5|5_5|5|5,
hit:4370_conv:8741,5,content,I like to relax on the weekends. It's nice not having anything to worry about.,513,That's kind of how I am. What kind of TV shows do you like?,3|5|5_5|5|5,
hit:4372_conv:8744,1,embarrassed,My car got soaked from the rain I sat in my seats and got my pants wet and had to walk in the gas station I was ashamed it looked like I wet myself lol.,284,I can be so forgetful I left my windows down in my car and my seats was full of water.,5|5|5_5|5|5,
hit:4372_conv:8744,2,embarrassed,My car got soaked from the rain I sat in my seats and got my pants wet and had to walk in the gas station I was ashamed it looked like I wet myself lol.,327,Oh no!! How did you dry them?,5|5|5_5|5|5,
hit:4372_conv:8744,3,embarrassed,My car got soaked from the rain I sat in my seats and got my pants wet and had to walk in the gas station I was ashamed it looked like I wet myself lol.,284,They are still air drying what makes it worse I sat in the sets and it looked like I wet myself lol a little embarrassing. ,5|5|5_5|5|5,
hit:4372_conv:8744,4,embarrassed,My car got soaked from the rain I sat in my seats and got my pants wet and had to walk in the gas station I was ashamed it looked like I wet myself lol.,327,Oh that sounds awful! But at least no one stole anything from your car.,5|5|5_5|5|5,
hit:4372_conv:8745,1,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,327,I found a hair in my pizza_comma_ and now I can't eat,5|5|5_5|5|5,
hit:4372_conv:8745,2,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,284,Did you take it back that is so disgusting. ,5|5|5_5|5|5,
hit:4372_conv:8745,3,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,327,I called them and told them about it. They said the next pizza would be free. I said no thanks.,5|5|5_5|5|5,
hit:4372_conv:8745,4,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,284,I wouldn't want another either it takes one time for me to find hair_comma_ bug etc and I am done with the restaurant.  ,5|5|5_5|5|5,
hit:4372_conv:8745,5,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,327,Yes indeed.  I think i am done with eating out for a while,5|5|5_5|5|5,
hit:4372_conv:8745,6,disgusted,I found a hair in my pizza_comma_ and now I can't eat.,284,Best way to ensure your food is clean and free of hair cook it yourself.,5|5|5_5|5|5,
hit:4375_conv:8751,1,prepared,I packed for a long road trip_comma_ and had everything I would need to hopefully avoid excessive stops. ,513,I recently went on a long trip by car. For the first time_comma_ it wasn't stressful because I packed everything I needed for it to run smoothly.,3|3|3_5|5|5,
hit:4375_conv:8751,2,prepared,I packed for a long road trip_comma_ and had everything I would need to hopefully avoid excessive stops. ,469,That sounds great! I really pays off to be prepared. ,3|3|3_5|5|5,
hit:4375_conv:8751,3,prepared,I packed for a long road trip_comma_ and had everything I would need to hopefully avoid excessive stops. ,513,Yes it does. I was able to eliminate several meal breaks which cut over three hours off my trip time!,3|3|3_5|5|5,
hit:4375_conv:8751,4,prepared,I packed for a long road trip_comma_ and had everything I would need to hopefully avoid excessive stops. ,469,That sounds like some efficient planning! I hope you enjoy your trip!,3|3|3_5|5|5,
hit:4375_conv:8751,5,prepared,I packed for a long road trip_comma_ and had everything I would need to hopefully avoid excessive stops. ,513,Thank you!,3|3|3_5|5|5,
hit:4380_conv:8760,1,lonely,Besides my cats being with me I am always alone at home all the kids are grown and moved out.,284,Ever since my kids left and went to college I am such a loner.,5|5|5_5|5|5,
hit:4380_conv:8760,2,lonely,Besides my cats being with me I am always alone at home all the kids are grown and moved out.,155,I can understand you. But this is the cycle of life_comma_ right? Kids grow and they have to find their own way of life in this world,5|5|5_5|5|5,
hit:4380_conv:8760,3,lonely,Besides my cats being with me I am always alone at home all the kids are grown and moved out.,284,Yes you are right at least I have my cats to keep me company lol so I'm not to much alone.,5|5|5_5|5|5,
hit:4380_conv:8760,4,lonely,Besides my cats being with me I am always alone at home all the kids are grown and moved out.,155,Good for you. You can also call your friends to hang out if you have close ones,5|5|5_5|5|5,
hit:4381_conv:8763,1,disappointed,i didn't get to see my favorite character when i went to disney world.,327,i didn't get to see my favorite character when i went to disney world.,4|5|5_5|5|5,
hit:4381_conv:8763,2,disappointed,i didn't get to see my favorite character when i went to disney world.,1,Oh no_comma_ I'm sorry. Did you stay long?,4|5|5_5|5|5,
hit:4381_conv:8763,3,disappointed,i didn't get to see my favorite character when i went to disney world.,327,I was there for 4 days.  I wanted to see Cinderella but I kept missing her.,4|5|5_5|5|5,
hit:4381_conv:8763,4,disappointed,i didn't get to see my favorite character when i went to disney world.,1,Maybe she was around_comma_ but it wasn't time for the ball yet.,4|5|5_5|5|5,
hit:4381_conv:8763,5,disappointed,i didn't get to see my favorite character when i went to disney world.,327,Haha_comma_ maybe I didn't think of that!,4|5|5_5|5|5,
hit:4383_conv:8767,1,prepared,I arranged a dinner date for me and my husband for his birthday I can't wait for that night.,284,I have a special dinner arranged for me and my husband this week.,5|5|5_5|5|5,
hit:4383_conv:8767,2,prepared,I arranged a dinner date for me and my husband for his birthday I can't wait for that night.,1,That's wonderful. He'll be so excited when he finds out!,5|5|5_5|5|5,
hit:4383_conv:8767,3,prepared,I arranged a dinner date for me and my husband for his birthday I can't wait for that night.,284,Yes he will it's his birthday and we can have a night alone no kids just a nice dinner.,5|5|5_5|5|5,
hit:4383_conv:8767,4,prepared,I arranged a dinner date for me and my husband for his birthday I can't wait for that night.,1,I hope you guys have a great time. Just the two of you.,5|5|5_5|5|5,
hit:4383_conv:8767,5,prepared,I arranged a dinner date for me and my husband for his birthday I can't wait for that night.,284,We will thank you.,5|5|5_5|5|5,
hit:4386_conv:8772,1,surprised, I went to my first concert last month I seen beyonce I was so amazed such a good show.,284,I went to my first official concert last month I was so amazed it was a surprise to even go there. ,5|5|5_5|5|5,
hit:4386_conv:8772,2,surprised, I went to my first concert last month I seen beyonce I was so amazed such a good show.,1,Concerts can be a lot of fun! Will you be going to another one?,5|5|5_5|5|5,
hit:4386_conv:8772,3,surprised, I went to my first concert last month I seen beyonce I was so amazed such a good show.,284,Yes if they can outdo Queen Bey lol we saw Beyonce and she put on a awesome show a night to remember. ,5|5|5_5|5|5,
hit:4386_conv:8772,4,surprised, I went to my first concert last month I seen beyonce I was so amazed such a good show.,1,I've never been to a Beyonce concert. Always sold out! I'm glad you got to see her.,5|5|5_5|5|5,
hit:4392_conv:8784,1,content,I am really pleased with my job performance lately.  I will be getting that promotion for sure.,327,I am really pleased with my job performance lately. I will be getting that promotion for sure.,5|5|5_5|5|5,
hit:4392_conv:8784,2,content,I am really pleased with my job performance lately.  I will be getting that promotion for sure.,1,Congratulations! Good to hear your hard work is paying off.,5|5|5_5|5|5,
hit:4392_conv:8784,3,content,I am really pleased with my job performance lately.  I will be getting that promotion for sure.,327,Yes i am delighted and kinda proud of myself.  I have worked very hard.,5|5|5_5|5|5,
hit:4392_conv:8784,4,content,I am really pleased with my job performance lately.  I will be getting that promotion for sure.,1,You should definitely be proud of yourself. Pat yourself on the back.,5|5|5_5|5|5,
hit:4392_conv:8784,5,content,I am really pleased with my job performance lately.  I will be getting that promotion for sure.,327,Thanks! It feels good to be productive.,5|5|5_5|5|5,
hit:4395_conv:8791,1,hopeful,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works!,327,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works,5|5|5_5|5|5,
hit:4395_conv:8791,2,hopeful,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works!,469,Yeah I hope you get it! I'm sure they will recognize the extra time you've been doing.,5|5|5_5|5|5,
hit:4395_conv:8791,3,hopeful,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works!,327,i really hope so. I have worked 60+ hours a week for the last 6 months.,5|5|5_5|5|5,
hit:4395_conv:8791,4,hopeful,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works!,469,Wow_comma_ I hope you're getting paid for all that extra time!,5|5|5_5|5|5,
hit:4395_conv:8791,5,hopeful,I have put in so many hours at work hoping to get a promotion. Fingers crossed it works!,327,I am salary unfortunately. But_comma_ if i get this new promotion_comma_ it will be worth it.  Lots more money.,5|5|5_5|5|5,
hit:4402_conv:8804,1,disappointed,I felt let down by my girlfriend when she decided to move some money out of our joint account. She never told me about that.,391,My girlfriend moved some of our mutual money without letting me know,5|5|5_5|5|5,
hit:4402_conv:8804,2,disappointed,I felt let down by my girlfriend when she decided to move some money out of our joint account. She never told me about that.,416,Sorry to hear that my friend.. Was she facing an emergency?,5|5|5_5|5|5,
hit:4402_conv:8804,3,disappointed,I felt let down by my girlfriend when she decided to move some money out of our joint account. She never told me about that.,391,Not really. I dont know what she used it for_comma_ she has not told anything to me so far,5|5|5_5|5|5,
hit:4402_conv:8804,4,disappointed,I felt let down by my girlfriend when she decided to move some money out of our joint account. She never told me about that.,416,I know it sucks_comma_ but i think you should have an open conversation to your girl about this,5|5|5_5|5|5,
hit:4404_conv:8809,1,angry,im so mad! I can't find my hair brush!,327,im so mad! I can't find my hair brush!,5|5|5_5|5|5,
hit:4404_conv:8809,2,angry,im so mad! I can't find my hair brush!,209,That is a tragedy! Where do you think you left it? Did you check your pockets?,5|5|5_5|5|5,
hit:4404_conv:8809,3,angry,im so mad! I can't find my hair brush!,327,i have checked everywhere.  i think my dog may have snatched it.,5|5|5_5|5|5,
hit:4404_conv:8809,4,angry,im so mad! I can't find my hair brush!,209,Interesting. I hope you do find it. I don't know how you brush your hair.,5|5|5_5|5|5,
hit:4404_conv:8809,5,angry,im so mad! I can't find my hair brush!,327,i am not brushing it_comma_ and it looks a mess. ,5|5|5_5|5|5,
hit:4412_conv:8825,1,jealous,my husband has been texting a woman he works with.,327,my husband has been texting a woman he works with,5|5|5_5|5|5,
hit:4412_conv:8825,2,jealous,my husband has been texting a woman he works with.,1,How does that make you feel?,5|5|5_5|5|5,
hit:4412_conv:8825,3,jealous,my husband has been texting a woman he works with.,327,very very bad_comma_ and suspicious.,5|5|5_5|5|5,
hit:4412_conv:8825,4,jealous,my husband has been texting a woman he works with.,1,I am so sorry to hear you're going through this. Hopefully_comma_ it's nothing.,5|5|5_5|5|5,
hit:4412_conv:8825,5,jealous,my husband has been texting a woman he works with.,327,hopefully.  I have not read the texts_comma_ but seen the name pop up.,5|5|5_5|5|5,
hit:4415_conv:8831,1,prepared,I always get all the ingredients on the table before i start to cook. i want to know that i am all set for it,391,I want to see all the ingredients on the table before i start to cook,5|5|5_5|5|5,
hit:4415_conv:8831,2,prepared,I always get all the ingredients on the table before i start to cook. i want to know that i am all set for it,416,Oh_comma_ so you always prepare well huh?,5|5|5_5|5|5,
hit:4415_conv:8831,3,prepared,I always get all the ingredients on the table before i start to cook. i want to know that i am all set for it,391,Yeah_comma_ i am one of those people who want to know everything is set before starting something,5|5|5_5|5|5,
hit:4415_conv:8831,4,prepared,I always get all the ingredients on the table before i start to cook. i want to know that i am all set for it,416,I always wanted that level of discipline. good for you :),5|5|5_5|5|5,
hit:4420_conv:8841,1,joyful,i am so happy that i am going to be a grandma,327,i am so happy that i am going to be a grandma,5|5|5_5|5|5,
hit:4420_conv:8841,2,joyful,i am so happy that i am going to be a grandma,27,Congratulations! boy or girl,5|5|5_5|5|5,
hit:4420_conv:8841,3,joyful,i am so happy that i am going to be a grandma,327,we don't know yet_comma_ she is only a few months along,5|5|5_5|5|5,
hit:4420_conv:8841,4,joyful,i am so happy that i am going to be a grandma,27,well_comma_ as long as its a happy healthy baby you guys will be fine,5|5|5_5|5|5,
hit:4420_conv:8841,5,joyful,i am so happy that i am going to be a grandma,327,yes that is all i want really,5|5|5_5|5|5,
hit:4420_conv:8841,6,joyful,i am so happy that i am going to be a grandma,27,Yep. Happy and healthy is a blessing,5|5|5_5|5|5,
hit:4425_conv:8850,1,sad,i am not going to be able to see my favorite band when they come to town.,327,i am not going to be able to see my favorite band when they come to town,5|5|5_5|5|5,
hit:4425_conv:8850,2,sad,i am not going to be able to see my favorite band when they come to town.,27,What's the band?,5|5|5_5|5|5,
hit:4425_conv:8850,3,sad,i am not going to be able to see my favorite band when they come to town.,327,def leppard.  they are going to be with reo speedwagon and tesla too.,5|5|5_5|5|5,
hit:4425_conv:8850,4,sad,i am not going to be able to see my favorite band when they come to town.,27,Wow. That must suck. Do you like the band incubus? I missed them a couple of times but I saw them this year,5|5|5_5|5|5,
hit:4425_conv:8850,5,sad,i am not going to be able to see my favorite band when they come to town.,327,i do_comma_ they did a song called wish you were here that i loved.,5|5|5_5|5|5,
hit:4425_conv:8850,6,sad,i am not going to be able to see my favorite band when they come to town.,27,yes. I wish I could send you a clip of it from the concert. It was awesome. Maybe you could catch def leppard next time. ,5|5|5_5|5|5,
hit:4428_conv:8856,1,nostalgic,I went to a comic book convention last month. Standing among all those super heroes and fantasy world_comma_ i felt like i am a kid again,391,I went to a comic book convention last month,5|5|5_5|5|5,
hit:4428_conv:8856,2,nostalgic,I went to a comic book convention last month. Standing among all those super heroes and fantasy world_comma_ i felt like i am a kid again,322,That sounds really cool.,5|5|5_5|5|5,
hit:4428_conv:8856,3,nostalgic,I went to a comic book convention last month. Standing among all those super heroes and fantasy world_comma_ i felt like i am a kid again,391,Yeah. i felt like i was a child again_comma_ standing amidst all those superheroes,5|5|5_5|5|5,
hit:4428_conv:8856,4,nostalgic,I went to a comic book convention last month. Standing among all those super heroes and fantasy world_comma_ i felt like i am a kid again,322,Oh_comma_ such great nostalgia. ,5|5|5_5|5|5,
hit:4429_conv:8858,1,jealous,i caught my husband looking at another woman,327,i caught my husband looking at another woman,3|2|3_5|5|5,
hit:4429_conv:8858,2,jealous,i caught my husband looking at another woman,63,oof that sucks.,3|2|3_5|5|5,
hit:4429_conv:8858,3,jealous,i caught my husband looking at another woman,327,yes.  i wasn't too happy about it.,3|2|3_5|5|5,
hit:4429_conv:8858,4,jealous,i caught my husband looking at another woman,63,yea you shouldnt be,3|2|3_5|5|5,
hit:4429_conv:8858,5,jealous,i caught my husband looking at another woman,327,i questioned him about it and he denied it.  but i caught him red handed.,3|2|3_5|5|5,
hit:4429_conv:8858,6,jealous,i caught my husband looking at another woman,63,well hopefully you find a better path in life,3|2|3_5|5|5,
hit:4430_conv:8860,1,anticipating,I am looking forward to this season of the Lakers. With Lebron coming in_comma_ i really think this will turn out to be a fantastic season for them,391,I am really looking forward to this season of the Lakers_comma_ with Lebron coming in,4|5|5_5|5|5,
hit:4430_conv:8860,2,anticipating,I am looking forward to this season of the Lakers. With Lebron coming in_comma_ i really think this will turn out to be a fantastic season for them,416,Oh wow_comma_ i did not even know about the transfer. I bet Laker fans are delighted,4|5|5_5|5|5,
hit:4430_conv:8860,3,anticipating,I am looking forward to this season of the Lakers. With Lebron coming in_comma_ i really think this will turn out to be a fantastic season for them,391,They are on cloud 9! With such a legend on their team_comma_ who wouldn't be_comma_ right?,4|5|5_5|5|5,
hit:4430_conv:8860,4,anticipating,I am looking forward to this season of the Lakers. With Lebron coming in_comma_ i really think this will turn out to be a fantastic season for them,416,So true. I might become a lakers fan myself at this point,4|5|5_5|5|5,
hit:4433_conv:8867,1,sad,im feeling blue because my days and nights are mixed up.,327,im feeling blue because my days and nights are mixed up,5|5|5_5|5|5,
hit:4433_conv:8867,2,sad,im feeling blue because my days and nights are mixed up.,209,That really sucks. Have you tried taking some melatonin?,5|5|5_5|5|5,
hit:4433_conv:8867,3,sad,im feeling blue because my days and nights are mixed up.,327,i have tried taking benadryl and some other sleep aid.  i should try melatonin.,5|5|5_5|5|5,
hit:4433_conv:8867,4,sad,im feeling blue because my days and nights are mixed up.,209,Yeah_comma_ it works wonders for me when I can't sleep.,5|5|5_5|5|5,
hit:4434_conv:8868,1,lonely,I felt really alone when my dog passed away last month. He was always by my side all the time.,391,I felt really lonely when my dog tom passed away last month,5|5|5_5|5|5,
hit:4434_conv:8868,2,lonely,I felt really alone when my dog passed away last month. He was always by my side all the time.,416,Oh god.. so sorry to hear that.. May i ask how did Tom pass?,5|5|5_5|5|5,
hit:4434_conv:8868,3,lonely,I felt really alone when my dog passed away last month. He was always by my side all the time.,391,He was old. He was my childhood dog and he was always with me all the time,5|5|5_5|5|5,
hit:4434_conv:8868,4,lonely,I felt really alone when my dog passed away last month. He was always by my side all the time.,416,I can feel your emotions. Find peace in knowing that Tom is at a better place now,5|5|5_5|5|5,
hit:4440_conv:8881,1,grateful,i am so thankful for my family.  they are the best!,327,i am so thankful for my family. they are the best!,1|3|3_5|5|5,
hit:4440_conv:8881,2,grateful,i am so thankful for my family.  they are the best!,209,Why are you so thankful? Is it Thanksgiving?,1|3|3_5|5|5,
hit:4440_conv:8881,3,grateful,i am so thankful for my family.  they are the best!,327,they are very good to me. they are always there for me.,1|3|3_5|5|5,
hit:4440_conv:8881,4,grateful,i am so thankful for my family.  they are the best!,209,That's great to hear. It's always good to have a great family support structure when times are hard.,1|3|3_5|5|5,
hit:4442_conv:8884,1,furious,I was so angry when my brother tied a firecracker to our dog's tail. I went straight ahead and gave him a punch.,391,I was so angry when my brother played a prank by tying a firecracker on our dog's tail,4|4|5_5|5|5,
hit:4442_conv:8884,2,furious,I was so angry when my brother tied a firecracker to our dog's tail. I went straight ahead and gave him a punch.,322,Oh no_comma_ that is a terrible thing to do.,4|4|5_5|5|5,
hit:4442_conv:8884,3,furious,I was so angry when my brother tied a firecracker to our dog's tail. I went straight ahead and gave him a punch.,391,Yeah.. He always plays pranks_comma_ but this one made me furious. I went to him and gave him a punch so that he learns to respect animals,4|4|5_5|5|5,
hit:4442_conv:8884,4,furious,I was so angry when my brother tied a firecracker to our dog's tail. I went straight ahead and gave him a punch.,322,I applaud your response.,4|4|5_5|5|5,
hit:4444_conv:8888,1,annoyed,my cat vomited in the floor and i stepped in it!,327,my cat vomited in the floor and i stepped in it!,5|5|5_5|5|5,
hit:4444_conv:8888,2,annoyed,my cat vomited in the floor and i stepped in it!,389,Oh no!!!  Did it smell bad?,5|5|5_5|5|5,
hit:4444_conv:8888,3,annoyed,my cat vomited in the floor and i stepped in it!,327,no just really gross_comma_ all warm and mushy between my toes. ,5|5|5_5|5|5,
hit:4444_conv:8888,4,annoyed,my cat vomited in the floor and i stepped in it!,389,Ewwwww!  Hope the cat is okay!,5|5|5_5|5|5,
hit:4447_conv:8895,1,prepared,I am ready for school to start! I can't wait to teach my first class.,327,I am ready for school to start! I can't wait to teach my first class.,5|5|5_5|5|5,
hit:4447_conv:8895,2,prepared,I am ready for school to start! I can't wait to teach my first class.,322,Sounds exciting.  I wish you the best of luck!,5|5|5_5|5|5,
hit:4447_conv:8895,3,prepared,I am ready for school to start! I can't wait to teach my first class.,327,Thank you! I am a little nervous_comma_ but I am sure everything will be great.,5|5|5_5|5|5,
hit:4447_conv:8895,4,prepared,I am ready for school to start! I can't wait to teach my first class.,322,Don't worry everyone is nervous on the first day.,5|5|5_5|5|5,
hit:10142_conv:20284,1,ashamed,I haven't been following my healthy eating regiment as strictly as I should. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,580, "I haven't been following my healthy eating regiment as strictly as I should.,5|5|5_5|5|5,
hit:10142_conv:20284,2,ashamed,I haven't been following my healthy eating regiment as strictly as I should. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,203,Why not_comma_ have you have many temptations?  I know how difficult it can be at times to eat healthy_comma_ I struggle to do it too.,5|5|5_5|5|5,
hit:10142_conv:20284,3,ashamed,I haven't been following my healthy eating regiment as strictly as I should. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,580,Yeah. Carbs like pasta and bread get me every time. ,5|5|5_5|5|5,
hit:10142_conv:20284,4,ashamed,I haven't been following my healthy eating regiment as strictly as I should. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,203,I know_comma_ fresh bread is a big temptation to me_comma_ and what about sweets and desserts_comma_ do you have a weakness for those too?,5|5|5_5|5|5,
hit:10142_conv:20284,5,ashamed,I haven't been following my healthy eating regiment as strictly as I should. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,580,Yup. Even though I don't need to lose any weight_comma_ I want to be healthy and cheating on this diet makes me feel bad.,5|5|5_5|5|5,
hit:10143_conv:20287,1,trusting,I told my best friends about my suicidal thoughts. I knew I could have faith in them_comma_ and they were super supportive.,580,I told my best friends about my suicidal thoughts.,5|5|5_5|5|5,
hit:10143_conv:20287,2,trusting,I told my best friends about my suicidal thoughts. I knew I could have faith in them_comma_ and they were super supportive.,659,It's really a good start to recognized that you should talk about it. We all need someone to just listen once in awhile.,5|5|5_5|5|5,
hit:10143_conv:20287,3,trusting,I told my best friends about my suicidal thoughts. I knew I could have faith in them_comma_ and they were super supportive.,580,Yeah you're right. I had faith in them and they ended up being really supportive.,5|5|5_5|5|5,
hit:10143_conv:20287,4,trusting,I told my best friends about my suicidal thoughts. I knew I could have faith in them_comma_ and they were super supportive.,659,Oh that's a good friend. I hope your feeling better now.,5|5|5_5|5|5,
hit:10147_conv:20294,1,faithful,One time I thought about getting Dominoes Pizza_comma_ but then I remembered I was loyal to Pizza Hut,547,One time I thought about getting Dominoes Pizza_comma_ but then I remembered I was loyal to Pizza Hut,5|5|5_4|5|3,
hit:10147_conv:20294,2,faithful,One time I thought about getting Dominoes Pizza_comma_ but then I remembered I was loyal to Pizza Hut,241,I like dominos better,5|5|5_4|5|3,
hit:10147_conv:20294,3,faithful,One time I thought about getting Dominoes Pizza_comma_ but then I remembered I was loyal to Pizza Hut,547,It tastes pretty good now_comma_ but I still like the oily goodness of Pizza Hut!,5|5|5_4|5|3,
hit:10147_conv:20294,4,faithful,One time I thought about getting Dominoes Pizza_comma_ but then I remembered I was loyal to Pizza Hut,241,Dominoes_comma_ IMO_comma_ has better prices and better pizza. I dont like all that oil,5|5|5_4|5|3,
hit:10172_conv:20344,1,caring,One time my mom brought me lunch all the way from her work!,547,One time my mom brought me lunch all the way from the restaurant where she works!,5|5|5_5|5|5,
hit:10172_conv:20344,2,caring,One time my mom brought me lunch all the way from her work!,561,That's thoughtful of her. Nice to know that your mom was thinking of you in the middle of the day.,5|5|5_5|5|5,
hit:10172_conv:20344,3,caring,One time my mom brought me lunch all the way from her work!,547,Yup_comma_ she is the best! I shall make it up to her one day!,5|5|5_5|5|5,
hit:10172_conv:20344,4,caring,One time my mom brought me lunch all the way from her work!,561,Take her out to dinner! That would be a treat.,5|5|5_5|5|5,
hit:10182_conv:20365,1,embarrassed,One time I was at the grocery store and I accidentally walked out with a item without paying for it,547,One time I was at the grocery store and I accidentally walked out with a item without paying for it,5|5|5_5|4|5,
hit:10182_conv:20365,2,embarrassed,One time I was at the grocery store and I accidentally walked out with a item without paying for it,557,Oh no! You must've felt ashamed. did you return it?,5|5|5_5|4|5,
hit:10182_conv:20365,3,embarrassed,One time I was at the grocery store and I accidentally walked out with a item without paying for it,547,Yes_comma_ I got caught and had to walk back in shame!,5|5|5_5|4|5,
hit:10182_conv:20365,4,embarrassed,One time I was at the grocery store and I accidentally walked out with a item without paying for it,557,Hopefully they were understanding that it was accidental?,5|5|5_5|4|5,
hit:10186_conv:20372,1,ashamed,I walked into work very late and the clinic was very busy. I was so embarrassed. ,567,I was very late to work today and they were so busy with patients.,43|4|4_3|4|5,
hit:10186_conv:20372,2,ashamed,I walked into work very late and the clinic was very busy. I was so embarrassed. ,650,Did you get wrote up or a lecture when you got their.,43|4|4_3|4|5,
hit:10186_conv:20372,3,ashamed,I walked into work very late and the clinic was very busy. I was so embarrassed. ,567,Well I will find out tomorrow. I know better than that is what she told me_comma_ especially on Fridays_comma_ when we are the busiest. ,43|4|4_3|4|5,
hit:10186_conv:20372,4,ashamed,I walked into work very late and the clinic was very busy. I was so embarrassed. ,650,Its always best to be on time _comma_ I hope they understand no one is perfect.,43|4|4_3|4|5,
hit:10186_conv:20372,5,ashamed,I walked into work very late and the clinic was very busy. I was so embarrassed. ,567,I agree. I feel so bad and embarrassed about it. Even the doctor scolded me. ,43|4|4_3|4|5,
hit:10188_conv:20377,1,apprehensive,My boss tells me I need to go to school to become a doctor instead of nurse_comma_ but I feel its alot of time and money to do so.,567,My boss told me to go to school to become a doctor instead of a nurse. I felt like it was a compliment. Im worried about how much and how long medical school will take.,5|5|5_5|5|5,
hit:10188_conv:20377,2,apprehensive,My boss tells me I need to go to school to become a doctor instead of nurse_comma_ but I feel its alot of time and money to do so.,679,The journey there is always the toughest_comma_ its years of practice and studying but in the end it will be totally worth it,5|5|5_5|5|5,
hit:10188_conv:20377,3,apprehensive,My boss tells me I need to go to school to become a doctor instead of nurse_comma_ but I feel its alot of time and money to do so.,567,I agree. I guess I will weigh my options_comma_ its just im getting old and watn to start a family too_comma_ so I dont know.,5|5|5_5|5|5,
hit:10188_conv:20377,4,apprehensive,My boss tells me I need to go to school to become a doctor instead of nurse_comma_ but I feel its alot of time and money to do so.,679,With whatever you decide in life just know that you are a very skilled and intelligent person! Im sure you will find your path in life eventually with time,5|5|5_5|5|5,
hit:10188_conv:20377,5,apprehensive,My boss tells me I need to go to school to become a doctor instead of nurse_comma_ but I feel its alot of time and money to do so.,567,Thank you for that! You just made my day!,5|5|5_5|5|5,
hit:10190_conv:20380,1,faithful,I got in a heated argument with my friend about religion and it made me question my beliefs.,567,My friend and I got in an argument about religion today. It made me question my faith. ,3|4|2_4|4|4,
hit:10190_conv:20380,2,faithful,I got in a heated argument with my friend about religion and it made me question my beliefs.,686,Wow_comma_  that was serious then,3|4|2_4|4|4,
hit:10190_conv:20380,3,faithful,I got in a heated argument with my friend about religion and it made me question my beliefs.,567,Yeah. I want to stay true to my beliefs_comma_ but she made some very valid points. ,3|4|2_4|4|4,
hit:10190_conv:20380,4,faithful,I got in a heated argument with my friend about religion and it made me question my beliefs.,686,OK_comma_  so what is your plan now? ,3|4|2_4|4|4,
hit:10190_conv:20380,5,faithful,I got in a heated argument with my friend about religion and it made me question my beliefs.,567,I will do research about her theories and then make really think about sticking to my beliefs. ,3|4|2_4|4|4,
hit:10196_conv:20393,1,apprehensive,When I started college I moved to a new state where I didn't know anyone.,573,I remember back when I started college. I moved to a new state where I didn't know anyone and I had no idea what to expect.,5|5|5_5|5|5,
hit:10196_conv:20393,2,apprehensive,When I started college I moved to a new state where I didn't know anyone.,687,Wow that sounds scary. Did you make good friends?,5|5|5_5|5|5,
hit:10196_conv:20393,3,apprehensive,When I started college I moved to a new state where I didn't know anyone.,573,Yes_comma_ I did eventually. There's something to be said for living in a dorm at a time when you want to meet new people and fit in.,5|5|5_5|5|5,
hit:10196_conv:20393,4,apprehensive,When I started college I moved to a new state where I didn't know anyone.,687,Yes it can be really helpful! I went to school 3 hours from my home and it was still hard to meet new people,5|5|5_5|5|5,
hit:10197_conv:20395,1,devastated,Today I had a patient who found out she could not have babies. She has been trying for 6 months now.,567,Today I had to tell a patient about her lab results. Come to find out she could not have babies. She was very upset.,5|5|5_5|5|5,
hit:10197_conv:20395,2,devastated,Today I had a patient who found out she could not have babies. She has been trying for 6 months now.,684,Oh_comma_ that must have been a hard thing to do. I'd hate to be the one to deliver such a bad news...,5|5|5_5|5|5,
hit:10197_conv:20395,3,devastated,Today I had a patient who found out she could not have babies. She has been trying for 6 months now.,567,Yeah I felt bad. She started crying so much. Its like her world ended there.,5|5|5_5|5|5,
hit:10197_conv:20395,4,devastated,Today I had a patient who found out she could not have babies. She has been trying for 6 months now.,684,So sad...that makes me want to cry_comma_ too_comma_ just hearing about her reaction.,5|5|5_5|5|5,
hit:10199_conv:20398,1,sentimental,I love going back to visit the house I grew up in.,573,Whenever I go back to my hometown I love driving by the house I grew up in. I have such great memories.,5|5|5_3|5|5,
hit:10199_conv:20398,2,sentimental,I love going back to visit the house I grew up in.,688,I bet it does! Do you go by there often?,5|5|5_3|5|5,
hit:10199_conv:20398,3,sentimental,I love going back to visit the house I grew up in.,573,I get to visit about once a year. I always wish that I knew who lived there now so I could see the inside of the house.,5|5|5_3|5|5,
hit:10199_conv:20398,4,sentimental,I love going back to visit the house I grew up in.,688,I bet it's much different than it was back when you lived there.,5|5|5_3|5|5,
hit:10199_conv:20398,5,sentimental,I love going back to visit the house I grew up in.,573,A little_comma_ but mostly I love seeing all the things that are the same.,5|5|5_3|5|5,
hit:10211_conv:20423,1,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,599,I found out some bad news the other day.,5|5|5_5|5|5,
hit:10211_conv:20423,2,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,687,Oh no :( Can you tell me? It's okay if it's too private.,5|5|5_5|5|5,
hit:10211_conv:20423,3,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,599,I just got back my grade for my final for my class and I failed it. I studied real hard_comma_ but unfortunately it didn't seem to matter.,5|5|5_5|5|5,
hit:10211_conv:20423,4,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,687,Oh my gosh I'm so sorry that really sucks. :( Can you talk to the professor and try to make it up or find out what you did wrong?,5|5|5_5|5|5,
hit:10211_conv:20423,5,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,599,I'm going to have to at least try. Maybe they'll be understanding and let me attempt again.,5|5|5_5|5|5,
hit:10211_conv:20423,6,ashamed,I failed one of my classes' finals. I definitely did not feel good about it when I found out I failed.,687,Good luck to you! I wish you all the best <3,5|5|5_5|5|5,
hit:10214_conv:20428,1,ashamed,Someone twisted what I said earlier today to make it seem like I was saying something else. I felt really bad_comma_ even though they were lying. ,549,Someone twisted what I Said earlier today to make it seem like i was saying something else. I felt reall ybad.,5|5|5_5|5|5,
hit:10214_conv:20428,2,ashamed,Someone twisted what I said earlier today to make it seem like I was saying something else. I felt really bad_comma_ even though they were lying. ,599,Oh no why would they do such a thing,5|5|5_5|5|5,
hit:10214_conv:20428,3,ashamed,Someone twisted what I said earlier today to make it seem like I was saying something else. I felt really bad_comma_ even though they were lying. ,549,They were angry and wanted to be aggressive. ,5|5|5_5|5|5,
hit:10214_conv:20428,4,ashamed,Someone twisted what I said earlier today to make it seem like I was saying something else. I felt really bad_comma_ even though they were lying. ,599,Definitely doesn't excuse what they did.,5|5|5_5|5|5,
hit:10214_conv:20429,1,faithful,Some lady I knew kept trying to hit on me knowing that I had a girlfriend. I knew I would never do anything like that though.,599,Some lady I know tried to hit on me even though she very well knew I am in a relationship!,5|5|5_5|5|5,
hit:10214_conv:20429,2,faithful,Some lady I knew kept trying to hit on me knowing that I had a girlfriend. I knew I would never do anything like that though.,549,That's not cool. I bet it felt awkward.,5|5|5_5|5|5,
hit:10214_conv:20429,3,faithful,Some lady I knew kept trying to hit on me knowing that I had a girlfriend. I knew I would never do anything like that though.,599,It really did but I would never do something to jeopardize my relationship.,5|5|5_5|5|5,
hit:10214_conv:20429,4,faithful,Some lady I knew kept trying to hit on me knowing that I had a girlfriend. I knew I would never do anything like that though.,549,You are a stand up person!,5|5|5_5|5|5,
hit:10219_conv:20439,1,content,I spent this evening with my family.  This makes me realize how blessed I am.,305,I had a nice quiet evening at home tonight.,4|5|5_5|5|5,
hit:10219_conv:20439,2,content,I spent this evening with my family.  This makes me realize how blessed I am.,437,Nice_comma_ silence is golden,4|5|5_5|5|5,
hit:10219_conv:20439,3,content,I spent this evening with my family.  This makes me realize how blessed I am.,305,Well_comma_ my family was here_comma_ too.  Toddlers don't make life quiet_comma_ but they make me feel fulfilled.,4|5|5_5|5|5,
hit:10219_conv:20439,4,content,I spent this evening with my family.  This makes me realize how blessed I am.,437,Full of what?,4|5|5_5|5|5,
hit:10219_conv:20439,5,content,I spent this evening with my family.  This makes me realize how blessed I am.,305,Full of meaning.,4|5|5_5|5|5,
hit:10220_conv:20440,1,ashamed,I was supposed to go to a meeting today.  However_comma_ I skipped it.,305,There was a meeting scheduled today that I thought was utterly pointless.,5|5|5_5|5|5,
hit:10220_conv:20440,2,ashamed,I was supposed to go to a meeting today.  However_comma_ I skipped it.,138,Oh no_comma_ was it one of those meetings that should have been an email?,5|5|5_5|5|5,
hit:10220_conv:20440,3,ashamed,I was supposed to go to a meeting today.  However_comma_ I skipped it.,305,It definitely was_comma_ so I ended up skipping it.,5|5|5_5|5|5,
hit:10220_conv:20440,4,ashamed,I was supposed to go to a meeting today.  However_comma_ I skipped it.,138,Hopefully you don't get in trouble but I definitely know the feeling. ,5|5|5_5|5|5,
hit:10220_conv:20440,5,ashamed,I was supposed to go to a meeting today.  However_comma_ I skipped it.,305,I regretted skipping the meeting a little bit_comma_ but probably not as much as if I'd gone to it.,5|5|5_5|5|5,
hit:10223_conv:20447,1,trusting,I told a close friend of mine a very big secret. I don't think that they will tell anyone.  I'm hoping.,599,I told my close friend a very big secret yesterday.,5|5|5_5|5|5,
hit:10223_conv:20447,2,trusting,I told a close friend of mine a very big secret. I don't think that they will tell anyone.  I'm hoping.,628,Oh really_comma_ were you supposed to tell them?,5|5|5_5|5|5,
hit:10223_conv:20447,3,trusting,I told a close friend of mine a very big secret. I don't think that they will tell anyone.  I'm hoping.,599,It was my choice. I really don't think they'll tell anyone! They're good people.,5|5|5_5|5|5,
hit:10223_conv:20447,4,trusting,I told a close friend of mine a very big secret. I don't think that they will tell anyone.  I'm hoping.,628,You can never be to sure now a days_comma_ they might post your secret to social media.,5|5|5_5|5|5,
hit:10224_conv:20448,1,confident,One time I was really sure that I was going to ace a test because I studied all night,547,One time I was really sure that I was going to ace a test because I studied all night,5|5|5_5|5|5,
hit:10224_conv:20448,2,confident,One time I was really sure that I was going to ace a test because I studied all night,683,Did you ace your test?,5|5|5_5|5|5,
hit:10224_conv:20448,3,confident,One time I was really sure that I was going to ace a test because I studied all night,547,Yup_comma_ I was worried during the test too_comma_ but I am glad everything turned out okay,5|5|5_5|5|5,
hit:10224_conv:20448,4,confident,One time I was really sure that I was going to ace a test because I studied all night,683,I always worry when I have to take tests. Unfortunately_comma_ I am horrible with tests. I am glad you aced it!,5|5|5_5|5|5,
hit:10225_conv:20450,1,jealous,A friend of mine recently got a decent pay raise_comma_ after he got one last year.  I wish I knew what to do to get a raise.,305,A friend at work recently got a raise.  That's the 2nd one in the last year for him!,3|5|5_5|5|5,
hit:10225_conv:20450,2,jealous,A friend of mine recently got a decent pay raise_comma_ after he got one last year.  I wish I knew what to do to get a raise.,437,Nice_comma_ what a legend and a gentleman,3|5|5_5|5|5,
hit:10225_conv:20450,3,jealous,A friend of mine recently got a decent pay raise_comma_ after he got one last year.  I wish I knew what to do to get a raise.,305,Yeah_comma_ but I haven't had one in 3 years.,3|5|5_5|5|5,
hit:10225_conv:20450,4,jealous,A friend of mine recently got a decent pay raise_comma_ after he got one last year.  I wish I knew what to do to get a raise.,437,Word_comma_ you should ask him what the secret is,3|5|5_5|5|5,
hit:10234_conv:20469,1,faithful,My ex boyfriend and I were together for 4 years.  I was faithful to him the whole four years,587,I was totally committed to him in every way I could be.   HOwever he ended up cheating on me,5|5|5_4|4|5,
hit:10234_conv:20469,2,faithful,My ex boyfriend and I were together for 4 years.  I was faithful to him the whole four years,691,That kind of cold disregard for someone you're supposed to love is totally alien to me_comma_ the guilt would destroy me.,5|5|5_4|4|5,
hit:10234_conv:20469,3,faithful,My ex boyfriend and I were together for 4 years.  I was faithful to him the whole four years,587,It would me too_comma_ not only that but he just up and got with her like I never existed to him.   Its funny because he always accused me of cheating on him and I had no clue why.  It was him the whole time!,5|5|5_4|4|5,
hit:10234_conv:20469,4,faithful,My ex boyfriend and I were together for 4 years.  I was faithful to him the whole four years,691,Yeah Freud called it Projection_comma_ it's too bad people tend to hide their true natures so well.,5|5|5_4|4|5,
hit:10236_conv:20472,1,trusting,I had to kick my 18 year old son out of house.    He has been gone almost a month now.,587,He kept doing drugs_comma_ and taking things that didnt belong to him.  It broke my heart but I didnt know what else to do.,5|5|5_5|5|5,
hit:10236_conv:20472,2,trusting,I had to kick my 18 year old son out of house.    He has been gone almost a month now.,684,It's awful! I know so many families are dealing with similar situation. Sometimes we don't know how to help someone close to us. Heart breaking...,5|5|5_5|5|5,
hit:10236_conv:20472,3,trusting,I had to kick my 18 year old son out of house.    He has been gone almost a month now.,587,Yes you are right about that.  I dealt with it for a long time and then I just couldn't anymore.  Over and over things would disappear and he would of course always deny it.  He became violent at time to his sister and me and I couldn't put them at risk.,5|5|5_5|5|5,
hit:10236_conv:20472,4,trusting,I had to kick my 18 year old son out of house.    He has been gone almost a month now.,684,You can't help him until he wants to really change. Sometimes all you can do is pray.,5|5|5_5|5|5,
hit:10239_conv:20479,1,faithful,my wife,568,today after coming back from work i saw my wife and she was asking me why are you late with questioning eyes_comma_ i told her that i wen't to go and get her favourite meal_comma_ i felt so faithful ,5|5|5_3|3|4,
hit:10239_conv:20479,2,faithful,my wife,548,Did she feel bad and apologize after she realized what you had done?,5|5|5_3|3|4,
hit:10239_conv:20479,3,faithful,my wife,568,she was joking about it_comma_ but she was so happy afterwards and we ate together_comma_ it was such a nice day,5|5|5_3|3|4,
hit:10239_conv:20479,4,faithful,my wife,548,It's good to get to spend time with those you love. I am happy for you.,5|5|5_3|3|4,
hit:10243_conv:20487,1,trusting,i trust myself,568,after a long day from work_comma_ i was stuck at this math problem_comma_ but then i decided to go for a walk instead of working on the problem_comma_ i trust my guts_comma_ after i had come back from the walk_comma_ the solution kicked in and i was able to do the problem,5|5|5_5|5|5,
hit:10243_conv:20487,2,trusting,i trust myself,437,That's inspiring_comma_ has that worked for you in the past in the same way?,5|5|5_5|5|5,
hit:10243_conv:20487,3,trusting,i trust myself,568,yes always_comma_ it is something that works for me,5|5|5_5|5|5,
hit:10243_conv:20487,4,trusting,i trust myself,437,That's so cool_comma_ you seem very intuitive,5|5|5_5|5|5,
hit:10244_conv:20489,1,apprehensive,walking at night,568,it was 3am_comma_ i was walking at the utter night... i was scared out of my mind... anything could happen_comma_ my walking speed increased... and everything i was seeing suddenly became more and more scary,5|5|5_2|5|5,
hit:10244_conv:20489,2,apprehensive,walking at night,630,That sounds frightening. Did you get home safely?,5|5|5_2|5|5,
hit:10244_conv:20489,3,apprehensive,walking at night,568,in the end yes.. but the journey was sure hell,5|5|5_2|5|5,
hit:10244_conv:20489,4,apprehensive,walking at night,630,Why were you walking so late at night?,5|5|5_2|5|5,
hit:10245_conv:20490,1,content,i am more content in my life,568,today i realised the small details of my life and i feel more satisfied,5|5|5_4|4|5,
hit:10245_conv:20490,2,content,i am more content in my life,547,Nice_comma_ what are some things you realized?,5|5|5_4|4|5,
hit:10245_conv:20490,3,content,i am more content in my life,568,that i have everything i wished for.,5|5|5_4|4|5,
hit:10245_conv:20490,4,content,i am more content in my life,547,You don't have me yet <3,5|5|5_4|4|5,
hit:10245_conv:20491,1,prepared,One time I bought water and snacks before a hurricane came to wreck my city,547,One time I bought water and snacks before a hurricane came to wreck my city,4|4|5_5|5|5,
hit:10245_conv:20491,2,prepared,One time I bought water and snacks before a hurricane came to wreck my city,568,oh god_comma_ how did you . have the courage to even go outside?,4|4|5_5|5|5,
hit:10245_conv:20491,3,prepared,One time I bought water and snacks before a hurricane came to wreck my city,547,My house flooded so I had to get on my roof !,4|4|5_5|5|5,
hit:10245_conv:20491,4,prepared,One time I bought water and snacks before a hurricane came to wreck my city,568,that sounds like an adventurous day ,4|4|5_5|5|5,
hit:10247_conv:20494,1,furious,One time they put cheese in my hamburger when I told them not to!,547,One time they put cheese in my hamburger when I told them not to!,5|5|5_5|5|5,
hit:10247_conv:20494,2,furious,One time they put cheese in my hamburger when I told them not to!,437,Wow_comma_ how could they do that?,5|5|5_5|5|5,
hit:10247_conv:20494,3,furious,One time they put cheese in my hamburger when I told them not to!,547,They are complete monsters I tell ya!,5|5|5_5|5|5,
hit:10247_conv:20494,4,furious,One time they put cheese in my hamburger when I told them not to!,437,Someone's gotta put an end to that poor service!,5|5|5_5|5|5,
hit:10248_conv:20496,1,embarrassed,i lied to my mother today ,568,i lied to my mother and i feel very uneasy,5|5|5_3|3|4,
hit:10248_conv:20496,2,embarrassed,i lied to my mother today ,548,You should. No matter what you can't lie to your Mom. It will always come back on you.,5|5|5_3|3|4,
hit:10248_conv:20496,3,embarrassed,i lied to my mother today ,568,i just realised this now,5|5|5_3|3|4,
hit:10248_conv:20496,4,embarrassed,i lied to my mother today ,548,Come clean and fix it. It will make you feel better.,5|5|5_3|3|4,
hit:10249_conv:20498,1,confident,confident with my abilities,568,i am beginning to realise in my abilities and i feel so confident in them_comma_ i am able to anything,5|5|5_4|4|5,
hit:10249_conv:20498,2,confident,confident with my abilities,547,Nice! Do you have any superpowers?,5|5|5_4|4|5,
hit:10249_conv:20498,3,confident,confident with my abilities,568,In a way yes_comma_yes i do the power of my will and mind,5|5|5_4|4|5,
hit:10249_conv:20498,4,confident,confident with my abilities,547,I like your confidence! Fighting!,5|5|5_4|4|5,
hit:10249_conv:20499,1,nostalgic,When I drink coffee_comma_ memories from my past keep playing in my head,547,When I drink coffee_comma_ memories from my past keep playing in my head,4|4|5_5|5|5,
hit:10249_conv:20499,2,nostalgic,When I drink coffee_comma_ memories from my past keep playing in my head,568,oh nice_comma_ what kind of memories?,4|4|5_5|5|5,
hit:10249_conv:20499,3,nostalgic,When I drink coffee_comma_ memories from my past keep playing in my head,547,Memories of talking to strangers in chatroom. Lots of things ,4|4|5_5|5|5,
hit:10249_conv:20499,4,nostalgic,When I drink coffee_comma_ memories from my past keep playing in my head,568,how does this make you feel like?,4|4|5_5|5|5,
hit:10250_conv:20501,1,hopeful,I have been working on my programming skills and I feel like I can get a new job!,547,I have been working on my programming skills and I feel like I can get a new job now!,5|5|5_5|5|5,
hit:10250_conv:20501,2,hopeful,I have been working on my programming skills and I feel like I can get a new job!,630,That's awesome! You're going to be rolling in the dough with those skills,5|5|5_5|5|5,
hit:10250_conv:20501,3,hopeful,I have been working on my programming skills and I feel like I can get a new job!,547,Hell yeah_comma_ no more living in mom's basement!,5|5|5_5|5|5,
hit:10250_conv:20501,4,hopeful,I have been working on my programming skills and I feel like I can get a new job!,630,That's exciting. How long have you been learning programming?,5|5|5_5|5|5,
hit:10251_conv:20502,1,caring,A couple of days ago there was a man at the gas pump at our local gas station.  He was  was trying to find a way to get himself some gas so he could get home.,587,From time to time I see an older man around our town driving his lawn mower from place to place..  SOmetimes I see him at  McDonalds_comma_or the Dollar General Store.  ,4|5|4_5|5|5,
hit:10251_conv:20502,2,caring,A couple of days ago there was a man at the gas pump at our local gas station.  He was  was trying to find a way to get himself some gas so he could get home.,568,an interesting one_comma_ what does this make you feel ?,4|5|4_5|5|5,
hit:10251_conv:20502,3,caring,A couple of days ago there was a man at the gas pump at our local gas station.  He was  was trying to find a way to get himself some gas so he could get home.,587,Well at first I thought it was cute until I saw him at the gas station the other day.  I overheard him asking someone that was getting gas if he happened to have a little change he could have.  He needed to get home in his lawnmower but didn't have the money to do so.   I ended up giving him 3 bucks.,4|5|4_5|5|5,
hit:10251_conv:20502,4,caring,A couple of days ago there was a man at the gas pump at our local gas station.  He was  was trying to find a way to get himself some gas so he could get home.,568,oh did he say anything afterwards? what happend next?,4|5|4_5|5|5,
hit:10251_conv:20503,1,caring,looking after my cats,568,i believe i am a really caring person_comma_ because i always take care of cats and feed them and lately they are more and more close to me,5|5|5_4|5|4,
hit:10251_conv:20503,2,caring,looking after my cats,587,That is so kind of you_comma_ I also a have a soft heart for animals.  I happen to be a dog person_comma_ but I do have a cat that was my ex boyfriends.  ,5|5|5_4|5|4,
hit:10251_conv:20503,3,caring,looking after my cats,568,i am a both person ahaha i also have a dog_comma_ but he is always so mean ,5|5|5_4|5|4,
hit:10251_conv:20503,4,caring,looking after my cats,587,I can't say I have had any mean dogs as of yet.  My favorite breed is  a pitbull.   THey are so loyal  and sweet although  lots of people claim they are mean and vicious .,5|5|5_4|5|4,
hit:10252_conv:20505,1,prepared,holidy,568,towels_comma_ clothes_comma_ underwear and phone all checked! and ready for my holiday #feelingsoprepeared,5|5|5_3|4|3,
hit:10252_conv:20505,2,prepared,holidy,693,where are you going!,5|5|5_3|4|3,
hit:10252_conv:20505,3,prepared,holidy,568,TO SEE MY BABY GIRL,5|5|5_3|4|3,
hit:10252_conv:20505,4,prepared,holidy,693,thats awesome i hope you have a good time with her,5|5|5_3|4|3,
hit:10253_conv:20506,1,disappointed,One time I bought a scratch lottery ticket and I didn't win anything!,547,One time I bought a scratch lottery ticket and I didn't win anything!,5|5|5_5|5|5,
hit:10253_conv:20506,2,disappointed,One time I bought a scratch lottery ticket and I didn't win anything!,77,Wow_comma_ that must've been disappointing.,5|5|5_5|5|5,
hit:10253_conv:20506,3,disappointed,One time I bought a scratch lottery ticket and I didn't win anything!,547,Yes_comma_ I was off by one number. I was going to be a millionaire!,5|5|5_5|5|5,
hit:10253_conv:20506,4,disappointed,One time I bought a scratch lottery ticket and I didn't win anything!,77,Oh! So close. Just to think about that missed opportunity.,5|5|5_5|5|5,
hit:10255_conv:20510,1,nostalgic,weather,568,every day the weather makes me feel so nostalgic_comma_ the colors_comma_ the wind all bring back a certain memory from my past,5|5|5_5|5|5,
hit:10255_conv:20510,2,nostalgic,weather,437,Yeah_comma_ it's like time passes but the smells and everything stays the same,5|5|5_5|5|5,
hit:10255_conv:20510,3,nostalgic,weather,568,yes! i am so glad you get it_comma_ it's beautiful but also sad at the same time,5|5|5_5|5|5,
hit:10255_conv:20510,4,nostalgic,weather,437,Yup_comma_ and I feel there's a part of us that is eternal as well,5|5|5_5|5|5,
hit:10256_conv:20513,1,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,547,One time I went into the public restroom and saw poop smeared everywhere! Ew!,5|4|5_4|5|5,
hit:10256_conv:20513,2,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,694,I think I would leave if I saw something like that.,5|4|5_4|5|5,
hit:10256_conv:20513,3,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,547,I left as fast as possible!,5|4|5_4|5|5,
hit:10256_conv:20513,4,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,694,Did you tell someone about it so they could take care of it?,5|4|5_4|5|5,
hit:10256_conv:20513,5,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,547,Nope_comma_ just you!,5|4|5_4|5|5,
hit:10256_conv:20513,6,disgusted,One time I went into the public restroom and saw poop smeared everywhere! Ew!,694,It seems most responsible to alert someone to clean it.,5|4|5_4|5|5
"""

In [4]:
valid_issues = """hit:10931_conv:21862,1,surprised,It was my birthday and my whole family threw a party for me after work. It was so unexpected!,725,"It was my birthday and my whole family threw a party for me after work. It was so unexpected!,5|5|5_5|5|5,
hit:10931_conv:21862,2,surprised,It was my birthday and my whole family threw a party for me after work. It was so unexpected!,231,That's so cool! Did they get you anything nice?,5|5|5_5|5|5,
hit:10931_conv:21862,3,surprised,It was my birthday and my whole family threw a party for me after work. It was so unexpected!,725,They got me a bunch of gift cards to my favorite stores. They are all so wonderful. I love them so much!,5|5|5_5|5|5,
hit:10931_conv:21862,4,surprised,It was my birthday and my whole family threw a party for me after work. It was so unexpected!,231,Time to go shoppin lol!,5|5|5_5|5|5,
hit:10933_conv:21866,1,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,318,At the beginning of the year I missed a lot of work.  I was sick but I didnt feel like a team player at the time.,5|5|5_5|5|5,
hit:10933_conv:21866,2,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,423,Aw Im sorry. Are you doing better now?,5|5|5_5|5|5,
hit:10933_conv:21866,3,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,318,yeah I'm doing much better thanks.  I'll for sure be getting a flu shot next year!,5|5|5_5|5|5,
hit:10933_conv:21866,4,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,423,That's a good idea. I haven't done that in years. I always feel like I barely dodge a bullet by not getting it.,5|5|5_5|5|5,
hit:10933_conv:21866,5,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,318,I have been lucky in the past but I dont remember the last time I have felt that sick. I definitely dont want to go through that again.,5|5|5_5|5|5,
hit:10933_conv:21866,6,ashamed,I missed a lot of work.  It was because I was sick but I dont feel like a team player when I miss work.,423,Glad you are doing better!,5|5|5_5|5|5,
hit:10934_conv:21869,1,afraid,i have to go out of town_comma_ i am not looking forward to flying,675,i have to go out of town_comma_ i am not looking forward to flying,4|5|5_5|5|5,
hit:10934_conv:21869,2,afraid,i have to go out of town_comma_ i am not looking forward to flying,128,I feel you_comma_ but at least it should be shorter than driving and the odds of a mishap are much lower than regular transportation.,4|5|5_5|5|5,
hit:10934_conv:21869,3,afraid,i have to go out of town_comma_ i am not looking forward to flying,675,i keep telling myself flying is safe but I am petrified of getting on that plane,4|5|5_5|5|5,
hit:10934_conv:21869,4,afraid,i have to go out of town_comma_ i am not looking forward to flying,128,I see_comma_ I think you may consider a tylenol or some type of sleep pill for the flight_comma_ to ensure that you don't feel uncomfortable or paranoid during travel.,4|5|5_5|5|5,
hit:10935_conv:21871,1,proud,My father has been such a great father. He is such a hard worker and loves his family so much. He is such a great man.,725,My father has been such a great father. He is such a hard worker and loves his family so much. He is such a great man.,5|5|5_5|5|5,
hit:10935_conv:21871,2,proud,My father has been such a great father. He is such a hard worker and loves his family so much. He is such a great man.,423,That's awesome. I was always a daddy's boy growing up.,5|5|5_5|5|5,
hit:10935_conv:21871,3,proud,My father has been such a great father. He is such a hard worker and loves his family so much. He is such a great man.,725,Awww_comma_ yeah Daddy's can be the best! I wish all children could have good daddy's.,5|5|5_5|5|5,
hit:10935_conv:21871,4,proud,My father has been such a great father. He is such a hard worker and loves his family so much. He is such a great man.,423,Amen. Mine was a good man. He taught me a lot of good life lessons.,5|5|5_5|5|5,
hit:10936_conv:21873,1,sentimental,I have a stuffed animal that my old dog used to lay by.  He's dead now but I still keep that stuffed animal in my room.,318,I have a stuffed animal that my old dog used to lay by/on as a pillow. My dog is dead but I still keep that stuffed animal to remind me of him,5|5|5_5|5|5,
hit:10936_conv:21873,2,sentimental,I have a stuffed animal that my old dog used to lay by.  He's dead now but I still keep that stuffed animal in my room.,35,I am sorry your dog passed away_comma_ I totally understand wanting to keep a momento to remind you of him. Was he old when he passed?,5|5|5_5|5|5,
hit:10936_conv:21873,3,sentimental,I have a stuffed animal that my old dog used to lay by.  He's dead now but I still keep that stuffed animal in my room.,318,He was about 12.  He was my first dog though! He was a sweetheart.  It is nice keeping something to remind me of him.,5|5|5_5|5|5,
hit:10936_conv:21873,4,sentimental,I have a stuffed animal that my old dog used to lay by.  He's dead now but I still keep that stuffed animal in my room.,35,I'm glad you have something that helps keep his memory alive. :),5|5|5_5|5|5,
hit:10936_conv:21873,5,sentimental,I have a stuffed animal that my old dog used to lay by.  He's dead now but I still keep that stuffed animal in my room.,318,I do too! ,5|5|5_5|5|5,
hit:10937_conv:21874,1,devastated,I was hoping to get an interview for a position and was told I would at least get that chance.  However_comma_ the recruiter didnt look at any internal candidates and by the time it was reported they had already hired someone from the outside.,318,I was hoping to get an interview for a higher position and was told I would get that at least.  However_comma_ the recruiter didnt look at internal candidates and by the time it was reported they had hired someone from the outside.,5|5|5_5|5|5,
hit:10937_conv:21874,2,devastated,I was hoping to get an interview for a position and was told I would at least get that chance.  However_comma_ the recruiter didnt look at any internal candidates and by the time it was reported they had already hired someone from the outside.,423,Aw what a sham! Did they give any reason why they hired from the outside?,5|5|5_5|5|5,
hit:10937_conv:21874,3,devastated,I was hoping to get an interview for a position and was told I would at least get that chance.  However_comma_ the recruiter didnt look at any internal candidates and by the time it was reported they had already hired someone from the outside.,318,My boss said the recruiter is an idiot and just never looked at internal candidates. It was a bummer but I figure I'll look for something else in the company soon,5|5|5_5|5|5,
hit:10937_conv:21874,4,devastated,I was hoping to get an interview for a position and was told I would at least get that chance.  However_comma_ the recruiter didnt look at any internal candidates and by the time it was reported they had already hired someone from the outside.,423,Well that's a good attitude to have. I don't blame you for being mad. I would be.,5|5|5_5|5|5,
hit:10937_conv:21874,5,devastated,I was hoping to get an interview for a position and was told I would at least get that chance.  However_comma_ the recruiter didnt look at any internal candidates and by the time it was reported they had already hired someone from the outside.,318,I was a little upset.  My boss and her boss were more angry than me though so I felt good they were at least on my side!,5|5|5_5|5|5,
hit:10938_conv:21877,1,excited,I am getting a new car this week! I can't wait to smell the new leather seats!,725,I am getting a new car this week! I can't wait to smell the new leather seats!,5|5|5_5|5|5,
hit:10938_conv:21877,2,excited,I am getting a new car this week! I can't wait to smell the new leather seats!,287,Oh man_comma_ that is exciting! Do you know what kind?,5|5|5_5|5|5,
hit:10938_conv:21877,3,excited,I am getting a new car this week! I can't wait to smell the new leather seats!,725,It's a beautiful 2018 yellow jeep! The car of my dreams! So stoked!,5|5|5_5|5|5,
hit:10938_conv:21877,4,excited,I am getting a new car this week! I can't wait to smell the new leather seats!,287,That's awesome!! I love jeeps!  congratulations_comma_ that's great news!,5|5|5_5|5|5,
hit:10940_conv:21881,1,faithful,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,725,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,5|5|5_5|5|5,
hit:10940_conv:21881,2,faithful,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,349,That's so sweet.  What kind of bird is it?,5|5|5_5|5|5,
hit:10940_conv:21881,3,faithful,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,725,I think it is a love bird. She thinks I am the mother and it is so cool when she walks all over me and comes when I call her. I have never had a pet bird before.,5|5|5_5|5|5,
hit:10940_conv:21881,4,faithful,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,349,Aww_comma_ I'd love to see a video of that.,5|5|5_5|5|5,
hit:10940_conv:21881,5,faithful,A bird fell out of a tree next to my house. I am now the mother and taking care of her. We are developing such a beautiful relationship.,725,Ok_comma_ I will make one for you. :),5|5|5_5|5|5,
hit:10942_conv:21885,1,trusting,I met a cow the other day and I went to go pet her. Something inside me told me to get away. Later that day the owner told me to stay away from her because she is known to attack people.,725,I met a cow the other day and I went to go pet her. Something inside me told me to get away. Later that day the owner told me to stay away from her because she is known to attack people.,5|5|5_5|5|5,
hit:10942_conv:21885,2,trusting,I met a cow the other day and I went to go pet her. Something inside me told me to get away. Later that day the owner told me to stay away from her because she is known to attack people.,349,You're lucky the cow didnt hurt you.,5|5|5_5|5|5,
hit:10942_conv:21885,3,trusting,I met a cow the other day and I went to go pet her. Something inside me told me to get away. Later that day the owner told me to stay away from her because she is known to attack people.,725,Yes I think so. That probably could have killed me. But I guess it was not my time. I angels saved me.,5|5|5_5|5|5,
hit:10942_conv:21885,4,trusting,I met a cow the other day and I went to go pet her. Something inside me told me to get away. Later that day the owner told me to stay away from her because she is known to attack people.,349,Good thing you listened to your gut feeling,5|5|5_5|5|5,
hit:10944_conv:21888,1,apprehensive,I need to go to the grocery store but it is storming outside. Not sure if I should go or not.,725,I need to go to the grocery store but it is storming outside. Not sure if I should go or not.,5|5|5_5|5|5,
hit:10944_conv:21888,2,apprehensive,I need to go to the grocery store but it is storming outside. Not sure if I should go or not.,423,Hopefully it will stop soon. That's the worst having to get creative with the little food you have on hand.,5|5|5_5|5|5,
hit:10944_conv:21888,3,apprehensive,I need to go to the grocery store but it is storming outside. Not sure if I should go or not.,725,Actually I have no food and I am starving. Maybe I should order a pizza. But then the delivery guy will have the same problem.,5|5|5_5|5|5,
hit:10944_conv:21888,4,apprehensive,I need to go to the grocery store but it is storming outside. Not sure if I should go or not.,423,Well that is what they get paid for. Just be sure to tip him well.,5|5|5_5|5|5,
hit:10946_conv:21893,1,furious,i came home yesterday_comma_ i found my sofa torn up_comma_ let's say this I am not happy with my pet dog,675,i came home yesterday_comma_ i found my sofa torn up_comma_ let's say this I am not very  happy with my pet dog,5|5|5_5|5|5,
hit:10946_conv:21893,2,furious,i came home yesterday_comma_ i found my sofa torn up_comma_ let's say this I am not happy with my pet dog,659,Uh oh. I bet he knew he was in trouble. Did you get him a crate?,5|5|5_5|5|5,
hit:10946_conv:21893,3,furious,i came home yesterday_comma_ i found my sofa torn up_comma_ let's say this I am not happy with my pet dog,675,i was on the verge of something more serious and probably would have regret it,5|5|5_5|5|5,
hit:10946_conv:21893,4,furious,i came home yesterday_comma_ i found my sofa torn up_comma_ let's say this I am not happy with my pet dog,659,Yeah_comma_ I'm glad you didn't do anything you would regret. I hope he is better for you in the future.,5|5|5_5|5|5,
hit:10947_conv:21895,1,devastated,I am so hurt that my baby goat died. My dogs were playing to rough and killer her. Man I should have stopped it sooner. It is all my fault. :(,725,I am so hurt that my baby goat died. My dogs were playing to rough and killer her. Man I should have stopped it sooner. It is all my fault. :(,5|5|5_5|5|5,
hit:10947_conv:21895,2,devastated,I am so hurt that my baby goat died. My dogs were playing to rough and killer her. Man I should have stopped it sooner. It is all my fault. :(,670,Oh no! I'm so sorry about that. Don't blame yourself_comma_ animals like to play with each other!,5|5|5_5|5|5,
hit:10947_conv:21895,3,devastated,I am so hurt that my baby goat died. My dogs were playing to rough and killer her. Man I should have stopped it sooner. It is all my fault. :(,725,Yes but I think I could have avoided it. I am training my dogs now to stay away from the goats. I hope it doesn't happen again.,5|5|5_5|5|5,
hit:10947_conv:21895,4,devastated,I am so hurt that my baby goat died. My dogs were playing to rough and killer her. Man I should have stopped it sooner. It is all my fault. :(,670,That sounds like a good plan. You definitely don't want that to happen again. Sometimes bad things happen in order for us to learn and avoid it again.,5|5|5_5|5|5,
hit:10948_conv:21897,1,joyful,I went fishing with my son a month ago_comma_ and didn't know how he'd do.  He caught 4 fish and was really excited!,305,I went fishing with my 4 year-old about a month ago.,5|5|5_5|5|5,
hit:10948_conv:21897,2,joyful,I went fishing with my son a month ago_comma_ and didn't know how he'd do.  He caught 4 fish and was really excited!,35,That sounds great_comma_ I can remeber going fishing with my dad at that age. Did your 4 year-old have a good time?,5|5|5_5|5|5,
hit:10948_conv:21897,3,joyful,I went fishing with my son a month ago_comma_ and didn't know how he'd do.  He caught 4 fish and was really excited!,305,He did!  He caught 4 fish all by himself.,5|5|5_5|5|5,
hit:10948_conv:21897,4,joyful,I went fishing with my son a month ago_comma_ and didn't know how he'd do.  He caught 4 fish and was really excited!,35,Wow_comma_ that's impressive! I'm glad he had a good time.,5|5|5_5|5|5,
hit:10950_conv:21900,1,caring,My dog was stung by a bug.  His lip got HUGE.  It looked pretty funny but I felt so bad for him!,318,My dog was stung by a bug.  His lip got huge and looked really funny but I felt terrible for him!,5|5|5_5|5|5,
hit:10950_conv:21900,2,caring,My dog was stung by a bug.  His lip got HUGE.  It looked pretty funny but I felt so bad for him!,35,Poor doggy! Did you have to take him to the vet?,5|5|5_5|5|5,
hit:10950_conv:21900,3,caring,My dog was stung by a bug.  His lip got HUGE.  It looked pretty funny but I felt so bad for him!,318,I did end up taking him in. Luckily it wasnt too serious and he was fine a couple days later.  He looked so sad with his big lip!,5|5|5_5|5|5,
hit:10950_conv:21900,4,caring,My dog was stung by a bug.  His lip got HUGE.  It looked pretty funny but I felt so bad for him!,35,I bet he did_comma_ I'm glad he's okay though.,5|5|5_5|5|5,
hit:10951_conv:21903,1,grateful,my best friend friend helped me through some rough spots with my marriage and job. I am gland he was there for me,675,my best friend friend helped me through some rough spots with my marriage and job. I am gland he was there for me,5|5|5_5|5|5,
hit:10951_conv:21903,2,grateful,my best friend friend helped me through some rough spots with my marriage and job. I am gland he was there for me,659,Thats really nice you have a friend like that.That's hard to come by. I hope everything is better for you now.,5|5|5_5|5|5,
hit:10951_conv:21903,3,grateful,my best friend friend helped me through some rough spots with my marriage and job. I am gland he was there for me,675,thank you_comma_ i do not know how i will repay him back_comma_ I will always be in his debt,5|5|5_5|5|5,
hit:10951_conv:21903,4,grateful,my best friend friend helped me through some rough spots with my marriage and job. I am gland he was there for me,659,I'm sure he doesn't expect it but thats a really nice thing he did.,5|5|5_5|5|5,
hit:10953_conv:21906,1,sentimental,I feel so bad to see so many homeless people on the streets these days. I hope I am never homeless.,725,I feel so bad to see so many homeless people on the streets these days. I hope I am never homeless.,5|5|5_5|5|5,
hit:10953_conv:21906,2,sentimental,I feel so bad to see so many homeless people on the streets these days. I hope I am never homeless.,318,I hope you arent and I'm not too! It would be rough living on the streets.,5|5|5_5|5|5,
hit:10953_conv:21906,3,sentimental,I feel so bad to see so many homeless people on the streets these days. I hope I am never homeless.,725,Yeah especially when it is snowing outside. Can you just imagine having to sleep outside like that . Oh! So sad!,5|5|5_5|5|5,
hit:10953_conv:21906,4,sentimental,I feel so bad to see so many homeless people on the streets these days. I hope I am never homeless.,318,It would be terrible in the winter.  I cant imagine sleeping outside when its freezing,5|5|5_5|5|5,
hit:10953_conv:21907,1,content,I went out to eat at a new place with my girlfriend last weekend. It was awesome and the food was great. ,318,I went out to eat with my girlfriend last week.  It was an awesome atmosphere and the food was great!,5|5|5_5|5|5,
hit:10953_conv:21907,2,content,I went out to eat at a new place with my girlfriend last weekend. It was awesome and the food was great. ,725,That sounds fun! Where did you go?,5|5|5_5|5|5,
hit:10953_conv:21907,3,content,I went out to eat at a new place with my girlfriend last weekend. It was awesome and the food was great. ,318,We went to a greek place.  There were belly dancers and a ton of good meats and appetizers. I think we'll check it out again this weekend since it was so fun!,5|5|5_5|5|5,
hit:10953_conv:21907,4,content,I went out to eat at a new place with my girlfriend last weekend. It was awesome and the food was great. ,725,Mmmmm_comma_ I love greek food! That sounds appetizing! I think I will take my girl there too this week!,5|5|5_5|5|5,
hit:10955_conv:21911,1,lonely,as I walk through my house_comma_ the rooms are empty_comma_ There is no one there for me,675,as I walk through my house_comma_ the rooms are empty_comma_ There is no one there for me,5|5|5_5|5|5,
hit:10955_conv:21911,2,lonely,as I walk through my house_comma_ the rooms are empty_comma_ There is no one there for me,536,I'm so sorry you feel that way. Maybe you should talk to someone you trust. A family member or a friend.,5|5|5_5|5|5,
hit:10955_conv:21911,3,lonely,as I walk through my house_comma_ the rooms are empty_comma_ There is no one there for me,675,that is the problem there is no one I can talk to or share with,5|5|5_5|5|5,
hit:10955_conv:21911,4,lonely,as I walk through my house_comma_ the rooms are empty_comma_ There is no one there for me,536,I'm sorry. You could seek out a professional to talk to. Or you could just try meeting some new people.,5|5|5_5|5|5,
hit:10956_conv:21913,1,caring,I have no children but I have a ton of animals. I love them all so much. They are my children.,725,I have no children but I have a ton of animals. I love them all so much. They are my children.,5|5|5_5|5|5,
hit:10956_conv:21913,2,caring,I have no children but I have a ton of animals. I love them all so much. They are my children.,659,What kinds of pets do you have?,5|5|5_5|5|5,
hit:10956_conv:21913,3,caring,I have no children but I have a ton of animals. I love them all so much. They are my children.,725,I have chickens_comma_ ducks_comma_ parrots_comma_ cockatiels_comma_ budgies_comma_ peacocks_comma_ guinea pigs_comma_ dogs_comma_ goats and cows. haha ,5|5|5_5|5|5,
hit:10956_conv:21913,4,caring,I have no children but I have a ton of animals. I love them all so much. They are my children.,659,oh wow! you did say a ton! I love animals too. wish I lived somewhere I could have more.,5|5|5_5|5|5,
hit:10956_conv:21913,5,caring,I have no children but I have a ton of animals. I love them all so much. They are my children.,725,Yeah_comma_ it is so much fun to live with animals. :),5|5|5_5|5|5,
hit:10957_conv:21914,1,afraid,When my son was born_comma_ I was pretty worried.  We knew that something wasn't quite right_comma_ so we weren't sure what to expect.,305,When my son was born_comma_ we knew that something was up before he came out.  It was pretty scary.,1|3|5_5|5|5,
hit:10957_conv:21914,2,afraid,When my son was born_comma_ I was pretty worried.  We knew that something wasn't quite right_comma_ so we weren't sure what to expect.,543,Did he have superpowers?,1|3|5_5|5|5,
hit:10957_conv:21914,3,afraid,When my son was born_comma_ I was pretty worried.  We knew that something wasn't quite right_comma_ so we weren't sure what to expect.,305,Hah - no.  But he did have the cord wrapped around his neck twice_comma_ and he'd tied an overhand knot in it_comma_ too.,1|3|5_5|5|5,
hit:10957_conv:21914,4,afraid,When my son was born_comma_ I was pretty worried.  We knew that something wasn't quite right_comma_ so we weren't sure what to expect.,543,That's pretty impressive.,1|3|5_5|5|5,
hit:10959_conv:21919,1,embarrassed,I was in a rush this morning and I dressed my self in stained clothes. I feel so dirty today. :(,725,I was in a rush this morning and I dressed my self in stained clothes. I feel so dirty today. :(,5|5|5_5|5|5,
hit:10959_conv:21919,2,embarrassed,I was in a rush this morning and I dressed my self in stained clothes. I feel so dirty today. :(,387,Oh wow_comma_ are you going to be in front of anyone important?,5|5|5_5|5|5,
hit:10959_conv:21919,3,embarrassed,I was in a rush this morning and I dressed my self in stained clothes. I feel so dirty today. :(,725,Not today. I just feel insecure about it_comma_ like everyone is looking at me. It is probably just all in my head.,5|5|5_5|5|5,
hit:10959_conv:21919,4,embarrassed,I was in a rush this morning and I dressed my self in stained clothes. I feel so dirty today. :(,387,Usually_comma_ you are the one who notices it the most. ,5|5|5_5|5|5,
hit:10963_conv:21927,1,anticipating,My cows are finally pregnant. I can't wait for my baby calves to be born!,725,My cows are finally pregnant. I can't wait for my baby calves to be born!,5|5|5_4|5|5,
hit:10963_conv:21927,2,anticipating,My cows are finally pregnant. I can't wait for my baby calves to be born!,290,wow_comma_ that must be an interesting thing to be involved with. lots of milk too_comma_ i presume.,5|5|5_4|5|5,
hit:10963_conv:21927,3,anticipating,My cows are finally pregnant. I can't wait for my baby calves to be born!,725,Oh_comma_ yes! Fresh raw milk is so wonderful and one of the best foods we can eat.,5|5|5_4|5|5,
hit:10963_conv:21927,4,anticipating,My cows are finally pregnant. I can't wait for my baby calves to be born!,290,very cool_comma_ wish i could get into something like that. sounds fun,5|5|5_4|5|5,
hit:10964_conv:21929,1,sad,i miss my old pet dog_comma_ i feel so empty without her around,675,i miss my old pet dog_comma_ i feel so empty without her around,5|5|5_5|5|5,
hit:10964_conv:21929,2,sad,i miss my old pet dog_comma_ i feel so empty without her around,470,Awww_comma_ I understand. Dogs are sooo great to be around and they are super friendly. ,5|5|5_5|5|5,
hit:10964_conv:21929,3,sad,i miss my old pet dog_comma_ i feel so empty without her around,675,i know that what makes it harder not having her around_comma_ no companionship,5|5|5_5|5|5,
hit:10964_conv:21929,4,sad,i miss my old pet dog_comma_ i feel so empty without her around,470,Maybe also looking into getting a new dog_comma_ I know it's hard_comma_ but you can get through it! Your friends and family are always there as well.,5|5|5_5|5|5,
hit:10966_conv:21933,1,jealous,I am so envious of those that can travel the world and get paid for it.,725,I am so envious of those that can travel the world and get paid for it.,5|5|5_5|5|5,
hit:10966_conv:21933,2,jealous,I am so envious of those that can travel the world and get paid for it.,387,That sounds like such an enjoyable life. If only it was so easy!,5|5|5_5|5|5,
hit:10966_conv:21933,3,jealous,I am so envious of those that can travel the world and get paid for it.,725,Yeah_comma_ maybe in my next life I guess_comma_ haha.,5|5|5_5|5|5,
hit:10966_conv:21933,4,jealous,I am so envious of those that can travel the world and get paid for it.,387,Haha_comma_ that is true. But we have to make the most of this one. ,5|5|5_5|5|5,
hit:10967_conv:21934,1,embarrassed,When I was in high school there was a senior I had a crush on...I was a sophomore. I ended up asking her out and got rejected but later in college we actually became pretty good friends.,318,When I was in high school there was a senior I had a crush on and I was a sophomore.  I ended up asking her out but got rejected. Later on in college we actually became pretty good friends.,5|5|5_5|5|5,
hit:10967_conv:21934,2,embarrassed,When I was in high school there was a senior I had a crush on...I was a sophomore. I ended up asking her out and got rejected but later in college we actually became pretty good friends.,554,Oh man heart breaker. Thats good yall are friends though. You never know maybe one day yall will be married,5|5|5_5|5|5,
hit:10967_conv:21934,3,embarrassed,When I was in high school there was a senior I had a crush on...I was a sophomore. I ended up asking her out and got rejected but later in college we actually became pretty good friends.,318,Haha_comma_ I hope not...my current girlfriend would probably be a little upset! She's a nice person (the one from high school) but I dont think I could ever marry her,5|5|5_5|5|5,
hit:10967_conv:21934,4,embarrassed,When I was in high school there was a senior I had a crush on...I was a sophomore. I ended up asking her out and got rejected but later in college we actually became pretty good friends.,554,you dont still have any feelings at all for her?,5|5|5_5|5|5,
hit:10969_conv:21939,1,confident,I am learning how sew my own clothes. I am studying hard and I think I will be able to make my first outfit!,725,I am learning how sew my own clothes. I am studying hard and I think I will be able to make my first outfit!,5|4|4_5|5|5,
hit:10969_conv:21939,2,confident,I am learning how sew my own clothes. I am studying hard and I think I will be able to make my first outfit!,659,Ooh_comma_ I've always wanted to learn how to do that. You must be so excited!,5|4|4_5|5|5,
hit:10969_conv:21939,3,confident,I am learning how sew my own clothes. I am studying hard and I think I will be able to make my first outfit!,725,Yes_comma_ I have so many ideas! How cool it would be to not have to buy any more expensive clothes and make my own! ,5|4|4_5|5|5,
hit:10969_conv:21939,4,confident,I am learning how sew my own clothes. I am studying hard and I think I will be able to make my first outfit!,659,Omg_comma_ that would be awesome. I might just have to learn how to do that myself.,5|4|4_5|5|5,
hit:10971_conv:21943,1,prepared,I just made a huge feast and now I am excited for all my guests to arrive to my baby shower!,725,I just made a huge feast and now I am excited for all my guests to arrive to my baby shower!,5|5|5_5|5|5,
hit:10971_conv:21943,2,prepared,I just made a huge feast and now I am excited for all my guests to arrive to my baby shower!,670,Ooo! That's exciting! Congrats on your baby! I hope your guests enjoy their time. ,5|5|5_5|5|5,
hit:10971_conv:21943,3,prepared,I just made a huge feast and now I am excited for all my guests to arrive to my baby shower!,725,Thank you! I have so many fun games planned and lots of great food.,5|5|5_5|5|5,
hit:10971_conv:21943,4,prepared,I just made a huge feast and now I am excited for all my guests to arrive to my baby shower!,670,Yay! That sounds like so much fun. Baby showers are a great way to bring everyone together.,5|5|5_5|5|5,
hit:10972_conv:21944,1,trusting,I trust my own intuitions when it comes to my health.,588,I trust the way my body feels to know if I'm generally healthy or not,4|4|5_2|1|23,
hit:10972_conv:21944,2,trusting,I trust my own intuitions when it comes to my health.,387,Usually it will let you know if something is wrong. Part of how we are designed! ,4|4|5_2|1|23,
hit:10972_conv:21944,3,trusting,I trust my own intuitions when it comes to my health.,588,Exactly! The human body is interesting!,4|4|5_2|1|23,
hit:10972_conv:21944,4,trusting,I trust my own intuitions when it comes to my health.,387,It is_comma_ everything has to be balanced just right_comma_ or else we feel it.,4|4|5_2|1|23,
hit:10974_conv:21948,1,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,725,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,5|5|5_5|5|5,
hit:10974_conv:21948,2,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,387,Oh no_comma_ how did you make things right with the manager. I am sure he is just wanting to get paid. ,5|5|5_5|5|5,
hit:10974_conv:21948,3,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,725,Yes_comma_ he did want to get paid but I needed to go home and get my wallet. He got angry with me but what could I do? I had no money with me.,5|5|5_5|5|5,
hit:10974_conv:21948,4,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,387,That is a tough spot to be in_comma_ maybe you could of washed dishes for him. Or give him your phone for collateral. ,5|5|5_5|5|5,
hit:10974_conv:21948,5,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,725,I just left but I came straight back and paid. All was fine in the end. But I will never go to that restaurant again.,5|5|5_5|5|5,
hit:10974_conv:21948,6,furious,I went to a restaurant and after I got the bill I realized that I did not have my wallet on me! The manager was yelling at me demanding me to pay before I left.,387,That is good_comma_ such an embarrassing situation to be in.,5|5|5_5|5|5,
hit:10975_conv:21951,1,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,318,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before it was in stock at the store,5|5|5_5|5|5,
hit:10975_conv:21951,2,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,554,Oh man I want that too! did you get it?,5|5|5_5|5|5,
hit:10975_conv:21951,3,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,318,I did!  It is by far the coolest thing I have ever bought!,5|5|5_5|5|5,
hit:10975_conv:21951,4,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,554,Do you use your computer?,5|5|5_5|5|5,
hit:10975_conv:21951,5,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,318,yeah_comma_ I have it plugged into my computer.  I cant tell you_comma_ or even begin to explain_comma_ how real everything feels when you use it.,5|5|5_5|5|5,
hit:10975_conv:21951,6,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,554,How much do they go for?,5|5|5_5|5|5,
hit:10975_conv:21951,7,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,318,It was $400_comma_ having a computer that can run it is the expensive piece,5|5|5_5|5|5,
hit:10975_conv:21951,8,anticipating,I really wanted an Oculus Rift.  I saved up enough and had to wait about a week before the store got them in stock.,554,thats not bad. I have a rx 280 ,5|5|5_5|5|5
"""

In [5]:
split_lines = valid_issues.split(",")
cleaned_split_lines = []

for line in split_lines:
    cleaned_split_lines.append(line.strip())

conversations = []
for i in range(0,len(cleaned_split_lines),7):
    conversations.append(cleaned_split_lines[i:i+7])
pd.DataFrame.from_records(conversations).to_csv("cleaned_up_conv_valid.csv",index=False)

In [70]:
test_issues = """
hit:2803_conv:5607,1,anticipating,I am waiting to receive my final grade for a summer college class that I just finished. I took the final exam yesterday_comma_ and I am not sure what score I will receive.,320,"I am waiting to receive my final grade for a summer college class that I just finished. I took the final exam yesterday_comma_ and I am not sure what score I will receive.,5|5|5_5|5|5,
hit:2803_conv:5607,2,anticipating,I am waiting to receive my final grade for a summer college class that I just finished. I took the final exam yesterday_comma_ and I am not sure what score I will receive.,123,I remember how that felt! I always hated having to wait for grades.,5|5|5_5|5|5,
hit:2803_conv:5607,3,anticipating,I am waiting to receive my final grade for a summer college class that I just finished. I took the final exam yesterday_comma_ and I am not sure what score I will receive.,320,I have_comma_ too. I actually took two classes this summer. I took an online final exam about an hour ago_comma_ and I scored the grade that I was needing. The grade I am concerned about is a Strategic Management course that I also was taking. The professor was a bit of a stickler when it came to grading.,5|5|5_5|5|5,
hit:2803_conv:5607,4,anticipating,I am waiting to receive my final grade for a summer college class that I just finished. I took the final exam yesterday_comma_ and I am not sure what score I will receive.,123,Awesome! Strategic Management sounds difficult by itself_comma_ I hope you get the grade you need!,5|5|5_5|5|5,
hit:2812_conv:5624,1,ashamed,I had sex with somebody that I hate. I feel terrible about myself and my values.,64,I recently had intercourse with somebody that I hate. I feel terrible.,5|5|5_5|5|5,
hit:2812_conv:5624,2,ashamed,I had sex with somebody that I hate. I feel terrible about myself and my values.,422,That sounds awful. Are you going to see them again?,5|5|5_5|5|5,
hit:2812_conv:5624,3,ashamed,I had sex with somebody that I hate. I feel terrible about myself and my values.,64,No_comma_ I do not think I should. This person doesn't make me happy.,5|5|5_5|5|5,
hit:2812_conv:5624,4,ashamed,I had sex with somebody that I hate. I feel terrible about myself and my values.,422,It's probably best that you don't see them_comma_ then. Good thinking.,5|5|5_5|5|5,
hit:2813_conv:5627,1,devastated,When my mom got really ill and was in the hospital I wasn't able to visit her for about a week. It just also happened to be the week that she died. I was broken. I couldn't be there when she needed me the most. ,313,My mom got really sick for a long time before she died. I was doing my best to see are as often as I could but I had work and lived 6 hours away. ,5|5|5_5|5|5,
hit:2813_conv:5627,2,devastated,When my mom got really ill and was in the hospital I wasn't able to visit her for about a week. It just also happened to be the week that she died. I was broken. I couldn't be there when she needed me the most. ,250,That must have been a difficult time for you.,5|5|5_5|5|5,
hit:2813_conv:5627,3,devastated,When my mom got really ill and was in the hospital I wasn't able to visit her for about a week. It just also happened to be the week that she died. I was broken. I couldn't be there when she needed me the most. ,313,The worst thing happened though. I wasn't able to see her for about a week because of not being allowed a bunch of time off,5|5|5_5|5|5,
hit:2813_conv:5627,4,devastated,When my mom got really ill and was in the hospital I wasn't able to visit her for about a week. It just also happened to be the week that she died. I was broken. I couldn't be there when she needed me the most. ,250,The company you work for is horrible. They should have gave you that time off because it was a family emergency.,5|5|5_5|5|5,
hit:2814_conv:5628,1,disgusted,I recently went to eat a restaurant that was not very clean. The whole place had an odd odor and made me uncomfortable.,320,I recently went to eat a restaurant that was not very clean. The whole place had an odd odor and made me uncomfortable.,5|5|5_5|5|5,
hit:2814_conv:5628,2,disgusted,I recently went to eat a restaurant that was not very clean. The whole place had an odd odor and made me uncomfortable.,123,Ew_comma_ that is not good! I wouldn't have been able to stay and eat.,5|5|5_5|5|5,
hit:2814_conv:5628,3,disgusted,I recently went to eat a restaurant that was not very clean. The whole place had an odd odor and made me uncomfortable.,320,I took about two bites from my plate and couldn't eat anymore. I will not be returning.,5|5|5_5|5|5,
hit:2814_conv:5628,4,disgusted,I recently went to eat a restaurant that was not very clean. The whole place had an odd odor and made me uncomfortable.,123,I do not blame you!,5|5|5_5|5|5,
hit:2815_conv:5631,1,nostalgic,I recently returned to my childhood home_comma_ and it brought back a flood of wonderful memories. I am feeling quite sentimental.,320,I recently returned to my childhood home_comma_ and it brought back a flood of wonderful memories. I am feeling quite sentimental.,5|5|5_4|5|5,
hit:2815_conv:5631,2,nostalgic,I recently returned to my childhood home_comma_ and it brought back a flood of wonderful memories. I am feeling quite sentimental.,44,that is awesome_comma_ i feel the same when i drive by my grandparents old home,5|5|5_4|5|5,
hit:2815_conv:5631,3,nostalgic,I recently returned to my childhood home_comma_ and it brought back a flood of wonderful memories. I am feeling quite sentimental.,320,Yes_comma_ it definitely can be an emotional experience. Were/are you close with your grandparents?,5|5|5_4|5|5,
hit:2815_conv:5631,4,nostalgic,I recently returned to my childhood home_comma_ and it brought back a flood of wonderful memories. I am feeling quite sentimental.,44,yes i was,5|5|5_4|5|5,
hit:2817_conv:5635,1,sentimental,I was going through boxes the other day. I came across old pictures I didn't know I still had. It really made me think back.,135,I was going through some boxes the other day. I found some old pictures of my kids I thought were gone,5|5|5_5|5|5,
hit:2817_conv:5635,2,sentimental,I was going through boxes the other day. I came across old pictures I didn't know I still had. It really made me think back.,123,That is exciting! I love having pictures to look back on. ,5|5|5_5|5|5,
hit:2817_conv:5635,3,sentimental,I was going through boxes the other day. I came across old pictures I didn't know I still had. It really made me think back.,135,Yes_comma_ definitely! It really made us all start talking about those and other memories_comma_ good times.,5|5|5_5|5|5,
hit:2817_conv:5635,4,sentimental,I was going through boxes the other day. I came across old pictures I didn't know I still had. It really made me think back.,123,I love being able to reminisce on the past. Time goes by so fast. ,5|5|5_5|5|5,
hit:2821_conv:5643,1,prepared,We had to evacuate our home due to a hurricane. We knew it might happen so we got organized and packed what we would need and put it all in the car.,343,We had to get out of the house because of Hurricane Matthew. It was a scary experience_comma_ but luckily we'd already packed what we needed into the car_comma_ so could just leave.,5|5|5_5|5|5,
hit:2821_conv:5643,2,prepared,We had to evacuate our home due to a hurricane. We knew it might happen so we got organized and packed what we would need and put it all in the car.,227,I had Hurricane Matthew come through where I live too! NE FL_comma_ I know how you feel_comma_ only I couldn't get gas for my car because there was no gas anywhere!,5|5|5_5|5|5,
hit:2821_conv:5643,3,prepared,We had to evacuate our home due to a hurricane. We knew it might happen so we got organized and packed what we would need and put it all in the car.,343,Oh_comma_ you poor thing! Thankfully_comma_ we'd gassed up and were good to go. The house survived too. Were you ok?,5|5|5_5|5|5,
hit:2821_conv:5643,4,prepared,We had to evacuate our home due to a hurricane. We knew it might happen so we got organized and packed what we would need and put it all in the car.,227,Yeah you thought ahead_comma_ me not so much. Our house was fine but a HUGE mess to clean up in the yard and no power for 2 days. I hope your house was ok too!,5|5|5_5|5|5,
hit:2821_conv:5643,5,prepared,We had to evacuate our home due to a hurricane. We knew it might happen so we got organized and packed what we would need and put it all in the car.,343,Ugh. We had to take the walls back to the studs_comma_ it was so bad inside. But thankfully we have wonderful friends_comma_ who accommodated us and helped with the clean-up!,5|5|5_5|5|5,
hit:2822_conv:5645,1,anxious,I was recently waiting to hear the results of a medical exam. Fortunately_comma_ in the end it all turned out ok. But it was nerve wracking.,423,I recently had a doctors appointment that made me very nervous.,5|5|1_5|5|5,
hit:2822_conv:5645,2,anxious,I was recently waiting to hear the results of a medical exam. Fortunately_comma_ in the end it all turned out ok. But it was nerve wracking.,260,Was it a checkup or did you have some health issue?,5|5|1_5|5|5,
hit:2822_conv:5645,3,anxious,I was recently waiting to hear the results of a medical exam. Fortunately_comma_ in the end it all turned out ok. But it was nerve wracking.,423,It started off as a checkup_comma_ but then they requested additional testing.,5|5|1_5|5|5,
hit:2822_conv:5645,4,anxious,I was recently waiting to hear the results of a medical exam. Fortunately_comma_ in the end it all turned out ok. But it was nerve wracking.,260,Did everything go well or do you not know yet?,5|5|1_5|5|5,
hit:2823_conv:5646,1,impressed,My neighbors built a treehouse for their children in just one week! I couldn't believe how fast they completed the project!,320,My neighbors built a treehouse for their children in just one week! I couldn't believe how fast they completed the project!,5|5|5_4|4|5,
hit:2823_conv:5646,2,impressed,My neighbors built a treehouse for their children in just one week! I couldn't believe how fast they completed the project!,126,Oh man_comma_ I bet that is fun,5|5|5_4|4|5,
hit:2823_conv:5646,3,impressed,My neighbors built a treehouse for their children in just one week! I couldn't believe how fast they completed the project!,320,Yeah if only to be a kid again! Have you ever built a treehouse?,5|5|5_4|4|5,
hit:2823_conv:5646,4,impressed,My neighbors built a treehouse for their children in just one week! I couldn't believe how fast they completed the project!,126,No_comma_ I have always wanted one though,5|5|5_4|4|5,
hit:2825_conv:5651,1,sad,I was partially raised by my grandparents and my grandmother recently passed away. She had a good long life though.,423,I recently had to attend a funeral. That's never a fun day.,5|5|5_5|5|5,
hit:2825_conv:5651,2,sad,I was partially raised by my grandparents and my grandmother recently passed away. She had a good long life though.,260,Was it someone you know? Sorry for your loss if it was.,5|5|5_5|5|5,
hit:2825_conv:5651,3,sad,I was partially raised by my grandparents and my grandmother recently passed away. She had a good long life though.,423,Unfortunately yes. It was my grandmother.,5|5|5_5|5|5,
hit:2825_conv:5651,4,sad,I was partially raised by my grandparents and my grandmother recently passed away. She had a good long life though.,260,I'm sorry did you get to see her a lot before she passed?,5|5|5_5|5|5,
hit:2828_conv:5657,1,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,103,I once went to a doc appointment where I literally sat for three hours in the waiting room and then the doc only spent 5 min with me,5|5|5_4|4|4,
hit:2828_conv:5657,2,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,421,Omg! That is terrible. They should really be considerate of people's time,5|5|5_4|4|4,
hit:2828_conv:5657,3,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,103,I wouldn't have minded as much if he actually seemed to care.  If they make you wait and then spend alot of time with you that is one thing,5|5|5_4|4|4,
hit:2828_conv:5657,4,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,421,Right. Did he at least apologize for the long wait,5|5|5_4|4|4,
hit:2828_conv:5657,5,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,103,Oh no he acted like I was bothering him by taking up his time!  It was so annoying!,5|5|5_4|4|4,
hit:2828_conv:5657,6,annoyed,There was this time when I was suppose to have a doctors appointment at this certain time.  well i sat there for literally 3 hours waiting to see the doctor.  Which honestly I wouldn't have minded if he would have spent time with me at the appointment but all he did was spend 5 min and act like he didn't want to be there.  It was annoying.,421,Wow! I would write a review_comma_ and choose another doctor who actually cares about his patient!,5|5|5_4|4|4,
hit:2829_conv:5659,1,anticipating,I am waiting to take a big test coming up! I feel prepared_comma_ yet I am anxious.,320,I am waiting to take a big test coming up! I feel prepared_comma_ yet I am anxious.,5|5|5_5|5|5,
hit:2829_conv:5659,2,anticipating,I am waiting to take a big test coming up! I feel prepared_comma_ yet I am anxious.,260,Good luck! What is it on?,5|5|5_5|5|5,
hit:2829_conv:5659,3,anticipating,I am waiting to take a big test coming up! I feel prepared_comma_ yet I am anxious.,320,Thank you! It is my final exam for a summer Database Applications/Design class that I am taking. ,5|5|5_5|5|5,
hit:2829_conv:5659,4,anticipating,I am waiting to take a big test coming up! I feel prepared_comma_ yet I am anxious.,260,How much studying did you do leading up to it?,5|5|5_5|5|5,
hit:2831_conv:5663,1,excited,I am taking a vacation to Orange Beach_comma_ Alabama in a week! My wife and I visited the area about two years ago_comma_ and we loved it!,320,I am taking a vacation to Orange Beach_comma_ Alabama in a week! My wife and I visited the area about two years ago_comma_ and we loved it!,5|5|5_4|5|4,
hit:2831_conv:5663,2,excited,I am taking a vacation to Orange Beach_comma_ Alabama in a week! My wife and I visited the area about two years ago_comma_ and we loved it!,126,Oh man_comma_ I bet that will be fun,5|5|5_4|5|4,
hit:2831_conv:5663,3,excited,I am taking a vacation to Orange Beach_comma_ Alabama in a week! My wife and I visited the area about two years ago_comma_ and we loved it!,320,We can't wait! Have you ever been to the ocean?,5|5|5_4|5|4,
hit:2831_conv:5663,4,excited,I am taking a vacation to Orange Beach_comma_ Alabama in a week! My wife and I visited the area about two years ago_comma_ and we loved it!,126,I have_comma_ I love going when I can,5|5|5_4|5|4,
hit:2833_conv:5666,1,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,103,One time my husband surprised me on our anniversary with a really pretty necklace and a trip to a Bed and Breakfast!,5|5|5_5|5|5,
hit:2833_conv:5666,2,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,260,That's really nice of him! How long have you been married?,5|5|5_5|5|5,
hit:2833_conv:5666,3,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,103,We have been married for 14 years.  Another reason it was so surprising was b/c he usually just takes me to dinner!,5|5|5_5|5|5,
hit:2833_conv:5666,4,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,260,Well you made the right choice in man to marry lol. What kind of necklace?,5|5|5_5|5|5,
hit:2833_conv:5666,5,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,103,It was one of those keys to your heart type.  Do you know what I mean?,5|5|5_5|5|5,
hit:2833_conv:5666,6,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,260,That either sounds like a locket_comma_ or a heart shaped key. Am I close at all?,5|5|5_5|5|5,
hit:2833_conv:5666,7,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,103,Yes the ones from Kay Jewelers that is a key with the heart at the top,5|5|5_5|5|5,
hit:2833_conv:5666,8,surprised,Once my husband surprised me on my anniversary with a really pretty necklace and a trip to a Bed and Breakfast.   I was so suprised b/c he usually only takes me to dinner for our anniversary.,260,Cool I think I saw a commercial for that actually.,5|5|5_5|5|5,
hit:2834_conv:5669,1,embarrassed,I went to the store first thing the other morning for some sodas and had a rough time. I dropped my money_comma_ ran into the door_comma_ and the bag busted sending a 2 liter bouncing away.,135,I went to the store for sodas and it ended up being a rough trip.,5|5|5_5|5|5,
hit:2834_conv:5669,2,embarrassed,I went to the store first thing the other morning for some sodas and had a rough time. I dropped my money_comma_ ran into the door_comma_ and the bag busted sending a 2 liter bouncing away.,160,Ut oh.. what happened?,5|5|5_5|5|5,
hit:2834_conv:5669,3,embarrassed,I went to the store first thing the other morning for some sodas and had a rough time. I dropped my money_comma_ ran into the door_comma_ and the bag busted sending a 2 liter bouncing away.,135,I dropped my money while my hands were full_comma_ ran into the door_comma_ and the bag busted sending a 2 liter bouncing through the parking lot. And of course there were a lot of people that saw.,5|5|5_5|5|5,
hit:2834_conv:5669,4,embarrassed,I went to the store first thing the other morning for some sodas and had a rough time. I dropped my money_comma_ ran into the door_comma_ and the bag busted sending a 2 liter bouncing away.,160,hehehe.. don't mind if I giggle do you?  You need to remember... don't feel embarrassed bout silly stuff like that.  Those people don't know you or really care.  They laugh like I did because its fun to see things happen to other people that we fear. ,5|5|5_5|5|5,
hit:2835_conv:5670,1,jealous,esterday my husband was very well arranged for a party alone_comma_ I'm suspecting,264,I suspect that my husband is fixing a lot to go to a party_comma_ I suspect he is going to see someone,4|5|4_4|4|5,
hit:2835_conv:5670,2,jealous,esterday my husband was very well arranged for a party alone_comma_ I'm suspecting,126,What are you going to do about it,4|5|4_4|4|5,
hit:2835_conv:5670,3,jealous,esterday my husband was very well arranged for a party alone_comma_ I'm suspecting,264,I'm going to take some time and I'll be looking at it with caution,4|5|4_4|4|5,
hit:2835_conv:5670,4,jealous,esterday my husband was very well arranged for a party alone_comma_ I'm suspecting,126,Well be careful for sure,4|5|4_4|4|5,
hit:2835_conv:5670,5,jealous,esterday my husband was very well arranged for a party alone_comma_ I'm suspecting,264,sure ... thanks,4|5|4_4|4|5,
hit:2837_conv:5674,1,jealous,My sister is always the sibling that my parents talk about. Sometimes I wonder if they even notice me.,320,My sister is always the sibling that my parents talk about. Sometimes I wonder if they even notice me.,5|5|5_4|4|4,
hit:2837_conv:5674,2,jealous,My sister is always the sibling that my parents talk about. Sometimes I wonder if they even notice me.,75,I am sorry to hear that. Have you ever talked to your parents about that?,5|5|5_4|4|4,
hit:2837_conv:5674,3,jealous,My sister is always the sibling that my parents talk about. Sometimes I wonder if they even notice me.,320,I haven't. I'm scared that it would just make the situation worse. Do you think I should tell them how I feel?,5|5|5_4|4|4,
hit:2837_conv:5674,4,jealous,My sister is always the sibling that my parents talk about. Sometimes I wonder if they even notice me.,75,Yes_comma_ you do.Otherwise_comma_ you are going to carry that pain the rest of your life.,5|5|5_4|4|4,
hit:2838_conv:5677,1,grateful,I recently received a raise at work. It came at a good time and was much needed.,423,My boss finally realized all the hard work I've been doing lately.,5|5|5_5|5|5,
hit:2838_conv:5677,2,grateful,I recently received a raise at work. It came at a good time and was much needed.,243,That is great that your hard work has paid off.,5|5|5_5|5|5,
hit:2838_conv:5677,3,grateful,I recently received a raise at work. It came at a good time and was much needed.,423,Things have been slow at work and no one has gotten any raises lately_comma_ but mine finally came through at a good time!,5|5|5_5|5|5,
hit:2838_conv:5677,4,grateful,I recently received a raise at work. It came at a good time and was much needed.,243,Now you having something to celebrate. Congrats!,5|5|5_5|5|5,
hit:2838_conv:5677,5,grateful,I recently received a raise at work. It came at a good time and was much needed.,423,Thanks. And it's the weekend. Perfect timing.,5|5|5_5|5|5,
hit:2840_conv:5681,1,angry,I was on an airplane last week_comma_ and someone's baby would not stop crying. I know it isn't right to be upset about such things_comma_ but it really bothered me.,320,I was on an airplane last week_comma_ and someone's baby would not stop crying. I know it isn't right to be upset about such things_comma_ but it really bothered me.,5|5|5_4|5|5,
hit:2840_conv:5681,2,angry,I was on an airplane last week_comma_ and someone's baby would not stop crying. I know it isn't right to be upset about such things_comma_ but it really bothered me.,260,Dont worry_comma_ anyone would have felt the same way. If it makes you feel better remember that we were all babies once.,5|5|5_4|5|5,
hit:2840_conv:5681,3,angry,I was on an airplane last week_comma_ and someone's baby would not stop crying. I know it isn't right to be upset about such things_comma_ but it really bothered me.,320,I tried to think about that_comma_ but I was still aggravated. On the plus side_comma_ at least I was self aware that it was somewhat irrational.,5|5|5_4|5|5,
hit:2840_conv:5681,4,angry,I was on an airplane last week_comma_ and someone's baby would not stop crying. I know it isn't right to be upset about such things_comma_ but it really bothered me.,260,Yeah thats important_comma_ did you have music you could have listened to or something?,5|5|5_4|5|5,
hit:2841_conv:5682,1,faithful,I was given a job offer at a new company. I decided to stay with my current organization out of loyalty.,320,I was given a job offer at a new company. I decided to stay with my current organization out of loyalty.,5|5|5_4|5|4,
hit:2841_conv:5682,2,faithful,I was given a job offer at a new company. I decided to stay with my current organization out of loyalty.,126,Loyalty is a good thing,5|5|5_4|5|4,
hit:2841_conv:5682,3,faithful,I was given a job offer at a new company. I decided to stay with my current organization out of loyalty.,320,Yes_comma_ I think so as well. The new company offered me approximately $10_comma_000 more a year than my current business is paying though.,5|5|5_4|5|4,
hit:2841_conv:5682,4,faithful,I was given a job offer at a new company. I decided to stay with my current organization out of loyalty.,126,Oh wow_comma_ that is loyalty for sure,5|5|5_4|5|4,
hit:2842_conv:5684,1,grateful,My grandma sent me a birthday card in the mail! She is such a sweet lady.,320,My grandma sent me a birthday card in the mail! She is such a sweet lady.,5|5|5_5|5|5,
hit:2842_conv:5684,2,grateful,My grandma sent me a birthday card in the mail! She is such a sweet lady.,160,I wish I had a grandma.. did she send you anything extra?,5|5|5_5|5|5,
hit:2842_conv:5684,3,grateful,My grandma sent me a birthday card in the mail! She is such a sweet lady.,320,I can understand that. She is the only grandparent that I have left. She only sent a card_comma_ but she is living on a fixed income.,5|5|5_5|5|5,
hit:2842_conv:5684,4,grateful,My grandma sent me a birthday card in the mail! She is such a sweet lady.,160,That is still really sweet.  I never knew any of mine.  Treasure her while you can!,5|5|5_5|5|5,
hit:2843_conv:5686,1,terrified,A tornado blew through my town last week. My family hid in our storm shelter_comma_ but I wasn't sure if we were going to survive it unscathed.,320,A tornado blew through my town last week. My family hid in our storm shelter_comma_ but I wasn't sure if we were going to survive it unscathed.,5|5|5_5|5|5,
hit:2843_conv:5686,2,terrified,A tornado blew through my town last week. My family hid in our storm shelter_comma_ but I wasn't sure if we were going to survive it unscathed.,274,Wow_comma_ must have been rather frightening. Glad you are ok!,5|5|5_5|5|5,
hit:2843_conv:5686,3,terrified,A tornado blew through my town last week. My family hid in our storm shelter_comma_ but I wasn't sure if we were going to survive it unscathed.,320,Thank you! We were rather frightened_comma_ and we feel very fortunate. Tornadoes are quite the force of nature. No pun intended.,5|5|5_5|5|5,
hit:2843_conv:5686,4,terrified,A tornado blew through my town last week. My family hid in our storm shelter_comma_ but I wasn't sure if we were going to survive it unscathed.,274,I hope you have all the emergency essentials you need for next time.,5|5|5_5|5|5,
hit:2844_conv:5688,1,caring,the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,264,Hello the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,4|5|4_4|5|5,<HI>
hit:2844_conv:5688,2,caring,the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,126,Gardening can be peaceful,4|5|4_4|5|5,
hit:2844_conv:5688,3,caring,the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,264,Especially when you feel something special by nature,4|5|4_4|5|5,
hit:2844_conv:5688,4,caring,the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,126,Nature is a wonderful thing,4|5|4_4|5|5,
hit:2844_conv:5688,5,caring,the plants in my garden I water them_comma_ I move the earth_comma_ I try to keep them happy,264,I hope this feeling increases,4|5|4_4|5|5,
hit:2845_conv:5690,1,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,423,My wife has a big birthday coming up and I've got a great weekend all laid out for her.,5|5|5_5|5|5,
hit:2845_conv:5690,2,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,103,Oh how exciting!  What do you have planned?,5|5|5_5|5|5,
hit:2845_conv:5690,3,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,423,We are going to eat tonight. Then tomorrow we are having a big surprise party with her best friend. I even had a couple of her friends fly into town that she hasnt seen in years.,5|5|5_5|5|5,
hit:2845_conv:5690,4,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,103,I bet she is going to be so surprised!  Is she hard to surpise?,5|5|5_5|5|5,
hit:2845_conv:5690,5,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,423,Yes she kind of is. I dont think she knows_comma_ but I can't really tell for sure.,5|5|5_5|5|5,
hit:2845_conv:5690,6,prepared,My wife's birthday is coming up. I got her a gift and the party planned out way ahead of time this year.,103,Well I'm sure everything will go great!,5|5|5_5|5|5,
hit:2845_conv:5691,1,guilty,Once when I was in college this guy that I really didn't like more than a friend asked me on a date to go get some ice cream.  ,103,Once in college this guy I was just friends with asked me out to get ice cream.  I could tell he really liked me and I didn't want to lead him on so I told him I had a headache.  I know that he knew I was lying and I still feel guilty to this day about that!,5|5|5_5|5|5,
hit:2845_conv:5691,2,guilty,Once when I was in college this guy that I really didn't like more than a friend asked me on a date to go get some ice cream.  ,423,yeah that's tough. But can you imagine how bad you might feel if he had a REALLY good time on the date and really thought things were going somewhere with the two of you?,5|5|5_5|5|5,
hit:2845_conv:5691,3,guilty,Once when I was in college this guy that I really didn't like more than a friend asked me on a date to go get some ice cream.  ,103,Yes_comma_ I know thats what I was thinking.  I just don't know if I should have said the truth that I didn't like him that way instead of lying and saying I had a headache.,5|5|5_5|5|5,
hit:2845_conv:5691,4,guilty,Once when I was in college this guy that I really didn't like more than a friend asked me on a date to go get some ice cream.  ,423,That is true. Either way it seems like he definitely got the hint. ,5|5|5_5|5|5,
hit:2851_conv:5702,1,embarrassed,Last week I had Thai food for lunch_comma_ and it didn't sit well with my stomach. I had some explosive issues later in the day at work_comma_ and the bathroom was near all of my coworkers' cubicles. Needless to say_comma_ the stench ended the workday a little early.,320,Last week I had Thai food for lunch_comma_ and it didn't sit well with my stomach. I had some explosive issues later in the day at work_comma_ and the bathroom was near all of my coworkers' cubicles. Needless to say_comma_ the stench ended the workday a little early.,5|5|5_4|4|5,
hit:2851_conv:5702,2,embarrassed,Last week I had Thai food for lunch_comma_ and it didn't sit well with my stomach. I had some explosive issues later in the day at work_comma_ and the bathroom was near all of my coworkers' cubicles. Needless to say_comma_ the stench ended the workday a little early.,126,Oh wow_comma_ I have heard that can happen with that food,5|5|5_4|4|5,
hit:2851_conv:5702,3,embarrassed,Last week I had Thai food for lunch_comma_ and it didn't sit well with my stomach. I had some explosive issues later in the day at work_comma_ and the bathroom was near all of my coworkers' cubicles. Needless to say_comma_ the stench ended the workday a little early.,320,Yes_comma_ it was quite the uncomfortable experience. Now all of my coworkers have given me the nickname 'Diarrhea Joe',5|5|5_4|4|5,
hit:2851_conv:5702,4,embarrassed,Last week I had Thai food for lunch_comma_ and it didn't sit well with my stomach. I had some explosive issues later in the day at work_comma_ and the bathroom was near all of my coworkers' cubicles. Needless to say_comma_ the stench ended the workday a little early.,126,Ha_comma_ that is funny,5|5|5_4|4|5,
hit:2852_conv:5705,1,disgusted,I was on a road trip last week and had to stop a gas station to use the restroom. It looked like they had never cleaned the bathroom_comma_ ever!,320,I was on a road trip last week and had to stop at gas station to use the restroom.,5|5|5_3|3|3,
hit:2852_conv:5705,2,disgusted,I was on a road trip last week and had to stop a gas station to use the restroom. It looked like they had never cleaned the bathroom_comma_ ever!,68,ewww! that is just super gross dude!,5|5|5_3|3|3,
hit:2852_conv:5705,3,disgusted,I was on a road trip last week and had to stop a gas station to use the restroom. It looked like they had never cleaned the bathroom_comma_ ever!,320,I know! The worst part is that I have irritable bowel syndrome_comma_ and I made the bathroom about twice as bad as it was initially!,5|5|5_3|3|3
hit:2852_conv:5705,4,disgusted,I was on a road trip last week and had to stop a gas station to use the restroom. It looked like they had never cleaned the bathroom_comma_ ever!,68,Oh man that is just crazy! Feel bad for the person who has to clean it.,5|5|5_3|3|3,



"""

In [71]:
split_lines = test_issues.split(",")
cleaned_split_lines = []

for line in split_lines:
    cleaned_split_lines.append(line.strip())

conversations = []
for i in range(0,len(cleaned_split_lines),7):
    conversations.append(cleaned_split_lines[i:i+7])
pd.DataFrame.from_records(conversations).to_csv("cleaned_up_conv_test.csv",index=False)